<h1><b>QOF</b></h1>

** The Quality Indicator Indicator Groups may change on an annual basis. 

** To prevent the need to manually update these they have been assigned proxy values QI_1, QI_2 and QI_3.

## Table of Contents


[Import packages](#Import-packages)

[1. Import data](#1.-Import-data)

[2. General Excel template inputs](#2.-General-Excel-template-inputs)

[3. Validate data](#3.-Validate-data)

[4. CSV outputs](#4.-CSV-outputs)

[5. Create Excel table content](#5.-Create-Excel-table-content)

[6. Populate Excel templates](#6.-Populate-Excel-templates)

* [Template 3 Achievement Excel table](#Template-3-Achievement-Excel-table)
* [Template 4 PCA Excel table](#Template-4-PCA-Excel-table)
* [Template 6 National and Regional Excel table](#Template-6-National-and-Regional-Excel-table)
* [Template 8 ICB (STP) Excel table](#Template-8-ICB-(STP)-Excel-table)
* [Template 9 Sub ICB (CCG) Excel table](#Template-9-Sub-ICB-(CCG)-Excel-table)
* [Template 10 CV practice Excel table](#Template-10-CV-practice-Excel-table)
* [Template 11 RESP practice Excel table](#Template-11-RESP-practice-Excel-table)
* [Template 12 LS practice Excel table](#Template-12-LS-practice-Excel-table)
* [Template 13 HD practice Excel table](#Template-13-HD-practice-Excel-table)
* [Template 14 NEU practice Excel table](#Template-14-NEU-practice-Excel-table)
* [Template 15 MS practice Excel table](#Template-15-MS-practice-Excel-table)
* [Template 16 FER OBS GYN practice Excel table](#Template-16-FER-OBS-GYN-practice-Excel-table)
* [Template 17 QI practice Excel table](#Template-17-QI-practice-Excel-table)
* [Template 19 VI practice Excel table](#Template-19-VI-practice-Excel-table)





## Import packages
[Return to contents](#Table-of-Contents)

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Is this a test run?

In [ ]:
## A test run will only output files to the local drive for testing
## Select False for publication

test_run = True
#test_run = False

Import all required packages

In [ ]:
import pyodbc as dbc
import pandas as pd
import numpy as np
import json
import os
import getpass
import datetime
import openpyxl
from dateutil.relativedelta import relativedelta
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, Fill
pd.set_option('display.max_rows', None)


In [ ]:
#Loading config file in
with open("config.json") as f:
    json_data = f.read()
config = json.loads(json_data)

server = config["Server"]
database = config["Database"]

#since I cant stick a literal path in the config, I have to change the double backlashes to single
root = os.path.normpath(config["root"])

In [ ]:
def create_test_folder(test_run):

    # If test equals true this function creates a test folder located in the current 
    # working directory if one doesn't already exist.
    
    # If test equals false this function does nothing and outputs will be written to 
    # the network drive.

    if test_run == True:
        try: 
            os.mkdir('.\\test_folder')
            print('Test folder created')
        except FileExistsError:
            print('Test folder already exists')
    else:
        print('WARNING: Outputs will be written to network drive')
            
create_test_folder(test_run)

In [ ]:
## Create connection string
def connect(conn_str):
    try:
        conn = dbc.connect (conn_str)
    except:
        raise Exception("Database connection unsuccessful")
        conn = None
    return conn

## INPUTS
# SQL connection string

## OUTPUTS
# connection

In [ ]:
## Connect to SQL server
ref_data_string = ("Driver=SQL Server;"
                  f"Server={server};"
                  f"Database={database};"
                  "Trusted_Connection=yes;")
sql_ref_data = connect(ref_data_string)

# Define file paths

In [ ]:
inputs = rf"{root}\LIVE\PROCESS_1\INPUTS" 
templates = rf"{root}\LIVE\PROCESS_1\INPUTS\Templates"
outputs = rf"{root}\LIVE\PROCESS_1\OUTPUTS"
dnp = rf"{root}\LIVE\PROCESS_1\OUTPUTS\DO NOT PUBLISH"

checking = rf"{root}\CHECKING\Additional information"

In [ ]:
"""
Creates a data frame of required age bands including 'TOTAL_LIST_SIZE' used by 'Validation rule 4' 
and 'Final list size' code blocks.

Individual age bands are used to create data columns 'ADD_PAT_LIST' (with value 'ADD_LIST_SIZE') 
and 'ALT_PAT_LIST' (with value 'ALT_LIST_SIZE'), used by code block in 'Alternative list size' section
"""

def create_list_size(list_size_extract_date):  
    ## Import data from list size table where 'EXTRACT_DATE' is the variable 'list_size_extract_date'
    ## created in 'Date variables' code block below
    list_size_query = """SELECT * FROM [GP_PATIENT_LIST]
                         WHERE EXTRACT_DATE >= ?"""
    
    list_size = pd.read_sql_query(list_size_query, sql_ref_data, params=(list_size_extract_date,))
    
    ## Aggregate age bands and build the data frame from them
   
    list_size_cols_m = ["MALE_" + str(i) + "_" + str(i + 1) for i in range(0, 120)]
    list_size_cols_f = ["FEMALE_" + str(i) + "_" + str(i + 1) for i in range(0, 120)]

    list_size["UN01"] = list_size[list_size_cols_m[0:1]+list_size_cols_f[0:1]].sum(axis=1)
    list_size["1_2"] = list_size[list_size_cols_m[1:3]+list_size_cols_f[1:3]].sum(axis=1)
    list_size["4_5"] = list_size[list_size_cols_m[4:6]+list_size_cols_f[4:6]].sum(axis=1)
    list_size["79_80"] = list_size[list_size_cols_m[79:81]+list_size_cols_f[79:81]].sum(axis=1)
    list_size["06OV"] = list_size[list_size_cols_m[6:]+list_size_cols_f[6:]].sum(axis=1)
    list_size["06_19"] = list_size[list_size_cols_m[6:20]+list_size_cols_f[6:20]].sum(axis=1)
    list_size["15OV"] = list_size[list_size_cols_m[15:]+list_size_cols_f[15:]].sum(axis=1)
    list_size["16OV"] = list_size[list_size_cols_m[16:]+list_size_cols_f[16:]].sum(axis=1)
    list_size["17OV"] = list_size[list_size_cols_m[17:]+list_size_cols_f[17:]].sum(axis=1)
    list_size["18OV"] = list_size[list_size_cols_m[18:]+list_size_cols_f[18:]].sum(axis=1)
    list_size["40OV"] = list_size[list_size_cols_m[40:]+list_size_cols_f[40:]].sum(axis=1)
    list_size["45OV"] = list_size[list_size_cols_m[45:]+list_size_cols_f[45:]].sum(axis=1)
    list_size["50OV"] = list_size[list_size_cols_m[50:]+list_size_cols_f[50:]].sum(axis=1)
    list_size["79UN"] = list_size[list_size_cols_m[:80]+list_size_cols_f[:80]].sum(axis=1)
    list_size["80OV"] = list_size[list_size_cols_m[80:]+list_size_cols_f[80:]].sum(axis=1)
    list_size["25_49_F"] = list_size[list_size_cols_f[25:50]].sum(axis=1)
    list_size["50_64_F"] = list_size[list_size_cols_f[50:65]].sum(axis=1)
    list_size["25_64_F"] = list_size[list_size_cols_f[25:65]].sum(axis=1)
    list_size["TOTAL_LIST_SIZE"] = list_size[list_size_cols_m + list_size_cols_f].sum(axis=1)

    list_size = list_size[[
    "PRACTICE_CODE","EXTRACT_DATE"
    ,"UN01","1_2","4_5","79_80"
    ,"06OV","06_19","15OV","16OV"
    ,"17OV","18OV","40OV","45OV"
    ,"50OV","79UN","80OV","25_49_F"
    ,"50_64_F","25_64_F","TOTAL_LIST_SIZE"
    ]]
    
    return list_size


## INPUTS
# None

## OUTPUTS
# list_size

## 1. Import data
[Return to contents](#Table-of-Contents)

In [ ]:
## Import all data needed for QOF publication

QOF_Mapping_Table = pd.read_csv(inputs+
                                "\GEOG_MAPPINGS\QOF_MANUAL_MAPPINGS.csv"
                                ,encoding="windows-1252")

Control_File = pd.read_csv(inputs+
                           "\CONTROL_FILES\CONTROL_FILE.csv")


PCA_Ref_Table = pd.read_csv(inputs+
                            "\CONTROL_FILES\PCA_REFERENCE.csv"
                            ,encoding="windows-1252")

Validation_Rules_Table = pd.read_csv(inputs+
                                     "\CONTROL_FILES\VALIDATION_RULES.csv")

Indicator_Reference_Control_Table = pd.read_csv(inputs+
                                                
                                                "\CONTROL_FILES\INDICATOR_REFERENCE_CONTROL.csv")
Indicator_Change_Message_Table = pd.read_csv(inputs+
                                             "\CONTROL_FILES\INDICATOR_GROUP_CHANGE_MESSAGE.csv")

practice_list = pd.read_csv(inputs+
                            "\CQRS_DATA\FINAL_PRACTICE_LIST.csv", usecols=["SP_ORG_CODE"])

Indicator_Values_Table_base = pd.read_csv(inputs+ "\CQRS_DATA\INDICATOR_VALUES.csv"
                                          ,usecols=["ORG_CODE","IND_CODE","FIELD_NAME","VALUE","LAST_SUBMISSION"]
                                          ,error_bad_lines=False, engine="python")

Indicator_Achieved_Points_Table = pd.read_csv(inputs + 
                                              "\CQRS_DATA\INDICATOR_ACHIEVED_POINTS.csv"
                                              , usecols=["ORG_CODE","IND_CODE","POINTS","APPROVED"])

Previous_Prevalence_Non_Register_Indicators = pd.read_csv(inputs + 
                                                          "\PREVIOUS_YEAR\PREVIOUS_PREVALENCE_NON_REGISTER_INDICATORS.csv"
                                                          ,encoding="windows-1252")

Previous_Prevalence_Non_Register_Alt_Indicators = pd.read_csv(inputs + 
                                                              "\PREVIOUS_YEAR\PREVIOUS_PREVALENCE_NON_REGISTER_ALT_INDICATORS.csv"
                                                              , encoding="windows-1252")

Previous_Prevalence = pd.read_csv(inputs + 
                                  "\PREVIOUS_YEAR\PREVIOUS_PREVALENCE.csv"
                                  , encoding="windows-1252")

Previous_Achievements_PCAs = pd.read_csv(inputs + 
                                         "\PREVIOUS_YEAR\PREVIOUS_ACHIEVEMENTS_PCAS.csv"
                                         ,encoding="windows-1252")

Previous_Indicator_Control = pd.read_csv(inputs + 
                                         "\PREVIOUS_YEAR\PREVIOUS_INDICATOR_CONTROL.csv"
                                         ,encoding="windows-1252")

Previous_Indicator_Mappings = pd.read_csv(inputs + 
                                          "\PREVIOUS_YEAR\PREVIOUS_INDICATOR_MAPPINGS.csv"
                                          ,encoding="windows-1252")

Previous_Organisation_Reference = pd.read_csv(inputs+ 
                                              "\PREVIOUS_YEAR\PREVIOUS_ORGANISATION_REFERENCE.csv"
                                              ,encoding="windows-1252")


## Date variables
#### Creates a range of date variables used through out the code

In [ ]:
## Financial year YYYY/YY
Fyear = Control_File.at[0, "FYEAR"]

## Reformat financial year as YYYY
Short_year_str = str(Fyear)
Short_start_year = Short_year_str[2:4]
Short_end_year = Short_year_str[5:7]
short_year = Short_start_year+Short_end_year

## Publication date reformated
Pub_date = Control_File.at[0, "PubDate"]
Pub_date = datetime.datetime.strptime(Pub_date, "%d/%m/%Y").date()
Publication_date = Pub_date.strftime("%d %B %Y")

## Copyright statement with correct year
Copyright_date = Control_File.at[0, "PubDate"]
Copyright_date = Copyright_date[-4:]
Copyright = "Copyright © {}, NHS England.".format(Copyright_date)

## Previous year variable
Previous_year = Control_File.at[0, "PREVIOUS_FYEAR"]

## Mapping date - used by 8. Mapping
mapping_date = Control_File.at[0, "PRAC_MAPPING_DATE"] 
mapping_date = datetime.datetime.strptime(mapping_date, "%d/%m/%Y").date()
mapping_date = mapping_date.strftime("%d %B %Y")

## List size date usually 1 April - used by Validation rule 4
list_size_date = Control_File.at[0, "LIST_SIZE_DATE"]
list_size_extract_date = (pd.to_datetime(list_size_date) ).strftime('%Y-%d-%m')


## List size date minus 3 months is usually 1 January - used by
list_size_date_minus_3mths = datetime.datetime.strptime(list_size_date, "%d/%m/%Y").date()
list_size_date_minus_3mths = datetime.date(list_size_date_minus_3mths.year
                                           ,list_size_date_minus_3mths.month
                                           ,list_size_date_minus_3mths.day) - relativedelta(months=3)

## PRAC_MAPPING_DATE is the date at which the practices status ('A' = active, 'C' = closed, 'D' = dormant, 
# 'B' = retired, 'P' = proposed) was taken - reformated and used by validation rule 3
prac_mapping_date = Control_File.at[0, "PRAC_MAPPING_DATE"]
prac_mapping_date = datetime.datetime.strptime(prac_mapping_date, "%d/%m/%Y").strftime("%Y-%m-%d")
prac_mapping_date = "'{0}'".format(prac_mapping_date)


<h3>Quality Improvement indicator groups assigned <br>generic key to allow for changes to groups year on year</h3>

In [ ]:
## Filter for QI group codes and indicator codes
QI_df = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["HIGHER_GROUP_CODE"] == "QI"
                                              ,["GROUP_CODE","INDICATOR_CODE"]].reset_index(drop = True)
                                              

## QI indicator codes only - used in Indicator Values.csv
QI_codes = QI_df[["INDICATOR_CODE"]]

## QI group codes used to create generic QI key ('GrC_KEY')
QI_gr_codes_with_key = pd.DataFrame(QI_df, columns=["GROUP_CODE"]
                                    ).drop_duplicates("GROUP_CODE"
                                    ).reset_index(drop=True)

## Assigns the index value to a new column 'GrC_KEY'
QI_gr_codes_with_key["GrC_KEY"] = QI_gr_codes_with_key.index

## Changes column 'GrC_KEY' to an integer and increments it by one, this removes the 
## zero starting point and replaces it by a one
QI_gr_codes_with_key["GrC_KEY"] = QI_gr_codes_with_key["GrC_KEY"].astype(int)+1

## Changes column 'GrC_KEY' to a string and prefaces it with 'QI_'
## e.g. QI_1
QI_gr_codes_with_key["GrC_KEY"] = "QI_"+ QI_gr_codes_with_key["GrC_KEY"].astype(str)

<b><h2>CQRS Data</h2></b>
#### FINAL_PRACTICE_LIST.csv

In [ ]:
""" 
Data taken from CQRS output FINAL_PRACTICE_LIST 
Used in 
Validation rules 1,3 and 4; 
4. Final Practice List; 
6. Achievement Data

"""

practice_list = practice_list.rename(columns={"SP_ORG_CODE":"PRACTICE_CODE"}
                            ).sort_values(by = ["PRACTICE_CODE"]
                            ).drop_duplicates("PRACTICE_CODE"
                            ).reset_index(drop=True)                           

## Number of practices imported before validation rules are applied
print(len(practice_list.index))

#### INDICATOR_VALUES.csv

In [ ]:
"""
Data taken from CQRS output INDICATOR_VALUES
Convert Y/N values for Quality Improvement indicators to numeric values
Quality Improvement indicators have boolean values Y/N which are either 
converted to the indicator values in the case of Y values or data row removed in the case of N values.

"""

Indicator_Values_Table_base = Indicator_Values_Table_base.loc[(Indicator_Values_Table_base["LAST_SUBMISSION"]=="Y"
                                                               )&(Indicator_Values_Table_base["VALUE"]!="N")]

## Data frame contains 'PRACTICE_CODE', 'INDICATOR_CODE', 'SHORT_NAME', 'VALUE'
Indicator_Values_Table_base = Indicator_Values_Table_base.rename(columns={"ORG_CODE":"PRACTICE_CODE"
                                                                          ,"FIELD_NAME":"SHORT_NAME"
                                                                          ,"IND_CODE":"INDICATOR_CODE"}
                                                                 ).drop(columns=["LAST_SUBMISSION"])

ind_ref_cont_with_ind_values = pd.merge(Indicator_Values_Table_base
                                        ,Indicator_Reference_Control_Table
                                        ,left_on=["INDICATOR_CODE"]
                                        ,right_on=["INDICATOR_CODE"]
                                        ,how="left")

## Replaces string values for QI indicators with their numeric value equivalent
ind_ref_cont_with_ind_values["VALUE"] = np.where((ind_ref_cont_with_ind_values["INDICATOR_CODE"].isin(QI_codes["INDICATOR_CODE"])
                                                  )&(ind_ref_cont_with_ind_values["VALUE"]=="Y")
                                                 ,ind_ref_cont_with_ind_values["INDICATOR_POINT_VALUE"]
                                                 ,ind_ref_cont_with_ind_values["VALUE"])


ind_ref_cont_with_ind_values["VALUE"] = ind_ref_cont_with_ind_values["VALUE"].astype(int)


## Replaces 'SHORT_NAME' for QI codes with the indicator code
ind_ref_cont_with_ind_values["SHORT_NAME"] = np.where(ind_ref_cont_with_ind_values["SHORT_NAME"] == "Yes/No Flag"
                                                      ,ind_ref_cont_with_ind_values["INDICATOR_CODE"]
                                                      ,ind_ref_cont_with_ind_values["SHORT_NAME"])

## Defines columns in final data frame
ind_ref_cont_with_ind_cols = ["PRACTICE_CODE","INDICATOR_CODE","SHORT_NAME","VALUE"]
ind_ref_cont_with_ind_values = ind_ref_cont_with_ind_values[ind_ref_cont_with_ind_cols]

#### INDICATOR_ACHIEVED_POINTS.csv

In [ ]:
"""
Remove any anomalous data (often additional rows of MI data not used by the publication) by only importing rows 
where the indicator code is pesent in the Indicator_Reference_Control_Table

"""

## Renames columns 
Indicator_Achieved_Points_Table = Indicator_Achieved_Points_Table.rename(columns={"ORG_CODE":"PRACTICE_CODE"
                                                                                  ,"IND_CODE":"INDICATOR_CODE"
                                                                                  ,"POINTS":"INDICATOR_POINTS"})

## only selects INDICATOR_CODE from the data frame
ind_ref_control_table_ind_code_cols = ["INDICATOR_CODE"]
ind_ref_control_table_ind_code = Indicator_Reference_Control_Table[ind_ref_control_table_ind_code_cols]

## Inner join ensures any anomalous data rows are excluded
ind_ach_points_df = pd.merge(Indicator_Achieved_Points_Table
                             ,ind_ref_control_table_ind_code
                             ,left_on=["INDICATOR_CODE"]
                             ,right_on=["INDICATOR_CODE"]
                             ,how="inner")

<b><h3>Previous Years Data</h3> 

In [ ]:
"""
Calculates the indicator group points values for the previous year - used in the excel table outputs 

"""

previous_gr_points_cols = ["INDICATOR_GROUP_CODE","INDICATOR_POINT_VALUE"]
previous_gr_points = Previous_Indicator_Mappings[previous_gr_points_cols]

previous_gr_points = previous_gr_points.groupby(by = ["INDICATOR_GROUP_CODE"]
                                          ,as_index=False
                                          ).sum().rename(columns={"INDICATOR_POINT_VALUE":"GROUP_POINT_VALUE"
                                                                  ,"INDICATOR_GROUP_CODE":"GROUP_CODE"})
                                                           

In [ ]:
"""
Calculates the domain points values for the previous year - used in the excel table outputs 
2. General excel templates inputs

"""

previous_dom_points_cols = ["DOMAIN_CODE","INDICATOR_POINT_VALUE"]
previous_dom_points = Previous_Indicator_Mappings[previous_dom_points_cols]

previous_dom_points = previous_dom_points.groupby(by = ["DOMAIN_CODE"]
                                            ,as_index=False
                                            ).sum().rename(columns={"INDICATOR_POINT_VALUE":"DOMAIN_POINT_VALUE"})
                                            
                                            

<b><h2>Reference variables</h2>

In [ ]:
"""
Creates a 'FLAG' value to indicate if the indicator group table needs a change message  
under the excel table heading in the outputs.

Indicator groups requring a message have a numeric value in INDICATOR_GROUP_CHANGE_NOTE  
column of the Indicator_Reference_Control_Table.

Change messages are stored in Indicator_Change_Message_Table with a numeric key that 
corresponds to the number in the INDICATOR_GROUP_CHANGE_NOTE column.

"""

grp_change_flag_cols = ["GROUP_CODE", "INDICATOR_GROUP_CHANGE_NOTE"]
grp_change_flag = Indicator_Reference_Control_Table[grp_change_flag_cols]

grp_change_flag = pd.merge(grp_change_flag
                           ,Indicator_Change_Message_Table
                           ,left_on="INDICATOR_GROUP_CHANGE_NOTE"
                           ,right_on="INDICATOR_GROUP_CHANGE_NOTE"
                           ,how="left")

In [ ]:
"""
Creates a group code data frame "PUB_ORDER","GROUP_CODE", "GROUP_DESCRIPTION", "HIGHER_GROUP_CODE", 
"HIGHER_GROUP_DESCRIPTION","SUB_DOMAIN_CODE", "DOMAIN_CODE".

Thia data frame is used by
'POINTS_BY_INDICATOR';
'13 Revised Maximun Points';
'Create variables used in table titles;
Regional and National, ICB, SUB_ICB and Practice tables';
'Create T10 to T19 Practice Table titles'

"""

gr_ref_fil_df_cols = ["GROUP_CODE", "INDICATOR_POINT_VALUE"]
gr_ref_fil_df = Indicator_Reference_Control_Table[gr_ref_fil_df_cols]

gr_ref_fil_df = gr_ref_fil_df.groupby(by = ["GROUP_CODE"]
                                      ,as_index=False
                                      ).INDICATOR_POINT_VALUE.sum()

ind_ref_control_df_cols = ["PUB_ORDER","GROUP_CODE","GROUP_DESCRIPTION","HIGHER_GROUP_CODE"
                           ,"HIGHER_GROUP_DESCRIPTION","SUB_DOMAIN_CODE","DOMAIN_CODE"]

gr_ref_df = Indicator_Reference_Control_Table[ind_ref_control_df_cols].drop_duplicates()

gr_ref_table_df = pd.merge(gr_ref_fil_df
                            ,gr_ref_df
                            ,left_on=["GROUP_CODE"]
                            ,right_on=["GROUP_CODE"]
                            ,how="inner")

## 2. General Excel template inputs
[Return to contents](#Table-of-Contents)

In [ ]:
""" 
Calculate domain point values by sub domain code
Used to populate domain point variables used in the excel templates
Variable previous_dom_points calculated in Previous Year Data

"""
dom_points_by_sub_domain_cols = ["SUB_DOMAIN_CODE","INDICATOR_POINT_VALUE"]
dom_points_by_sub_domain = Indicator_Reference_Control_Table[dom_points_by_sub_domain_cols]

dom_points_by_sub_domain = dom_points_by_sub_domain.groupby(by = ["SUB_DOMAIN_CODE"]
                                                      ,as_index=False
                                                      ).sum().rename(columns={"INDICATOR_POINT_VALUE":"DOMAIN_POINT_VALUE"})
                                                                     
                                                      

cl_ach_points = dom_points_by_sub_domain.loc[dom_points_by_sub_domain["SUB_DOMAIN_CODE"]=="CL"]
ph_ach_points = dom_points_by_sub_domain.loc[dom_points_by_sub_domain["SUB_DOMAIN_CODE"]=="PH"].reset_index(drop=True)
phas_ach_points = dom_points_by_sub_domain.loc[dom_points_by_sub_domain["SUB_DOMAIN_CODE"]=="PHAS"].reset_index(drop=True)
phvi_ach_points = dom_points_by_sub_domain.loc[dom_points_by_sub_domain["SUB_DOMAIN_CODE"]=="PHVI"].reset_index(drop=True)
qi_ach_points = dom_points_by_sub_domain.loc[dom_points_by_sub_domain["SUB_DOMAIN_CODE"]=="QI"].reset_index(drop=True)

previous_cl_ach_points = previous_dom_points.loc[previous_dom_points["DOMAIN_CODE"]=="CL"]
previous_ph_ach_points = previous_dom_points.loc[previous_dom_points["DOMAIN_CODE"]=="PH"].reset_index(drop=True)
previous_phas_ach_points = previous_dom_points.loc[previous_dom_points["DOMAIN_CODE"]=="PHAS"].reset_index(drop=True)
previous_phvi_ach_points = previous_dom_points.loc[previous_dom_points["DOMAIN_CODE"]=="PHVI"].reset_index(drop=True)


In [ ]:
"""
Creates text variables used by the excel templates

"""

QOF_title = "Quality and Outcomes Framework {}".format(Fyear)

URL = Control_File.at[0, "Friendly URL"]

Source = "Source: Data source, NHS England"

notes_date = datetime.datetime.strptime(list_size_date, "%d/%m/%Y").date()
notes_date = notes_date.strftime("%d %B %Y")

Notes = "Practices have been mapped to their respective PCNs and Sub ICB Locations using reference data current at %s. \
This mapping has been applied to data for both the current and previous reporting year; this should be borne in mind \
when making comparisons between years. Please be aware that the aggregated (i.e. non-practice level) figures presented \
for %s in this workbook will not match those published in %s, as all figures have been recalculated using practice level \
data that can be mapped to current NHS geographies." % (list_size_date, Previous_year, Previous_year) 



PCA_footnote = "As of 1 April 2019 exception reporting was replaced with a Personalised Care Adjustment (PCA) \
which will better reflect individual clinical situations and patients wishes. \
More information can be found at"
PCN_footnote = "Primary Care Networks (PCNs) are based on general practice registered lists, typically serving \
natural communities of around 30,000 to 50,000 patients. \
They should be small enough to provide the personal care valued by both patients and healthcare professionals, \
but large enough to have impact and economies of scale \
through better collaboration between general practices and others in the local health and social care system, \
including community pharmacies."

number_of_indicators = Control_File.at[0, "NUMBER_INDICATORS"] 

max_points = Control_File.at[0, "MAX_POINTS"].astype(str)
previous_max_points = Control_File.at[0, "PREV_MAX_POINTS"].astype(str)

RespStat_name = Control_File.at[0, "RESP_STAT"]
RS_name = "Lead Analyst "+RespStat_name


## 3. Validate data
[Return to contents](#Table-of-Contents)

<h3>Validation rule 1

In [ ]:
"""
Validation rule 1 = The total number of points achieved by a GP practice was less than or equal to 
the total number of QOF points that can be achieved for indicators which require a manual response (boolean measure).

"""

## Calculate the total number of manual points from all indicators
manual_points_value_df = Indicator_Reference_Control_Table.loc[(Indicator_Reference_Control_Table["AUTOMATIC_MANUAL_ENTRY"]=="M")]
manual_points_value_cols = ["INDICATOR_POINT_VALUE"]
total_manual_points_value = manual_points_value_df[manual_points_value_cols]
total_manual_points_value_df = total_manual_points_value["INDICATOR_POINT_VALUE"].sum()

## Calculate the total number of points achieved for each practice
ind_ach_points_cols = ["PRACTICE_CODE", "INDICATOR_POINTS"]
all_prac_total_points = Indicator_Achieved_Points_Table[ind_ach_points_cols]

all_prac_total_points = all_prac_total_points.groupby(by = ["PRACTICE_CODE"]
                                                ,as_index=False).sum()
                                                
prac_list_merged = pd.merge(practice_list
                            ,all_prac_total_points
                            ,left_on="PRACTICE_CODE"
                            ,right_on="PRACTICE_CODE"
                            ,how="left")

## Loops through the data frame adding the column 'TOTAL_MANUAL_INDICATOR_VALUE' to which the 'INDICATOR_POINTS' are compared
## If 'INDICATOR_POINTS' are greater than 'TOTAL_MANUAL_INDICATOR_VALUE' then the column 'FLAG' = Y if not N.

for x in range(len(prac_list_merged)):

    prac_list_merged["MANUAL"] = total_manual_points_value_df
    prac_list_merged["FLAG"] = np.where(prac_list_merged["INDICATOR_POINTS"] > prac_list_merged["MANUAL"], "Y", "N")
    
    if x > len(prac_list_merged.index):
            break

## Identify which practices fail validation rule 1   
prac_list_merged = prac_list_merged.loc[(prac_list_merged["FLAG"]=="N")]

## Adds new column 'RULE' with a value of one
## This indicaes these practices failed rule one
prac_list_merged.insert(loc = 1
                        ,column = "RULE"
                        ,value = 1)

prac_manual_points_cols = ["PRACTICE_CODE", "RULE"]
prac_manual_points_rule = prac_list_merged[prac_manual_points_cols]

## Number of practices that fail validation rule 1, this figure is used for data quality reporting purposes
print(len(prac_manual_points_rule.index))


<h3>Validation rules 2 and 3

In [ ]:
""" 
Practice code, name, status code ('A' = active, 'C' = closed, 'D' = dormant, 'B' = retired,  
'P' = proposed), prescribing setting (4 = prescribing practice), open and close dates  
are extracted from corporate reference table ODS_PRACTICE_V02

"""

def ods_prac_table():
                      
    ods_prac_query = """SELECT DSS_KEY
                        ,CODE AS 'PRACTICE_CODE'
                        ,NAME AS 'PRACTICE_NAME'
                        ,OPEN_DATE, CLOSE_DATE
                        ,STATUS_CODE, PRESCRIBING_SETTING
                        ,NATIONAL_GROUPING AS 'REGION_CODE' 
                        ,HIGH_LEVEL_HEALTH_GEOGRAPHY AS 'STP_CODE' 
                        ,COMMISSIONER_ORGANISATION_CODE AS 'CCG_CODE'
                        ,DSS_RECORD_START_DATE, DSS_RECORD_END_DATE 
                        ,DSS_SYSTEM_CREATED_DATE, DSS_SYSTEM_UPDATED_DATE
                        FROM [dbo].[ODS_PRACTICE_V02]"""
    
    ods_prac_df = pd.read_sql_query(ods_prac_query,sql_ref_data,)

    return ods_prac_df 

ods_prac_df = ods_prac_table()

print(len(ods_prac_df.index))

#### Format ODS_PRACTICE_V02 dates

In [ ]:
## Convert data frame string dates to date time to facilitate calculations for validation rules 2 to 4

ods_prac_df["OPEN_DATE"] = pd.to_datetime(ods_prac_df["OPEN_DATE"], format="%Y-%m-%d") 
ods_prac_df["CLOSE_DATE"] = pd.to_datetime(ods_prac_df["CLOSE_DATE"], format="%Y-%m-%d") 
ods_prac_df["DSS_RECORD_START_DATE"] = pd.to_datetime(ods_prac_df["DSS_RECORD_START_DATE"], format="%Y-%m-%d") 
ods_prac_df["DSS_RECORD_END_DATE"] = pd.to_datetime(ods_prac_df["DSS_RECORD_END_DATE"], format="%Y-%m-%d") 
ods_prac_df["DSS_SYSTEM_CREATED_DATE"] = pd.to_datetime(ods_prac_df["DSS_SYSTEM_CREATED_DATE"], format="%Y-%m-%d") 
ods_prac_df["DSS_SYSTEM_UPDATED_DATE"] = pd.to_datetime(ods_prac_df["DSS_SYSTEM_UPDATED_DATE"], format="%Y-%m-%d") 

<h3>Validation rule 3

In [ ]:
"""
GP practice status in NHS England reference data on 31 March of the reporting period was not equal to 'A' (Active)

"""

## Filter ods practice data frame by dss record end date criteria
ods_prac_end_date_fil = ods_prac_df.loc[(ods_prac_df["DSS_RECORD_END_DATE"] >= prac_mapping_date
                                         )|(ods_prac_df["DSS_RECORD_END_DATE"].isnull())]

## Get practice information from the filtered data frame by the earliest date available
ods_prac_end_date_fil = ods_prac_end_date_fil[ods_prac_end_date_fil["DSS_KEY"] == ods_prac_end_date_fil
                                              .groupby(by = ["PRACTICE_CODE"])["DSS_KEY"].transform('min')]

## Add practice list size to the practice information
prac_list_end_date_fil = pd.merge(ods_prac_end_date_fil
                                  ,practice_list
                                  ,left_on=["PRACTICE_CODE"]
                                  ,right_on=["PRACTICE_CODE"]
                                  ,how="inner")

## Filter merged data frame by practice status code - 'P' = proposed and 'D' = dormant
prac_end_date_status_fil = prac_list_end_date_fil.loc[(prac_list_end_date_fil["STATUS_CODE"]=="P"
                                                       )|(prac_list_end_date_fil["STATUS_CODE"]=="D")]

## Create data frame of practice codes filtered by dss record end date and practice status
## to identify which practices fail validation rule 3 
prac_end_date_status_fil_col = ["PRACTICE_CODE"]
status_not_active_rule = prac_end_date_status_fil[prac_end_date_status_fil_col]

status_not_active_rule.insert(loc = 1
                              ,column = "RULE"
                              ,value = 3)

## Number of practices that fail validation rule 3, this figure is used for data quality reporting purposes
print(len(status_not_active_rule.index))

<h3>Validation rule 2

In [ ]:
"""
GP practice closed before 1 April in the year of publication and this closure was recorded 
in the NHS England reference data before 1 July in the year of publication

"""

## Filter practice data frame from validation rule 3 by status code = 'C' (closed)
## to identify which practices fail validation rule 2 
closed_prac_rule = prac_list_end_date_fil.loc[(prac_list_end_date_fil["STATUS_CODE"]=="C")]

closed_prac_rule.insert(loc = 1
                        ,column = "RULE"
                        ,value = 2)

## Number of practices that fail validation rule 2, this figure is used for data quality reporting purposes
print(len(closed_prac_rule.index))

<h3><b>Validation rule 4</h3></b>

In [ ]:
"""
The number of registered patients at the GP practice was not available for any of the 3 
months prior to 31 March of the reporting year

"""

## Gets 'TOTAL_LIST_SIZE' and all age bands used by the publication for all months
## from list_size_extract_date onwards
gp_patient_list = create_list_size(list_size_extract_date) 

## Selects required columns
gp_patient_list_cols = ["PRACTICE_CODE", "EXTRACT_DATE", "TOTAL_LIST_SIZE"]
gp_patient_list_fil = gp_patient_list[gp_patient_list_cols]

## Creates a data frame that only contains practices in this publication
gp_patient_list_fil = pd.merge(gp_patient_list_fil
                               ,practice_list
                               ,left_on=["PRACTICE_CODE"]
                               ,right_on=["PRACTICE_CODE"]
                               , how="inner")

gp_patient_list_fil["EXTRACT_DATE"] = pd.to_datetime(gp_patient_list_fil["EXTRACT_DATE"], format="%Y-%m-%d")

## Gets the latest date list size data was provided by the practice
gp_patient_list_id_max_df = gp_patient_list_fil.loc[gp_patient_list_fil.groupby("PRACTICE_CODE").EXTRACT_DATE.idxmax()]

## Gets date 3 months prior to list size date and reformats
list_size_date_minus_3mths = pd.to_datetime(list_size_date_minus_3mths, format="%Y-%m-%d")

## Create data frame of practice codes where their latest list size extract date was less than 3 months prior 
## to the list size date (usually 1 January) to identify which practices failed validation rule 4 
reg_patient_rule_fail = gp_patient_list_id_max_df.loc[(gp_patient_list_id_max_df["EXTRACT_DATE"] < list_size_date_minus_3mths)]

reg_patient_rule_col = ["PRACTICE_CODE"]
reg_patient_rule = reg_patient_rule_fail[reg_patient_rule_col]

reg_patient_rule.insert(loc = 1
                        ,column = "RULE"
                        ,value = 4)

## Number of practices that fail validation rule 4, this figure is used for data quality reporting purposes
print(len(reg_patient_rule.index))


## Create list of practices that failed validation

In [ ]:
"""
Combine all the failed practice codes with the first validation rule they failed on, this will be the lowest rule number.
As practices are only reported on by the first validation rule they fail on

"""

## Join all the data frames containing practices that failed validation
prac_failed_validation_df = pd.concat([prac_manual_points_rule,closed_prac_rule
                                       ,status_not_active_rule,reg_patient_rule]
                                       ,ignore_index=True)

## Get the lowest number of the rule a practice failed on
prac_failed_with_validation_rule = prac_failed_validation_df[prac_failed_validation_df["RULE"] == prac_failed_validation_df
                                                             .groupby("PRACTICE_CODE")["RULE"]
                                                             .transform('min')]
                                                                                                                           
prac_failed_with_validation_rule_col = ["PRACTICE_CODE","RULE"]
remove_failed_prac_list = prac_failed_with_validation_rule[prac_failed_with_validation_rule_col]

## Removal flag is added to indicate which practices need removing from the final practice list of practices and which
## will be included in the publication
remove_failed_prac_list.insert(loc = 1
                              ,column = "REMOVAL_FLAG"
                              ,value = 1)

## Total number of practices that failed a validation rule, this figure is used for data quality reporting purposes
print(len(remove_failed_prac_list.index)) 


### PRACTICE_VALIDATION_OUTCOMES csv data frame

In [ ]:
"""
Create the data frame of practice codes with the rule number they first failed validation on with 
the text descripton of that rule.

Used in Export csvs for PRACTICE_VALIDATION_OUTCOMES

"""
prac_validation_outcomes = pd.merge(remove_failed_prac_list
                                    ,Validation_Rules_Table
                                    ,left_on=["RULE"]
                                    ,right_on=["RULE"]
                                    ,how="left"
                                    ).drop(columns = "REMOVAL_FLAG")


### Outputs for checking validation comments

In [ ]:
## Achievement points by practice used for checking only
ach_prac_points = ind_ach_points_df.groupby(by = ["PRACTICE_CODE"],as_index=False
                                           ).sum()
if test_run is True:
    prac_list_merged.to_csv(f"test_folder\\Achievement_points_by_practice_"+short_year+".csv", index = False)
    prac_list_end_date_fil.to_csv(f"test_folder\\Practice_status_"+short_year+".csv", index = False)

elif test_run is False:
    ach_prac_points.to_csv(checking+"\Achievement_points_by_practice_"+short_year+".csv", index = False)
    prac_list_end_date_fil.to_csv(checking+"\Practice_status_"+short_year+".csv", index = False)

<h1><b>4 Final Practice List</h1></b>

In [ ]:
"""
Creates the final list of practices whose data will be included in the publication after 
all practices that failed validation have been removed.

The number of practices remaining (len(final_prac_list.index)) plus the number of practices 
that failed validation (len(remove_failed_prac_list.index)) should add up to the number 
of practices in the 'FINAL_PRACTICE_LIST.csv' ((len(practice_list.index))

"""

## Final_practice_list
gp_patient_list_final_df = gp_patient_list

gp_patient_list_final_merged = pd.merge(gp_patient_list_final_df
                                        ,practice_list
                                        ,left_on = ["PRACTICE_CODE"]
                                        ,right_on = ["PRACTICE_CODE"]
                                        , how="inner")


gp_patient_list_final_with_failed_flag = pd.merge(gp_patient_list_final_merged
                                                  ,remove_failed_prac_list
                                                  ,left_on=["PRACTICE_CODE"]
                                                  ,right_on=["PRACTICE_CODE"]
                                                  , how="left")

## Change date formats used in filtering data
gp_patient_list_final_with_failed_flag["EXTRACT_DATE"] = pd.to_datetime(gp_patient_list_final_with_failed_flag["EXTRACT_DATE"]
                                                                        , format="%Y-%m-%d")

list_size_date_minus_3mths = pd.to_datetime(list_size_date_minus_3mths, format="%Y-%m-%d")
list_size_date = datetime.datetime.strptime(list_size_date, "%d/%m/%Y").strftime("%Y-%m-%d")

gp_patient_list_final = gp_patient_list_final_with_failed_flag.loc[(gp_patient_list_final_with_failed_flag["EXTRACT_DATE"]>=list_size_date_minus_3mths)&
                                                                   (gp_patient_list_final_with_failed_flag["EXTRACT_DATE"] <= list_size_date)]

gp_patient_list_final = gp_patient_list_final.loc[gp_patient_list_final.groupby("PRACTICE_CODE").EXTRACT_DATE.idxmax()]

final_prac_list = gp_patient_list_final.loc[(gp_patient_list_final["REMOVAL_FLAG"]!=1)]


final_prac_list = final_prac_list.rename(columns={"TOTAL_LIST_SIZE":"LIST_SIZE"})
final_prac_list_col = ["PRACTICE_CODE", "LIST_SIZE"]
final_prac_list = final_prac_list[final_prac_list_col]


## Number of practices in the final practice list plus the number of practices that failed 
## validation should be the same as the number of practices imported
print(len(final_prac_list.index))


<h1><b>5 Coverage</h1></b>

In [ ]:
"""
Coverage is the number of practices who supplied data divided by the number of practices 
who could have supplied data multiplied by 100

"""

## Define 'Coverage' data frame
coverage ={}
coverage = pd.DataFrame(columns=["CONTRIBUTING_PRACTICES", "ACTIVE_PRACTICES", "COVERAGE"])


In [ ]:
"""
ods_prac_df is imported from corporate reference data table 'ODS_PRACTICE_V02' in code block 'Validation rules 2 and 3'

"""

ods_prac_coverage = ods_prac_df

## Create variable which is 12 months prior to the list size data
list_size_date_reformatted = datetime.datetime.strptime(list_size_date, "%Y-%m-%d").date()
list_size_date_minus_12mths = datetime.date(list_size_date_reformatted.year, list_size_date_reformatted.month, 
                                            list_size_date_reformatted.day) - relativedelta(months=12)
list_size_date_minus_12mths = pd.to_datetime(list_size_date_minus_12mths, format="%Y-%m-%d")

## Get all qualifying practices in the specified 12 month period
ods_prac_coverage_fil = ods_prac_coverage.loc[(ods_prac_coverage["DSS_RECORD_START_DATE"] < list_size_date) &
                                              ((ods_prac_coverage["DSS_RECORD_END_DATE"] >= list_size_date_minus_12mths) | 
                                              (ods_prac_coverage["DSS_RECORD_END_DATE"].isnull())) & 
                                              (ods_prac_coverage["STATUS_CODE"] == "A") & 
                                              (ods_prac_coverage["PRESCRIBING_SETTING"] == "4") & 
                                              ((ods_prac_coverage["CLOSE_DATE"] >= list_size_date_minus_12mths) | 
                                              (ods_prac_coverage["CLOSE_DATE"].isnull()))]
                                                      
## Get the most recent practice list size data
ods_prac_coverage_max = ods_prac_coverage_fil.loc[ods_prac_coverage_fil.groupby("PRACTICE_CODE").DSS_RECORD_START_DATE.idxmax()]

## Practices who supplied data
ods_prac_coverage_max = ods_prac_coverage_max[["PRACTICE_CODE"]]
ods_prac_coverage_max = ods_prac_coverage_max.rename(columns={"PRACTICE_CODE":"0"})
coverage["CONTRIBUTING_PRACTICES"] = ods_prac_coverage_max.count()

## Practices who could have supplied data
active_prac = final_prac_list[["PRACTICE_CODE"]]
active_prac = active_prac.rename(columns={"PRACTICE_CODE":"0"})
coverage["ACTIVE_PRACTICES"] = active_prac.count()

## Coverage calculated to be exported as a csv which is used in publication documents
coverage["COVERAGE"] = (coverage["ACTIVE_PRACTICES"]/coverage["CONTRIBUTING_PRACTICES"])*100

<h1><b>6 Achievement Data</h1></b>

In [ ]:
"""
Data quality checks for the achievement points data frame.
These two checks rectify known past data quality issues.

"""

"""
Check 1 makes sure the same practices are present in the 
INDICATOR_ACHIEVED_POINTS.csv and FINAL_PRACTICE_LIST.csv

"""

## Ensures data imported in two different CSVs uses the same practice codes, 
## this is a safe guard that prevents processing problems
ind_ach_points_checked = pd.merge(ind_ach_points_df
                            ,practice_list
                            ,left_on=["PRACTICE_CODE"]
                            ,right_on=["PRACTICE_CODE"]
                            , how="inner"
                            ).drop(columns=["APPROVED"]
                            ).rename(columns={"INDICATOR_POINTS":"ACH_POINTS"
                                              ,"INDICATOR_CODE":"INDICATOR"})

In [ ]:
"""
Check 2 makes sure a practice has not submitted data more than once and if 
this happens removes the earlier data entries.

This is accomplished by identifying practices with more than one entry 
removing these from the INDICATOR_ACHIEVED_POINTS data frame.

Cleaning the duplicated data (by removing ealier data entries) and 
adding the cleaned data back onto INDICATOR_ACHIEVED_POINTS data

"""

## Compares the expected number of indicator codes by practice against 
## the actual number of indicator codes submitted.
ind_code_count_check_col = ["PRACTICE_CODE", "INDICATOR"]
ind_code_count_check = ind_ach_points_checked[ind_code_count_check_col]

ind_code_count_check = ind_code_count_check.groupby(by = ["PRACTICE_CODE"]
                                                    ,as_index=False).count()
                                                                                              

## Identifies which practice codes have more indicator codes than expected submitted
prac_invalid_count = ind_code_count_check.loc[(ind_code_count_check["INDICATOR"]>number_of_indicators)]
prac_invalid_count = prac_invalid_count.rename(columns={"INDICATOR":"COUNT_OF_IND_CODE"})
prac_invalid_count["NUMBER_INDICATORS"] = number_of_indicators

In [ ]:
## Achievement data that has passed check 1 and only contains practices codes 
## that are also present in the FINAL_PRACTICE_LIST.csv
## Grouped by practice code and with count of indicator codes
ind_code_count_check_col = ["PRACTICE_CODE", "INDICATOR"]
ind_code_count_check = ind_ach_points_checked[ind_code_count_check_col]

ind_code_count_check = ind_code_count_check.groupby(by = ["PRACTICE_CODE"]
                                                    ,as_index=False).count()
                                                    
                                          
## Filters out practices with more than the required number of indicator codes
prac_invalid_count = ind_code_count_check.loc[(ind_code_count_check["INDICATOR"]>number_of_indicators)]
prac_invalid_count = prac_invalid_count.drop(columns = "INDICATOR")
prac_invalid_count["PRAC_CODE"] = prac_invalid_count["PRACTICE_CODE"]

In [ ]:
## Achievement points by indicator code for practices identified as having more 
## than the required number of achievement points
ind_ach_multiples = pd.merge(ind_ach_points_checked
                             ,prac_invalid_count
                             ,left_on  =["PRACTICE_CODE"]
                             ,right_on = ["PRACTICE_CODE"]
                             ,how="inner"
                             ).rename(columns={"INDICATOR":"INDICATOR_CODE"})

## Filter out any records where the achievement point value is null
ind_ach_remove_blanks = ind_ach_multiples.loc[ind_ach_multiples["ACH_POINTS"].notnull()]

## Latest achievement points value by indicator code and practice code
ind_ach_cleaned = ind_ach_remove_blanks.loc[ind_ach_remove_blanks.groupby(["PRACTICE_CODE"
                                                                           ,"INDICATOR_CODE"]
                                                                          ,sort=False)["ACH_POINTS"].idxmax()][["PRACTICE_CODE"
                                                                                                                ,"INDICATOR_CODE"
                                                                                                                ,"ACH_POINTS"]]
                                                                          
                                                                            


## Combine achievement data with list of practices with multiple entries and filter out all rows where PRAC_COUNT is null.
## This will remove all data entries for practices with an invalid indicator count
ind_ach_invalid_count = pd.merge(ind_ach_points_checked
                                 ,prac_invalid_count
                                 ,left_on=["PRACTICE_CODE"]
                                 ,right_on=["PRACTICE_CODE"]
                                 ,how="left")

ind_ach = ind_ach_invalid_count.loc[ind_ach_invalid_count["PRAC_CODE"].isnull()]

ind_ach = ind_ach.rename(columns={"INDICATOR":"INDICATOR_CODE"}).drop(columns = "PRAC_CODE")
                
ind_ach_points_final = pd.concat([ind_ach, ind_ach_cleaned], ignore_index=True)

## Combines the original achievement data frame minus the duplicate entries and the 
## cleaned data for practices with multiple enties to make the final achievement 
## points data frame used to create the publication
ind_ach_points_final = pd.concat([ind_ach, ind_ach_cleaned], ignore_index=True)


<h1><b>7 Register Sizes</h1></b>

In [ ]:
"""
Applies solution to known DQ isue and identifies which 'PRACTICE_CODE', 'INDICATOR_CODE' 
combinations are registers and the associated 'VALUE'.

"""
"""
Remove rows of duplicate data and keeps only the most recent rows of data.
This is a known DQ issue.

"""

## Final practice list after all practices that failed validation have been 
## removed combined with indicator data
ind_values_final_prac = pd.merge(final_prac_list
                                ,ind_ref_cont_with_ind_values
                                ,left_on=["PRACTICE_CODE"]
                                ,right_on=["PRACTICE_CODE"]
                                ,how="left"
                                ).drop(columns=["LIST_SIZE"]
                                ).rename(columns={"SHORT_NAME":"FIELD_NAME"})


## Remove rows for practices that contain duplicate data
ind_values_prac_with_dup = pd.merge(ind_values_final_prac
                                    ,prac_invalid_count
                                    ,left_on=["PRACTICE_CODE"]
                                    ,right_on=["PRACTICE_CODE"]
                                    ,how="left")

ind_values_prac_with_dup_fil = ind_values_prac_with_dup.loc[ind_values_prac_with_dup["PRAC_CODE"].isnull()]

ind_values_prac_with_dup_fil = ind_values_prac_with_dup_fil.drop(columns=["PRAC_CODE"])


## Create data frame of rows for practices that contain duplicate data
ind_values_prac_invalid = pd.merge(ind_values_final_prac
                                ,prac_invalid_count
                                ,left_on=["PRACTICE_CODE"]
                                ,right_on=["PRACTICE_CODE"]
                                , how="left")

ind_values_prac_invalid_fil = ind_values_prac_invalid.loc[ind_values_prac_invalid["PRAC_CODE"].notnull()]

## Keep only the most recent version of the duplicate data rows
ind_values_prac_valid_df = ind_values_prac_invalid_fil.loc[ind_values_prac_invalid_fil.groupby(["PRACTICE_CODE"
                                                                                                ,"INDICATOR_CODE"
                                                                                                ,"FIELD_NAME"]
                                                                                               ,sort=False)["VALUE"].idxmax()][["PRACTICE_CODE"
                                                                                                                                ,"INDICATOR_CODE"
                                                                                                                                ,"FIELD_NAME","VALUE"]]
                                                                                                                                                                                   
## Combine the data frame with no duplicate rows with data frame with latest data only to create final data set
ind_values_final = ind_values_prac_with_dup_fil.append(ind_values_prac_valid_df, ignore_index=True)


In [ ]:
"""
Identifies which PRACTICE_CODE, INDICATOR_CODE, VALUE are for register indicators

"""
register_sizes = pd.merge(Indicator_Reference_Control_Table
                          ,ind_values_final
                          ,left_on=["INDICATOR_CODE"]
                          ,right_on=["INDICATOR_CODE"]
                          ,how="inner")

register_sizes = register_sizes[["PRACTICE_CODE", "INDICATOR_CODE", "VALUE"
                                 ,"REGISTER_FLAG","FIELD_NAME"]]

register_sizes = register_sizes.loc[((register_sizes["REGISTER_FLAG"].notnull()) & 
                                     (register_sizes["FIELD_NAME"].str.contains ("egister")))]

register_sizes_cols = ["PRACTICE_CODE", "INDICATOR_CODE", "VALUE"]
register_sizes = register_sizes[register_sizes_cols]

register_sizes = register_sizes.rename(columns={"VALUE":"REGISTER_SIZE"}
                              ).reset_index(drop=True)


<h1><b>8 Mapping</h1></b>

In [ ]:
"""
Adds NHS geographies to the list of practice codes to be used in the publication

"""

## Defines data frame used in this code block and the one below 9 Points by indicator group
final_prac_list_cols = ["PRACTICE_CODE"]
final_prac_codes_df = final_prac_list[final_prac_list_cols]

## Correct possible formatting issues in the imported mapping table
QOF_Mapping_Table_cols = ["PRACTICE_CODE", "PRACTICE_NAME", "PCN_ODS_CODE", "PCN_NAME"
                          ,"CCG_ODS_CODE", "CCG_ONS_CODE", "CCG_NAME"
                          ,"STP_ODS_CODE", "STP_ONS_CODE", "STP_NAME"
                          ,"REGION_ODS_CODE", "REGION_ONS_CODE", "REGION_NAME"
                          ,"NAT_ONS_CODE", "NAT_CODE", "NAT_COUNTRY"]

QOF_map_df = QOF_Mapping_Table[QOF_Mapping_Table_cols]
QOF_map_df["PRACTICE_NAME"] = QOF_map_df["PRACTICE_NAME"].str.title()
QOF_map_df["PRACTICE_NAME"] = QOF_map_df["PRACTICE_NAME"].str.replace(" Pms "," PMS ")
QOF_map_df["PCN_NAME"] = QOF_map_df["PCN_NAME"].str.title()
QOF_map_df["PCN_NAME"] = QOF_map_df["PCN_NAME"].str.replace(" Pcn"," PCN")

## Add geographies to list of practices
geog_mappings = pd.merge(final_prac_codes_df,QOF_map_df,on=["PRACTICE_CODE"], how="left")



<h1><b>9 Points by Indicator Group</h1></b>

In [ ]:
"""
Combines INDICATOR_ACHIEVED_POINTS.csv with the Indicator Reference Control Table. 
To give a data frame contains all the practice code by indicators and the associated data.

"""

## Defines data frames to be used
## final_practice_list data frame from code block above 8 Mapping
prac_only_df = final_prac_codes_df

ind_ref_control_ind_code_col = ["INDICATOR_CODE"]
ind_codes_only_df = Indicator_Reference_Control_Table[ind_ref_control_ind_code_col]

## Combines data frames in an outer join to give a data frame that ensures all practices have 
## a full set of indicator codes allocated to them regardles of whether they submitted data for that indicator
prac_ind_ref_df = prac_only_df.assign(key=1).merge(ind_codes_only_df.assign(key=1)
                                                   ,how="outer"
                                                   ,on="key")

prac_ind_ref_df = prac_ind_ref_df.drop(columns=["key"])


In [ ]:
## Adds the Group code and maximum points achievable to the Practice code and Indicator code data frame
ind_ref_control_gr_code_col = ["INDICATOR_CODE", "GROUP_CODE", "INDICATOR_POINT_VALUE"]
ind_ref_control_gr_code_df = Indicator_Reference_Control_Table[ind_ref_control_gr_code_col]

prac_ind_ref_gr_ref_df = pd.merge(prac_ind_ref_df
                                  ,ind_ref_control_gr_code_df
                                  ,left_on=["INDICATOR_CODE"]
                                  ,right_on=["INDICATOR_CODE"]
                                  ,how="left"
                                  ).rename(columns={"GROUP_CODE":"INDICATOR_GROUP"
                                                    ,"INDICATOR_POINT_VALUE":"MAX_INDICATOR_POINTS"})

In [ ]:
## Data frames with only valid deduplicated values 
## Join PRATICE_CODE, INDICATOR_CODE, FIELD_NAME to VALUE
ind_ref_values_points_df = pd.merge(ind_values_final
                                    ,ind_ach_points_final
                                    ,left_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                                    ,right_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                                    ,how="inner")

## Adds group code to the data frame
ind_ref_gr_code_col = ["INDICATOR_CODE", "GROUP_CODE"]
ind_ref_gr_code_df = Indicator_Reference_Control_Table[ind_ref_gr_code_col]

## Join PRATICE_CODE, INDICATOR_CODE, FIELD_NAME, VALUE to ACH_POINTS
ind_ref_values_points_df = pd.merge(ind_ref_values_points_df
                                    ,ind_ref_gr_code_df
                                    ,left_on=["INDICATOR_CODE"]
                                    ,right_on=["INDICATOR_CODE"]
                                    ,how="inner"
                                    ).rename(columns={"GROUP_CODE":"INDICATOR_GROUP"})

In [ ]:
## Create data frame of Numerator by Practice Code and Indicator Code
ind_by_prac_num_df = ind_ref_values_points_df
ind_by_prac_num_df = ind_by_prac_num_df.loc[ind_by_prac_num_df["FIELD_NAME"] == "Numerator"]
ind_by_prac_num_df = ind_by_prac_num_df.drop(columns=["FIELD_NAME","INDICATOR_GROUP","ACH_POINTS"]
                                             ).rename(columns={"VALUE":"NUMERATOR"})
                                      

## Create data frame of Denominator by Practice Code and Indicator Code
ind_by_prac_denom_df = ind_ref_values_points_df
ind_by_prac_denom_df = ind_by_prac_denom_df.loc[ind_by_prac_denom_df["FIELD_NAME"] == "Denominator"]
ind_by_prac_denom_df = ind_by_prac_denom_df.drop(columns=["FIELD_NAME","INDICATOR_GROUP","ACH_POINTS"]
                                                ).rename(columns={"VALUE":"DENOMINATOR"})

## Combine Numerator and Denominator data frames
ind_by_prac_num_denom_df = pd.merge(ind_by_prac_num_df
                                    ,ind_by_prac_denom_df
                                    ,left_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                                    ,right_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                                    ,how="left")

In [ ]:
## Join Numerators and Denominators to data frame containing practices with all indicator codes
prac_ind_gr_ref_num_denom_df = pd.merge(prac_ind_ref_gr_ref_df
                                        ,ind_by_prac_num_denom_df
                                        ,left_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                                        ,right_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                                        ,how="left")

In [ ]:
## Join data frame containing Numerators and Denominators to practices with all indicator 
## codes to indicators by practice with points ind_ref_gr_code_col previously defined
ind_by_prac_points_ref_df = Indicator_Reference_Control_Table[ind_ref_gr_code_col]
ind_by_practice_points_df = ind_ach_points_final

ind_by_prac_points_df = pd.merge(ind_by_prac_points_ref_df
                                 ,ind_by_practice_points_df
                                 ,left_on=["INDICATOR_CODE"]
                                 ,right_on=["INDICATOR_CODE"]
                                 ,how="inner")

ind_all_pracs = pd.merge(prac_ind_gr_ref_num_denom_df
                         ,ind_by_prac_points_df
                         ,left_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                         ,right_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                         , how="left")
## Drop unwanted column
ind_all_pracs = ind_all_pracs.drop(columns=["GROUP_CODE"])



In [ ]:
## Adds fields MAX_GROUP_POINTS, SUC_DOMAIN_CODE, DOMAIN_CODE to the indicators with all practices data frame.
## Renames columns to be more useful in the Master table
points_by_ind_merged_df = pd.merge(ind_all_pracs,
                                   gr_ref_table_df
                                   ,left_on = "INDICATOR_GROUP"
                                   ,right_on = "GROUP_CODE"
                                   ,how = "left"
                                   ).drop(columns = ["GROUP_CODE","PUB_ORDER","GROUP_DESCRIPTION"
                                                     ,"HIGHER_GROUP_CODE","HIGHER_GROUP_DESCRIPTION"]
                                                     ).rename(columns={"INDICATOR_POINT_VALUE":"MAX_GROUP_POINTS"
                                                                       ,"NUMERATOR":"ACHIEVEMENT_NUMERATOR"
                                                                       ,"DENOMINATOR":"ACHIEVEMENT_DENOMINATOR"
                                                                       ,"INDICATOR_GROUP":"GROUP_CODE"
                                                                       ,"ACH_POINTS":"ACHIEVED_POINTS"})
                        
points_by_ind_col = ["PRACTICE_CODE", "INDICATOR_CODE", "GROUP_CODE", "DOMAIN_CODE"
                     ,"SUB_DOMAIN_CODE","ACHIEVEMENT_NUMERATOR", "ACHIEVEMENT_DENOMINATOR"
                     ,"ACHIEVED_POINTS", "MAX_GROUP_POINTS", "MAX_INDICATOR_POINTS"]
points_by_ind_df = points_by_ind_merged_df[points_by_ind_col]


points_by_ind_df = points_by_ind_df.sort_values(by=["PRACTICE_CODE", "INDICATOR_CODE"]
                                                ,ascending = [True, True]).reset_index(drop=True)


<h1><b>10 Prevalence</h1></b>

In [ ]:
"""
Combines list of practice codes with register sizes and indicator reference Control Table fields
Used as part of the Master table

"""
prevalence_by_ind = pd.merge(final_prac_list
                             ,register_sizes
                             ,left_on=["PRACTICE_CODE"]
                             ,right_on=["PRACTICE_CODE"]
                             ,how="left")

prevalence_by_ind = pd.merge(prevalence_by_ind
                             ,ind_ref_control_table_ind_code
                             ,left_on=["INDICATOR_CODE"]
                             ,right_on=["INDICATOR_CODE"]
                             ,how="inner").drop(columns=["LIST_SIZE"])


<h1><b>11 PCA Output</h1></b>

In [ ]:
"""
Create a PCA data frame PCA csv to which suppression can be applied 
Data frame is picked up in the Suppression section of code

"""

## PCA reference table imported csv filtered to create a data frame of PCA names
pca_ref_table_cols = ["SHORT_NAME", "NAME", "TYPE"]
pca_ref_df = PCA_Ref_Table[pca_ref_table_cols]

pca_ref_fil_df = pca_ref_df.loc[pca_ref_df["TYPE"] == "PCA"]
pca_ref_fil_df = pca_ref_fil_df.rename(columns={"SHORT_NAME":"FIELD_NAME"})

In [ ]:
"""
Creates denominator data frame

"""

## Creates a data frame of denominators by Practice code and Indicator code
pca_prac_denom_df = pd.merge(prac_only_df
                             ,ind_by_prac_denom_df
                             ,left_on=["PRACTICE_CODE"]
                             ,right_on=["PRACTICE_CODE"]
                             ,how="left")

## Creates a data frame of denominator by Practice code, Indicator code and adds Group code
pca_prac_gr_denom_df = pd.merge(pca_prac_denom_df
                                ,ind_by_prac_points_ref_df
                                ,left_on=["INDICATOR_CODE"]
                                ,right_on=["INDICATOR_CODE"]
                                ,how="inner")   

In [ ]:
"""
PCA count data frame without PCA name

"""

## Add indicator code and practice code to PCA reference table data
prac_ind_name_df = pd.merge(ind_values_final
                            ,pca_ref_fil_df
                            ,left_on=["FIELD_NAME"]
                            ,right_on=["FIELD_NAME"]
                            ,how="left")

## Filter for PCA rows only
prac_ind_pca_name_df = prac_ind_name_df.loc[prac_ind_name_df["TYPE"] == "PCA"]

## Drop PCA names and type
prac_ind_pca_name_df = prac_ind_pca_name_df.drop(columns=["FIELD_NAME", "NAME", "TYPE"])

prac_ind_pca_name_df["VALUE"] = prac_ind_pca_name_df["VALUE"].astype(int)

## PCA values summed by practice code and indicator code
prac_ind_pca_summed_df = prac_ind_pca_name_df.groupby(by = ["PRACTICE_CODE","INDICATOR_CODE"]
                                                      ,as_index=False )["VALUE"].sum()
                                                     
## Group code added to PCA values
prac_ind_pca_gr_df = pd.merge(prac_ind_pca_summed_df
                              ,ind_by_prac_points_ref_df
                              ,left_on=["INDICATOR_CODE"]
                              ,right_on=["INDICATOR_CODE"]
                              ,how="left")

prac_ind_pca_gr_df = prac_ind_pca_gr_df.rename(columns={"VALUE":"PCA_COUNT"})

pca_count_df = pd.merge(prac_ind_pca_gr_df
                     ,prac_only_df
                     ,left_on=["PRACTICE_CODE"]
                     ,right_on=["PRACTICE_CODE"]
                     ,how="left")

In [ ]:
"""
PCA count data frame with PCA name without group code

"""

prac_ind_name_copy_df = prac_ind_name_df.drop(columns=["FIELD_NAME", "TYPE"]
                                       ).rename(columns={"VALUE":"PCA_COUNT"})

pca_count_name_df = prac_ind_name_copy_df.groupby(by = ["PRACTICE_CODE","INDICATOR_CODE","NAME"]
                                                  ,as_index=False)["PCA_COUNT"].sum()

In [ ]:
## Creates a distinct list of practice codes with indicator code and group code
## to which denominators can be applied

pca_prac_gr_denom_col = ["PRACTICE_CODE", "GROUP_CODE", "INDICATOR_CODE"]
pca_denom_prac_df = pca_prac_gr_denom_df[pca_prac_gr_denom_col]

pca_count_col = ["PRACTICE_CODE", "GROUP_CODE", "INDICATOR_CODE"]
pca_count_fil_df = pca_count_df[pca_count_col]

pca_denom_all_pracs = pd.concat([pca_denom_prac_df, pca_count_fil_df]
                                ,ignore_index=True, sort = False)

pca_denom_pracs_dist_df = pca_denom_all_pracs.drop_duplicates()

In [ ]:
## Combines distinct practice code, group code, indicator code with denominator and pca count
query_pca_prac_gr_denom_df = pd.merge(pca_denom_pracs_dist_df
                             ,pca_prac_gr_denom_df
                             ,left_on=["PRACTICE_CODE","GROUP_CODE","INDICATOR_CODE"]
                             ,right_on=["PRACTICE_CODE","GROUP_CODE","INDICATOR_CODE"]
                             ,how="left")


query_pcas = pd.merge(query_pca_prac_gr_denom_df
                      ,pca_count_df
                      ,left_on=["PRACTICE_CODE","GROUP_CODE","INDICATOR_CODE"]
                      ,right_on=["PRACTICE_CODE","GROUP_CODE","INDICATOR_CODE"]
                      ,how="left"
                      ).drop(columns=["GROUP_CODE"]
                      ).rename(columns={"DENOMINATOR":"PCA_DENOMINATOR"})

In [ ]:
""" 
Creates the outputs for PCA csv to which suppression can be applied 

"""

prac_pca_csv_df = pd.merge(query_pca_prac_gr_denom_df
                            ,pca_count_name_df
                            ,left_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                            ,right_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                            ,how="left"
                            ).rename(columns={"NAME":"PCA_NAME"})

## Filters out null values
prac_pca_csv_df = prac_pca_csv_df.loc[prac_pca_csv_df["PCA_COUNT"].notnull()]

## Creates total number of PCAs by indicator group and practice code
prac_pca_gr_df = prac_pca_csv_df.groupby(by = ["PRACTICE_CODE","INDICATOR_CODE"]
                                         ,as_index=False)["PCA_COUNT"].sum(
                                                                     ).rename(columns={"PCA_COUNT":"TOT_PCA_COUNT"}) 
                                        
## Adds the total number of PCAs by indicator group to practice pca data frame
prac_pcas_df = pd.merge(prac_pca_csv_df
                        ,prac_pca_gr_df
                        ,left_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                        ,right_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                        ,how="left").drop(columns=["GROUP_CODE"])


<h1><b>12 Alternative Patient List Sizes</h1></b>

In [ ]:
"""
Takes the list size age bands used in the publication and divides them into two columns 
ALT_PAT_LIST (with value 'ALT_LIST_SIZE') and ADD_PAT_LIST (with value 'ADD_LIST_SIZE').

ALT_PAT_LIST = 'alternative patient list size' these are age group denominators for 
indicator groups that have an age requirement for example Asthma (AST) only applies 
to patients aged 6 or over.

ADD_PAT_LIST = 'additional patient list size' and applies to indicators that have an 
age requirement. The indicator group may have have its own age criteria 
for example Asthma (AST) or it may not for example Stroke and transient ischaemic 
attack (STIA) which applies to all ages. The 'additional patient list size' is provided 
in the excel tables in addition to the generic age group for the indicator group.

"""

list_size_age_bands_df = create_list_size(list_size_extract_date)

## Filters age band data frame to list_size_date (usually 1 April after the data collection period ended)
list_size_age_bands_df = list_size_age_bands_df.loc[list_size_age_bands_df["EXTRACT_DATE"] == list_size_extract_date]

## Filters age band data frame to practices used in the publication
all_lists_wide = pd.merge(final_prac_codes_df
                          ,list_size_age_bands_df
                          ,left_on=["PRACTICE_CODE"]
                          ,right_on=["PRACTICE_CODE"]
                          ,how="left"
                         ).drop(columns=["EXTRACT_DATE","TOTAL_LIST_SIZE"])

## The short broad data frame is converted into a long thin data frame and the age bands 
## are stacked in a column called 'ALT_PAT_LIST'
all_lists_wide_stacked = (all_lists_wide.set_index(["PRACTICE_CODE"])
                                        .stack()
                                        .ffill(axis=0)
                                        .bfill(axis=0, downcast='infer')
                                        .reset_index()
                                        .rename({"level_1": "ALT_PAT_LIST", 0:"VALUE"}, axis=1))

In [ ]:
## To refine the age bands to only those used as an ALT_PAT_LIST = 'alternative patient list size'
## Join abge band data frame to the indicator reference control table 
alt_pat_list_by_ind_ref_control = Indicator_Reference_Control_Table[["INDICATOR_CODE", "ALT_PAT_LIST"]]

## Adds the INDICATOR_CODE column onto the data frame
alt_pat_list_by_ind = pd.merge(all_lists_wide_stacked
                               ,alt_pat_list_by_ind_ref_control
                               ,left_on=["ALT_PAT_LIST"]
                               ,right_on=["ALT_PAT_LIST"]
                               ,how="left")

## If the INDICATOR_CODE is null then the INDICATOR_CODE has no ALT_PAT_LIST = 'alternative patient list size' 
## and therefore the ALT_PAT_LIST is replaced by a null.
alt_pat_list_by_ind["ALT_PAT_LIST"] = np.where(alt_pat_list_by_ind["INDICATOR_CODE"].isnull()
                                               ,alt_pat_list_by_ind["INDICATOR_CODE"]
                                               ,alt_pat_list_by_ind["ALT_PAT_LIST"])

## VALUE column is redefined as 'ALT_LIST_SIZE'
alt_pat_list_by_ind = alt_pat_list_by_ind.rename(columns={"VALUE":"ALT_LIST_SIZE"})

print(len(alt_pat_list_by_ind))


In [ ]:
## To refine the age bands to only those used as an ADD_PAT_LIST = 'additional patient list size'
## Join age band data frame to the indicator reference control table 
add_pat_list_by_ind_ref_control = Indicator_Reference_Control_Table[["INDICATOR_CODE", "ADD_PAT_LIST"]]

## Adds the INDICATOR_CODE column onto the data frame and converts 'ALT_PAT_LIST' bands to 'ADD_PAT_LIST' bands
add_pat_list_by_ind = pd.merge(all_lists_wide_stacked
                               ,add_pat_list_by_ind_ref_control
                               ,left_on = "ALT_PAT_LIST"
                               ,right_on = "ADD_PAT_LIST"
                               ,how = "left").drop(columns = ["ALT_PAT_LIST"])

## VALUE column is redefined as 'ADD_LIST_SIZE'
add_pat_list_by_ind = add_pat_list_by_ind.rename(columns={"VALUE":"ADD_LIST_SIZE"})

print(len(add_pat_list_by_ind))


<h1><b>13 Revised Maximum Points</h1></b>

In [ ]:
"""
Practice who are unable to achieve the maximum points due to local arrangements have a revised 
'Maximum Points' value calculated.

"""

## Filters data frame for 'Register' fields with 0 value
ind_values_final_fil_df = ind_values_final.loc[((ind_values_final["FIELD_NAME"].str.contains ("Register")) 
                                                & (ind_values_final["VALUE"]== 0))]

## Reduces Indicator_Reference_Control_Table to required columns where 'ZERO_REGISTER_FLAG' has a value
ind_ref_col = ["INDICATOR_CODE", "GROUP_CODE", "ZERO_REGISTER_FLAG"]
ind_gr_reg_flag_df = Indicator_Reference_Control_Table[ind_ref_col]

ind_gr_reg_flag_df = ind_gr_reg_flag_df.loc[ind_gr_reg_flag_df["ZERO_REGISTER_FLAG"].notnull()]

## Reduces group reference table to required columns 
gr_points_col = ["GROUP_CODE", "INDICATOR_POINT_VALUE"]
gr_points_df = gr_ref_table_df[gr_points_col]

gr_points_df = gr_points_df.rename(columns={"INDICATOR_POINT_VALUE":"TOTAL_GROUP_POINTS"})

## Adds 'INDICATOR_CODE' and 'ZERO_REGISTER_FLAG' to columns 'GROUP_CODE', 'TOTAL_GROUP_POINTS'
ind_gr_points_zero_reg_df = pd.merge(gr_points_df
                                     ,ind_gr_reg_flag_df
                                     ,left_on=["GROUP_CODE"]
                                     ,right_on=["GROUP_CODE"]
                                     ,how="inner")

## Adds 'PRACTICE_CODE' and 'VALUE' to 'GROUP_CODE', 'TOTAL_GROUP_POINTS', 'INDICATOR_CODE' and 'ZERO_REGISTER_FLAG' (as 'NO_REGISTER_FLAG' )
zero_reg_sizes_df = pd.merge(ind_gr_points_zero_reg_df
                             ,ind_values_final_fil_df
                             ,left_on=["INDICATOR_CODE"]
                             ,right_on=["INDICATOR_CODE"]
                             ,how="inner"
                             ).rename(columns = {"ZERO_REGISTER_FLAG":"NO_REGISTER_FLAG"} 
                             ).drop(columns = "FIELD_NAME")


zero_reg_sizes_df["NO_REGISTER_FLAG"] = zero_reg_sizes_df["NO_REGISTER_FLAG"].astype('int') 

zero_reg_sizes_col = ["PRACTICE_CODE", "GROUP_CODE", "INDICATOR_CODE", "VALUE", "TOTAL_GROUP_POINTS", "NO_REGISTER_FLAG"]
zero_reg_sizes_df = zero_reg_sizes_df[zero_reg_sizes_col]


In [ ]:
## Create data frame with 'NO_REGISTER_FLAG' used to create part of the 'Master table'
prac_gr_no_reg_flag_col = ["PRACTICE_CODE", "GROUP_CODE", "NO_REGISTER_FLAG"]
prac_gr_no_reg_flag_df = zero_reg_sizes_df[prac_gr_no_reg_flag_col]

In [ ]:
## Create data frame of 'TOTAL_GROUP_POINTS' by 'PRACTICE_CODE'
revised_max_points_df  = zero_reg_sizes_df.groupby(by = ["PRACTICE_CODE"],
                                                   as_index=False)["TOTAL_GROUP_POINTS"].sum()
                                                   

## Loops through the data frame adding the column 'MAX_POINTS' which is a variable value 
## from 'CONTROL_FILE' and REVISED_MAX_POINTS which is 'MAX_POINTS' - 'TOTAL_GROUP_POINTS'
for y in range (len(revised_max_points_df.index)):
    revised_max_points_df["MAX_POINTS"] = int(max_points)
    revised_max_points_df["REVISED_TOTAL_MAX_POINTS"] = revised_max_points_df["MAX_POINTS"]-revised_max_points_df["TOTAL_GROUP_POINTS"]
    
    if y > len(revised_max_points_df.index):
            break

In [ ]:
## Create data frame total 'ACH_POINTS' by practice code
revised_max_points_total_df  = ind_ach_points_final.groupby(by = ["PRACTICE_CODE"]
                                                            ,as_index=False)["ACH_POINTS"].sum()
                                                           

## Combine 'ACH_POINTS' by practice code and indicator code with 'TOTAL_ACH_POINTS'
revised_max_points_total_df = pd.merge(revised_max_points_total_df
                                       ,revised_max_points_df
                                       ,left_on=["PRACTICE_CODE"]
                                       ,right_on=["PRACTICE_CODE"]
                                       ,how="left").drop(columns=["TOTAL_GROUP_POINTS"])

## Loops through the data frame adding the column 'MAX_POINTS' which is a variable value from 
## 'CONTROL_FILE' and when 'REVISED_MAXIMUM_POINTS' equals null it is replaced by 'MAX_POINTS' 
## otherwise the use the REVISED_MAXIMUM_POINTS
for z in range (len(revised_max_points_total_df.index)):
    revised_max_points_total_df["MAX_POINTS"] = max_points
    revised_max_points_total_df["REVISED_MAX_POINTS"] = np.where(revised_max_points_total_df["REVISED_TOTAL_MAX_POINTS"].isnull()
                                                                 ,revised_max_points_total_df["MAX_POINTS"]
                                                                 ,revised_max_points_total_df["REVISED_TOTAL_MAX_POINTS"])

    if z == len(revised_max_points_total_df.index):
            break

## Drop unwanted columns
revised_max_points_total_df = revised_max_points_total_df.drop(columns=["REVISED_TOTAL_MAX_POINTS"
                                                                        ,"MAX_POINTS"])



<h1><b>14 Master Table</h1></b>

In [ ]:
"""
Creates a 'Master table' of all the data for the current reporting year used in the publication by NHS geographies
The data frames created previously are brought together to build the 'Master table'

"""

## Adds 'LIST_SIZE' to all rows of data by 'PRACTICE_CODE' and 'INDICATOR_CODE' 
## forming the foundation of the 'Master table'
master_prac_ind_list_df = pd.merge(points_by_ind_df
                                      ,final_prac_list
                                      ,left_on=["PRACTICE_CODE"]
                                      ,right_on=["PRACTICE_CODE"]
                                      ,how="left")

## Adds 'REGISTER_SIZE' for register indicator codes to data frame for 'Master table'
master_reg_size_df = pd.merge(master_prac_ind_list_df
                              ,prevalence_by_ind
                              ,left_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                              ,right_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                              ,how="left")

## Adds 'PCA_DENOMINATOR' and 'PCA_COUNT' to data frame for 'Master table' 
master_pcas_df = pd.merge(master_reg_size_df
                          ,query_pcas
                          ,left_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                          ,right_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                          ,how="left")

## Adds 'NO_REGISTER_FLAG' to data frame for 'Master table' 
master_no_reg_flag_df = pd.merge(master_pcas_df
                                 ,prac_gr_no_reg_flag_df
                                 ,left_on=["PRACTICE_CODE", "GROUP_CODE"]
                                 ,right_on=["PRACTICE_CODE", "GROUP_CODE"]
                                 ,how="left")

## Adds 'REVISED_MAX_POINTS' and 'ACH_POINTS' to data frame for 'Master table' 
master_revised_max_point_df = pd.merge(master_no_reg_flag_df
                                  ,revised_max_points_total_df
                                  ,left_on=["PRACTICE_CODE"]
                                  ,right_on=["PRACTICE_CODE"]
                                  ,how="left")

## Adds 'ALT_PAT_LIST' and 'ALT_LIST_SIZE' to data frame for 'Master table' 
master_alt_lists_df = pd.merge(master_revised_max_point_df
                               ,alt_pat_list_by_ind
                               ,left_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                               ,right_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                               ,how="left")

## Adds 'ADD_PAT_LIST' and 'ADD_LIST_SIZE' to data frame for 'Master table'
master_add_lists_df = pd.merge(master_alt_lists_df
                               ,add_pat_list_by_ind
                               ,left_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                               ,right_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                               ,how="left")

## Adds NHS geographies to data frame for 'Master table'
## These have the old geography terms i.e. STP instead of ICB. These will be converted on output.
master_nhs_geog_df = pd.merge(master_add_lists_df
                              ,geog_mappings
                              ,left_on=["PRACTICE_CODE"]
                              ,right_on=["PRACTICE_CODE"]
                              ,how="left")

## Filters the Indicator Reference Control Table to required field for 'Master table'
master_ind_ref_fil_col = ["INDICATOR_CODE", "REGISTER_FLAG", "HIGHER_GROUP_CODE"
                          ,"NO_PREVALENCE_FLAG","INDICATOR_GROUP_CHANGE_NOTE"]
master_ind_ref_fil_df = Indicator_Reference_Control_Table[master_ind_ref_fil_col]

## Adds required fields from Indicator Reference Control Table to 'Master table' data frame
qof_master = pd.merge(master_nhs_geog_df
                      ,master_ind_ref_fil_df
                      ,left_on=["INDICATOR_CODE"]
                      ,right_on=["INDICATOR_CODE"]
                      ,how="left")

## Final outputs for 'Master table' reordered for ease of use
qof_master_col = ["NAT_ONS_CODE","NAT_CODE","NAT_COUNTRY"
                  ,"REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME"
                  ,"STP_ODS_CODE","STP_ONS_CODE","STP_NAME","CCG_ODS_CODE"
                  ,"CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME"
                  ,"PRACTICE_CODE","PRACTICE_NAME","DOMAIN_CODE","SUB_DOMAIN_CODE"
                  ,"HIGHER_GROUP_CODE","GROUP_CODE","INDICATOR_CODE","ACHIEVEMENT_NUMERATOR"
                  ,"ACHIEVEMENT_DENOMINATOR","ACHIEVED_POINTS","MAX_GROUP_POINTS"
                  ,"MAX_INDICATOR_POINTS","NO_REGISTER_FLAG","REGISTER_FLAG","LIST_SIZE"
                  ,"ALT_LIST_SIZE","ALT_PAT_LIST","ADD_PAT_LIST","ADD_LIST_SIZE"
                  ,"REGISTER_SIZE","PCA_COUNT","PCA_DENOMINATOR","REVISED_MAX_POINTS"
                  ,"NO_PREVALENCE_FLAG","INDICATOR_GROUP_CHANGE_NOTE"]
qof_master = qof_master[qof_master_col]

<h1><b>15 Previous Year</h1></b>

In [ ]:
## Divides imported 'Previous_Prevalence' data frame into three separate data frames
## Rejoins them in required format

## Reference columns to which previous prevalence totals are joined
reference_col = ["PRACTICE_CODE", "GROUP_CODE", "REGISTER"]
reference_prevalence_df = Previous_Prevalence[reference_col]

## Calculating previous prevalence totals by practice code
previous_prevalence_total_col = ["PRACTICE_CODE", "PATIENT_LIST_TYPE","PRACTICE_LIST_SIZE"]
previous_prevalence_total_df = Previous_Prevalence[previous_prevalence_total_col]

previous_prevalence_total_df = previous_prevalence_total_df.loc[previous_prevalence_total_df["PATIENT_LIST_TYPE"]=="TOTAL"]

previous_prevalence_total_df = previous_prevalence_total_df.drop(columns=["PATIENT_LIST_TYPE"]
                                                                 ).drop_duplicates().reset_index(drop=True) 
                                                                  
## Join previous prevalence totals to reference columns
previous_prevalence_df = pd.merge(reference_prevalence_df
                                  ,previous_prevalence_total_df
                                  ,left_on=["PRACTICE_CODE"]
                                  ,right_on=["PRACTICE_CODE"]
                                  ,how="left")

## Calculating previous prevalence alternative total list size by practice code and group code 
previous_prevalence_gr_total_col = ["PRACTICE_CODE", "GROUP_CODE", "PATIENT_LIST_TYPE","PRACTICE_LIST_SIZE"]
previous_prevalence_gr_total_df = Previous_Prevalence[previous_prevalence_gr_total_col] 

## Filters data frame
previous_prevalence_gr_total_df = previous_prevalence_gr_total_df.loc[previous_prevalence_gr_total_df["PATIENT_LIST_TYPE"]!="TOTAL"]

previous_prevalence_gr_total_df = previous_prevalence_gr_total_df.rename(columns={"PRACTICE_LIST_SIZE":"ALT_LIST_SIZE"}
                                                                         ).drop(columns=["PATIENT_LIST_TYPE"])


## Joins previous prevalence totals by practice code to previous prevalence alternative 
## total list size by practice code and group code 
previous_prevalence_list_size_df = pd.merge(previous_prevalence_df
                                     ,previous_prevalence_gr_total_df
                                     ,left_on=["PRACTICE_CODE", "GROUP_CODE"]
                                     ,right_on=["PRACTICE_CODE", "GROUP_CODE"]
                                     ,how="left")

In [ ]:
## Reconfigures the 'Previous_Achievements_PCAs' imported data frame
## to an indicator code data frame with associated values and practice code

## Creates a distinct list of practice codes from the previous year with a 'JOIN_FLAG' column
previous_year_pra_col = ["PRACTICE_CODE"]
previous_year_prac_df = Previous_Achievements_PCAs[previous_year_pra_col]

previous_year_prac_df = previous_year_prac_df.drop_duplicates()
previous_year_prac_df["JOIN_FLAG"] = 1



## Creates an indicator code data frame with associated values and a 'JOIN_FLAG' to facilitate a join
prev_ind_gr_df = pd.merge(Previous_Indicator_Control
                          ,Previous_Indicator_Mappings
                          ,left_on=["GROUP_CODE"]
                          ,right_on=["INDICATOR_GROUP_CODE"]
                          ,how="left")

## Adds a field to join 'previous_year_prac_df' to
prev_ind_gr_df["JOIN_FLAG"] = 1

## Reorders 'prev_ind_gr_df' for ease of use
prev_ind_gr_cols = ["INDICATOR_CODE", "INDICATOR_POINT_VALUE", "HIGHER_GROUP_CODE"
                    ,"INDICATOR_GROUP_CODE", "INDICATOR_GROUP_DESCRIPTION"
                    ,"DOMAIN_CODE", "DOMAIN_DESCRIPTION", "PATIENT_LIST_TYPE"
                    ,"NO_PREVALENCE_FLAG", "JOIN_FLAG"]

prev_mappings_with_no_ind_flag = prev_ind_gr_df[prev_ind_gr_cols] 

## Joins together two data frames to create an indicator code data frame with 
## associated values with practice code
current_pracs_prev_map = pd.merge(previous_year_prac_df
                                  ,prev_mappings_with_no_ind_flag
                                  ,left_on=["JOIN_FLAG"]
                                  ,right_on=["JOIN_FLAG"]
                                  ,how="left"
                                  ).drop(columns=["JOIN_FLAG"])

In [ ]:
## Converts long-thin imported 'Previous_Achievements_PCAs' data frame into a short broad data frame
## 'ACHIEVED_POINTS','DENOMINATOR','NUMERATOR','PCAS','REGISTER' by 'PRACTICE_CODE','INDICATOR'
previous_ach_pca_piv_df = Previous_Achievements_PCAs.pivot_table(index=["PRACTICE_CODE", "INDICATOR_CODE"]
                                                                          ,columns="MEASURE" 
                                                                          ,values="VALUE"
                                                                          ,fill_value=0).reset_index().rename_axis(None, axis=1)

## Joins indicator code data frame with associated values and practice code 
## to short broad Previous_Achievements_PCAs data frame
previous_prac_data_df = pd.merge(current_pracs_prev_map
                                         ,previous_ach_pca_piv_df
                                         ,left_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                                         ,right_on=["PRACTICE_CODE", "INDICATOR_CODE"]
                                         ,how="left"
                                         ).drop(columns=["NUMERATOR", "REGISTER"])

In [ ]:
## Divides previous_prac_data_df into six data frames and rejojns them on 'PRACTICE_CODE', 
## 'INDICATOR_GROUP_CODE', 'PATIENT_LIST_TYPE'
## To create practice prevalence for the previous year

## Creates a reference data frame without duplicates to which other data frames can be joined
previous_prac_ref_col = ["PRACTICE_CODE","HIGHER_GROUP_CODE","INDICATOR_GROUP_CODE"
                         ,"DOMAIN_CODE","DOMAIN_DESCRIPTION","PATIENT_LIST_TYPE"]
previous_prac_ref_df = previous_prac_data_df[previous_prac_ref_col] 

previous_prac_ref_df = previous_prac_ref_df.drop_duplicates()

## Creates a data frame 
previous_prac_ref_gr_points_col = ["PRACTICE_CODE","INDICATOR_GROUP_CODE"
                                ,"PATIENT_LIST_TYPE","INDICATOR_POINT_VALUE"]
previous_prac_ref_gr_points_df = previous_prac_data_df[previous_prac_ref_gr_points_col]

previous_prac_ref_gr_points_df = previous_prac_ref_gr_points_df.groupby(by = ["PRACTICE_CODE","INDICATOR_GROUP_CODE","PATIENT_LIST_TYPE"]
                                                                  ,as_index=False)["INDICATOR_POINT_VALUE"].sum(                                                                      
                                                                  ).rename(columns={"INDICATOR_POINT_VALUE":"TOTAL_INDICATOR_GROUP_POINTS"})

## Creates a data frame of  'PRACTICE_CODE', 'INDICATOR_GROUP_CODE', 'PATIENT_LIST_TYPE' and 'NO_PREVALENCE_FLAG'                                                                                                                                                                                                       
previous_prac_ref_no_prevalence_col = ["PRACTICE_CODE", "INDICATOR_GROUP_CODE","PATIENT_LIST_TYPE","NO_PREVALENCE_FLAG"]
previous_prac_ref_no_prevalence_df = previous_prac_data_df[previous_prac_ref_no_prevalence_col]

previous_prac_ref_no_prevalence_df = previous_prac_ref_no_prevalence_df.loc[previous_prac_ref_no_prevalence_df.groupby(["PRACTICE_CODE"]
                                                                                                                       ,sort=False)["NO_PREVALENCE_FLAG"].idxmax()][["PRACTICE_CODE"
                                                                                                                                                                    ,"INDICATOR_GROUP_CODE"
                                                                                                                                                                    ,"PATIENT_LIST_TYPE"
                                                                                                                                                                    ,"NO_PREVALENCE_FLAG"]]
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
                                                                                                                          
                                                                                                                                                        
## Creates a data frame of 'PRACTICE_CODE', 'INDICATOR_GROUP_CODE', 'PATIENT_LIST_TYPE' and 'ACHIEVED_POINTS'
previous_prac_ref_ach_points_col =  ["PRACTICE_CODE", "INDICATOR_GROUP_CODE", "PATIENT_LIST_TYPE", "ACHIEVED_POINTS"]                                                                                                                         
previous_prac_ref_ach_points_df = previous_prac_data_df[previous_prac_ref_ach_points_col]

previous_prac_ref_ach_points_df = previous_prac_ref_ach_points_df.groupby(by = ["PRACTICE_CODE","INDICATOR_GROUP_CODE","PATIENT_LIST_TYPE"]
                                                                    ,as_index=False)["ACHIEVED_POINTS"].sum()
                                                                          
## Creates a data frame of 'PRACTICE_CODE', 'INDICATOR_GROUP_CODE', 'PATIENT_LIST_TYPE' and 'DENOMINATOR'
previous_prac_ref_denom_col = ["PRACTICE_CODE", "INDICATOR_GROUP_CODE", "PATIENT_LIST_TYPE", "DENOMINATOR"]
previous_prac_ref_denom_df = previous_prac_data_df[previous_prac_ref_denom_col]

previous_prac_ref_denom_df = previous_prac_ref_denom_df.groupby(by = ["PRACTICE_CODE", "INDICATOR_GROUP_CODE", "PATIENT_LIST_TYPE"]
                                                                ,as_index=False)["DENOMINATOR"].sum()
                                                                                                                             
## Creates a data frame of 'PRACTICE_CODE', 'INDICATOR_GROUP_CODE', 'PATIENT_LIST_TYPE' and 'PCAS'
previous_prac_ref_pca_col = ["PRACTICE_CODE", "INDICATOR_GROUP_CODE", "PATIENT_LIST_TYPE", "PCAS"]
previous_prac_ref_pca_df = previous_prac_data_df[previous_prac_ref_pca_col]

previous_prac_ref_pca_df = previous_prac_ref_pca_df.groupby(by = ["PRACTICE_CODE", "INDICATOR_GROUP_CODE", "PATIENT_LIST_TYPE"]
                                                            ,as_index=False)["PCAS"].sum()
                                                             

## Joins all the data frames together
previous_prac_ach_pca_df = pd.merge(previous_prac_ref_df
                                    ,previous_prac_ref_gr_points_df
                                    ,left_on=["PRACTICE_CODE","INDICATOR_GROUP_CODE","PATIENT_LIST_TYPE"]
                                    ,right_on=["PRACTICE_CODE","INDICATOR_GROUP_CODE","PATIENT_LIST_TYPE"]
                                    ,how="left")

previous_prac_ach_pca_df = pd.merge(previous_prac_ach_pca_df
                                    ,previous_prac_ref_no_prevalence_df
                                    ,left_on=["PRACTICE_CODE","INDICATOR_GROUP_CODE","PATIENT_LIST_TYPE"]
                                    ,right_on=["PRACTICE_CODE","INDICATOR_GROUP_CODE","PATIENT_LIST_TYPE"]
                                    ,how="left")

previous_prac_ach_pca_df = pd.merge(previous_prac_ach_pca_df
                                    ,previous_prac_ref_ach_points_df
                                    ,left_on=["PRACTICE_CODE","INDICATOR_GROUP_CODE","PATIENT_LIST_TYPE"]
                                    ,right_on=["PRACTICE_CODE","INDICATOR_GROUP_CODE","PATIENT_LIST_TYPE"]
                                    ,how="left")

previous_prac_ach_pca_df = pd.merge(previous_prac_ach_pca_df
                                    ,previous_prac_ref_denom_df
                                   ,left_on=["PRACTICE_CODE","INDICATOR_GROUP_CODE","PATIENT_LIST_TYPE"]
                                    ,right_on=["PRACTICE_CODE","INDICATOR_GROUP_CODE","PATIENT_LIST_TYPE"]
                                    ,how="left")

previous_prac_ach_pca_df = pd.merge(previous_prac_ach_pca_df
                                    ,previous_prac_ref_pca_df
                                    ,left_on=["PRACTICE_CODE","INDICATOR_GROUP_CODE","PATIENT_LIST_TYPE"]
                                    ,right_on=["PRACTICE_CODE","INDICATOR_GROUP_CODE","PATIENT_LIST_TYPE"]
                                    ,how="left")

previous_prac_ach_pca_df = previous_prac_ach_pca_df.rename(columns={ "INDICATOR_GROUP_CODE":"GROUP_CODE"})


In [ ]:
""" 
Creates the data frame for LVSD indicator group, which comprises of indicator code HF003 and is appended
to the previous years practice data frame

"""

## 'previous_prac_data_df' filtered for indicator HF003
previous_prac_data_denom_pca_col = ["PRACTICE_CODE", "INDICATOR_CODE", "DENOMINATOR", "PCAS"]
previous_prac_data_denom_pca_df = previous_prac_data_df[previous_prac_data_denom_pca_col]

previous_prac_data_denom_pca_df = previous_prac_data_denom_pca_df.loc[previous_prac_data_denom_pca_df["INDICATOR_CODE"] == "HF003"]


## Creates a data frame of 'PRACTICE_CODE' and 'PRACTICE_LIST_SIZE' for HF indicator group
hf_prac_list_for_lvsd_col = ["PRACTICE_CODE", "PRACTICE_LIST_SIZE", "GROUP_CODE"]
hf_prac_list_for_lvsd_df = previous_prevalence_list_size_df[hf_prac_list_for_lvsd_col]

hf_prac_list_for_lvsd_df = hf_prac_list_for_lvsd_df.loc[hf_prac_list_for_lvsd_df["GROUP_CODE"] == "HF"].drop(columns = ("GROUP_CODE"))


## Creates the LVSD data frame
LVSD_prac_ind_denom_pca_df = pd.merge(previous_prac_data_denom_pca_df
                                      ,hf_prac_list_for_lvsd_df
                                      ,left_on=["PRACTICE_CODE"]
                                      ,right_on=["PRACTICE_CODE"]
                                      ,how="left")

## Define the remaining data columns to align with the previous practice data frame
LVSD_prac_ind_denom_pca_df["DOMAIN_CODE"] = "CL"
LVSD_prac_ind_denom_pca_df["HIGHER_GROUP_CODE"] = "CV"
LVSD_prac_ind_denom_pca_df["GROUP_CODE"] = "LVSD"
LVSD_prac_ind_denom_pca_df["PREV_REGISTER_SIZE"] = np.where(LVSD_prac_ind_denom_pca_df["PCAS"] > 0
                                                            ,(LVSD_prac_ind_denom_pca_df["DENOMINATOR"]+LVSD_prac_ind_denom_pca_df["PCAS"])
                                                            , LVSD_prac_ind_denom_pca_df["DENOMINATOR"])


LVSD_prac_ind_denom_pca_df["PREV_PREVALENCE"] = np.where(LVSD_prac_ind_denom_pca_df["PCAS"] > 0
                                                         ,(((LVSD_prac_ind_denom_pca_df["DENOMINATOR"]+LVSD_prac_ind_denom_pca_df["PCAS"]
                                                             )/LVSD_prac_ind_denom_pca_df["PRACTICE_LIST_SIZE"])*100)
                                                         ,(LVSD_prac_ind_denom_pca_df["DENOMINATOR"]/LVSD_prac_ind_denom_pca_df["PRACTICE_LIST_SIZE"])*100)


LVSD_prac_ind_denom_pca_df = LVSD_prac_ind_denom_pca_df.rename(columns={"PRACTICE_LIST_SIZE":"PREV_PRACTICE_LIST_SIZE"})
LVSD_prac_ind_denom_pca_df["PREV_ACHIEVED_POINTS"] = np.nan
LVSD_prac_ind_denom_pca_df["PREV_TOTAL_INDICATOR_GROUP_POINTS"] = np.nan
LVSD_prac_ind_denom_pca_df["PREV_ACHIEVEMENT_RATE"] = np.nan
LVSD_prac_ind_denom_pca_df["PREV_PCA_COUNT"] = np.nan
LVSD_prac_ind_denom_pca_df["PREV_DENOMINATOR"] = np.nan
LVSD_prac_ind_denom_pca_df["PREV_PCA_RATE"] = np.nan
LVSD_prac_ind_denom_pca_df["PREV_ALT_PAT_LIST"] = np.nan

## Reorganise the columns to align with the previous practice data frame to which it is appended
LVSD_data_col = ["PRACTICE_CODE","DOMAIN_CODE","HIGHER_GROUP_CODE","GROUP_CODE"
                 ,"PREV_PRACTICE_LIST_SIZE","PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE"
                 ,"PREV_PREVALENCE","PREV_ACHIEVED_POINTS","PREV_TOTAL_INDICATOR_GROUP_POINTS"
                 ,"PREV_ACHIEVEMENT_RATE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"]

LVSD_data = LVSD_prac_ind_denom_pca_df[LVSD_data_col]



In [ ]:
"""
Creates a data frame of the previous years data by all NHS geographies 
Thia is used to calculate data by NHS geographies i.e. Region

"""
## Combines achievement pca data frame with 'ALT_LIST_SIZE'
previous_prevalence_prac_data_df = pd.merge(previous_prac_ach_pca_df
                                            ,previous_prevalence_list_size_df
                                            ,left_on=["PRACTICE_CODE", "GROUP_CODE"]
                                            ,right_on=["PRACTICE_CODE", "GROUP_CODE"]
                                            ,how="left"
                                            ).rename(columns={"PATIENT_LIST_TYPE":"ALT_PAT_LIST"
                                                              ,"REGISTER":"REGISTER_SIZE", "PCAS":"PCA_COUNT"})

## Reorder region data frame columns
previous_prevalence_prac_data_col = ["PRACTICE_CODE","HIGHER_GROUP_CODE","GROUP_CODE","TOTAL_INDICATOR_GROUP_POINTS"
                                     ,"DOMAIN_CODE","ALT_PAT_LIST","NO_PREVALENCE_FLAG","PRACTICE_LIST_SIZE"
                                     , "ALT_LIST_SIZE","REGISTER_SIZE","ACHIEVED_POINTS","PCA_COUNT","DENOMINATOR"]                                                    
previous_prevalence_prac_data_df  = previous_prevalence_prac_data_df [previous_prevalence_prac_data_col]


In [ ]:
## Adds missing 'ALT_LIST_SIZE' values to region data frame
previous_prevalence_alt_prac_data_df = pd.merge(previous_prevalence_prac_data_df
                                                ,Previous_Prevalence_Non_Register_Alt_Indicators
                                                ,left_on=["PRACTICE_CODE", "ALT_PAT_LIST"]
                                                ,right_on=["PRACTICE_CODE", "ALT_PAT_LIST"]
                                                ,how="left")

previous_prevalence_alt_prac_data_df["ALT_LIST_SIZE"] = np.where(previous_prevalence_alt_prac_data_df["ALT_LIST"].notnull()
                                                              ,previous_prevalence_alt_prac_data_df["ALT_LIST"]
                                                              ,previous_prevalence_alt_prac_data_df["ALT_LIST_SIZE"])

previous_prevalence_alt_prac_data_col = ["PRACTICE_CODE","HIGHER_GROUP_CODE","GROUP_CODE","TOTAL_INDICATOR_GROUP_POINTS"
                                         ,"DOMAIN_CODE","ALT_PAT_LIST","NO_PREVALENCE_FLAG","PRACTICE_LIST_SIZE"
                                         ,"ALT_LIST_SIZE","REGISTER_SIZE","ACHIEVED_POINTS","PCA_COUNT","DENOMINATOR"]                                                
previous_prevalence_alt_prac_data_df = previous_prevalence_alt_prac_data_df[previous_prevalence_alt_prac_data_col]

## Add NHS geographies to previous practice data frame
previous_prevalence_with_geog_df = pd.merge(geog_mappings
                                            ,previous_prevalence_alt_prac_data_df
                                            ,left_on=["PRACTICE_CODE"]
                                            ,right_on=["PRACTICE_CODE"]
                                            ,how="inner")

In [ ]:
"""
Creates data frame for national level with previous years data

"""

## Creates national level reference columns to which data can be joined
previous_prevalence_national_gr_col = ["NAT_COUNTRY", "DOMAIN_CODE", "HIGHER_GROUP_CODE", "GROUP_CODE"]
previous_prevalence_national_gr_df = previous_prevalence_with_geog_df[previous_prevalence_national_gr_col]

## Column 'NAT_COUNTRY' is renamed as 'REGION_NAME' as National level data is appended to 
## region data to be output in the same excel tables
previous_prevalence_national_gr_df = previous_prevalence_national_gr_df.rename(columns={ "NAT_COUNTRY":"REGION_NAME"}
                                                                               ).drop_duplicates()


## Selects columns needed for national data frame and excludes unwanted NHS geography columns
previous_prevalence_with_national_col = ["GROUP_CODE","PRACTICE_CODE","ALT_LIST_SIZE","REGISTER_SIZE"
                                         ,"PRACTICE_LIST_SIZE","ACHIEVED_POINTS"
                                         ,"TOTAL_INDICATOR_GROUP_POINTS","PCA_COUNT","DENOMINATOR"]                                        
previous_prevalence_with_national_df = previous_prevalence_with_geog_df[previous_prevalence_with_national_col]

## Sums numeric data items to provide national level data
previous_prevalence_with_national_df = previous_prevalence_with_national_df.groupby(by = ["GROUP_CODE"],as_index=False
                                                                                    ).agg({"PRACTICE_CODE" : [pd.Series.count]
                                                                                           ,"PRACTICE_LIST_SIZE" : [pd.Series.sum]
                                                                                           ,"ALT_LIST_SIZE" : [pd.Series.sum]
                                                                                           ,"REGISTER_SIZE" : [pd.Series.sum]
                                                                                           ,"ACHIEVED_POINTS" : [pd.Series.sum]
                                                                                           ,"TOTAL_INDICATOR_GROUP_POINTS" : [pd.Series.sum]
                                                                                           ,"PCA_COUNT" : [pd.Series.sum]
                                                                                           ,"DENOMINATOR" : [pd.Series.sum]
                                                                                           })                                                                                    
previous_prevalence_with_national_df.columns = previous_prevalence_with_national_df.columns.droplevel(1)

## Renames columns to show they contain last years data
previous_prevalence_with_national_df = previous_prevalence_with_national_df.rename(columns={"PRACTICE_CODE":"PREV_NUMBER_PRACTICES"
                                                                                            ,"PRACTICE_LIST_SIZE":"PREV_PRACTICE_LIST_SIZE"
                                                                                            ,"ALT_LIST_SIZE":"PREV_ALT_PAT_LIST"
                                                                                            ,"REGISTER_SIZE":"PREV_REGISTER_SIZE"
                                                                                            ,"ACHIEVED_POINTS":"PREV_ACHIEVED_POINTS"
                                                                                            ,"TOTAL_INDICATOR_GROUP_POINTS":"PREV_TOTAL_INDICATOR_GROUP_POINTS"
                                                                                            ,"PCA_COUNT":"PREV_PCA_COUNT"
                                                                                            ,"DENOMINATOR":"PREV_DENOMINATOR"})
                                               
## Calculates the previous prevalence data at national level
previous_prevalence_with_national_df["PREV_PREVALENCE"] = np.where(previous_prevalence_with_national_df["PREV_ALT_PAT_LIST"] == 0
                                                                   ,(previous_prevalence_with_national_df["PREV_REGISTER_SIZE"]/
                                                                     previous_prevalence_with_national_df["PREV_PRACTICE_LIST_SIZE"])*100
                                                                   ,(previous_prevalence_with_national_df["PREV_REGISTER_SIZE"]/
                                                                     previous_prevalence_with_national_df["PREV_ALT_PAT_LIST"])*100)
                                                                                                                                   
## Calculates the previous achievement rate data at national level
previous_prevalence_with_national_df["PREV_ACHIEVEMENT_RATE"] = (previous_prevalence_with_national_df["PREV_ACHIEVED_POINTS"]/
                                                                 previous_prevalence_with_national_df["PREV_TOTAL_INDICATOR_GROUP_POINTS"])*100

## Calculates the previous PCA rate data at national level
previous_prevalence_with_national_df["PREV_PCA_RATE"] = (previous_prevalence_with_national_df["PREV_PCA_COUNT"]/
                                                         (previous_prevalence_with_national_df["PREV_DENOMINATOR"]+
                                                          previous_prevalence_with_national_df["PREV_PCA_COUNT"]))*100
                                                         
## Join national level data to national level reference columns
previous_prevalence_national_data_df = pd.merge(previous_prevalence_national_gr_df
                                                ,previous_prevalence_with_national_df
                                                ,left_on=["GROUP_CODE"]
                                                ,right_on=["GROUP_CODE"]
                                                , how="left")

## Reorder data frame columns so they can be appended to the region level data frame
previous_prevalence_national_data_col = ["REGION_NAME","DOMAIN_CODE","HIGHER_GROUP_CODE","GROUP_CODE"
                                         ,"PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE","PREV_ALT_PAT_LIST"
                                         ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","PREV_ACHIEVED_POINTS"
                                         ,"PREV_TOTAL_INDICATOR_GROUP_POINTS","PREV_ACHIEVEMENT_RATE"
                                         ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"]
previous_prevalence_national_data_df = previous_prevalence_national_data_df[previous_prevalence_national_data_col]

In [ ]:
"""
Creates data frame for region level with previous years data

"""

## Creates region level reference columns to which data can be joined
previous_prevalence_region_gr_col = ["REGION_NAME", "DOMAIN_CODE", "HIGHER_GROUP_CODE", "GROUP_CODE"]
previous_prevalence_region_gr_df = previous_prevalence_with_geog_df[previous_prevalence_region_gr_col]

previous_prevalence_region_gr_df = previous_prevalence_region_gr_df.drop_duplicates()

## Selects columns needed for region data frame and excludes unwanted NHS geography columns
previous_prevalence_with_region_col = ["REGION_NAME","GROUP_CODE", "PRACTICE_CODE", "ALT_LIST_SIZE", "REGISTER_SIZE"
                                       ,"PRACTICE_LIST_SIZE","ACHIEVED_POINTS" ,"TOTAL_INDICATOR_GROUP_POINTS"
                                       ,"PCA_COUNT", "DENOMINATOR"]                                  
previous_prevalence_with_region_df = previous_prevalence_with_geog_df[previous_prevalence_with_region_col]

## Sums numeric data items to provide region level data
previous_prevalence_with_region_df = previous_prevalence_with_region_df.groupby(by = ["REGION_NAME","GROUP_CODE"],as_index=False
                                                                                ).agg({"PRACTICE_CODE" : [pd.Series.count]
                                                                                       ,"PRACTICE_LIST_SIZE" : [pd.Series.sum]
                                                                                       ,"ALT_LIST_SIZE" : [pd.Series.sum]
                                                                                       ,"REGISTER_SIZE" : [pd.Series.sum]
                                                                                       ,"ACHIEVED_POINTS" : [pd.Series.sum]
                                                                                       ,"TOTAL_INDICATOR_GROUP_POINTS" : [pd.Series.sum]
                                                                                       ,"PCA_COUNT" : [pd.Series.sum]
                                                                                       ,"DENOMINATOR" : [pd.Series.sum]
                                                                                       })
previous_prevalence_with_region_df.columns = previous_prevalence_with_region_df.columns.droplevel(1)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
## Renames columns to show they contain last years data
previous_prevalence_with_region_df = previous_prevalence_with_region_df.rename(columns={"PRACTICE_CODE":"PREV_NUMBER_PRACTICES"
                                                                                        ,"PRACTICE_LIST_SIZE":"PREV_PRACTICE_LIST_SIZE"
                                                                                        ,"ALT_LIST_SIZE":"PREV_ALT_PAT_LIST"
                                                                                        ,"REGISTER_SIZE":"PREV_REGISTER_SIZE"
                                                                                        ,"ACHIEVED_POINTS":"PREV_ACHIEVED_POINTS"
                                                                                        ,"TOTAL_INDICATOR_GROUP_POINTS":"PREV_TOTAL_INDICATOR_GROUP_POINTS"
                                                                                        ,"PCA_COUNT":"PREV_PCA_COUNT","DENOMINATOR":"PREV_DENOMINATOR"})

## Calculates the previous prevalence data at region level
previous_prevalence_with_region_df["PREV_PREVALENCE"] = np.where(previous_prevalence_with_region_df["PREV_ALT_PAT_LIST"] == 0
                                                                 ,(previous_prevalence_with_region_df["PREV_REGISTER_SIZE"]/
                                                                   previous_prevalence_with_region_df["PREV_PRACTICE_LIST_SIZE"])*100
                                                                 ,(previous_prevalence_with_region_df["PREV_REGISTER_SIZE"]/
                                                                   previous_prevalence_with_region_df["PREV_ALT_PAT_LIST"])*100)

## Calculates the previous achievement rate data at region level
previous_prevalence_with_region_df["PREV_ACHIEVEMENT_RATE"] = (previous_prevalence_with_region_df["PREV_ACHIEVED_POINTS"]/
                                                               previous_prevalence_with_region_df["PREV_TOTAL_INDICATOR_GROUP_POINTS"])*100

## Calculates the previous PCA rate data at region level
previous_prevalence_with_region_df["PREV_PCA_RATE"] = (previous_prevalence_with_region_df["PREV_PCA_COUNT"]/
                                                       (previous_prevalence_with_region_df["PREV_DENOMINATOR"]+
                                                        previous_prevalence_with_region_df["PREV_PCA_COUNT"]))*100

## Join region level data to region level reference columns
previous_prevalence_region_data_df = pd.merge(previous_prevalence_region_gr_df
                                              ,previous_prevalence_with_region_df
                                              ,left_on=["REGION_NAME", "GROUP_CODE"]
                                              ,right_on=["REGION_NAME", "GROUP_CODE"]
                                              ,how="left")

## Reorder data frame columns so they can be appended to the national data frame
previous_prevalence_region_data_col = ["REGION_NAME","DOMAIN_CODE","HIGHER_GROUP_CODE","GROUP_CODE"
                                       ,"PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE","PREV_ALT_PAT_LIST"
                                       ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","PREV_ACHIEVED_POINTS"
                                       ,"PREV_TOTAL_INDICATOR_GROUP_POINTS","PREV_ACHIEVEMENT_RATE"
                                       ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"]
previous_prevalence_region_data_df = previous_prevalence_region_data_df[previous_prevalence_region_data_col]

In [ ]:
"""
Append national and region data frames to form combined National and Region excel tables

"""
previous_out_national_region_joined = previous_prevalence_national_data_df.append(previous_prevalence_region_data_df
                                                                                  ).reset_index(drop = True)

previous_out_national_region_joined = previous_out_national_region_joined[["REGION_NAME","DOMAIN_CODE"
                                                                           ,"HIGHER_GROUP_CODE","GROUP_CODE"
                                                                           ,"PREV_NUMBER_PRACTICES"
                                                                           ,"PREV_PRACTICE_LIST_SIZE","PREV_ALT_PAT_LIST"
                                                                           ,"PREV_PREVALENCE","PREV_REGISTER_SIZE"
                                                                           ,"PREV_ACHIEVED_POINTS","PREV_TOTAL_INDICATOR_GROUP_POINTS"
                                                                           ,"PREV_ACHIEVEMENT_RATE","PREV_PCA_COUNT"
                                                                           ,"PREV_DENOMINATOR","PREV_PCA_RATE"]]

In [ ]:
"""
Creates data frame for ICB (formerly STP) level with previous years data
As this code is reused year-on-year and was written prior to the change in NHS geogrphy group titles
the old names persist in the code but are renamed in the outputs.

"""

## Creates ICB (STP) level reference columns to which data can be joined
previous_prevalence_stp_gr_col = ["STP_ODS_CODE", "DOMAIN_CODE", "HIGHER_GROUP_CODE", "GROUP_CODE"]
previous_prevalence_stp_gr_df = previous_prevalence_with_geog_df[previous_prevalence_stp_gr_col]

previous_prevalence_stp_gr_df = previous_prevalence_stp_gr_df.drop_duplicates()

## Selects columns needed for ICB (STP) data frame and excludes unwanted NHS geography columns
previous_prevalence_with_stp_col = ["STP_ODS_CODE","GROUP_CODE","PRACTICE_CODE","ALT_LIST_SIZE"
                                    ,"REGISTER_SIZE","PRACTICE_LIST_SIZE","ACHIEVED_POINTS"
                                    ,"TOTAL_INDICATOR_GROUP_POINTS","PCA_COUNT","DENOMINATOR"]
previous_prevalence_with_stp_df = previous_prevalence_with_geog_df[previous_prevalence_with_stp_col]

previous_prevalence_with_stp_df = previous_prevalence_with_stp_df.groupby(by = ["STP_ODS_CODE","GROUP_CODE"]
                                                                          ,as_index=False
                                                                          ).agg({"PRACTICE_CODE" : [pd.Series.count]
                                                                                 ,"PRACTICE_LIST_SIZE" : [pd.Series.sum]
                                                                                 ,"ALT_LIST_SIZE" : [pd.Series.sum]
                                                                                 ,"REGISTER_SIZE" : [pd.Series.sum]
                                                                                 ,"ACHIEVED_POINTS" : [pd.Series.sum]
                                                                                 ,"TOTAL_INDICATOR_GROUP_POINTS" : [pd.Series.sum]
                                                                                 ,"PCA_COUNT" : [pd.Series.sum]
                                                                                 ,"DENOMINATOR" : [pd.Series.sum]
                                                                                 })                                                                            
previous_prevalence_with_stp_df.columns = previous_prevalence_with_stp_df.columns.droplevel(1)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
## Renames columns to show they contain last years data
previous_prevalence_with_stp_df = previous_prevalence_with_stp_df.rename(columns={"PRACTICE_CODE":"PREV_NUMBER_PRACTICES"
                                                                                  ,"PRACTICE_LIST_SIZE":"PREV_PRACTICE_LIST_SIZE"
                                                                                  ,"ALT_LIST_SIZE":"PREV_ALT_PAT_LIST"
                                                                                  ,"REGISTER_SIZE":"PREV_REGISTER_SIZE"
                                                                                  ,"ACHIEVED_POINTS":"PREV_ACHIEVED_POINTS"
                                                                                  ,"TOTAL_INDICATOR_GROUP_POINTS":"PREV_TOTAL_INDICATOR_GROUP_POINTS"
                                                                                  ,"PCA_COUNT":"PREV_PCA_COUNT"
                                                                                  ,"DENOMINATOR":"PREV_DENOMINATOR"})

## Calculates the previous prevalence data at ICB (STP) level                                               
previous_prevalence_with_stp_df["PREV_PREVALENCE"] = np.where(previous_prevalence_with_stp_df["PREV_ALT_PAT_LIST"] == 0
                                                              ,(previous_prevalence_with_stp_df["PREV_REGISTER_SIZE"]/
                                                                previous_prevalence_with_stp_df["PREV_PRACTICE_LIST_SIZE"])*100
                                                              ,(previous_prevalence_with_stp_df["PREV_REGISTER_SIZE"]/
                                                                previous_prevalence_with_stp_df["PREV_ALT_PAT_LIST"])*100)

## Calculates the previous achievement rate data at ICB (STP) level
previous_prevalence_with_stp_df["PREV_ACHIEVEMENT_RATE"] = (previous_prevalence_with_stp_df["PREV_ACHIEVED_POINTS"]/
                                                            previous_prevalence_with_stp_df["PREV_TOTAL_INDICATOR_GROUP_POINTS"])*100

## Calculates the previous PCA rate data at ICB (STP) level
previous_prevalence_with_stp_df["PREV_PCA_RATE"] = (previous_prevalence_with_stp_df["PREV_PCA_COUNT"]/
                                                    (previous_prevalence_with_stp_df["PREV_DENOMINATOR"]+
                                                     previous_prevalence_with_stp_df["PREV_PCA_COUNT"]))*100

## Join national level data to ICB (STP) level reference columns
previous_prevalence_stp_data_df = pd.merge(previous_prevalence_stp_gr_df
                                           ,previous_prevalence_with_stp_df
                                           ,left_on=["STP_ODS_CODE", "GROUP_CODE"]
                                           ,right_on=["STP_ODS_CODE", "GROUP_CODE"]
                                           ,how="left")

## Reorder data frame columns
previous_prevalence_stp_data_col = ["STP_ODS_CODE","DOMAIN_CODE","HIGHER_GROUP_CODE","GROUP_CODE"
                                    ,"PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
                                    ,"PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE"
                                    ,"PREV_ACHIEVED_POINTS","PREV_TOTAL_INDICATOR_GROUP_POINTS"
                                    ,"PREV_ACHIEVEMENT_RATE","PREV_PCA_COUNT","PREV_DENOMINATOR"
                                    ,"PREV_PCA_RATE"]
previous_prevalence_stp_data_df = previous_prevalence_stp_data_df[previous_prevalence_stp_data_col]


In [ ]:
"""
Creates data frame for SUB ICB (formerly CCG) level with previous years data
As this code is reused year-on-year and was written prior to the change in NHS geogrphy group titles
the old names persist in the code but are renamed in the outputs.

"""

## Creates SUB ICB (CCG) level reference columns to which data can be joined
previous_prevalence_ccg_gr_col = ["CCG_ODS_CODE", "DOMAIN_CODE", "HIGHER_GROUP_CODE", "GROUP_CODE"]
previous_prevalence_ccg_gr_df = previous_prevalence_with_geog_df[previous_prevalence_ccg_gr_col]

previous_prevalence_ccg_gr_df = previous_prevalence_ccg_gr_df.drop_duplicates()

## Selects columns needed for SUB ICB (CCG) data frame and excludes unwanted NHS geography columns
previous_prevalence_with_ccg_col = ["CCG_ODS_CODE","GROUP_CODE","PRACTICE_CODE","ALT_LIST_SIZE"
                                    ,"REGISTER_SIZE","PRACTICE_LIST_SIZE","ACHIEVED_POINTS"
                                    ,"TOTAL_INDICATOR_GROUP_POINTS","PCA_COUNT","DENOMINATOR"]                                         
previous_prevalence_with_ccg_df  = previous_prevalence_with_geog_df [previous_prevalence_with_ccg_col]

## Sums numeric data items to provide SUB ICB (CCG) level data
previous_prevalence_with_ccg_df = previous_prevalence_with_ccg_df.groupby(by =["CCG_ODS_CODE","GROUP_CODE"]
                                                                          ,as_index=False
                                                                          ).agg({"PRACTICE_CODE" : [pd.Series.count]
                                                                                 ,"PRACTICE_LIST_SIZE" : [pd.Series.sum]
                                                                                 ,"ALT_LIST_SIZE" : [pd.Series.sum]
                                                                                 ,"REGISTER_SIZE" : [pd.Series.sum]
                                                                                 ,"ACHIEVED_POINTS" : [pd.Series.sum]
                                                                                 ,"TOTAL_INDICATOR_GROUP_POINTS" : [pd.Series.sum]
                                                                                 ,"PCA_COUNT" : [pd.Series.sum]
                                                                                 ,"DENOMINATOR" : [pd.Series.sum]
                                                                                 })                                                                                                                                                                                                                                                                                                                                                                                                                                                             
previous_prevalence_with_ccg_df.columns = previous_prevalence_with_ccg_df.columns.droplevel(1)                                                                         
                                                                        
## Renames columns to show they contain last years data
previous_prevalence_with_ccg_df = previous_prevalence_with_ccg_df.rename(columns={"PRACTICE_CODE":"PREV_NUMBER_PRACTICES"
                                                                                  ,"PRACTICE_LIST_SIZE":"PREV_PRACTICE_LIST_SIZE"
                                                                                  ,"ALT_LIST_SIZE":"PREV_ALT_PAT_LIST"
                                                                                  ,"REGISTER_SIZE":"PREV_REGISTER_SIZE"
                                                                                  ,"ACHIEVED_POINTS":"PREV_ACHIEVED_POINTS"
                                                                                  ,"TOTAL_INDICATOR_GROUP_POINTS":"PREV_TOTAL_INDICATOR_GROUP_POINTS"
                                                                                  ,"PCA_COUNT":"PREV_PCA_COUNT"
                                                                                  ,"DENOMINATOR":"PREV_DENOMINATOR"})

## Calculates the previous prevalence data at SUB ICB (CCG) level
previous_prevalence_with_ccg_df["PREV_PREVALENCE"] = np.where(previous_prevalence_with_ccg_df["PREV_ALT_PAT_LIST"] == 0
                                                              ,(previous_prevalence_with_ccg_df["PREV_REGISTER_SIZE"]/
                                                                previous_prevalence_with_ccg_df["PREV_PRACTICE_LIST_SIZE"])*100
                                                              ,(previous_prevalence_with_ccg_df["PREV_REGISTER_SIZE"]/
                                                                previous_prevalence_with_ccg_df["PREV_ALT_PAT_LIST"])*100)

## Calculates the previous achievement rate at SUB ICB (CCG) level
previous_prevalence_with_ccg_df["PREV_ACHIEVEMENT_RATE"] = (previous_prevalence_with_ccg_df["PREV_ACHIEVED_POINTS"]/
                                                            previous_prevalence_with_ccg_df["PREV_TOTAL_INDICATOR_GROUP_POINTS"])*100

## Calculates the previous PCA rate at SUB ICB (CCG) level
previous_prevalence_with_ccg_df["PREV_PCA_RATE"] = (previous_prevalence_with_ccg_df["PREV_PCA_COUNT"]/
                                                    (previous_prevalence_with_ccg_df["PREV_DENOMINATOR"]+
                                                     previous_prevalence_with_ccg_df["PREV_PCA_COUNT"]))*100

## Join national level data to SUB ICB (CCG) level reference columns
previous_prevalence_ccg_data_df  = pd.merge(previous_prevalence_ccg_gr_df
                                            ,previous_prevalence_with_ccg_df
                                            ,left_on=["CCG_ODS_CODE","GROUP_CODE"]
                                            ,right_on=["CCG_ODS_CODE","GROUP_CODE"]
                                            ,how="left")

## Reorder data frame columns
previous_prevalence_ccg_data_col = ["CCG_ODS_CODE","DOMAIN_CODE","HIGHER_GROUP_CODE","GROUP_CODE"
                                    ,"PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE","PREV_ALT_PAT_LIST"
                                    ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","PREV_ACHIEVED_POINTS"
                                    ,"PREV_TOTAL_INDICATOR_GROUP_POINTS","PREV_ACHIEVEMENT_RATE"
                                    ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"]
previous_prevalence_ccg_data_df  = previous_prevalence_ccg_data_df [previous_prevalence_ccg_data_col]

In [ ]:
"""
Creates data frame for practice level with previous years data
LVSD data frame is appended to the practice level with previous years data
Practice level data is the only level that contains LVSD data

"""

## Creates practice level reference columns to which data can be joined
## In addition it contains data for 'ALT_LIST_SIZE' and 'PRACTICE_LIST_SIZE' which are not summed
previous_prevalence_prac_gr_col = ["PRACTICE_CODE","DOMAIN_CODE","HIGHER_GROUP_CODE"
                                   ,"GROUP_CODE","ALT_LIST_SIZE","PRACTICE_LIST_SIZE"]
previous_prevalence_prac_gr_df = previous_prevalence_with_geog_df[previous_prevalence_prac_gr_col]

previous_prevalence_prac_gr_df = previous_prevalence_prac_gr_df.drop_duplicates()


## Selects columns needed for practice data frame and excludes unwanted NHS geography columns
previous_prevalence_with_prac_col = ["PRACTICE_CODE", "GROUP_CODE","REGISTER_SIZE","ACHIEVED_POINTS"
                                    ,"TOTAL_INDICATOR_GROUP_POINTS","PCA_COUNT","DENOMINATOR"]
previous_prevalence_with_prac_df  = previous_prevalence_with_geog_df[previous_prevalence_with_prac_col]


## Sums numeric data items to provide practice level data excluding 'ALT_LIST_SIZE' and 'PRACTICE_LIST_SIZE'
previous_prevalence_with_prac_df = previous_prevalence_with_prac_df.groupby(by = ["PRACTICE_CODE","GROUP_CODE"]
                                                                            ,as_index=False
                                                                            ).agg({"REGISTER_SIZE" : [pd.Series.sum]
                                                                                   ,"ACHIEVED_POINTS" : [pd.Series.sum]
                                                                                   ,"TOTAL_INDICATOR_GROUP_POINTS" : [pd.Series.sum]
                                                                                   ,"PCA_COUNT" : [pd.Series.sum]
                                                                                   ,"DENOMINATOR" : [pd.Series.sum]
                                                                                   })                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
previous_prevalence_with_prac_df.columns = previous_prevalence_with_prac_df.columns.droplevel(1)

## Join practice level data to practice level reference columns
previous_prevalence_prac_data_df  = pd.merge(previous_prevalence_prac_gr_df
                                             ,previous_prevalence_with_prac_df
                                             ,left_on=["PRACTICE_CODE", "GROUP_CODE"]
                                             ,right_on=["PRACTICE_CODE", "GROUP_CODE"] 
                                             ,how="left")

## Renames columns to show they contain last years data
previous_prevalence_prac_data_df = previous_prevalence_prac_data_df.rename(columns={"PRACTICE_LIST_SIZE":"PREV_PRACTICE_LIST_SIZE"
                                                                                    ,"ALT_LIST_SIZE":"PREV_ALT_PAT_LIST"
                                                                                    ,"REGISTER_SIZE":"PREV_REGISTER_SIZE"
                                                                                    ,"ACHIEVED_POINTS":"PREV_ACHIEVED_POINTS"
                                                                                    ,"TOTAL_INDICATOR_GROUP_POINTS":"PREV_TOTAL_INDICATOR_GROUP_POINTS"
                                                                                    ,"PCA_COUNT":"PREV_PCA_COUNT"
                                                                                    ,"DENOMINATOR":"PREV_DENOMINATOR"})
                                                
## Calculates the previous prevalence data at practice level
previous_prevalence_prac_data_df["PREV_PREVALENCE"] = np.where(previous_prevalence_prac_data_df["PREV_ALT_PAT_LIST"].isnull()
                                                               ,(previous_prevalence_prac_data_df["PREV_REGISTER_SIZE"]/
                                                                 previous_prevalence_prac_data_df["PREV_PRACTICE_LIST_SIZE"])*100
                                                               ,(previous_prevalence_prac_data_df["PREV_REGISTER_SIZE"]/
                                                                 previous_prevalence_prac_data_df["PREV_ALT_PAT_LIST"])*100)
                                              
## Calculates the previous achievement rate at practice level
previous_prevalence_prac_data_df["PREV_ACHIEVEMENT_RATE"] = (previous_prevalence_prac_data_df["PREV_ACHIEVED_POINTS"]/
                                                             previous_prevalence_prac_data_df["PREV_TOTAL_INDICATOR_GROUP_POINTS"])*100

## Calculates the previous PCA rate at practice level
previous_prevalence_prac_data_df["PREV_PCA_RATE"] = (previous_prevalence_prac_data_df["PREV_PCA_COUNT"]/
                                                     (previous_prevalence_prac_data_df["PREV_DENOMINATOR"]+
                                                      previous_prevalence_prac_data_df["PREV_PCA_COUNT"]))*100

## Reorder data frame columns
previous_prevalence_prac_data_col = ["PRACTICE_CODE","DOMAIN_CODE","HIGHER_GROUP_CODE","GROUP_CODE"
                                     ,"PREV_PRACTICE_LIST_SIZE","PREV_ALT_PAT_LIST"
                                     ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","PREV_ACHIEVED_POINTS"
                                     ,"PREV_TOTAL_INDICATOR_GROUP_POINTS","PREV_ACHIEVEMENT_RATE"
                                     ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"]
previous_prevalence_prac_data_df = previous_prevalence_prac_data_df[previous_prevalence_prac_data_col]


## Replace a zero (0) in 'PREV_PRACTICE_LIST_SIZE', 'PREV_ALT_PAT_LIST', and 'PREV_REGISTER_SIZE' with 'nan'.
previous_prevalence_prac_data_df["PREV_PRACTICE_LIST_SIZE"] = np.where(previous_prevalence_prac_data_df["PREV_PRACTICE_LIST_SIZE"] == 0
                                                                       , np.nan
                                                                       , previous_prevalence_prac_data_df["PREV_PRACTICE_LIST_SIZE"])

previous_prevalence_prac_data_df["PREV_ALT_PAT_LIST"] = np.where(previous_prevalence_prac_data_df["PREV_ALT_PAT_LIST"] == 0
                                                                 , np.nan
                                                                 , previous_prevalence_prac_data_df["PREV_ALT_PAT_LIST"])

previous_prevalence_prac_data_df["PREV_REGISTER_SIZE"] = np.where(previous_prevalence_prac_data_df["PREV_REGISTER_SIZE"] == 0
                                                                  , np.nan
                                                                  , previous_prevalence_prac_data_df["PREV_REGISTER_SIZE"])

## Joins the LVSD data frame to the previous practice data frame for output as part of the indicator groups/indicators in the excel tables
previous_out_prac_df = previous_prevalence_prac_data_df.append(LVSD_data).reset_index(drop = True)


<h2><b>Suppression</b></h2>

Suppression is applied to practice level data when:
1. For fractional indicators with 1 PCA specified: where denominator <2 and PCA > 0, suppress the PCA and the denominator for that indicator.
2. For fractional indicators with >1 PCA specified: where denominator = 0 and the sum of all PCAs is equal to the value of any one PCA, suppress all the PCAs for that indicator.

Suppressed values are replaced by '*'.


In [ ]:
"""
This is the foundation (base) data frame that contains values for practices and indicators codes 
with both 1 PCA specified and more than 1 PCA specified

"""

## Create base table without suppression to which suppression can be applied
prac_pcas_base_col = ["PRACTICE_CODE","INDICATOR_CODE","PCA_NAME","PCA_COUNT"]
prac_pcas_base_df = prac_pcas_df[prac_pcas_base_col]

## Add number of PCAs per indicator (PCACOUNT from Indicator Control Table) to 
## 'PRACTICE_CODE','INDICATOR_CODE','DENOMINATOR','PCA_NAME','PCA_COUNT','PCA_COUNT'
## This allows for the data frame to be filtered by 'PCA_COUNT_1' = 1
pca_count_df = Indicator_Reference_Control_Table[["INDICATOR_CODE","PCA_COUNT_1"]]
prac_pcas_flag_df = pd.merge(prac_pcas_df
                             ,pca_count_df
                             ,left_on=["INDICATOR_CODE"]
                             ,right_on=["INDICATOR_CODE"]
                             ,how="left")



In [ ]:
"""
For fractional indicators with 1 PCA specified: where denominator <2 and PCA > 0, 
suppress the PCA and the denominator for that indicator. Suppressed values are replaced by '*'.

This code deals with Indicators that only have 1 PCA.

** Since no fractional indicators with 1 PCA specified required primary suppression (checks were made to ensure that is correct), 
** Secondary suppression was not needed
** Secondary suppression code has still to be written and will need to be added at a later date.

"""

## Filter for indicators with 1 PCA
prac_count_one_pca_df = prac_pcas_flag_df.loc[prac_pcas_flag_df["PCA_COUNT_1"] == 1]

        
## Identifies which combinations of data suppression should be applied to and adds a 'SUP_FLAG' 
## value of 1 if suppression is required
conditions = [
    ((prac_count_one_pca_df["DENOMINATOR"] < 2)&(prac_count_one_pca_df["PCA_COUNT"] > 0))]
choices = [1]
prac_count_one_pca_df["SUP_FLAG"] = np.select(conditions, choices, default=0)

## Create a data frame of data that requires suppression 
prac_one_pca_sup_flag_df = prac_count_one_pca_df.loc[prac_count_one_pca_df["SUP_FLAG"] == 1]

prac_one_pca_sup_flag_col = ["INDICATOR_CODE","PRACTICE_CODE","SUP_FLAG"]
prac_one_pca_sup_flag_df = prac_one_pca_sup_flag_df[prac_one_pca_sup_flag_col]

prac_one_pca_sup_flag_df = prac_one_pca_sup_flag_df.drop_duplicates()

prac_count_one_pca_df = prac_count_one_pca_df.drop(columns=["SUP_FLAG"])

## Combine suppression data frame with data for indicators with 1 PCA
prac_pca_one_pca_sup_df = pd.merge(prac_count_one_pca_df
                                   ,prac_one_pca_sup_flag_df
                                   ,left_on=["PRACTICE_CODE","INDICATOR_CODE"]
                                   ,right_on=["PRACTICE_CODE","INDICATOR_CODE"]
                                   ,how="left")

## Apply suppression character to flagged data for 'PCA_COUNT' and 'DENOMINATOR' values
conditions = [
    (prac_pca_one_pca_sup_df["SUP_FLAG"] == 1)]
choices = ["*"]

prac_pca_one_pca_sup_df["PCA_COUNT"] = np.select(conditions
                                                 ,choices
                                                 ,default=prac_pca_one_pca_sup_df["PCA_COUNT"])

prac_pca_one_pca_sup_df["DENOMINATOR"] = np.select(conditions
                                                   ,choices
                                                   ,default=prac_pca_one_pca_sup_df["DENOMINATOR"])


In [ ]:
"""
For fractional indicators with >1 PCA specified: where denominator = 0 and the 
sum of all PCAs is equal to the value of any one PCA, suppress all the PCAs 
for that indicator. Suppressed values are replaced by '*'.

This code deals with indicators that do not only have 1 PCA

"""

## Filter for indicators who do not have only 1 PCA
prac_count_two_pca_df = prac_pcas_flag_df.loc[prac_pcas_flag_df["PCA_COUNT_1"] != 1]


## Identifies which combinations of data suppression should be applied to and adds 
## a 'SUP_FLAG' value of 2 if suppression is required
conditions = [
    ((prac_count_two_pca_df["DENOMINATOR"] == 0)&(prac_count_two_pca_df["TOT_PCA_COUNT"] >0)&
     (prac_count_two_pca_df["PCA_COUNT"] == prac_count_two_pca_df["TOT_PCA_COUNT"]))]
choices = [2]
prac_count_two_pca_df["SUP_FLAG"] = np.select(conditions
                                              ,choices
                                              ,default=0)

## Filter for data with a 'SUP_FLAG' = 2
prac_count_two_pca_fil_df = prac_count_two_pca_df.loc[prac_count_two_pca_df["SUP_FLAG"] == 2]

## Select required columns and remove duplicate data
prac_two_pca_sup_flag_col = ["INDICATOR_CODE","PRACTICE_CODE","SUP_FLAG"]
prac_two_pca_sup_flag_df = prac_count_two_pca_fil_df[prac_two_pca_sup_flag_col]

prac_two_pca_sup_df = prac_two_pca_sup_flag_df.drop_duplicates()  

prac_count_two_pca_df = prac_count_two_pca_df.drop(columns=["SUP_FLAG"])
                                                
## Combine suppression data frame with data for indicators who do not have only 1 PCA
prac_pca_two_pca_sup_df = pd.merge(prac_count_two_pca_df
                                   ,prac_two_pca_sup_df
                                   ,left_on=["PRACTICE_CODE","INDICATOR_CODE"]
                                   ,right_on=["PRACTICE_CODE","INDICATOR_CODE"]
                                   ,how="left")

## Apply suppression character to flagged data for 'PCA_COUNT' values
conditions = [
    (prac_pca_two_pca_sup_df["SUP_FLAG"] == 2)]
choices = ["*"]

prac_pca_two_pca_sup_df["PCA_COUNT"] = np.select(conditions
                                                 ,choices
                                                 ,default=prac_pca_two_pca_sup_df["PCA_COUNT"])

In [ ]:
"""
Appends the two data frames containing data to be suppressed.
As a practice code indicator code combination cannot be present in both data frames.
This will create a data frame of all the practice code indicator code 
combinations with suppression.

This is used as the output for the PRACTICE_PCA csvs by region

"""

prac_pcas_sup_df = prac_pca_one_pca_sup_df.append(prac_pca_two_pca_sup_df)

# 4. CSV outputs
[Return to contents](#Table-of-Contents)

### ACHIEVEMENT.CSV

In [ ]:
"""
Create the ouput needed for the 'ACHIEVEMENT.csv'  

"""

## Select columns for csv
qof_ach_csv_master_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME"
                          ,"PRACTICE_CODE","INDICATOR_CODE","REGISTER_SIZE"
                          ,"ACHIEVEMENT_NUMERATOR","ACHIEVEMENT_DENOMINATOR"
                          ,"PCA_COUNT","ACHIEVED_POINTS"]
qof_ach_csv_master_df = qof_master[qof_ach_csv_master_col]


## Rename columns and replace 'nan' values with a blank
qof_ach_csv_master_df = qof_ach_csv_master_df.rename(columns={"REGISTER_SIZE":"REGISTER"
                                                              ,"ACHIEVEMENT_NUMERATOR":"NUMERATOR"
                                                              ,"ACHIEVEMENT_DENOMINATOR":"DENOMINATOR"
                                                              ,"PCA_COUNT":"PCAS"}).replace(np.nan,"")
                                            

## The short broad data frame is converted into a long thin data frame and the non-index columns 
## are stacked in a column called 'MEASURES'
qof_ach_csv_stacked = (qof_ach_csv_master_df.set_index(["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME"
                                                        ,"PRACTICE_CODE","INDICATOR_CODE"])
                                                        .stack()
                                                        .ffill(axis=0)
                                                        .bfill(axis=0, downcast='infer')
                                                        .reset_index()
                                                        .rename({"level_5": "MEASURE", 0:"VALUE"}, axis=1))
                                            

## Filter for indicators that do not have a blank value
## 'qof_ach_parallel_csv_df' is produced as a single output for Parallel the company 
## who produce our external QOF website
qof_ach_parallel_csv_df = qof_ach_csv_stacked.loc[qof_ach_csv_stacked["VALUE"]!=""]

## Format the column 'REGION_NAME' ready to be ouput as separate region files
ach_csv_df = qof_ach_parallel_csv_df
ach_csv_df["REGION_NAME"] = ach_csv_df["REGION_NAME"].str.upper().str.replace(" ","_")        
                                                                
## Output used by 'Export CSVs' code
ach_region_df = ach_csv_df.groupby("REGION_NAME")


### PREVALENCE.CSV

In [ ]:
"""
Create the ouput needed for the 'PREVALENCE.csv'
"""

## Select columns for qof master data frame
qof_prevalence_csv_master_col = ["PRACTICE_CODE","LIST_SIZE","GROUP_CODE","ALT_PAT_LIST"
                                 ,"REGISTER_SIZE","NO_PREVALENCE_FLAG","REGISTER_FLAG"]
qof_prevalence_csv_master_df = qof_master[qof_prevalence_csv_master_col]

In [ ]:
## Reshapes data frame 'qof_prevalence_csv_master_df' into three columns
total_list_by_prac_df = qof_prevalence_csv_master_df.drop(columns=["GROUP_CODE","ALT_PAT_LIST","REGISTER_SIZE"
                                                                   ,"NO_PREVALENCE_FLAG","REGISTER_FLAG"]
                                                                   ).drop_duplicates()
                                                                                                                             
total_list_by_prac_df["LIST_TYPE"] = "TOTAL"


## Rename columns in the data frame that contains alternative list sizes
all_alt_lists_df = all_lists_wide_stacked.rename(columns={"ALT_PAT_LIST":"LIST_TYPE"
                                                          ,"VALUE":"LIST_SIZE"})


## Append 'all_alt_lists_df' to 'total_list_by_prac_df'
pat_list_csv_out_df = total_list_by_prac_df.append(all_alt_lists_df
                                                   ,ignore_index=True
                                                   ).sort_values(by=["PRACTICE_CODE","LIST_TYPE"]
                                                                 ,ascending = [True, True])
                                                                
pat_list_csv_out_col = ["PRACTICE_CODE","LIST_SIZE","LIST_TYPE"]
pat_list_csv_out_df = pat_list_csv_out_df[pat_list_csv_out_col]


In [ ]:
## Filters prevalence master data frame
reg_by_grp_and_prac_df = qof_prevalence_csv_master_df.loc[((qof_prevalence_csv_master_df["NO_PREVALENCE_FLAG"]!=1)&
                                                           (qof_prevalence_csv_master_df["REGISTER_FLAG"]==1))]

## replace all 'nan' values with a blank
reg_by_grp_and_prac_df = reg_by_grp_and_prac_df.replace(np.nan,"")

## Define contents of new columns 'LIST_TYPE' - 'TOTAL' as all ages and 'ALT_PAT_LIST' as alternate age bands
reg_by_grp_and_prac_df["LIST_TYPE"] = np.where(reg_by_grp_and_prac_df["ALT_PAT_LIST"]== ""
                                               ,"TOTAL"
                                               ,reg_by_grp_and_prac_df["ALT_PAT_LIST"])

## Drop unwanted columns
reg_by_grp_and_prac_df = reg_by_grp_and_prac_df.drop(columns=["LIST_SIZE","NO_PREVALENCE_FLAG"
                                                              ,"ALT_PAT_LIST","REGISTER_FLAG"])

In [ ]:

## Combines data frames 'reg_by_grp_and_prac_df' and 'pat_list_csv_out_df' 
## to create the contents of the 'PREVALENCE.csv'
prevalence_csv_df = pd.merge(reg_by_grp_and_prac_df
                             ,pat_list_csv_out_df
                             ,left_on = ["PRACTICE_CODE","LIST_TYPE"]
                             ,right_on = ["PRACTICE_CODE","LIST_TYPE"]
                             ,how ="left")

prevalence_csv_df = prevalence_csv_df.rename(columns={"REGISTER_SIZE":"REGISTER"
                                                      ,"LIST_TYPE":"PATIENT_LIST_TYPE"
                                                      ,"LIST_SIZE":"PRACTICE_LIST_SIZE"}
                                                      ).sort_values(by=["PRACTICE_CODE","GROUP_CODE"]
                                                                    ,ascending = [True, True]).replace("",0)                                   
                                    

### ORGANISATION_REFERENCE.CSV

In [ ]:
"""
Create the ouput needed for the 'ORGANISATION_REFERENCE.csv'

"""

## Select columns for qof master data frame
org_ref_csv_col = ["PRACTICE_CODE","REVISED_MAX_POINTS"]
org_ref_csv_df = qof_master[org_ref_csv_col]

## Formats data frame ready for export
org_ref_csv_df = org_ref_csv_df.drop_duplicates().sort_values(by=["PRACTICE_CODE"]
                                                              ,ascending = [True]
                                                              ).reset_index(drop = True)                                            

### INDICATOR_MAPPING.CSV

In [ ]:
"""
Create the ouput needed for the 'INDICATOR_MAPPING.csv'

"""

## Select required columns
ind_map_csv_col = ["INDICATOR_CODE","INDICATOR_POINT_VALUE","GROUP_CODE","GROUP_DESCRIPTION"
                   ,"DOMAIN_CODE","DOMAIN_DESCRIPTION","ALT_PAT_LIST","ADD_PAT_LIST"]
ind_map_csv_df = Indicator_Reference_Control_Table[ind_map_csv_col]

## Replace 'nan' values with a blank
ind_map_csv_df = ind_map_csv_df.replace(np.nan,"")

## When there is no value in the 'ADD_PAT_LIST' column use the value from 
## the 'ALT_PAT_LIST' to create new column 'PAT_LIST_TYPE'
ind_map_csv_df["PAT_LIST_TYPE"] = np.where(ind_map_csv_df["ADD_PAT_LIST"]== ""
                                           ,ind_map_csv_df["ALT_PAT_LIST"]
                                           ,ind_map_csv_df["ADD_PAT_LIST"])

## When there is no value in the 'PAT_LIST_TYPE' column use the 
## value 'TOTAL' to create new column 'PATIENT_LIST_TYPE'
ind_map_csv_df["PATIENT_LIST_TYPE"] = np.where(ind_map_csv_df["PAT_LIST_TYPE"]== ""
                                               ,"TOTAL"
                                               ,ind_map_csv_df["PAT_LIST_TYPE"])

## Drop unwanted columns used in the calculation creating a data frame ready for export
ind_map_csv_df = ind_map_csv_df.drop(columns=["ALT_PAT_LIST","ADD_PAT_LIST","PAT_LIST_TYPE"])


### PRACTICE_PCA.CSV

In [ ]:
"""
Create the ouput needed for the 'PRACTICE_PCA.csv'. 
The data frame of values is brought down from the 'Suppression' code block 
and contains values already suppressed

"""

## Select required columns
prac_pcas_excl_csv_col = ["PRACTICE_CODE","INDICATOR_CODE","PCA_NAME","PCA_COUNT"]
prac_pcas_excl_csv_df = prac_pcas_sup_df[["PRACTICE_CODE","INDICATOR_CODE","PCA_NAME","PCA_COUNT"]]

prac_pcas_mapping_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PRACTICE_CODE"]
prac_pcas_mapping_df = geog_mappings[["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PRACTICE_CODE"]]


## Merge the two data frames to add region nhs geographies
## prac_pcas_mapping_parallel_df is produced as a single output for Parallel 
## the company who produce our external website
prac_pcas_mapping_parallel_df = pd.merge(prac_pcas_mapping_df
                                         ,prac_pcas_excl_csv_df
                                         ,left_on=["PRACTICE_CODE"]
                                         ,right_on=["PRACTICE_CODE"]
                                         ,how="right")

## Format the column 'REGION_NAME' ready to be ouput as separate region files
prac_pcas_mapping_df  = prac_pcas_mapping_parallel_df 
prac_pcas_mapping_df["REGION_NAME"] = prac_pcas_mapping_df["REGION_NAME"].str.upper().str.replace(" ","_")
                                                                        
prac_pcas_region_df = prac_pcas_mapping_df.groupby("REGION_NAME")

### MAPPING_NHS_GEOGRAPHIES.CSV

In [ ]:
"""
Create the ouput needed for the ' MAPPING_NHS_GEOGRAHIES.csv'. 

"""

## Select required columns
mapping_nhs_geogs_col = ["NAT_ONS_CODE","NAT_CODE","NAT_COUNTRY"
                         ,"REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME"
                         ,"STP_ODS_CODE","STP_ONS_CODE","STP_NAME"
                         ,"CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME"
                         ,"PCN_ODS_CODE","PCN_NAME"
                         ,"PRACTICE_CODE","PRACTICE_NAME"]                                  
mapping_nhs_geogs_df = geog_mappings[mapping_nhs_geogs_col]

## Renamed columns from old nhs geographies areas to new nhs geographies areas
mapping_nhs_geogs_df = mapping_nhs_geogs_df.rename(columns={"STP_ODS_CODE":"ICB_ODS_CODE"
                                                            ,"STP_ONS_CODE":"ICB_ONS_CODE"
                                                            ,"STP_NAME":"ICB_NAME"
                                                            ,"CCG_ODS_CODE":"SUB_ICB_ODS_CODE"
                                                            ,"CCG_ONS_CODE":"SUB_ICB_ONS_CODE"
                                                            ,"CCG_NAME":"SUB_ICB_NAME"})

## Export CSVs

In [ ]:
"""    
Export CSV files to appropriate folders.
ACHIEVEMENT and PRACTICE_PCA data are output as separate regional files as 
the source data frame is to large for the publication. These files are 
exported for Parallel before being divided by region.

"""

## If the test_ run flag at the beginning of the code is set to true. 
# The data will be exported to a 'test folder' on the local drive.

## If it is set to false the data will be exported to publication OUTPUTS folders as per root directory in config file

if test_run is True:
    for name, group in ach_region_df:
        group.to_csv(f"test_folder\\ACHIEVEMENT_{name}_"+short_year+".csv", index=False)
    for name, group in prac_pcas_region_df:
        group.to_csv(f"test_folder\\PRACTICE_PCA_{name}_"+short_year+".csv", index=False)

    qof_ach_parallel_csv_df.to_csv(f"test_folder\\ACHIEVEMENT_"+short_year+".csv", index = False)
    prevalence_csv_df.to_csv(f"test_folder\\PREVALENCE_"+short_year+".csv", index = False)
    org_ref_csv_df.to_csv(f"test_folder\\ORGANISATION_REFERENCE_"+short_year+".csv", index = False)
    ind_map_csv_df.to_csv(f"test_folder\\MAPPING_INDICATORS_"+short_year+".csv", index = False)
    prac_pcas_mapping_parallel_df.to_csv(f"test_folder\\PRACTICE_PCA_"+short_year+".csv", index = False)
    prac_validation_outcomes.to_csv(f"test_folder\\PRACTICE_VALIDATION_OUTCOMES_"+short_year+".csv", index = False)
    mapping_nhs_geogs_df.to_csv(f"test_folder\\MAPPING_NHS_GEOGRAPHIES_"+short_year+".csv", index = False)
    prac_pcas_base_df.to_csv(f"test_folder\\PRACTICE_PCA_UNSUP_"+short_year+".csv", index = False)
    coverage.to_csv(f"test_folder\\PRACTICE_PCA_UNSUP_"+short_year+".csv", index = False)

elif test_run is False:
    for name, group in ach_region_df:
        group.to_csv(outputs+"\CSVs\ACHIEVEMENT_{name}_"+short_year+".csv", index=False)
    for name, group in prac_pcas_region_df:
        group.to_csv(outputs+"\CSVs\PRACTICE_PCA_{name}_"+short_year+".csv", index=False)

    qof_ach_parallel_csv_df.to_csv(outputs+"\CSVs\PARALLEL_ONLY\ACHIEVEMENT_"+short_year+".csv", index = False)
    prevalence_csv_df.to_csv(outputs+"\CSVs\PREVALENCE_"+short_year+".csv", index = False)
    org_ref_csv_df.to_csv(outputs+"\CSVs\ORGANISATION_REFERENCE_"+short_year+".csv", index = False)
    ind_map_csv_df.to_csv(outputs+"\CSVs\MAPPING_INDICATORS_"+short_year+".csv", index = False)
    prac_pcas_mapping_parallel_df.to_csv(outputs+"\CSVs\PARALLEL_ONLY\PRACTICE_PCA_"+short_year+".csv", index = False)
    prac_validation_outcomes.to_csv(outputs+"\CSVs\PRACTICE_VALIDATION_OUTCOMES_"+short_year+".csv", index = False)
    mapping_nhs_geogs_df.to_csv(outputs+"\CSVs\MAPPING_NHS_GEOGRAPHIES_"+short_year+".csv", index = False)
    prac_pcas_base_df.to_csv(dnp+"\PRACTICE_PCA_UNSUP_do_not_publish_"+short_year+".csv", index = False)
    coverage.to_csv(dnp+"\Coverage_do_not_publish_"+short_year+".csv", index = False)

<h1><b>OUTPUTS - NEXT YEARS CSVs</b></h1>

### PREVIOUS_INDICATOR_CONTROL.CSV

In [ ]:
"""
Create the ouput needed for the ' PREVIOUS_INDICATOR_CONTROL.csv'. 

"""

## Select required columns from 'Indicator_Reference_Control_Table'
## Remove duplicate entries
## Replace 'nan' values with a blank
next_years_ind_control_csv_col = ["GROUP_CODE","HIGHER_GROUP_CODE","NO_PREVALENCE_FLAG"]
next_years_ind_control_csv_df = Indicator_Reference_Control_Table[next_years_ind_control_csv_col]

next_years_ind_control_csv_df = next_years_ind_control_csv_df.drop_duplicates(
                                                            ).replace(np.nan,""
                                                            ).reset_index(drop=True)


### PREVIOUS_INDICATOR_MAPPINGS.CSV

In [ ]:
"""
Create the ouput needed for the ' PREVIOUS_INDICATOR_CONTROL.csv'. 

"""

## Select required columns from 'Indicator_Reference_Control_Table'
next_years_ind_map_csv_col = ["INDICATOR_CODE","INDICATOR_POINT_VALUE"
                              ,"GROUP_CODE","GROUP_DESCRIPTION"
                              ,"HIGHER_GROUP_CODE","HIGHER_GROUP_DESCRIPTION"
                              ,"ALT_PAT_LIST","ADD_PAT_LIST"]
next_years_ind_map_csv_df = Indicator_Reference_Control_Table[next_years_ind_map_csv_col]

## Rename columns
next_years_ind_map_csv_df = next_years_ind_map_csv_df.rename(columns={"GROUP_CODE":"INDICATOR_GROUP_CODE"
                                                                      ,"GROUP_DESCRIPTION":"INDICATOR_GROUP_DESCRIPTION"
                                                                      ,"HIGHER_GROUP_CODE":"DOMAIN_CODE"
                                                                      ,"HIGHER_GROUP_DESCRIPTION":"DOMAIN_DESCRIPTION"}
                                                                      ).replace(np.nan,"")
                                                    


## When there is no value in the 'ADD_PAT_LIST' column use the value from the 'ALT_PAT_LIST' 
## to create new column 'PAT_LIST_TYPE'
next_years_ind_map_csv_df["PATIENT_LIST_TYPE"] = np.where(next_years_ind_map_csv_df["ALT_PAT_LIST"]== ""
                                                          ,"TOTAL"
                                                          ,next_years_ind_map_csv_df["ALT_PAT_LIST"])

## When there is no value in the 'PAT_LIST_TYPE' column use the value 'TOTAL' 
## to create new column 'PATIENT_LIST_TYPE'
next_years_ind_map_csv_df["PATIENT_ADD_LIST_TYPE"] = np.where(next_years_ind_map_csv_df["ADD_PAT_LIST"]== ""
                                                              ,"TOTAL"
                                                              ,next_years_ind_map_csv_df["ADD_PAT_LIST"])

## Drop unwanted columns used in the calculation creating a data frame ready for export
next_years_ind_map_csv_df = next_years_ind_map_csv_df.drop(columns=["ALT_PAT_LIST","ADD_PAT_LIST"])

### PREVIOUS_PREVALENCE_NON_REGISTER_ALT_INDICATORS.CSV

In [ ]:
"""
Create the ouput needed for the ' PREVIOUS_PREVALENCE_NON_REGISTER_ALT_INDICATORS.csv'. 

"""

## Drop unwanted column
## Remove duplicate values
## Remove 'nan' values from 'ALT_PAT_LIST' column which defines the type of 
## alternative patient list 'ALT_LIST' count refers to 

next_years_non_reg_alt_ind_df = alt_pat_list_by_ind.drop(columns=["INDICATOR_CODE"]
                                                         ).drop_duplicates(                                                             
                                                         ).rename(columns={"ALT_LIST_SIZE":"ALT_LIST"}
                                                                  ).dropna(subset=["ALT_PAT_LIST"])

In [ ]:
"""    
Export CSV files to appropriate folders.

"""

## If the test_ run flag at the beginning of the code is set to true. 
## The data will be exported to a 'test folder' on the local drive.
## If it is set to false the data will be exported to publication folders in network drive

if test_run is True:
    ach_csv_df.to_csv(f"test_folder\\PREVIOUS_ACHIEVEMENTS_PCAS.csv", index = False)
    prevalence_csv_df.to_csv(f"test_folder\\PREVIOUS_PREVALENCE.csv", index = False)
    org_ref_csv_df.to_csv(f"test_folder\\PREVIOUS_ORGANISATION_REFERENCE.csv", index = False)
    next_years_ind_control_csv_df.to_csv(f"test_folder\\PREVIOUS_INDICATOR_CONTROL.csv", index = False)
    next_years_ind_map_csv_df.to_csv(f"test_folder\\PREVIOUS_INDICATOR_MAPPINGS.csv", index = False)
    all_lists_wide.to_csv(f"test_folder\\PREVIOUS_PREVALENCE_NON_REGISTER_INDICATORS.csv", index = False)
    next_years_non_reg_alt_ind_df.to_csv(f"test_folder\\PREVIOUS_PREVALENCE_NON_REGISTER_ALT_INDICATORS.csv", index = False)

elif test_run is False:
    ach_csv_df.to_csv(outputs+"\\Next years PREVIOUS_YEAR publication folder\PREVIOUS_ACHIEVEMENTS_PCAS.csv", index = False)
    prevalence_csv_df.to_csv(outputs+"\\Next years PREVIOUS_YEAR publication folder\PREVIOUS_PREVALENCE.csv", index = False)
    org_ref_csv_df.to_csv(outputs+"\\Next years PREVIOUS_YEAR publication folder\PREVIOUS_ORGANISATION_REFERENCE.csv", index = False)
    next_years_ind_control_csv_df.to_csv(outputs+"\\Next years PREVIOUS_YEAR publication folder\PREVIOUS_INDICATOR_CONTROL.csv", index = False)
    next_years_ind_map_csv_df.to_csv(outputs+"\\Next years PREVIOUS_YEAR publication folder\PREVIOUS_INDICATOR_MAPPINGS.csv", index = False)
    all_lists_wide.to_csv(outputs+"\\Next years PREVIOUS_YEAR publication folder\PREVIOUS_PREVALENCE_NON_REGISTER_INDICATORS.csv", index = False)
    next_years_non_reg_alt_ind_df.to_csv(outputs+"\\Next years PREVIOUS_YEAR publication folder\PREVIOUS_PREVALENCE_NON_REGISTER_ALT_INDICATORS.csv", index = False)

### 5. Create Excel table content
[Return to contents](#Table-of-Contents)

<h1><b>16 OUTPUTS - T3-Achievement</h1></b>

Sub set of qof master created for Achievement data


In [ ]:
"""   
The 'Overall domain achievement' tab in the prac-dom-ach excel workbook consists of the 
following elements at practice level mapped to PCN and Sub ICB

Overall achievement for the current year
Achievement by sub-domain for the current year
Overall achievement for the previous year
Achievement by sub-domain for the previous year

""" 
## Select all the columns from the 'Master table' that are used by Template 3 Achievement
ach_master_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME"
                  ,"PRACTICE_CODE","PRACTICE_NAME","SUB_DOMAIN_CODE","GROUP_CODE"
                  ,"INDICATOR_CODE","NO_REGISTER_FLAG","LIST_SIZE","ACHIEVED_POINTS"
                  ,"MAX_INDICATOR_POINTS","MAX_GROUP_POINTS","REGISTER_FLAG"]                                 
ach_master_df = qof_master[ach_master_col]

<h4>OVERALL_DOMAIN_ACHIEVEMENT<br>
Overall achievement for current year

In [ ]:
"""   
 Calculate adjusted achievement percentages for the current year
 Maximum_points_achievable is taken from the variable 'max_points'

"""
## Select required columns
overall_ach_col = ["PRACTICE_CODE", "ACHIEVED_POINTS"]
overall_ach_df = ach_master_df[overall_ach_col]

## Sum 'ACHIEVED_POINTS' by practice code
overall_ach_df = overall_ach_df.groupby(by = ["PRACTICE_CODE"]
                                        ,as_index=False)["ACHIEVED_POINTS"].sum()
                                        
## Calculate the achievement percentage
overall_ach_df["ACH_PC_TOTAL"] = (overall_ach_df["ACHIEVED_POINTS"]/int(max_points))*100

In [ ]:
## Calculate maximum points available, by summing indicator points and excluding those with 0 register values.
# overall_ach_adj_table_1 = ach_master_df
# overall_ach_adj_table_1 = overall_ach_adj_table_1

## Filter master achievement data frame for indicators with 'NO_REGISTER_FLAG' not being flagged
overall_ach_adj_fil_df = ach_master_df.loc[(ach_master_df["NO_REGISTER_FLAG"]!=1)]

## Select required columns
overall_ach_adj_col = ["PRACTICE_CODE", "MAX_INDICATOR_POINTS"]
overall_ach_adj_df = overall_ach_adj_fil_df[overall_ach_adj_col]

## Calculate total maximum indicator points by practice
overall_ach_adj_df = overall_ach_adj_df.groupby(by = ["PRACTICE_CODE"]
                                                ,as_index=False)["MAX_INDICATOR_POINTS"].sum()                                               

In [ ]:
"""   
Calculate adjusted achievement percentages used to join sub-domain data to overall achievement

""" 

overall_ach_max_points_df = pd.merge(overall_ach_df
                                  ,overall_ach_adj_df
                                  ,left_on=["PRACTICE_CODE"]
                                  ,right_on=["PRACTICE_CODE"]
                                  ,how="inner")

## Calculate adjusted achievement percentages and rename columns
overall_ach_max_points_df["ACHIEVEMENT_ADJUSTED_PERCENT"] = (overall_ach_max_points_df["ACHIEVED_POINTS"]/
                                                             overall_ach_max_points_df["MAX_INDICATOR_POINTS"])*100

overall_ach_max_points_df = overall_ach_max_points_df.rename(columns={"MAX_INDICATOR_POINTS":"MAX_QOF_POINTS_AVAILABLE"
                                                                     ,"ACHIEVED_POINTS":"ACHIEVEMENT_SCORE"
                                                                     ,"ACH_PC_TOTAL":"ACHIEVEMENT_PERCENT"})

<h4>OVERALL_ACHIEVEMENT<br>
Sub-domain achievement for current year

In [ ]:
## Select data by sub-domain and practice and sum 'ACHIEVEMENT_SCORE' and 'MAX_INDICATOR_POINTS' to get totals
subdom_ach_all_col = ["PRACTICE_CODE", "SUB_DOMAIN_CODE", "ACHIEVED_POINTS", "MAX_INDICATOR_POINTS"]
subdom_ach_all_df = df = ach_master_df[subdom_ach_all_col]

subdom_ach_all_df = subdom_ach_all_df.groupby(by = ["PRACTICE_CODE", "SUB_DOMAIN_CODE"]
                                              ,as_index=False
                                              ).agg({"ACHIEVED_POINTS" : [pd.Series.sum]
                                                     ,"MAX_INDICATOR_POINTS" : [pd.Series.sum]}
                                                     ).rename(columns={"ACHIEVED_POINTS":"ACHIEVEMENT_SCORE"})                                                                                                                                                             
subdom_ach_all_df.columns = subdom_ach_all_df.columns.droplevel(1)

## Calculate total percentage achievement by practice and sub domain
subdom_ach_all_df["ACH_PC_TOTAL"] = (subdom_ach_all_df["ACHIEVEMENT_SCORE"]/
                                     subdom_ach_all_df["MAX_INDICATOR_POINTS"])*100


In [ ]:
"""   
The aim of this code is to create a series of new columns that consist of the values in 'SUB_DOMAIN_CODE' and 'FIELD'
being concatenated with a hyphen e.g. CL-ACHIEVEMENT_SCORE

"""

## The short broad data frame is converted into a long thin data frame and the non-index columns 
## are stacked in a column called 'FIELD'
subdom_ach_all_stacked_df = (subdom_ach_all_df.set_index(["PRACTICE_CODE","SUB_DOMAIN_CODE"]
                                                        ).stack(
                                                        ).ffill(axis=0
                                                        ).bfill(axis=0, downcast='infer'
                                                        ).reset_index(
                                                        ).rename({"level_2": "FIELD", 0:"VALUE"}, axis=1))

## New column header is created by joining the 'SUB_DOMAIN_CODE' to 'FIELD' with a hyphen
subdom_ach_all_stacked_df["HEADER"] = subdom_ach_all_stacked_df["SUB_DOMAIN_CODE"]+"-"+subdom_ach_all_stacked_df["FIELD"]

## Filter 'FIELD' where it doesn't equal value in 'MAX_INDICATOR_POINTS'
subdom_ach_all_stacked_df = subdom_ach_all_stacked_df.loc[(subdom_ach_all_stacked_df["FIELD"]!= "MAX_INDICATOR_POINTS")]

## Pivot data frame from long thin data frame into short broad data frame
subdom_ach_all_piv_df = subdom_ach_all_stacked_df.pivot_table(index=["PRACTICE_CODE"]
                                                              ,columns="HEADER"
                                                              ,values="VALUE"
                                                              ,fill_value=0).reset_index().rename_axis(None, axis=1)                                                    

In [ ]:
## Select patient list sizes for each practice
pat_list_by_prac_col = ["PRACTICE_CODE","LIST_SIZE"]
pat_list_by_prac_df = ach_master_df[pat_list_by_prac_col]

pat_list_by_prac_df = pat_list_by_prac_df.drop_duplicates()

## Select NHS geographies for each practice
nhs_geog_by_prac_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE"
                        ,"PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"]
nhs_geog_by_prac_df = ach_master_df[nhs_geog_by_prac_col]

nhs_geog_by_prac_df = nhs_geog_by_prac_df.drop_duplicates().reset_index(drop=True)

## Join current geographies to list size - this is used widely
qof_cur_geog_list_df = pd.merge(nhs_geog_by_prac_df
                                ,pat_list_by_prac_df
                                ,left_on=["PRACTICE_CODE"]
                                ,right_on=["PRACTICE_CODE"]
                                ,how="left")

## Join list size to sub domain achievement (subdom_ach_all_piv_df) 
## and nhs geographies (nhs_geog_by_prac)
## Used by code blocks to calculate achievement data by sub domain
subdom_ach_all_geog_df = pd.merge(subdom_ach_all_piv_df
                                  ,qof_cur_geog_list_df
                                  ,left_on=["PRACTICE_CODE"]
                                  ,right_on=["PRACTICE_CODE"]
                                  ,how="left")

In [ ]:
## Join sub-domain data to overall achievement for current year
ach_subdom_all_current_df = pd.merge(overall_ach_max_points_df
                                     ,subdom_ach_all_geog_df
                                     ,left_on=["PRACTICE_CODE"]
                                     ,right_on=["PRACTICE_CODE"]
                                     ,how="left")

## Reorder columns for output
ach_subdom_all_current_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME"
                              ,"PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                              ,"LIST_SIZE","ACHIEVEMENT_SCORE","ACHIEVEMENT_PERCENT"
                              ,"MAX_QOF_POINTS_AVAILABLE","ACHIEVEMENT_ADJUSTED_PERCENT"
                              ,"CL-ACHIEVEMENT_SCORE","CL-ACH_PC_TOTAL"
                              ,"PH-ACHIEVEMENT_SCORE","PH-ACH_PC_TOTAL"
                              ,"PHAS-ACHIEVEMENT_SCORE","PHAS-ACH_PC_TOTAL"
                              ,"QI-ACHIEVEMENT_SCORE","QI-ACH_PC_TOTAL"
                              ,"PHVI-ACHIEVEMENT_SCORE","PHVI-ACH_PC_TOTAL"]                              
ach_subdom_all_current_df = ach_subdom_all_current_df[ach_subdom_all_current_col]

<h4>OVERALL_DOMAIN_ACHIEVEMENT<br>
Overall achievement for previous year

In [ ]:
"""   
Creates a data frame for overall achievement for the previous year and joins 
data frame for overall achievement by sub domain to it, to create output format 
in prac-dom-ach.xlsx on 'Overall domain achievemnt' tab

"""

## Create data frame ('overall_ach_previous_raw_df') of overall achieved points 
## and list size for previous year
prev_revised_max_points_col = ["PRACTICE_CODE","REVISED_MAX_POINTS"]
prev_revised_max_points_df = Previous_Organisation_Reference[prev_revised_max_points_col]

overall_ach_previous_col = ["PRACTICE_CODE","PREV_PRACTICE_LIST_SIZE","PREV_ACHIEVED_POINTS"
                            ,"PREV_TOTAL_INDICATOR_GROUP_POINTS"]
overall_ach_previous_df = previous_out_prac_df[overall_ach_previous_col]

overall_ach_previous_df = overall_ach_previous_df.groupby(["PRACTICE_CODE"]
                                                          ).agg({"PREV_PRACTICE_LIST_SIZE" : [pd.Series.max]
                                                                 ,"PREV_ACHIEVED_POINTS" : [pd.Series.sum]
                                                                 ,"PREV_TOTAL_INDICATOR_GROUP_POINTS" : [pd.Series.sum]
                                                                 }).reset_index()                                                                        
overall_ach_previous_df.columns = overall_ach_previous_df.columns.droplevel(1)

overall_ach_previous_df = pd.merge(overall_ach_previous_df
                                  ,prev_revised_max_points_df
                                  ,left_on=["PRACTICE_CODE"]
                                  ,right_on=["PRACTICE_CODE"]
                                  ,how="left")


## Calculate new column 'PREV_ACHIEVEMENT_PERCENT'
overall_ach_previous_df["PREV_ACHIEVEMENT_PERCENT"] = (overall_ach_previous_df["PREV_ACHIEVED_POINTS"]/
                                                       overall_ach_previous_df["PREV_TOTAL_INDICATOR_GROUP_POINTS"])*100

## Calculate new column 'PREV_ACHIEVEMENT_ADJUSTED_PERCENT' 
## and rename existing column 'REVISED_MAX_POINTS' as 'PREV_REVISED_MAXIMUM_POINTS'
overall_ach_previous_df["PREV_ACHIEVEMENT_ADJUSTED_PERCENT"] = (overall_ach_previous_df["PREV_ACHIEVED_POINTS"]/
                                                                overall_ach_previous_df["REVISED_MAX_POINTS"])*100

overall_ach_previous_df = overall_ach_previous_df.rename(columns={"REVISED_MAX_POINTS":"PREV_REVISED_MAXIMUM_POINTS"})

<h4>OVERALL_ACHIEVEMENT<br>
Sub-domain achievement for previous year

In [ ]:
## Select previous years data by sub-domain - previous achieved points and previous total indicator group points
prev_subdom_ach_ind_points_col = ["PRACTICE_CODE","DOMAIN_CODE","PREV_TOTAL_INDICATOR_GROUP_POINTS"
                                 ,"PREV_ACHIEVED_POINTS"]
prev_subdom_ach_ind_points_df = previous_out_prac_df[prev_subdom_ach_ind_points_col]


prev_subdom_ach_points_df = prev_subdom_ach_ind_points_df.groupby(by = ["PRACTICE_CODE","DOMAIN_CODE"]
                                                                  ,as_index=False
                                                                  ).agg({"PREV_TOTAL_INDICATOR_GROUP_POINTS" : [pd.Series.sum]
                                                                  ,"PREV_ACHIEVED_POINTS" : [pd.Series.sum]})                                                                                                                                                             
prev_subdom_ach_points_df.columns = prev_subdom_ach_points_df.columns.droplevel(1)

## Calculate new column 'PREV_ACHIEVEMENT_RATE'
prev_subdom_ach_points_df["PREV_ACHIEVEMENT_RATE"] = (prev_subdom_ach_points_df["PREV_ACHIEVED_POINTS"]/
                                                      prev_subdom_ach_points_df["PREV_TOTAL_INDICATOR_GROUP_POINTS"])*100

## Drop unwanted columns
prev_subdom_ach_points_df = prev_subdom_ach_points_df.drop(columns=["PREV_TOTAL_INDICATOR_GROUP_POINTS"])

In [ ]:
"""   
The aim of this code is to create a series of new columns that consist of the
values in 'DOMAIN_CODE' and 'VARIABLE' being concatenated with a hyphen
e.g. CL-PREV_ACHIEVED_POINTS

"""

## The short broad data frame is converted into a long thin data frame 
## and the non-index columns are stacked in a column called 'VARIABLE'
prev_subdom_ach_all_stacked_df = prev_subdom_ach_points_df.melt(id_vars=["PRACTICE_CODE","DOMAIN_CODE"]
                                                                ,var_name="VARIABLE"
                                                                ,value_name="VALUE")

## New column 'DOMAIN_VARIABLE'' is created by joining the 'DOMAIN_CODE' 
## to 'VARIABLE' with a hyphen
prev_subdom_ach_all_stacked_df["DOMAIN_VARIABLE"] = (prev_subdom_ach_all_stacked_df["DOMAIN_CODE"]
                                                     )+"-"+(prev_subdom_ach_all_stacked_df["VARIABLE"])

## Drop unwanted columns
prev_subdom_ach_all_stacked_df = prev_subdom_ach_all_stacked_df.drop(columns=["DOMAIN_CODE","VARIABLE"])

## Pivot from long thin data frame to short broad data frame where the new columns 
## consist of the newly created 'DOMAIN_VARIABLE' column
prev_subdom_ach_all_piv_df = prev_subdom_ach_all_stacked_df.pivot_table(index=["PRACTICE_CODE"],columns="DOMAIN_VARIABLE"
                                                                                ,values="VALUE"
                                                                                ,fill_value=0).reset_index().rename_axis(None, axis=1)                                                            

In [ ]:
"""   
Combines all the data frames for the current and previous years for achievement by practice
to format output for prac-dom-ach excel workbook

"""

## Combine Overall Achievement data frames for previous year with sub-domain data for the same year
overall_ach_previous_df = pd.merge(overall_ach_previous_df
                                   ,prev_subdom_ach_all_piv_df
                                   ,left_on=["PRACTICE_CODE"]
                                   ,right_on=["PRACTICE_CODE"]
                                   ,how="left")

overall_ach_previous_df = overall_ach_previous_df.drop(columns=["PREV_TOTAL_INDICATOR_GROUP_POINTS"])

## Combine Overall Achievement data frames for current and previous years
overall_ach_current_previous_df = pd.merge(ach_subdom_all_current_df
                                           ,overall_ach_previous_df
                                           ,left_on=["PRACTICE_CODE"]
                                           ,right_on=["PRACTICE_CODE"]
                                           ,how="left")


overall_ach_current_previous_df["ACH_CHANGE"] = (overall_ach_current_previous_df["ACHIEVEMENT_ADJUSTED_PERCENT"]
                                                 ) - (overall_ach_current_previous_df["PREV_ACHIEVEMENT_ADJUSTED_PERCENT"])

<h3>OUTPUT - Overall Domain Achievement worksheet for Achievement workbook

In [ ]:
## Reorder and sort data frame columns for excel output
overall_ach_current_previous_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME"
                                    ,"PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                                    ,"PREV_PRACTICE_LIST_SIZE","LIST_SIZE"
                                    ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_PERCENT","PREV_REVISED_MAXIMUM_POINTS"
                                    ,"PREV_ACHIEVEMENT_ADJUSTED_PERCENT","ACHIEVEMENT_SCORE","ACHIEVEMENT_PERCENT"
                                    ,"MAX_QOF_POINTS_AVAILABLE","ACHIEVEMENT_ADJUSTED_PERCENT","ACH_CHANGE"                          
                                    ,"CL-PREV_ACHIEVED_POINTS","CL-PREV_ACHIEVEMENT_RATE","CL-ACHIEVEMENT_SCORE"
                                    ,"CL-ACH_PC_TOTAL","PH-PREV_ACHIEVED_POINTS","PH-PREV_ACHIEVEMENT_RATE"
                                    ,"PH-ACHIEVEMENT_SCORE","PH-ACH_PC_TOTAL","PHAS-PREV_ACHIEVED_POINTS"
                                    ,"PHAS-PREV_ACHIEVEMENT_RATE","PHAS-ACHIEVEMENT_SCORE","PHAS-ACH_PC_TOTAL"
                                    ,"PHVI-PREV_ACHIEVED_POINTS","PHVI-PREV_ACHIEVEMENT_RATE","PHVI-ACHIEVEMENT_SCORE"
                                    ,"PHVI-ACH_PC_TOTAL","QI-ACHIEVEMENT_SCORE","QI-ACH_PC_TOTAL"]
ach_all_overall_df = overall_ach_current_previous_df[overall_ach_current_previous_col]

ach_all_overall_df = ach_all_overall_df.sort_values(by=["CCG_ODS_CODE","PRACTICE_NAME"]
                                                    ,ascending = [True,True]).reset_index(drop = True)                                              

<b><h2>OVERALL_ACHIEVEMENT - Group data frame</h2></b>

In [ ]:
## Create practice base data frame from QOF Master from which data will be 
# filtered and processed by 'SUB_DOMAIN_CODE'
ach_gr_current_base_col = ["PRACTICE_CODE","GROUP_CODE","SUB_DOMAIN_CODE"
                           ,"ACHIEVED_POINTS"]
ach_gr_current_base_df = qof_master[ach_gr_current_base_col]

ach_gr_current_base_df = ach_gr_current_base_df.groupby(by = ["PRACTICE_CODE","GROUP_CODE","SUB_DOMAIN_CODE"]
                                                        ,as_index=False).agg({"ACHIEVED_POINTS" : [pd.Series.sum]})  
                                                                                                                                                                                                                               
ach_gr_current_base_df.columns = ach_gr_current_base_df.columns.droplevel(1)

<h4>OUTPUT - Clinical Domain (CL) Achievement worksheet for Achievement workbook

In [ ]:
## Indicator groups in the Clinical domain
ach_gr_current_cl_df = ach_gr_current_base_df.loc[(ach_gr_current_base_df["SUB_DOMAIN_CODE"]== "CL")]
ach_gr_current_cl_df = ach_gr_current_cl_df.drop(columns=["SUB_DOMAIN_CODE"])

ach_gr_current_cl_piv_df = ach_gr_current_cl_df.pivot_table(index=["PRACTICE_CODE"]
                                                                  ,columns="GROUP_CODE"
                                                                  ,values="ACHIEVED_POINTS"
                                                                  ,fill_value=0 ).reset_index(                                                                      
                                                                                ).rename_axis(None, axis=1)
                                                        


## Total Achievement Score and Percentage by Clinical domain
subdom_ach_current_cl_df = subdom_ach_all_stacked_df.loc[(subdom_ach_all_stacked_df["SUB_DOMAIN_CODE"]== "CL")]
subdom_ach_current_cl_df = subdom_ach_current_cl_df.drop(columns=["HEADER","SUB_DOMAIN_CODE"])

subdom_ach_current_cl_piv_df = subdom_ach_current_cl_df.pivot_table(index=["PRACTICE_CODE"]
                                                                    ,columns="FIELD"
                                                                    ,values="VALUE"
                                                                    ,fill_value=0).reset_index(                                                                        
                                                                                 ).rename_axis(None, axis=1)

In [ ]:
## Join indicator groups to overall achievement
ach_ind_gr_cl_df = pd.merge(ach_gr_current_cl_piv_df
                            ,subdom_ach_current_cl_piv_df
                            ,left_on=["PRACTICE_CODE"]
                            ,right_on=["PRACTICE_CODE"]
                            ,how="left")



## Add geographies and list size to indicator groups to overall achievement
ach_ind_gr_geog_cl_df = pd.merge(qof_cur_geog_list_df
                                 ,ach_ind_gr_cl_df
                                 ,left_on=["PRACTICE_CODE"]
                                 ,right_on=["PRACTICE_CODE"]
                                 ,how="left")

ach_all_cl_df = ach_ind_gr_geog_cl_df.sort_values(by = ["CCG_ODS_CODE","PRACTICE_NAME"]
                                                  ,ascending = [True,True]).reset_index(drop = True)                                                                                                                                          

<h4>OUTPUT - Public Health Domain (PH) Achievement worksheet for Achievement workbook

In [ ]:
## Indicator groups in the Public Health domain
ach_gr_current_ph_df = ach_gr_current_base_df.loc[(ach_gr_current_base_df["SUB_DOMAIN_CODE"]== "PH")]
ach_gr_current_ph_df = ach_gr_current_ph_df.drop(columns=["SUB_DOMAIN_CODE"])

ach_gr_current_ph_piv_df = ach_gr_current_ph_df.pivot_table(index=["PRACTICE_CODE"]
                                                            ,columns="GROUP_CODE"
                                                            ,values="ACHIEVED_POINTS"
                                                            ,fill_value=0).reset_index(                                                                
                                                                         ).rename_axis(None, axis=1)

## Total Achievement Score and Percentage by Public Health domain
subdom_ach_current_ph_df = subdom_ach_all_stacked_df.loc[(subdom_ach_all_stacked_df["SUB_DOMAIN_CODE"]== "PH")]
subdom_ach_current_ph_df = subdom_ach_current_ph_df.drop(columns=["HEADER","SUB_DOMAIN_CODE"])

subdom_ach_current_ph_df = subdom_ach_current_ph_df.pivot_table(index=["PRACTICE_CODE"]
                                                                ,columns="FIELD"
                                                                ,values="VALUE"
                                                                ,fill_value=0).reset_index(                                                                    
                                                                             ).rename_axis(None, axis=1)

In [ ]:
## Join indicator groups to overall achievement
ach_ind_gr_ph_df = pd.merge(ach_gr_current_ph_piv_df
                            ,subdom_ach_current_ph_df
                            ,left_on=["PRACTICE_CODE"]
                            ,right_on=["PRACTICE_CODE"]
                            ,how="left")

## Add geographies and list size to indicator groups to overall achievement
ach_all_ph_df = pd.merge(qof_cur_geog_list_df
                      ,ach_ind_gr_ph_df
                      ,left_on=["PRACTICE_CODE"]
                      ,right_on=["PRACTICE_CODE"]
                      ,how="left")


ach_all_ph_df = ach_all_ph_df.sort_values(by = ["CCG_ODS_CODE","PRACTICE_NAME"]
                                          ,ascending = [True,True]).reset_index(drop = True)
                                                                            

<h4>OUTPUT - Public Health Additional Service Domain (PHAS) Achievement worksheet for Achievement workbook

In [ ]:
## Indicator groups in the Public Health Additional Services domain
ach_gr_current_phas_df = ach_gr_current_base_df.loc[(ach_gr_current_base_df["SUB_DOMAIN_CODE"]== "PHAS")]
ach_gr_current_phas_df = ach_gr_current_phas_df.drop(columns=["SUB_DOMAIN_CODE"])

ach_gr_current_phas_piv_df = ach_gr_current_phas_df.pivot_table(index=["PRACTICE_CODE"]
                                                                ,columns="GROUP_CODE"
                                                                ,values="ACHIEVED_POINTS"
                                                                ,fill_value=0).reset_index(                                                                    
                                                                             ).rename_axis(None, axis=1)

## Total Achievement Score and Percentage by Public Health Additional Services domain
subdom_ach_current_phas_df = subdom_ach_all_stacked_df.loc[(subdom_ach_all_stacked_df["SUB_DOMAIN_CODE"]== "PHAS")]
subdom_ach_current_phas_df = subdom_ach_current_phas_df.drop(columns=["HEADER","SUB_DOMAIN_CODE"])

subdom_ach_current_phas_df = subdom_ach_current_phas_df.pivot_table(index=["PRACTICE_CODE"]
                                                                  ,columns="FIELD"
                                                                  ,values="VALUE"
                                                                  ,fill_value=0).reset_index(                                                                      
                                                                               ).rename_axis(None, axis=1)

In [ ]:
## Join Indicator groups to overall achievement
ach_ind_gr_phas_df = pd.merge(ach_gr_current_phas_piv_df
                              ,subdom_ach_current_phas_df
                              ,left_on=["PRACTICE_CODE"]
                              ,right_on=["PRACTICE_CODE"]
                              ,how="left")

## Add geographies and list size to indicator groups to overall achievement
ach_all_phas_df = pd.merge(qof_cur_geog_list_df
                           ,ach_ind_gr_phas_df
                           ,left_on=["PRACTICE_CODE"]
                           ,right_on=["PRACTICE_CODE"]
                           ,how="left")


ach_all_phas_df = ach_all_phas_df.sort_values(by = ["CCG_ODS_CODE","PRACTICE_NAME"]
                                              ,ascending = [True,True]).reset_index(drop = True)
                                                                            

<h4>OUTPUT - Quality Improvement Domain (QI) Achievement worksheet for Achievement workbook

In [ ]:
## Indicator groups in the Quality Improvement domain
ach_gr_current_qi_df = ach_gr_current_base_df.loc[(ach_gr_current_base_df["SUB_DOMAIN_CODE"]== "QI")]
ach_gr_current_qi_df = ach_gr_current_qi_df.drop(columns=["SUB_DOMAIN_CODE"])

ach_gr_current_qi_piv_df = ach_gr_current_qi_df.pivot_table(index=["PRACTICE_CODE"]
                                                            ,columns="GROUP_CODE"
                                                            ,values="ACHIEVED_POINTS"
                                                            ,fill_value=0).reset_index(                                                                
                                                                         ).rename_axis(None, axis=1)

## Total Achievement Score and Percentage by Quality Improvement domain
subdom_ach_current_qi_df = subdom_ach_all_stacked_df.loc[(subdom_ach_all_stacked_df["SUB_DOMAIN_CODE"]== "QI")]
subdom_ach_current_qi_df = subdom_ach_current_qi_df.drop(columns=["HEADER","SUB_DOMAIN_CODE"])

subdom_ach_current_qi_df = subdom_ach_current_qi_df.pivot_table(index=["PRACTICE_CODE"]
                                                                ,columns="FIELD"
                                                                ,values="VALUE"
                                                                ,fill_value=0).reset_index(                                                                    
                                                                             ).rename_axis(None, axis=1)

In [ ]:
## Join Indicator groups to overall achievement
ach_ind_gr_qi_df = pd.merge(ach_gr_current_qi_piv_df
                              ,subdom_ach_current_qi_df
                              ,left_on=["PRACTICE_CODE"]
                              ,right_on=["PRACTICE_CODE"]
                              ,how="left")

## Add geographies and list size to indicator groups to overall achievement
ach_all_qi_df = pd.merge(qof_cur_geog_list_df
                         ,ach_ind_gr_qi_df
                         ,left_on=["PRACTICE_CODE"]
                         ,right_on=["PRACTICE_CODE"]
                         ,how="left")

ach_all_qi_df = ach_all_qi_df.sort_values(by = ["CCG_ODS_CODE","PRACTICE_NAME"]
                                          ,ascending = [True,True]).reset_index(drop = True)
                                                                             

<h4> OUTPUT - Public Health Domain Vaccination and Immunisation (PHVI) Achievement worksheet for Achievement workbook

In [ ]:
## Indicator groups in the Public Health Domain Vaccination and Immunisation
ach_gr_current_vi_df = ach_gr_current_base_df.loc[(ach_gr_current_base_df["SUB_DOMAIN_CODE"]== "PHVI")]
ach_gr_current_vi_df = ach_gr_current_vi_df.drop(columns=["SUB_DOMAIN_CODE"])

ach_gr_current_vi_piv_df = ach_gr_current_vi_df.pivot_table(index=["PRACTICE_CODE"]
                                                            ,columns="GROUP_CODE"
                                                            ,values="ACHIEVED_POINTS"
                                                            ,fill_value=0).reset_index(                                                                
                                                                         ).rename_axis(None, axis=1)


## Total Achievement Score and Percentage by Public Health Domain Vaccination and Immunisation
subdom_ach_current_vi_df = subdom_ach_all_stacked_df.loc[(subdom_ach_all_stacked_df["SUB_DOMAIN_CODE"]== "PHVI")]
subdom_ach_current_vi_df = subdom_ach_current_vi_df.drop(columns=["HEADER","SUB_DOMAIN_CODE"])

subdom_ach_current_vi_df = subdom_ach_current_vi_df.pivot_table(index=["PRACTICE_CODE"]
                                                                ,columns="FIELD"
                                                                ,values="VALUE"
                                                                ,fill_value=0).reset_index(                                                                    
                                                                             ).rename_axis(None, axis=1)

In [ ]:
## Join Indicator groups to overall achievement
ach_ind_gr_qi_df = pd.merge(ach_gr_current_vi_piv_df
                            ,subdom_ach_current_vi_df
                            ,left_on=["PRACTICE_CODE"]
                            ,right_on=["PRACTICE_CODE"]
                            ,how="left")


## Add geographies and list size to indicator groups to overall achievement
ach_all_vi_df = pd.merge(qof_cur_geog_list_df
                         ,ach_ind_gr_qi_df
                         ,left_on=["PRACTICE_CODE"]
                         ,right_on=["PRACTICE_CODE"]                         
                         ,how="left")

ach_all_vi_df = ach_all_vi_df.sort_values(by = ["CCG_ODS_CODE","PRACTICE_NAME"]
                                          ,ascending = [True,True]).reset_index(drop = True)
                                                                             

### Create T3 Achievement Table titles


In [ ]:
## Reduced the number of Group Reference table columns - used by Region and National, 
## STP, CCG and Practice tables
ach_dom_col = ["SUB_DOMAIN_CODE","SUB_DOMAIN_DESCRIPTION"]
ach_dom_df= Indicator_Reference_Control_Table[ach_dom_col]

ach_dom_df = ach_dom_df.drop_duplicates().sort_values(by = ["SUB_DOMAIN_CODE"]
                                                      ,ascending = [True]).reset_index(drop = True)                                                                                

### Create the Table Number - used by Achievement and PCA tables

In [ ]:
## Reset index to create another column of numbers in ascending order
ach_dom_table_number_df = ach_dom_df.reset_index()

## Rename the duplicate index column as 'TABLE_NO'
ach_dom_table_number_df = ach_dom_table_number_df.rename(columns={"index":"TABLE_NO"})

## Recalculate 'TABLE_NO' to remove the zero value and 1 values, as table 1 is for overall domain
ach_dom_table_number_df["TABLE_NO"] = ach_dom_table_number_df["TABLE_NO"]+2

## Make 'SUB_DOMAIN_DESCRIPTION' column all lower case and add the word domain as this forms part of the table title
ach_dom_table_number_df["SUB_DOMAIN_DESCRIPTION"] = ach_dom_table_number_df["SUB_DOMAIN_DESCRIPTION"].str.lower()+" domain"

## Create number and part of the title for table 1
ach_dom_table_number_df.loc[len(ach_dom_table_number_df.index)] = [1,'OV', "overall"] 

## Sort values and drop unwanted column
ach_dom_table_number_df = ach_dom_table_number_df.sort_values(by = ["TABLE_NO"]
                                                              ,ascending = [True]).reset_index(drop = True)

ach_dom_table_number_df = ach_dom_table_number_df.drop(columns=["SUB_DOMAIN_CODE"])

In [ ]:
## Combine elements that contribute to achievment table title
ach_table_titles_df = ach_dom_table_number_df
ach_table_titles_df["TABLE"] = ach_table_titles_df["TABLE_NO"].astype(str)
ach_table_titles_df["FYEAR"] = Control_File.at[0, "FYEAR"]

## When table number = 1 then the table number will be included in the title text
ach_table_titles_df.loc[ach_table_titles_df["TABLE_NO"] == 1
                        ,"TABLE_TITLE"] = "Table 1: Achievement, "+\
                            ach_table_titles_df["SUB_DOMAIN_DESCRIPTION"]+\
                                " and by domain, "+ach_table_titles_df["FYEAR"]+", GP practice level"
                            
                                                                               

## When table number is not 1 then the  table number will be taken from the data frame
ach_table_titles_df.loc[ach_table_titles_df["TABLE_NO"] != 1
                        ,"TABLE_TITLE"] = "Table "+ach_table_titles_df["TABLE"]+\
                            ": Achievement by indicator group , "+ach_table_titles_df["SUB_DOMAIN_DESCRIPTION"]+\
                                ", "+ach_table_titles_df["FYEAR"]+", GP practice level"
                                                                                                                                                          
## Drop unwanted columns
ach_table_titles_df = ach_table_titles_df.drop(columns=["SUB_DOMAIN_DESCRIPTION","FYEAR","TABLE"])

<b><h1>17 OUTPUTS - T4-PCAs</h1></b><br><h2>Overall domain PCAs

In [ ]:
"""   
The 'Overall domain PCA' tab in the prac-dom-pca excel workbook consists of the 
following elements at practice level mapped to PCN and Sub ICB

Overall PCAs for the current year
PCAs by sub-domain for the current year
Overall PCAs for the previous year
PCAs by sub-domain for the previous year

""" 

## Calculate overall PCA rate for the cuuent year
ov_pca_rate_col = ["PRACTICE_CODE","PCA_COUNT","PCA_DENOMINATOR"]
ov_pca_rate_df = qof_master[ov_pca_rate_col]

ov_pca_rate_df = ov_pca_rate_df.groupby(by = ["PRACTICE_CODE"],as_index=False
                                        ).agg({"PCA_COUNT" : [pd.Series.sum]
                                               ,"PCA_DENOMINATOR" : [pd.Series.sum]
                                               })                                                                                                                                         
ov_pca_rate_df.columns = ov_pca_rate_df.columns.droplevel(1)

ov_pca_rate_df = ov_pca_rate_df.rename(columns={"PCA_COUNT":"PCAS"
                                                ,"PCA_DENOMINATOR":"DENOMINATORS"})

ov_pca_rate_df["PCA_RATE"] = ov_pca_rate_df["PCAS"]/(ov_pca_rate_df["DENOMINATORS"]+
                                                     ov_pca_rate_df["PCAS"])*100

In [ ]:
## Calculate sub-domain PCA rate for the current year
dom_pca_rate_col = ["PRACTICE_CODE","SUB_DOMAIN_CODE","PCA_COUNT","PCA_DENOMINATOR"]

dom_pca_rate_df = qof_master[dom_pca_rate_col]
dom_pca_rate_df = dom_pca_rate_df.groupby(["PRACTICE_CODE","SUB_DOMAIN_CODE"]
                                          ).agg({"PCA_COUNT" : [pd.Series.sum]
                                                 ,"PCA_DENOMINATOR" : [pd.Series.sum]
                                                 }).reset_index()                                                                                
dom_pca_rate_df.columns = dom_pca_rate_df.columns.droplevel(1)

dom_pca_rate_df = dom_pca_rate_df.rename(columns={"PCA_COUNT":"PCAS"
                                                  ,"PCA_DENOMINATOR":"DENOMINATORS"})

dom_pca_rate_df["PCA_RATE"] = dom_pca_rate_df["PCAS"]/(dom_pca_rate_df["DENOMINATORS"]+
                                                       dom_pca_rate_df["PCAS"])*100

## Replace 'nan' values with a zero
dom_pca_rate_df["PCA_RATE"] = dom_pca_rate_df["PCA_RATE"].replace(np.nan, 0)

In [ ]:
"""   
The aim of this code is to create a series of new columns that consist of the values in 'SUB_DOMAIN_CODE' and 'VARIABLE'
being concatenated with a hyphen e.g. CL-PCAS

"""

## The short broad data frame is converted into a long thin data frame and the 
## non-index columns are stacked in a column called 'VARIABLE'
dom_pca_rate_stacked_df = dom_pca_rate_df.melt(id_vars=["PRACTICE_CODE","SUB_DOMAIN_CODE"]
                                              ,var_name="VARIABLE"
                                              ,value_name="VALUE")

## New column 'DOMAIN_VARIABLE'' is created by joining the 'DOMAIN_CODE' to 
## 'VARIABLE' with a hyphen
dom_pca_rate_stacked_df["DOMAIN_VARIABLE"] = (dom_pca_rate_stacked_df["SUB_DOMAIN_CODE"]
                                              )+"-"+(dom_pca_rate_stacked_df["VARIABLE"])

## Drop unwanted columns
dom_pca_rate_stacked_df = dom_pca_rate_stacked_df.drop(columns=["SUB_DOMAIN_CODE","VARIABLE"])

## Pivot from long thin data frame to short broad data frame where the new 
## columns consist of the newly created 'DOMAIN_VARIABLE' column
dom_pca_rate_piv_df = dom_pca_rate_stacked_df.pivot_table(index=["PRACTICE_CODE"]
                                                          ,columns="DOMAIN_VARIABLE"
                                                          ,values="VALUE"
                                                          ,fill_value=0).reset_index().rename_axis(None, axis=1)

In [ ]:
"""   
Combines all the data frames for the current and previous years for achievement by practice
to format output for prac-dom-pca excel workbook

"""

## Combine Overall PCA data frames for previous year with sub-domain data for the same year
overall_pca_previous_df = pd.merge(qof_cur_geog_list_df
                                    ,ov_pca_rate_df
                                    ,left_on=["PRACTICE_CODE"]
                                    ,right_on=["PRACTICE_CODE"]
                                    ,how="left")

## Combine Overall PCA data frames for current and previous years
overall_pca_currrent_previous_df = pd.merge(overall_pca_previous_df
                                            ,dom_pca_rate_piv_df
                                            ,left_on=["PRACTICE_CODE"]
                                            ,right_on=["PRACTICE_CODE"]
                                            ,how="left")

<h2>OVERALL PCAs - Previous years data frame

In [ ]:
"""   
Creates a data frame for overall PCA for the previous year and joins data frame 
for overall PCA by sub domain to it, to create output format for prac-dom-pca.xlsx
on 'Overall domain achievement' tab

"""

## Calculate previous years overall PCA rate
previous_ov_pca_rate_col = ["PRACTICE_CODE","PREV_PRACTICE_LIST_SIZE","PREV_PCA_COUNT"
                            ,"PREV_DENOMINATOR"]
previous_overall_pca_rate_df = previous_out_prac_df[previous_ov_pca_rate_col]

previous_overall_pca_rate_df = previous_overall_pca_rate_df.groupby(by = ["PRACTICE_CODE"]
                                                                    ,as_index=False
                                                                    ).agg({"PREV_PRACTICE_LIST_SIZE" : [pd.Series.max]
                                                                           ,"PREV_PCA_COUNT" : [pd.Series.sum]
                                                                           ,"PREV_DENOMINATOR" : [pd.Series.sum]
                                                                           })                                                                                                                            
previous_overall_pca_rate_df.columns = previous_overall_pca_rate_df.columns.droplevel(1)

previous_overall_pca_rate_df = previous_overall_pca_rate_df.rename(columns={"PREV_PCA_COUNT":"PREV_PCAS"                                                                          
                                                                           ,"PREV_DENOMINATOR":"PREV_DENOMINATORS"})

## Calculate new data frame column 'PREV_PCA_RATE'
previous_overall_pca_rate_df["PREV_PCA_RATE"] = (previous_overall_pca_rate_df["PREV_PCAS"]
                                                 )/(previous_overall_pca_rate_df["PREV_DENOMINATORS"]+previous_overall_pca_rate_df["PREV_PCAS"])*100

                                                

In [ ]:
## Calculate sub-domain PCA rate
previous_dom_pca_rate_col = ["PRACTICE_CODE","DOMAIN_CODE","PREV_PCA_COUNT","PREV_DENOMINATOR"]
previous_dom_pca_rate = previous_out_prac_df[previous_dom_pca_rate_col]

previous_out_prac_df = previous_out_prac_df.groupby(by = ["PRACTICE_CODE","DOMAIN_CODE"]
                                                    ,as_index=False
                                                    ).agg({"PREV_PCA_COUNT" : [pd.Series.sum]
                                                           ,"PREV_DENOMINATOR" : [pd.Series.sum]
                                                           })                                                           
previous_out_prac_df.columns = previous_out_prac_df.columns.droplevel(1)

previous_out_prac_df = previous_out_prac_df.rename(columns={"PREV_PCA_COUNT":"PREV_PCAS"
                                                            ,"PREV_DENOMINATOR":"PREV_DENOMINATORS"})

## Calculate new data frame column 'PREV_PCA_RATE
previous_out_prac_df["PREV_PCA_RATE"] = (previous_out_prac_df["PREV_PCAS"]
                                         )/(previous_out_prac_df["PREV_DENOMINATORS"]+previous_out_prac_df["PREV_PCAS"])*100

## Replace 'nan' values with a zero
previous_out_prac_df["PREV_PCA_RATE"] = previous_out_prac_df["PREV_PCA_RATE"].replace(np.nan, 0)

In [ ]:
"""   
The aim of this code is to create a series of new columns that consist of the values in 'SUB_DOMAIN_CODE' and 'VARIABLE'
being concatenated with a hyphen e.g. CL-PREV_PCAS

"""

## The short broad data frame is converted into a long thin data frame and the non-index columns are 
## stacked in a column called 'VARIABLE'
previous_dom_pca_rate_stacked_df = previous_out_prac_df.melt(id_vars=["PRACTICE_CODE","DOMAIN_CODE"]
                                                            , var_name="VARIABLE"
                                                            , value_name="VALUE")

## The short broad data frame is converted into a long thin data frame and the non-index columns 
## are stacked in a column called 'VARIABLE'
previous_dom_pca_rate_stacked_df["DOMAIN_VARIABLE"] = (previous_dom_pca_rate_stacked_df["DOMAIN_CODE"]
                                                       )+"-"+(previous_dom_pca_rate_stacked_df["VARIABLE"])

## Drop unwanted columns
previous_dom_pca_rate_stacked_df = previous_dom_pca_rate_stacked_df.drop(columns=["DOMAIN_CODE","VARIABLE"])

## Pivot from long thin data frame to short broad data frame where the new columns consist of the newly 
## created 'DOMAIN_VARIABLE' column
previous_dom_pca_rate_piv_df = previous_dom_pca_rate_stacked_df.pivot_table(index=["PRACTICE_CODE"]
                                                                            ,columns="DOMAIN_VARIABLE"
                                                                            ,values="VALUE"
                                                                            ,fill_value=0).reset_index().rename_axis(None, axis=1)

In [ ]:
"""   
Combines all the data frames for the current and previous years for achievement by practice
to format output for prac-dom-pca excel workbook

"""

## Combine Overall PCA data frames for previous year with sub-domain data for the same year
previous_overall_dom_pca_df = pd.merge(previous_overall_pca_rate_df
                                       ,previous_dom_pca_rate_piv_df
                                       ,left_on=["PRACTICE_CODE"]
                                       ,right_on=["PRACTICE_CODE"]
                                       ,how="left")

## Join current PCA data frame to previous PCA data frame
overall_pca_df = pd.merge(overall_pca_currrent_previous_df
                          ,previous_overall_dom_pca_df
                          ,left_on=["PRACTICE_CODE"]
                          ,right_on=["PRACTICE_CODE"]
                          ,how="left")

In [ ]:

## For all and each domain where the denominator or PCA value equals zero replace the value with 'nan'
overall_pca_df["DENOMINATORS"] = np.where(overall_pca_df["DENOMINATORS"]== 0
                                          ,np.nan
                                          ,overall_pca_df["DENOMINATORS"])

overall_pca_df["PCAS"] = np.where(overall_pca_df["DENOMINATORS"].isnull()
                                  ,np.nan
                                  ,overall_pca_df["PCAS"])

overall_pca_df["CL-DENOMINATORS"] = np.where(overall_pca_df["CL-DENOMINATORS"]== 0
                                             ,np.nan
                                             ,overall_pca_df["CL-DENOMINATORS"])

overall_pca_df["CL-PCAS"] = np.where(overall_pca_df["CL-DENOMINATORS"].isnull()
                                     ,np.nan
                                     , overall_pca_df["CL-PCAS"])

overall_pca_df["PH-DENOMINATORS"] = np.where(overall_pca_df["PH-DENOMINATORS"]== 0
                                             ,np.nan
                                             ,overall_pca_df["PH-DENOMINATORS"])

overall_pca_df["PH-PCAS"] = np.where(overall_pca_df["PH-DENOMINATORS"].isnull()
                                     ,np.nan
                                     ,overall_pca_df["PH-PCAS"])

overall_pca_df["PHAS-DENOMINATORS"] = np.where(overall_pca_df["PHAS-DENOMINATORS"]== 0
                                               ,np.nan
                                               ,overall_pca_df["PHAS-DENOMINATORS"])

overall_pca_df["PHAS-PCAS"] = np.where(overall_pca_df["PHAS-DENOMINATORS"].isnull()
                                       ,np.nan
                                       ,overall_pca_df["PHAS-PCAS"])

overall_pca_df["PHVI-DENOMINATORS"] = np.where(overall_pca_df["PHVI-DENOMINATORS"]== 0
                                               ,np.nan
                                               ,overall_pca_df["PHVI-DENOMINATORS"])

overall_pca_df["PHVI-PCAS"] = np.where(overall_pca_df["PHVI-DENOMINATORS"].isnull()
                                       ,np.nan
                                       ,overall_pca_df["PHVI-PCAS"])

overall_pca_df["QI-DENOMINATORS"] = np.where(overall_pca_df["QI-DENOMINATORS"]== 0
                                             ,np.nan
                                             ,overall_pca_df["QI-DENOMINATORS"])

overall_pca_df["QI-PCAS"] = np.where(overall_pca_df["QI-DENOMINATORS"].isnull()
                                     ,np.nan
                                     ,overall_pca_df["QI-PCAS"])

In [ ]:
## Adds PCA change column to the data frame
## This provides the difference in percentage points between the current years PCA rate and 
## the previous years PCA rate for all domains overall and individual domains by practice

overall_pca_df["ALL_PCA_CHANGE"] = overall_pca_df["PCA_RATE"] - overall_pca_df["PREV_PCA_RATE"]
overall_pca_df["CL_PCA_CHANGE"] = overall_pca_df["CL-PCA_RATE"] - overall_pca_df["CL-PREV_PCA_RATE"]
overall_pca_df["PH_PCA_CHANGE"] = overall_pca_df["PH-PCA_RATE"] - overall_pca_df["PH-PREV_PCA_RATE"]
overall_pca_df["PHAS_PCA_CHANGE"] = overall_pca_df["PHAS-PCA_RATE"] - overall_pca_df["PHAS-PREV_PCA_RATE"]  
overall_pca_df["PHVI_PCA_CHANGE"] = overall_pca_df["PHVI-PCA_RATE"] - overall_pca_df["PHVI-PREV_PCA_RATE"] 
overall_pca_df["QI_PCA_CHANGE"] = overall_pca_df["QI-PCA_RATE"] - overall_pca_df["QI-PREV_PCA_RATE"]

<h3>OUTPUT - Overall Domain PCAs worksheet for PCA workbook

In [ ]:
#Overall PCA worksheet data frame used to populate outputs in template 4 prac-dom-pca
overall_pca_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE"
                    ,"PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                    ,"PREV_PRACTICE_LIST_SIZE","LIST_SIZE"
                    ,"PREV_PCAS","PREV_DENOMINATORS","PREV_PCA_RATE"
                    ,"PCAS","DENOMINATORS","PCA_RATE","ALL_PCA_CHANGE"
                    ,"CL-PREV_PCAS","CL-PREV_DENOMINATORS","CL-PREV_PCA_RATE"
                    ,"CL-PCAS","CL-DENOMINATORS","CL-PCA_RATE","CL_PCA_CHANGE"                      
                    ,"PH-PREV_PCAS","PH-PREV_DENOMINATORS","PH-PREV_PCA_RATE"
                    ,"PH-PCAS","PH-DENOMINATORS","PH-PCA_RATE","PH_PCA_CHANGE"                       
                    ,"PHAS-PREV_PCAS","PHAS-PREV_DENOMINATORS","PHAS-PREV_PCA_RATE"
                    ,"PHAS-PCAS","PHAS-DENOMINATORS","PHAS-PCA_RATE","PHAS_PCA_CHANGE"
                    ,"PHVI-PREV_PCAS","PHVI-PREV_DENOMINATORS","PHVI-PREV_PCA_RATE"
                    ,"PHVI-PCAS", "PHVI-DENOMINATORS", "PHVI-PCA_RATE","PHVI_PCA_CHANGE"
                    ,"QI-PCAS", "QI-DENOMINATORS", "QI-PCA_RATE"
                    ]
overall_pca_df = overall_pca_df[overall_pca_col] 

overall_pca_df = overall_pca_df.sort_values(by=["CCG_ODS_CODE","PRACTICE_NAME"]
                                            ,ascending = [True,True]).reset_index(drop = True)

<h2>OVERALL_PCA - Group data frame

In [ ]:
## Practice data frame from QOF Master table
pca_gr_current_col = ["PRACTICE_CODE","GROUP_CODE","SUB_DOMAIN_CODE","PCA_COUNT","PCA_DENOMINATOR"]
pca_gr_current_df = qof_master[pca_gr_current_col]

pca_gr_current_df = pca_gr_current_df.groupby(by = ["PRACTICE_CODE","GROUP_CODE","SUB_DOMAIN_CODE"]
                                              ,as_index = False
                                              ).agg({"PCA_COUNT" : [pd.Series.sum]
                                                     ,"PCA_DENOMINATOR" : [pd.Series.sum]})                                                                                                                                                                                                                                                   
pca_gr_current_df.columns = pca_gr_current_df.columns.droplevel(1)

In [ ]:
## Create a list of group codes with PCAs which is used to add a column of  
## summed denominator by practice and group code. Where denominator does 
## not equal zero
pca_gr_codes_col = ["GROUP_CODE","PCA_DENOMINATOR"]
pca_gr_codes_df = pca_gr_current_df[pca_gr_codes_col]

pca_gr_codes_df =pca_gr_codes_df.groupby(by = ["GROUP_CODE"]
                                         ,as_index=False
                                         )["PCA_DENOMINATOR"].sum()

pca_gr_codes_df = pca_gr_codes_df.loc[(pca_gr_codes_df["PCA_DENOMINATOR"]!= 0)]
pca_gr_codes_df = pca_gr_codes_df.drop(columns=["PCA_DENOMINATOR"])

In [ ]:
## pca_gr_codes_df is used to provide summed 'PCA_DENOMINATOR' where there 
## are no zero values
pca_gr_current_df = pd.merge(pca_gr_current_df
                             ,pca_gr_codes_df
                             ,left_on = ["GROUP_CODE"]
                             ,right_on = ["GROUP_CODE"]
                             ,how ="inner")

## Clinical domain (CL)

In [ ]:
## Indicator groups in the Clinical domain from the PCA indicator group data frame
pca_gr_current_cl_df = pca_gr_current_df.loc[(pca_gr_current_df["SUB_DOMAIN_CODE"]== "CL")]


## Add PCA rate to the Clinical domain data frame 
pca_dom_current_cl_df = pca_gr_current_cl_df.drop(columns=["GROUP_CODE"])

pca_dom_current_cl_df = pca_dom_current_cl_df.groupby(by = ["PRACTICE_CODE","SUB_DOMAIN_CODE"]
                                                      ,as_index=False
                                                      ).agg({"PCA_COUNT" : [pd.Series.sum]
                                                             ,"PCA_DENOMINATOR" : [pd.Series.sum]})                                                                                                                                                                                                                                                                           
pca_dom_current_cl_df.columns = pca_dom_current_cl_df.columns.droplevel(1)

## Calculate 'PCA_RATE' and replace 'nan' values with zero
pca_dom_current_cl_df["PCA_RATE"] = (pca_dom_current_cl_df["PCA_COUNT"]
                                     )/(pca_dom_current_cl_df["PCA_DENOMINATOR"]+pca_dom_current_cl_df["PCA_COUNT"])*100

pca_dom_current_cl_df["PCA_RATE"] = pca_dom_current_cl_df["PCA_RATE"].replace(np.nan, 0)

In [ ]:
"""   
The aim of this code is to create a series of new columns that consist of the values in 'SUB_DOMAIN_CODE' and 'VARIABLE'
being concatenated with a hyphen in the middle e.g. CL-PCA_COUNT

"""

## The short broad data frame is converted into a long thin data frame and the non-index columns are 
## stacked in a column called 'VARIABLE'
pca_dom_current_cl_stacked_df = pca_dom_current_cl_df.melt(id_vars=["PRACTICE_CODE","SUB_DOMAIN_CODE"]
                                                             , var_name="VARIABLE"
                                                             , value_name="VALUE")

## New column 'DOMAIN_VARIABLE' is created by joining the 'SUB_DOMAIN_CODE' to 'VARIABLE' 
## with a hyphen and drop unwanted columns
pca_dom_current_cl_stacked_df["DOMAIN_VARIABLE"] = (pca_dom_current_cl_stacked_df["SUB_DOMAIN_CODE"]
                                                    )+"-"+(pca_dom_current_cl_stacked_df["VARIABLE"])

pca_dom_current_cl_stacked_df = pca_dom_current_cl_stacked_df.drop(columns=["SUB_DOMAIN_CODE","VARIABLE"])

## Pivot data frame from long thin data frame into short broad data frame
pca_dom_current_cl_piv_df = pca_dom_current_cl_stacked_df.pivot_table(index=["PRACTICE_CODE"]
                                                                      ,columns="DOMAIN_VARIABLE"
                                                                      ,values="VALUE"
                                                                      ,fill_value=0).reset_index(
                                                                                   ).rename_axis(None, axis=1)

In [ ]:
"""   
The aim of this code is to create a series of new columns that consist of the values in 'GROUP_CODE' and 'VARIABLE'
being concatenated with a hyphen in the middle e.g. AF-PCA_COUNT

""" 

## Now this column is no longer needed it can be dropped from the data frame
pca_gr_current_cl_df = pca_gr_current_cl_df.drop(columns=["SUB_DOMAIN_CODE"])

## The short broad data frame is converted into a long thin data frame and the non-index columns are 
## stacked in a column called 'VARIABLE'
pca_gr_current_cl_stacked_df = pca_gr_current_cl_df.melt(id_vars=["PRACTICE_CODE","GROUP_CODE"]
                                                         , var_name="VARIABLE"
                                                         , value_name="VALUE")

## New column 'GROUP_VARIABLE' is created by joining the 'SUB_DOMAIN_CODE' to 'VARIABLE' 
## with a hyphen and drop unwanted columns
pca_gr_current_cl_stacked_df["GROUP_VARIABLE"] = (pca_gr_current_cl_stacked_df["GROUP_CODE"]
                                                  )+"-"+(pca_gr_current_cl_stacked_df["VARIABLE"])

pca_gr_current_cl_stacked_df = pca_gr_current_cl_stacked_df.drop(columns=["GROUP_CODE","VARIABLE"])


## Pivot data frame from long thin data frame into short broad data frame
pca_gr_current_cl_piv_df = pca_gr_current_cl_stacked_df.pivot_table(index=["PRACTICE_CODE"]
                                                                            ,columns="GROUP_VARIABLE"
                                                                            ,values="VALUE"
                                                                            ,fill_value=0).reset_index(
                                                                                         ).rename_axis(None, axis=1)

### OUTPUT - Clinical Domain PCAs worksheet for PCA workbook

In [ ]:
## Add geographies and list size to individual PCA domain data
pca_cl_df = pd.merge(qof_cur_geog_list_df
                     ,pca_gr_current_cl_piv_df
                     ,left_on=["PRACTICE_CODE"]
                     ,right_on=["PRACTICE_CODE"]
                     ,how="left")


## Add overall PCA domain to individual domain data frame
pca_cl_df = pd.merge(pca_cl_df
                     ,pca_dom_current_cl_piv_df
                     ,left_on=["PRACTICE_CODE"]
                     ,right_on=["PRACTICE_CODE"]
                     ,how="left")

## Sort values for output
pca_cl_df = pca_cl_df.sort_values(by = ["CCG_ODS_CODE","PRACTICE_NAME"]
                                  ,ascending = [True,True]).reset_index(drop = True)

## Public Health domain (PH)

In [ ]:
## Indicator groups in the Public Health domain from the PCA indicator group data frame
pca_gr_current_ph_df = pca_gr_current_df.loc[(pca_gr_current_df["SUB_DOMAIN_CODE"]== "PH")]


## Add PCA rate to Public Health domain data frame
pca_dom_current_ph_df = pca_gr_current_ph_df.drop(columns=["GROUP_CODE"])

pca_dom_current_ph_df = pca_dom_current_ph_df.groupby(by = ["PRACTICE_CODE","SUB_DOMAIN_CODE"]
                                                      ,as_index=False
                                                      ).agg({"PCA_COUNT" : [pd.Series.sum]
                                                             ,"PCA_DENOMINATOR" : [pd.Series.sum]})                                                                                                                                                                                                                                                     
pca_dom_current_ph_df.columns = pca_dom_current_ph_df.columns.droplevel(1)

## Calculate 'PCA_RATE' and replace 'nan' values with zero
pca_dom_current_ph_df["PCA_RATE"] = (pca_dom_current_ph_df["PCA_COUNT"]
                                     )/(pca_dom_current_ph_df["PCA_DENOMINATOR"]+pca_dom_current_ph_df["PCA_COUNT"])*100

pca_dom_current_ph_df["PCA_RATE"] = pca_dom_current_ph_df["PCA_RATE"].replace(np.nan, 0)

In [ ]:
"""   
The aim of this code is to create a series of new columns that consist of the values in 'SUB_DOMAIN_CODE' and 'VARIABLE'
being concatenated with a hyphen in the middle e.g. PH-PCA_COUNT

"""

## The short broad data frame is converted into a long thin data frame and the non-index columns are 
## stacked in a column called 'VARIABLE'
pca_dom_current_ph_stacked_df = pca_dom_current_ph_df.melt(id_vars=["PRACTICE_CODE","SUB_DOMAIN_CODE"]
                                                           ,var_name="VARIABLE"
                                                           ,value_name="VALUE")

## New column 'DOMAIN_VARIABLE' is created by joining the 'SUB_DOMAIN_CODE' to 'VARIABLE' 
## with a hyphen and drop unwanted columns
pca_dom_current_ph_stacked_df["DOMAIN_VARIABLE"] = (pca_dom_current_ph_stacked_df["SUB_DOMAIN_CODE"]
                                                    )+"-"+(pca_dom_current_ph_stacked_df["VARIABLE"])

pca_dom_current_ph_stacked_df = pca_dom_current_ph_stacked_df.drop(columns=["SUB_DOMAIN_CODE","VARIABLE"])

## Pivot data frame from long thin data frame into short broad data frame
pca_dom_current_ph_piv_df = pca_dom_current_ph_stacked_df.pivot_table(index=["PRACTICE_CODE"]
                                                                      ,columns="DOMAIN_VARIABLE"
                                                                      ,values="VALUE"
                                                                      ,fill_value=0).reset_index(
                                                                                   ).rename_axis(None, axis=1)

In [ ]:
"""   
The aim of this code is to create a series of new columns that consist of the values in 'GROUP_CODE' 
and 'VARIABLE' being concatenated with a hyphen in the middle e.g. BP-PCA_COUNT

""" 

## Now this column is no longer needed it can be dropped from the data frame
pca_gr_current_ph_df = pca_gr_current_ph_df.drop(columns=["SUB_DOMAIN_CODE"])

## The short broad data frame is converted into a long thin data frame and the 
# non-index columns are stacked in a column called 'VARIABLE'
pca_gr_current_ph_stacked_df = pca_gr_current_ph_df.melt(id_vars=["PRACTICE_CODE","GROUP_CODE"]
                                                         ,var_name="VARIABLE"
                                                         ,value_name="VALUE")

## New column 'GROUP_VARIABLE' is created by joining the 'SUB_DOMAIN_CODE' to 'VARIABLE' 
## with a hyphen and drop unwanted columns
pca_gr_current_ph_stacked_df["GROUP_VARIABLE"] = (pca_gr_current_ph_stacked_df["GROUP_CODE"]
                                                  )+"-"+(pca_gr_current_ph_stacked_df["VARIABLE"])

pca_gr_current_ph_stacked_df = pca_gr_current_ph_stacked_df.drop(columns=["GROUP_CODE","VARIABLE"])

## Pivot data frame from long thin data frame into short broad data frame
pca_gr_current_ph_piv_df = pca_gr_current_ph_stacked_df.pivot_table(index=["PRACTICE_CODE"]
                                                                    ,columns="GROUP_VARIABLE"
                                                                    ,values="VALUE"
                                                                    ,fill_value=0).reset_index(
                                                                                 ).rename_axis(None, axis=1)

<h3>OUTPUT - Public Health Domain PCAs worksheet for PCA workbook

In [ ]:
## Add geographies and list size to individual PCA domain data
pca_ph_df = pd.merge(qof_cur_geog_list_df
                     ,pca_gr_current_ph_piv_df
                     ,left_on=["PRACTICE_CODE"]
                     ,right_on=["PRACTICE_CODE"]
                     ,how="left")


## Add overall PCA domain to individual domain data frame
pca_ph_df = pd.merge(pca_ph_df
                     ,pca_dom_current_ph_piv_df
                     ,left_on=["PRACTICE_CODE"]
                     ,right_on=["PRACTICE_CODE"]
                     ,how="left")


## Sort values for output
pca_ph_df = pca_ph_df.sort_values(by = ["CCG_ODS_CODE","PRACTICE_NAME"]
                                  ,ascending = [True,True]).reset_index(drop = True)

<h2>Public Health Additional Services domain (PHAS)

In [ ]:
## Indicator groups in the Public Health Additional Services domain from the PCA indicator group data frame
pca_gr_current_phas_df = pca_gr_current_df.loc[(pca_gr_current_df["SUB_DOMAIN_CODE"]== "PHAS")]

## Add PCA rate to the Public Health Additional Service domain data frame
pca_dom_current_phas_df = pca_gr_current_phas_df.drop(columns=["GROUP_CODE"])

pca_dom_current_phas_df = pca_dom_current_phas_df.groupby(by = ["PRACTICE_CODE","SUB_DOMAIN_CODE"]
                                                          ,as_index=False
                                                          ).agg({"PCA_COUNT" : [pd.Series.sum]
                                                                 ,"PCA_DENOMINATOR" : [pd.Series.sum]})                                                                                                                                                        
pca_dom_current_phas_df.columns = pca_dom_current_phas_df.columns.droplevel(1)

## Calculate 'PCA_RATE' and replace 'nan' values with zero
pca_dom_current_phas_df["PCA_RATE"] = (pca_dom_current_phas_df["PCA_COUNT"]
                                       )/(pca_dom_current_phas_df["PCA_DENOMINATOR"]+pca_dom_current_phas_df["PCA_COUNT"])*100

pca_dom_current_phas_df["PCA_RATE"] = pca_dom_current_phas_df["PCA_RATE"].replace(np.nan, 0)

In [ ]:
"""   
The aim of this code is to create a series of new columns that consist of the values in 'SUB_DOMAIN_CODE' and 'VARIABLE'
being concatenated with a hyphen in the middle e.g. PHAS-PCA_COUNT

"""

## The short broad data frame is converted into a long thin data frame and the non-index columns are 
## stacked in a column called 'VARIABLE'
pca_dom_current_phas_stacked_df = pca_dom_current_phas_df.melt(id_vars=["PRACTICE_CODE","SUB_DOMAIN_CODE"]
                                                               ,var_name="VARIABLE"
                                                               ,value_name="VALUE")

## New column 'DOMAIN_VARIABLE' is created by joining the 'SUB_DOMAIN_CODE' to 'VARIABLE' 
## with a hyphen and drop unwanted columns
pca_dom_current_phas_stacked_df["DOMAIN_VARIABLE"] = (pca_dom_current_phas_stacked_df["SUB_DOMAIN_CODE"]
                                                      )+"-"+(pca_dom_current_phas_stacked_df["VARIABLE"])

pca_dom_current_phas_stacked_df = pca_dom_current_phas_stacked_df.drop(columns=["SUB_DOMAIN_CODE","VARIABLE"])

pca_dom_current_phas_piv_df = pca_dom_current_phas_stacked_df.pivot_table(index=["PRACTICE_CODE"]
                                                                          ,columns="DOMAIN_VARIABLE"
                                                                          ,values="VALUE"
                                                                          ,fill_value=0).reset_index(
                                                                                       ).rename_axis(None, axis=1)

In [ ]:
"""   
The aim of this code is to create a series of new columns that consist of the values in 'GROUP_CODE' and 'VARIABLE'
being concatenated with a hyphen in the middle e.g. CS-PCA_COUNT

""" 

## Now this column is no longer needed it can be dropped from the data frame
pca_gr_current_phas_df = pca_gr_current_phas_df.drop(columns=["SUB_DOMAIN_CODE"])

## The short broad data frame is converted into a long thin data frame and the non-index columns are 
## stacked in a column called 'VARIABLE'
pca_gr_current_phas_stacked_df = pca_gr_current_phas_df.melt(id_vars=["PRACTICE_CODE","GROUP_CODE"]
                                                             ,var_name="VARIABLE"
                                                             ,value_name="VALUE")

## New column 'GROUP_VARIABLE' is created by joining the 'SUB_DOMAIN_CODE' to 'VARIABLE' 
## with a hyphen and drop unwanted columns
pca_gr_current_phas_stacked_df["GROUP_VARIABLE"] = (pca_gr_current_phas_stacked_df["GROUP_CODE"]
                                                    )+"-"+(pca_gr_current_phas_stacked_df["VARIABLE"])

pca_gr_current_phas_stacked_df = pca_gr_current_phas_stacked_df.drop(columns=["GROUP_CODE","VARIABLE"])

## Pivot data frame from long thin data frame into short broad data frame
pca_gr_current_phas_piv_df = pca_gr_current_phas_stacked_df.pivot_table(index=["PRACTICE_CODE"]
                                                                        ,columns="GROUP_VARIABLE"
                                                                        ,values="VALUE"
                                                                        ,fill_value=0).reset_index(
                                                                                     ).rename_axis(None, axis=1)

<h3>OUTPUT - Public Health Additional Services Domain PCAs worksheet for PCA workbook

In [ ]:
## Add geographies and list size to individual PCA domain data
pca_phas_df = pd.merge(qof_cur_geog_list_df
                       ,pca_gr_current_phas_piv_df
                       ,left_on=["PRACTICE_CODE"]
                       ,right_on=["PRACTICE_CODE"]
                       ,how="left")


## Add overall PCA domain to individual domain data frame
pca_phas_df = pd.merge(pca_phas_df
                        ,pca_dom_current_phas_piv_df
                        ,left_on=["PRACTICE_CODE"]
                        ,right_on=["PRACTICE_CODE"]
                        ,how="left")

## Sort values for output
pca_phas_df = pca_phas_df.sort_values(by = ["CCG_ODS_CODE","PRACTICE_NAME"]
                                      ,ascending = [True,True]).reset_index(drop = True)

## Public Health Vaccination and Immunisation domain (PHVI)

In [ ]:
## Indicator groups in the Public Health Vaccination and Immunisation domain from the PCA indicator group data frame
pca_gr_current_phvi_df = pca_gr_current_df.loc[(pca_gr_current_df["SUB_DOMAIN_CODE"]== "PHVI")]


## Add PCA rate to Public Health Additional Service domain data frame
pca_dom_current_phvi_df = pca_gr_current_phvi_df.drop(columns=["GROUP_CODE"])

pca_dom_current_phvi_df = pca_dom_current_phvi_df.groupby(by = ["PRACTICE_CODE","SUB_DOMAIN_CODE"]
                                                          ,as_index=False
                                                          ).agg({"PCA_COUNT" : [pd.Series.sum]
                                                                 ,"PCA_DENOMINATOR" : [pd.Series.sum]})                                                                                                                                                                                                                                                                                                  
pca_dom_current_phvi_df.columns = pca_dom_current_phvi_df.columns.droplevel(1)

## Calculate 'PCA_RATE' and replace 'nan' values with zero
pca_dom_current_phvi_df["PCA_RATE"] = (pca_dom_current_phvi_df["PCA_COUNT"]
                                       )/(pca_dom_current_phvi_df["PCA_DENOMINATOR"]+pca_dom_current_phvi_df["PCA_COUNT"])*100

pca_dom_current_phvi_df["PCA_RATE"] = pca_dom_current_phvi_df["PCA_RATE"].replace(np.nan, 0)

In [ ]:
"""   
The aim of this code is to create a series of new columns that consist of the values 
in 'SUB_DOMAIN_CODE' and 'VARIABLE' being concatenated with a hyphen e.g. PHVI-PCA_COUNT

"""

## The short broad data frame is converted into a long thin data frame and the non-index 
## columns are stacked in a column called 'VARIABLE'
pca_dom_current_phvi_stacked_df = pca_dom_current_phvi_df.melt(id_vars=["PRACTICE_CODE","SUB_DOMAIN_CODE"]
                                                               ,var_name="VARIABLE"
                                                               ,value_name="VALUE")

## New column 'DOMAIN_VARIABLE' is created by joining the 'SUB_DOMAIN_CODE' to 'VARIABLE' 
## with a hyphen and drop unwanted columns
pca_dom_current_phvi_stacked_df["DOMAIN_VARIABLE"] = (pca_dom_current_phvi_stacked_df["SUB_DOMAIN_CODE"]
                                                      )+"-"+(pca_dom_current_phvi_stacked_df["VARIABLE"])

pca_dom_current_phvi_stacked_df = pca_dom_current_phvi_stacked_df.drop(columns=["SUB_DOMAIN_CODE","VARIABLE"])

## Pivot data frame from long thin data frame into short broad data frame
pca_dom_current_phvi_piv_df = pca_dom_current_phvi_stacked_df.pivot_table(index=["PRACTICE_CODE"]
                                                                          ,columns="DOMAIN_VARIABLE"
                                                                          ,values="VALUE"
                                                                          ,fill_value=0).reset_index(
                                                                                       ).rename_axis(None, axis=1)

In [ ]:
"""   
The aim of this code is to create a series of new columns that consist of the values in 'GROUP_CODE' and 'VARIABLE'
being concatenated with a hyphen e.g. VI-PCA_COUNT
""" 

## Now this column is no longer needed it can be dropped from the data frame
pca_gr_current_phvi_df = pca_gr_current_phvi_df.drop(columns=["SUB_DOMAIN_CODE"])

## The short broad data frame is converted into a long thin data frame and the non-index columns are 
## stacked in a column called 'VARIABLE'
pca_gr_current_phvi_stacked_df = pca_gr_current_phvi_df.melt(id_vars=["PRACTICE_CODE","GROUP_CODE"]
                                                             ,var_name="VARIABLE"
                                                             ,value_name="VALUE")

## New column 'GROUP_VARIABLE' is created by joining the 'SUB_DOMAIN_CODE' to 'VARIABLE' 
## with a hyphen and drop unwanted columns
pca_gr_current_phvi_stacked_df["GROUP_VARIABLE"] = (pca_gr_current_phvi_stacked_df["GROUP_CODE"]
                                                    )+"-"+(pca_gr_current_phvi_stacked_df["VARIABLE"])

pca_gr_current_phvi_stacked_df = pca_gr_current_phvi_stacked_df.drop(columns=["GROUP_CODE","VARIABLE"])

## Pivot data frame from long thin data frame into short broad data frame
pca_gr_current_phvi_piv_df = pca_gr_current_phvi_stacked_df.pivot_table(index=["PRACTICE_CODE"]
                                                                        ,columns="GROUP_VARIABLE"
                                                                        ,values="VALUE"
                                                                        ,fill_value=0).reset_index(
                                                                                     ).rename_axis(None, axis=1)

<h3>OUTPUT - Public Health Additional Services Domain PCAs worksheet for PCA workbook

In [ ]:
## Add geographies and list size to individual PCA domain data
pca_phvi_df = pd.merge(qof_cur_geog_list_df
                       ,pca_gr_current_phvi_piv_df
                       ,left_on=["PRACTICE_CODE"]
                       ,right_on=["PRACTICE_CODE"]
                       ,how="left")

## Add overall PCA domain to individual domain data frame
pca_phvi_df = pd.merge(pca_phvi_df
                       ,pca_dom_current_phvi_piv_df
                       ,left_on=["PRACTICE_CODE"]
                       ,right_on=["PRACTICE_CODE"]
                       ,how="left")

## Sort values for output
pca_phvi_df = pca_phvi_df.sort_values(by = ["CCG_ODS_CODE","PRACTICE_NAME"]
                                      ,ascending = [True,True]).reset_index(drop = True)

## Quality Improvement domain (QI)

In [ ]:
## Indicator groups in the Quality Improvement domain from the PCA indicator group data frame
pca_gr_current_qi_df = pca_gr_current_df.loc[(pca_gr_current_df["SUB_DOMAIN_CODE"]== "QI")]


## Add PCA rate to Quality Improvement  domain data frame
pca_dom_current_qi_df = pca_gr_current_qi_df.drop(columns=["GROUP_CODE"])

pca_dom_current_qi_df = pca_dom_current_qi_df.groupby(by = ["PRACTICE_CODE","SUB_DOMAIN_CODE"]
                                                      ,as_index=False
                                                      ).agg({"PCA_COUNT" : [pd.Series.sum]
                                                             ,"PCA_DENOMINATOR" : [pd.Series.sum]})                                                                                                                                                                                                                   
pca_dom_current_qi_df.columns = pca_dom_current_qi_df.columns.droplevel(1)

## Calculate 'PCA_RATE' and replace 'nan' values with zero
pca_dom_current_qi_df["PCA_RATE"] = (pca_dom_current_qi_df["PCA_COUNT"]
                                     )/(pca_dom_current_qi_df["PCA_DENOMINATOR"]+pca_dom_current_qi_df["PCA_COUNT"])*100

pca_dom_current_qi_df["PCA_RATE"] = pca_dom_current_qi_df["PCA_RATE"].replace(np.nan, 0)

In [ ]:
"""   
The aim of this code is to create a series of new columns that consist of the values 
in 'SUB_DOMAIN_CODE' and 'VARIABLE' being concatenated with a hyphen e.g. QI-PCA_COUNT

"""

## The short broad data frame is converted into a long thin data frame and the non-index
## columns are stacked in a column called 'VARIABLE'
pca_dom_current_qi_stacked_df = pca_dom_current_qi_df.melt(id_vars=["PRACTICE_CODE","SUB_DOMAIN_CODE"]
                                                           ,var_name="VARIABLE"
                                                           ,value_name="VALUE")

## New column header is created by joining the 'SUB_DOMAIN_CODE' to 'VARIABLE' 
## with a hyphen and drop unwanted columns
pca_dom_current_qi_stacked_df["DOMAIN_VARIABLE"] = (pca_dom_current_qi_stacked_df["SUB_DOMAIN_CODE"]
                                                    )+"-"+(pca_dom_current_qi_stacked_df["VARIABLE"])

pca_dom_current_qi_stacked_df = pca_dom_current_qi_stacked_df.drop(columns=["SUB_DOMAIN_CODE","VARIABLE"])

## Pivot data frame from long thin data frame into short broad data frame
pca_dom_current_qi_piv_df = pca_dom_current_qi_stacked_df.pivot_table(index=["PRACTICE_CODE"]
                                                                      ,columns="DOMAIN_VARIABLE"
                                                                      ,values="VALUE"
                                                                      ,fill_value=0).reset_index(
                                                                                   ).rename_axis(None, axis=1)

In [ ]:
"""   
The aim of this code is to create a series of new columns that consist of the values in 'GROUP_CODE' and 'VARIABLE'
being concatenated with a hyphen e.g. QIPDD_MI-PCA_COUNT

""" 

## Now this column is no longer needed it can be dropped from the data frame
pca_gr_current_qi_df = pca_gr_current_qi_df.drop(columns=["SUB_DOMAIN_CODE"])


## The short broad data frame is converted into a long thin data frame and the non-index 
## columns are stacked in a column called 'VARIABLE'
pca_gr_current_qi_stacked_df = pca_gr_current_qi_df.melt(id_vars=["PRACTICE_CODE","GROUP_CODE"]
                                                         ,var_name="VARIABLE"
                                                         ,value_name="VALUE")

## New column 'GROUP_VARIABLE' is created by joining the 'SUB_DOMAIN_CODE' to 'VARIABLE' 
## with a hyphen and drop unwanted columns
pca_gr_current_qi_stacked_df["GROUP_VARIABLE"] = (pca_gr_current_qi_stacked_df["GROUP_CODE"]
                                                  )+"-"+(pca_gr_current_qi_stacked_df["VARIABLE"])

pca_gr_current_qi_stacked_df = pca_gr_current_qi_stacked_df.drop(columns=["GROUP_CODE","VARIABLE"])

## Pivot data frame from long thin data frame into short broad data frame
pca_gr_current_qi_piv_df = pca_gr_current_qi_stacked_df.pivot_table(index=["PRACTICE_CODE"]
                                                                    ,columns="GROUP_VARIABLE"
                                                                    ,values="VALUE"
                                                                    ,fill_value=0).reset_index(
                                                                                 ).rename_axis(None, axis=1)

<h3>OUTPUT - Quality Improvement Domain PCAs worksheet for PCA workbook

In [ ]:
## Add geographies and list size to individual PCA domain data
pca_qi_df = pd.merge(qof_cur_geog_list_df
                     ,pca_gr_current_qi_piv_df
                     ,left_on=["PRACTICE_CODE"]
                     ,right_on=["PRACTICE_CODE"]
                     ,how="left")

## Add overall PCA domain to individual domain data frame
pca_qi_df = pd.merge(pca_qi_df
                     ,pca_dom_current_qi_piv_df
                     ,left_on=["PRACTICE_CODE"]
                     ,right_on=["PRACTICE_CODE"]
                     ,how="left")

## Sort values for output
pca_qi_df = pca_qi_df.sort_values(by = ["CCG_ODS_CODE","PRACTICE_NAME"]
                                  ,ascending = [True,True]).reset_index(drop = True)

### Create T4 PCA Table titles

In [ ]:
## Use table title produced for template 3 and replace the word
## 'Achievement' with the words 'Personalised Care Adjustments' in table titles
pca_table_titles_df = ach_table_titles_df.copy()
pca_table_titles_df["TABLE_TITLE"] = pca_table_titles_df["TABLE_TITLE"].str.replace("Achievement","Personalised care adjustments")

<b><h1>19 OUTPUTS - T6-PrevAchPCA_reg and nat</h1></b><br><h2>Part 1 - QOF_MASTER for current indicators<br><h3>Region data frame from QOF Master</b>

In [ ]:
## Required columns from QOF Master table
t6_reg_current_base_col = ["REGION_ODS_CODE","GROUP_CODE","INDICATOR_CODE","ACHIEVED_POINTS"
                           ,"MAX_INDICATOR_POINTS","ACHIEVEMENT_NUMERATOR"
                           ,"ACHIEVEMENT_DENOMINATOR","PCA_COUNT"]
t6_reg_current_base_df = qof_master[t6_reg_current_base_col]

## Sum all non index columns
t6_reg_current_sum_df = t6_reg_current_base_df.groupby(by = ["REGION_ODS_CODE"
                                                             ,"GROUP_CODE"
                                                             ,"INDICATOR_CODE"]
                                                             ,as_index=False).agg({"ACHIEVED_POINTS" : [pd.Series.sum]
                                                                                    ,"MAX_INDICATOR_POINTS" : [pd.Series.sum]
                                                                                    ,"ACHIEVEMENT_NUMERATOR" : [pd.Series.sum]
                                                                                    ,"ACHIEVEMENT_DENOMINATOR" : [pd.Series.sum]
                                                                                    ,"PCA_COUNT" : [pd.Series.sum]})                                                                                                                                              
t6_reg_current_sum_df.columns = t6_reg_current_sum_df.columns.droplevel(1)

## Create all percentage columns
## Achievement Rate
t6_reg_current_sum_df["ACH_RATE"] = (t6_reg_current_sum_df["ACHIEVED_POINTS"]/
                                     t6_reg_current_sum_df["MAX_INDICATOR_POINTS"])*100

## Underlying Achievement net of PCAs
t6_reg_current_sum_df["UL_NET_ACH"] = (t6_reg_current_sum_df["ACHIEVEMENT_NUMERATOR"]/
                                       t6_reg_current_sum_df["ACHIEVEMENT_DENOMINATOR"])*100

## PCA Rate
t6_reg_current_sum_df["PCA_RATE"] = (t6_reg_current_sum_df["PCA_COUNT"]/
                                     (t6_reg_current_sum_df["ACHIEVEMENT_DENOMINATOR"]+
                                      t6_reg_current_sum_df["PCA_COUNT"]))*100

## Denominator plus PCAs
t6_reg_current_sum_df["DEN_PLUS_PCA"] = (t6_reg_current_sum_df["ACHIEVEMENT_DENOMINATOR"]+
                                         t6_reg_current_sum_df["PCA_COUNT"])

## Patients Receiving Intervention
t6_reg_current_sum_df["PRI"] = (t6_reg_current_sum_df["ACHIEVEMENT_NUMERATOR"]/
                                (t6_reg_current_sum_df["ACHIEVEMENT_DENOMINATOR"]+
                                 t6_reg_current_sum_df["PCA_COUNT"]))*100


## Drop unwanted column
t6_reg_current_sum_df = t6_reg_current_sum_df.drop(columns=["MAX_INDICATOR_POINTS"])


In [ ]:
## The short broad data frame is converted into a long thin data frame and the non-index columns are 
## stacked in a column called 'FIELD'
t6_reg_current_stacked_df = (t6_reg_current_sum_df.set_index(["REGION_ODS_CODE","GROUP_CODE","INDICATOR_CODE"])
                                                  .stack()                                                              
                                                  .ffill(axis=0)
                                                  .bfill(axis=0, downcast='infer')
                                                  .reset_index()                                                                
                                                  .rename({"level_3":"FIELD", 0:"VALUE"},axis=1))
                                                                                                                                                                                                                                                                                                                                                                                   
## New column 'INDICATOR' is created by joining the 'INDICATOR_CODE' to 'FIELD' 
## with a hyphen and drop unwanted columns e.g. AF001_ACHIEVED_POINTS
t6_reg_current_stacked_df["INDICATOR"] = (t6_reg_current_stacked_df["INDICATOR_CODE"]
                                          )+"_"+(t6_reg_current_stacked_df["FIELD"])

## Drop Unwanted columns leaving 'REGION_ODS_CODE','GROUP_CODE','INDICATOR','VALUE'
t6_reg_current_stacked_df = t6_reg_current_stacked_df.drop(columns=["INDICATOR_CODE","FIELD"])

<h3>National data frame from QOF Master

In [ ]:
## Required indicator and achievement columns from QOF Master table
t6_nat_current_base_col = ["NAT_CODE","GROUP_CODE","INDICATOR_CODE","ACHIEVED_POINTS"
                           ,"MAX_INDICATOR_POINTS" ,"ACHIEVEMENT_NUMERATOR"
                          ,"ACHIEVEMENT_DENOMINATOR","PCA_COUNT"]
t6_nat_current_base_df = qof_master[t6_nat_current_base_col]

## Sum all non-index columns
t6_nat_current_sum_df = t6_nat_current_base_df.groupby(by = ["NAT_CODE","GROUP_CODE","INDICATOR_CODE"]
                                                       ,as_index=False
                                                       ).agg({"ACHIEVED_POINTS" : [pd.Series.sum],
                                                              "MAX_INDICATOR_POINTS" : [pd.Series.sum],
                                                              "ACHIEVEMENT_NUMERATOR" : [pd.Series.sum],
                                                              "ACHIEVEMENT_DENOMINATOR" : [pd.Series.sum],
                                                              "PCA_COUNT" : [pd.Series.sum]})    
t6_nat_current_sum_df.columns = t6_nat_current_sum_df.columns.droplevel(1)

## Create all percentage columns
## Achievement Rate
t6_nat_current_sum_df["ACH_RATE"] = (t6_nat_current_sum_df["ACHIEVED_POINTS"]/
                                     t6_nat_current_sum_df["MAX_INDICATOR_POINTS"])*100

## Underlying Achievement net of PCAs
t6_nat_current_sum_df["UL_NET_ACH"] = (t6_nat_current_sum_df["ACHIEVEMENT_NUMERATOR"]/
                                       t6_nat_current_sum_df["ACHIEVEMENT_DENOMINATOR"])*100

## PCA Rate
t6_nat_current_sum_df["PCA_RATE"] = (t6_nat_current_sum_df["PCA_COUNT"]/
                                     (t6_nat_current_sum_df["ACHIEVEMENT_DENOMINATOR"]+
                                      t6_nat_current_sum_df["PCA_COUNT"]))*100

## Denominator plus PCAs
t6_nat_current_sum_df["DEN_PLUS_PCA"] = (t6_nat_current_sum_df["ACHIEVEMENT_DENOMINATOR"]+
                                         t6_nat_current_sum_df["PCA_COUNT"])

## Patients Receiving Intervention
t6_nat_current_sum_df["PRI"] = (t6_nat_current_sum_df["ACHIEVEMENT_NUMERATOR"]/
                                (t6_nat_current_sum_df["ACHIEVEMENT_DENOMINATOR"]+
                                 t6_nat_current_sum_df["PCA_COUNT"]))*100


## Drop unwanted column
t6_nat_current_sum_df = t6_nat_current_sum_df.drop(columns=["MAX_INDICATOR_POINTS"])

In [ ]:
## The short broad data frame is converted into a long thin data frame and the non-index 
## columns are stacked in a column called 'FIELD'
t6_nat_current_stacked_df = (t6_nat_current_sum_df.set_index(["NAT_CODE","GROUP_CODE","INDICATOR_CODE"])
                                                  .stack()                                                              
                                                  .ffill(axis=0)
                                                  .bfill(axis=0, downcast='infer')
                                                  .reset_index()                                                              
                                                  .rename({"level_3":"FIELD", 0:"VALUE"}, axis=1))                                                                                                   

## New column 'INDICATOR' is created by joining the 'INDICATOR_CODE' to 'FIELD' 
## with a hyphen and drop unwanted columns e.g. AF001_ACHIEVED_POINTS                                                                                              
t6_nat_current_stacked_df["INDICATOR"] = (t6_nat_current_stacked_df["INDICATOR_CODE"]
                                          )+"_"+(t6_nat_current_stacked_df["FIELD"])

## 'NAT_CODE' column is renamed 'REGION_ODS_CODE' to enable this data frame to be joined to the region data frame
t6_nat_current_stacked_df = t6_nat_current_stacked_df.rename(columns={"NAT_CODE":"REGION_ODS_CODE"})

## Drop unwanted columns leaving 'REGION_ODS_CODE','GROUP_CODE','INDICATOR','VALUE'
t6_nat_current_stacked_df = t6_nat_current_stacked_df.drop(columns=["INDICATOR_CODE","FIELD"])

In [ ]:
## Append Region data frame to National data frame to give current years indicator specific data frame
t6_nat_reg_current_col = [t6_reg_current_stacked_df, t6_nat_current_stacked_df]
t6_nat_reg_current_df = pd.concat(t6_nat_reg_current_col)

## Pivot data frame from long thin data frame into short broad data frame
## With each column containing an individual elements of each indicator 
## e.g. 'AF001_ACHIEVED_POINTS', 'AF001_ACH_RATE'
t6_nat_reg_current_piv_df = pd.pivot_table(t6_nat_reg_current_df
                                           ,values="VALUE"
                                           ,index=["REGION_ODS_CODE","GROUP_CODE"]
                                           ,columns="INDICATOR").reset_index(                                                
                                                               ).rename_axis(None, axis=1)

<h2>Part 2 - Join QOF Master to previous years National and Regional data frame<br><h3>National

In [ ]:
"""   
Creates a national level data frame that comprises data for both the previous and current years 

"""

## Drop unwanted nhs geographies columns from the QOF Master table
t6_nat_master_col =["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME"
                    ,"STP_ODS_CODE","STP_ONS_CODE","STP_NAME"
                    ,"CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME"
                    ,"PCN_ODS_CODE","PCN_NAME","PRACTICE_NAME"
                    ,"DOMAIN_CODE","SUB_DOMAIN_CODE"]
t6_nat_master_df = qof_master.drop(columns=t6_nat_master_col)
                                                  
## Drop unwanted column from the data frame that combines data from the previous year at National level
t6_prev_out_nat_joined_df = previous_out_national_region_joined.drop(columns=["HIGHER_GROUP_CODE"])

## The resulting data frame is used as the foundation for four other code blocks below
t6_nat_current_previous_df = pd.merge(t6_nat_master_df
                                     ,t6_prev_out_nat_joined_df
                                     ,left_on = ["NAT_COUNTRY","GROUP_CODE"]
                                     ,right_on = ["REGION_NAME","GROUP_CODE"]
                                     ,how = 'left') 

In [ ]:
## Select columns required for national group data frame for the previous year
## These columns are used as the index to which other columns are joined
t6_nat_previous_gr_col = ["NAT_CODE","NAT_ONS_CODE","NAT_COUNTRY","GROUP_CODE","ALT_PAT_LIST"
                          ,"PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE","PREV_ALT_PAT_LIST"
                          ,"PREV_PREVALENCE","PREV_REGISTER_SIZE","PREV_ACHIEVED_POINTS"
                          ,"PREV_TOTAL_INDICATOR_GROUP_POINTS","PREV_ACHIEVEMENT_RATE"
                          ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"]                                       
t6_nat_previous_gr_index_df = t6_nat_current_previous_df[t6_nat_previous_gr_col]

## Remove duplicate values
t6_nat_previous_gr_index_df = t6_nat_previous_gr_index_df.drop_duplicates().reset_index(drop=True)

In [ ]:
## Select columns required for national group data frame for the previous year
## Count distinct number of practices by indicator group
t6_nat_prac_gr_col = ["NAT_ONS_CODE","PRACTICE_CODE","GROUP_CODE"]
t6_nat_prac_gr_df = t6_nat_current_previous_df[t6_nat_prac_gr_col]
t6_nat_prac_gr_df = t6_nat_prac_gr_df.drop_duplicates()

t6_nat_prac_gr_df = t6_nat_prac_gr_df.groupby(by = ["NAT_ONS_CODE","GROUP_CODE"]
                                              ,as_index=False
                                              ).agg({"PRACTICE_CODE" : [pd.Series.count]})
t6_nat_prac_gr_df.columns = t6_nat_prac_gr_df.columns.droplevel(1)  

## Rename column 'PRACTICE_CODE' which contains a numeric count as 'NUMBER_PRACTICES'
t6_nat_prac_gr_df = t6_nat_prac_gr_df.rename(columns={"PRACTICE_CODE":"NUMBER_PRACTICES"})


In [ ]:
## Select columns required for national group data frame for the previous year
## Count distinct number of indicators by indicator group
t6_nat_ind_gr_col = ["NAT_ONS_CODE","INDICATOR_CODE","GROUP_CODE"]
t6_nat_ind_gr_df = t6_nat_current_previous_df[t6_nat_ind_gr_col]
t6_nat_ind_gr_df = t6_nat_ind_gr_df.drop_duplicates()

t6_nat_ind_gr_df = t6_nat_ind_gr_df.groupby(by = ["NAT_ONS_CODE","GROUP_CODE"]
                                            ,as_index=False
                                            ).agg({"INDICATOR_CODE" : [pd.Series.count]})
t6_nat_ind_gr_df.columns = t6_nat_ind_gr_df.columns.droplevel(1)  

## Rename column 'INDICATOR_CODE' which contains a numeric count as 'COUNT_DISTINCT_IND_CODE'
t6_nat_ind_gr_df = t6_nat_ind_gr_df.rename(columns={"INDICATOR_CODE":"COUNT_DISTINCT_IND_CODE"})

In [ ]:
## Select columns required for national group data frame for the previous year
## Aggregate data in non index columns
t6_nat_gr_current_aggregated_df = t6_nat_current_previous_df.groupby(by = ["NAT_ONS_CODE","GROUP_CODE"]
                                                                     ,as_index=False
                                                                     ).agg({"LIST_SIZE" : [pd.Series.sum]
                                                                            ,"ALT_LIST_SIZE" : [pd.Series.sum]
                                                                            ,"REGISTER_SIZE" : [pd.Series.sum]
                                                                            ,"ACHIEVED_POINTS" : [pd.Series.sum]
                                                                            ,"PCA_COUNT" : [pd.Series.sum]
                                                                            ,"ACHIEVEMENT_DENOMINATOR" : [pd.Series.sum]
                                                                            ,"MAX_GROUP_POINTS" : [pd.Series.max]})                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
t6_nat_gr_current_aggregated_df.columns = t6_nat_gr_current_aggregated_df.columns.droplevel(1)

## Rename columns 'LIST_SIZE' as @LIST_SIZE_SUMMED' and 'ALT_LIST_SIZE' as 'ALT_LIST_SIZE_SUMMED' which contain total counts
t6_nat_gr_current_aggregated_df = t6_nat_gr_current_aggregated_df.rename(columns={"LIST_SIZE":"LIST_SIZE_SUMMED"
                                                                                  ,"ALT_LIST_SIZE":"ALT_LIST_SIZE_SUMMED"})

In [ ]:
## Join aggregated data to geographies and measures that are not aggregated
t6_nat_gr_df = pd.merge(t6_nat_previous_gr_index_df
                        ,t6_nat_prac_gr_df
                        ,left_on = ["NAT_ONS_CODE","GROUP_CODE"]
                        ,right_on = ["NAT_ONS_CODE","GROUP_CODE"]
                        ,how = 'left')


t6_nat_gr_df = pd.merge(t6_nat_gr_df
                        ,t6_nat_ind_gr_df
                        ,left_on = ["NAT_ONS_CODE","GROUP_CODE"]
                        ,right_on = ["NAT_ONS_CODE","GROUP_CODE"]
                        ,how = 'left')


t6_nat_gr_df = pd.merge(t6_nat_gr_df
                        ,t6_nat_gr_current_aggregated_df
                        ,left_on = ["NAT_ONS_CODE","GROUP_CODE"]
                        ,right_on = ["NAT_ONS_CODE","GROUP_CODE"]
                        ,how = 'left')

In [ ]:
## Create finished t6 national data frame to be joined with region data

## Create 'LIST_SIZE' column and drop aggregated 'LIST_SIZE_SUMMED' as it is no longer needed
t6_nat_gr_df["LIST_SIZE"] = t6_nat_gr_df["LIST_SIZE_SUMMED"]/t6_nat_gr_df["COUNT_DISTINCT_IND_CODE"]
t6_nat_gr_df = t6_nat_gr_df.drop(columns=["LIST_SIZE_SUMMED"])

## Create column 'ALT_LIST_SIZE' and leave values blank if there is no value in the corresponding 'ALT_PAT_LIST'
t6_nat_gr_df["ALT_LIST_SIZE"] = np.where(t6_nat_gr_df["ALT_PAT_LIST"] != None
                                         ,(t6_nat_gr_df["ALT_LIST_SIZE_SUMMED"]/t6_nat_gr_df["COUNT_DISTINCT_IND_CODE"])
                                         ,"")

## Rename columns for national geographies ('NAT') columns  with 'REGION' to facilitate the join with region data. 
## Data columns that contain data totals with the prefix 'TOTAL'
t6_nat_gr_df = t6_nat_gr_df.rename(columns={"NAT_ONS_CODE":"REGION_ONS_CODE","NAT_CODE":"REGION_ODS_CODE"
                                            ,"NAT_COUNTRY":"REGION_NAME","ACHIEVED_POINTS":"TOTAL_ACH_SCORE"
                                            ,"PCA_COUNT":"TOTAL_PCAS","REGISTER_SIZE":"REGISTER"
                                            ,"ACHIEVEMENT_DENOMINATOR":"TOTAL_DENOMINATORS"
                                            })

## Select national level columns ordered to be appended to region data.
t6_nat_gr_col = ["GROUP_CODE","REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","NUMBER_PRACTICES"
                 ,"LIST_SIZE","ALT_LIST_SIZE","ALT_LIST_SIZE_SUMMED","COUNT_DISTINCT_IND_CODE","REGISTER"
                 ,"TOTAL_ACH_SCORE","MAX_GROUP_POINTS","TOTAL_PCAS","TOTAL_DENOMINATORS"
                 ,"PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE","PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE"
                 ,"PREV_PREVALENCE","PREV_ACHIEVED_POINTS","PREV_TOTAL_INDICATOR_GROUP_POINTS"
                 ,"PREV_ACHIEVEMENT_RATE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"]                                                                                       
t6_nat_gr_df = t6_nat_gr_df[t6_nat_gr_col]

<h3>Region

In [ ]:
"""   
Creates a region level data frame that comprises data for both the previous and current years 
without indicator data

"""

## Drop unwanted columns from the QOF Master table
t6_reg_master_col = ["NAT_ONS_CODE","NAT_CODE","NAT_COUNTRY"
                     ,"STP_ODS_CODE","STP_ONS_CODE","STP_NAME"
                     ,"CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME"
                     ,"PCN_ODS_CODE","PCN_NAME","PRACTICE_NAME"
                     ,"DOMAIN_CODE","SUB_DOMAIN_CODE"]                                                
t6_reg_master_df = qof_master.drop(columns= t6_reg_master_col)



## Drop unwanted column from the data frame that combines data from the previous year at Region level
t6_prev_out_nat_joined_df = previous_out_national_region_joined.drop(columns=["HIGHER_GROUP_CODE"])

## The resulting data frame is used as the foundation for four other code blocks below
t6_reg_current_previous_df = pd.merge(t6_reg_master_df
                                      ,t6_prev_out_nat_joined_df
                                      ,left_on = ["REGION_NAME","GROUP_CODE"]
                                      ,right_on = ["REGION_NAME","GROUP_CODE"]
                                      ,how = 'left') 

## Select columns required for region group data frame for the previous year
## These columns are used as the index to which other columns are joined
t6_reg_current_previous_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","GROUP_CODE"
                               ,"ALT_PAT_LIST","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
                               ,"PREV_ALT_PAT_LIST","PREV_PREVALENCE","PREV_REGISTER_SIZE"
                               ,"PREV_ACHIEVED_POINTS","PREV_TOTAL_INDICATOR_GROUP_POINTS"
                               ,"PREV_ACHIEVEMENT_RATE","PREV_PCA_COUNT","PREV_DENOMINATOR"
                               ,"PREV_PCA_RATE"]                                       
t6_reg_previous_gr_index_df = t6_reg_current_previous_df[t6_reg_current_previous_col]

## Remove duplicate values
t6_nat_previous_gt6_reg_previous_gr_index_dfr_index_df = t6_reg_previous_gr_index_df.drop_duplicates()


In [ ]:
## Select columns required for region group data frame for the previous year
## Count distinct number of practices by indicator group
t6_reg_prac_gr_col = ["REGION_ONS_CODE","PRACTICE_CODE","GROUP_CODE"]
t6_reg_prac_gr_df = t6_reg_current_previous_df[t6_reg_prac_gr_col]
t6_reg_prac_gr_df = t6_reg_prac_gr_df.drop_duplicates()

t6_reg_prac_gr_df = t6_reg_prac_gr_df.groupby(by = ["REGION_ONS_CODE","GROUP_CODE"]
                                              ,as_index=False
                                              ).agg({"PRACTICE_CODE" : [pd.Series.count]})       
t6_reg_prac_gr_df.columns = t6_reg_prac_gr_df.columns.droplevel(1)  

## Rename column 'PRACTICE_CODE' which contains a numeric count as 'NUMBER_PRACTICES'
t6_reg_prac_gr_df = t6_reg_prac_gr_df.rename(columns={"PRACTICE_CODE":"NUMBER_PRACTICES"})

In [ ]:
## Select columns required for region group data frame for the previous year
## Count distinct number of indicators by indicator group
t6_reg_ind_gr_col   = ["REGION_ONS_CODE","INDICATOR_CODE","GROUP_CODE"]
t6_reg_ind_gr_df = t6_reg_current_previous_df[t6_reg_ind_gr_col]
t6_reg_ind_gr_df = t6_reg_ind_gr_df.drop_duplicates()

t6_reg_ind_gr_df = t6_reg_ind_gr_df.groupby(by = ["REGION_ONS_CODE","GROUP_CODE"]
                                            ,as_index=False
                                            ).agg({"INDICATOR_CODE" : [pd.Series.count]})               
t6_reg_ind_gr_df.columns = t6_reg_ind_gr_df.columns.droplevel(1)  

## Rename column 'INDICATOR_CODE' which contains a numeric count as 'COUNT_DISTINCT_IND_CODE'
t6_reg_ind_gr_df = t6_reg_ind_gr_df.rename(columns={"INDICATOR_CODE":"COUNT_DISTINCT_IND_CODE"})

In [ ]:
## Select columns required for region group data frame
## Aggregate data in non index columns
t6_reg_gr_current_aggregated_df = t6_reg_current_previous_df.groupby(by = ["REGION_ONS_CODE","GROUP_CODE"]
                                                                     ,as_index=False
                                                                     ).agg({"LIST_SIZE" : [pd.Series.sum]
                                                                            ,"ALT_LIST_SIZE" : [pd.Series.sum]
                                                                            ,"REGISTER_SIZE" : [pd.Series.sum]
                                                                            ,"ACHIEVED_POINTS" : [pd.Series.sum]
                                                                            ,"PCA_COUNT" : [pd.Series.sum]
                                                                            ,"ACHIEVEMENT_DENOMINATOR" : [pd.Series.sum]
                                                                            ,"MAX_GROUP_POINTS" : [pd.Series.max]})                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
t6_reg_gr_current_aggregated_df.columns = t6_reg_gr_current_aggregated_df.columns.droplevel(1)

## Rename columns 'LIST_SIZE' as 'LIST_SIZE_SUMMED' and 'ALT_LIST_SIZE' as 'ALT_LIST_SIZE_SUMMED' which contains total counts
t6_reg_gr_current_aggregated_df = t6_reg_gr_current_aggregated_df.rename(columns={"LIST_SIZE":"LIST_SIZE_SUMMED"
                                                                                  ,"ALT_LIST_SIZE":"ALT_LIST_SIZE_SUMMED"})

In [ ]:
## Join aggregated data to geographies and measures that are not aggregated
t6_reg_gr_df  = pd.merge(t6_reg_previous_gr_index_df
                         ,t6_reg_prac_gr_df
                         ,left_on = ["REGION_ONS_CODE","GROUP_CODE"]
                         ,right_on = ["REGION_ONS_CODE","GROUP_CODE"]
                         ,how = 'left')
                        

t6_reg_gr_df = pd.merge(t6_reg_gr_df
                        ,t6_reg_ind_gr_df
                        ,left_on = ["REGION_ONS_CODE","GROUP_CODE"]
                        ,right_on = ["REGION_ONS_CODE","GROUP_CODE"]
                        ,how = 'left')


t6_reg_gr_df = pd.merge(t6_reg_gr_df
                        ,t6_reg_gr_current_aggregated_df
                        ,left_on = ["REGION_ONS_CODE","GROUP_CODE"]
                        ,right_on = ["REGION_ONS_CODE","GROUP_CODE"]
                        ,how = 'left')

In [ ]:
## Create finished t6 region data frame to be joined with national data

## Create 'LIST_SIZE' column and drop aggregated 'LIST_SIZE_SUMMED' as it is no longer needed
t6_reg_gr_df["LIST_SIZE"] = t6_reg_gr_df["LIST_SIZE_SUMMED"]/t6_reg_gr_df["COUNT_DISTINCT_IND_CODE"]
t6_reg_gr_df = t6_reg_gr_df.drop(columns=["LIST_SIZE_SUMMED"])

## Create column 'ALT_LIST_SIZE' and leave values blank if there is no value in the corresponding 'ALT_PAT_LIST'
t6_reg_gr_df["ALT_LIST_SIZE"] = np.where(t6_reg_gr_df["ALT_PAT_LIST"] != None
                                         ,(t6_reg_gr_df["ALT_LIST_SIZE_SUMMED"]/t6_reg_gr_df["COUNT_DISTINCT_IND_CODE"])
                                         ,"")

## Data columns that contain data totals with the prefix 'TOTAL'
t6_reg_gr_df = t6_reg_gr_df.rename(columns={ "ACHIEVED_POINTS":"TOTAL_ACH_SCORE"
                                            ,"PCA_COUNT":"TOTAL_PCAS"
                                            ,"ACHIEVEMENT_DENOMINATOR":"TOTAL_DENOMINATORS"
                                            ,"REGISTER_SIZE":"REGISTER"})
                                           
## Select region level columns ordered to be appended to national data.
t6_reg_gr_col = ["GROUP_CODE","REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","NUMBER_PRACTICES"
                 ,"LIST_SIZE","ALT_LIST_SIZE","ALT_LIST_SIZE_SUMMED","COUNT_DISTINCT_IND_CODE","REGISTER"
                 ,"TOTAL_ACH_SCORE","MAX_GROUP_POINTS","TOTAL_PCAS","TOTAL_DENOMINATORS"
                 ,"PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE","PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE"
                 ,"PREV_PREVALENCE","PREV_ACHIEVED_POINTS","PREV_TOTAL_INDICATOR_GROUP_POINTS"
                 ,"PREV_ACHIEVEMENT_RATE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"]                                                                                        
t6_reg_gr_df = t6_reg_gr_df[t6_reg_gr_col]

## Sort data before appending to national data
t6_reg_gr_df = t6_reg_gr_df.sort_values(by = ["REGION_ODS_CODE"]
                                        ,ascending = [True]).reset_index(drop = True)

<h3>Append National and Region data frames

In [ ]:
"""   
Region data is appended to the national data with the geography codes and data counts
After the data is appended Prevalence, Achievement Score, Achievement Percentage and Overall PCA rate
are calculated for both national and region data and comprise part of the final published data frames
in nat-reg-ach-prev-pca.xlsx

"""
t6_nat_reg_base_df = t6_nat_gr_df.append(t6_reg_gr_df, ignore_index=True)

## Change the data type of the column 'ALT_LIST_SIZE'used in the calculations
t6_nat_reg_base_df["ALT_LIST_SIZE"] = t6_nat_reg_base_df["ALT_LIST_SIZE"].astype(float)

## Calculate Prevalence
t6_nat_reg_base_df["PREVALENCE"] = np.where(t6_nat_reg_base_df["ALT_LIST_SIZE"] == 0
                                            ,(t6_nat_reg_base_df["REGISTER"]/t6_nat_reg_base_df["LIST_SIZE"])*100
                                            ,(t6_nat_reg_base_df["REGISTER"]/t6_nat_reg_base_df["ALT_LIST_SIZE"])*100)
                                                  
## Calculate Achievement Score
t6_nat_reg_base_df["AVG_ACH_SCORE"] = t6_nat_reg_base_df["TOTAL_ACH_SCORE"]/t6_nat_reg_base_df["NUMBER_PRACTICES"]

## Calculate Achievement Percentage
t6_nat_reg_base_df["ACH_PC"] = ((t6_nat_reg_base_df["TOTAL_ACH_SCORE"]/t6_nat_reg_base_df["NUMBER_PRACTICES"]
                                 )/t6_nat_reg_base_df["MAX_GROUP_POINTS"])*100

## Calculate Overall PCA rate
t6_nat_reg_base_df["OVERALL_PCA_RATE"] = (t6_nat_reg_base_df["TOTAL_PCAS"]/
                                          (t6_nat_reg_base_df["TOTAL_DENOMINATORS"]+
                                           t6_nat_reg_base_df["TOTAL_PCAS"]))*100

## Reorder columns to include new values
t6_nat_reg_base_col = ["GROUP_CODE","REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME"
                       ,"NUMBER_PRACTICES","LIST_SIZE","ALT_LIST_SIZE","REGISTER"
                       ,"PREVALENCE","TOTAL_ACH_SCORE","AVG_ACH_SCORE","ACH_PC"
                       ,"TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE"
                       ,"PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE","PREV_ALT_PAT_LIST"
                       ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","PREV_ACHIEVED_POINTS"
                       , "PREV_TOTAL_INDICATOR_GROUP_POINTS","PREV_ACHIEVEMENT_RATE"
                       ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"]                                                                                                                                                              
t6_nat_reg_base_df = t6_nat_reg_base_df[t6_nat_reg_base_col]


<h2>Part 3 - Create National and Region Additional List df<br><h3>National

In [ ]:
"""   
Addional practice list sizes 'ADD_PAT_SIZE' with the accompanying description column 'ADD_PAT_LIST'
are created at national and region level by indicator group and are joined to the data frame created in Part 2

"""

## National data frame defined this forms the foundation (base) of the data frame to be created
t6_nat_add_list_col = ["NAT_CODE","GROUP_CODE","ADD_PAT_LIST","ADD_LIST_SIZE","INDICATOR_CODE"]
t6_nat_add_list_base_df = t6_nat_current_previous_df[t6_nat_add_list_col]

In [ ]:
## Count distinct number of indicators 
t6_nat_add_list_ind_col = ["NAT_CODE","INDICATOR_CODE","GROUP_CODE"]
t6_nat_add_list_ind_df = t6_nat_add_list_base_df[t6_nat_add_list_ind_col]
t6_nat_add_list_ind_df = t6_nat_add_list_ind_df.drop_duplicates()

t6_nat_add_list_ind_df = t6_nat_add_list_ind_df.groupby(by = ["NAT_CODE","GROUP_CODE"]
                                                        ,as_index=False
                                                        ).agg({"INDICATOR_CODE" : [pd.Series.count]})                                                             
t6_nat_add_list_ind_df.columns = t6_nat_add_list_ind_df.columns.droplevel(1) 

t6_nat_add_list_ind_df = t6_nat_add_list_ind_df.rename(columns={"INDICATOR_CODE":"COUNT_DISTINCT_IND_CODE"})


## Sum ADD_LIST_SIZE (additional list size)
t6_nat_add_list_df = t6_nat_add_list_base_df.drop(columns=["INDICATOR_CODE"])
t6_nat_add_list_sum_df = t6_nat_add_list_df.groupby(by = ["NAT_CODE","GROUP_CODE","ADD_PAT_LIST"],
                                                    as_index=False
                                                    )["ADD_LIST_SIZE"].sum()

In [ ]:
## Combine aggregated data frames
t6_nat_add_list_df = pd.merge(t6_nat_add_list_sum_df
                              ,t6_nat_add_list_ind_df
                              ,left_on = ["NAT_CODE","GROUP_CODE"]
                              ,right_on = ["NAT_CODE","GROUP_CODE"]
                              ,how = 'left') 

## Define new column 'ADD_LIST' e.g. ADD_LIST_06OV
t6_nat_add_list_df["ADD_LIST"] = "ADD_LIST_"+t6_nat_add_list_df["ADD_PAT_LIST"]

## Drop unwanted columns
t6_nat_reg_list_col = ["ADD_PAT_LIST","COUNT_DISTINCT_IND_CODE"]
t6_nat_add_list_df = t6_nat_add_list_df.drop(columns=t6_nat_reg_list_col)

## Rename 'NAT_CODE' column as 'REGION_ODS_CODE' to facilitate appending to region data
t6_nat_add_list_df = t6_nat_add_list_df.rename(columns={"NAT_CODE":"REGION_ODS_CODE"})

## Reorder column ready for appending
t6_nat_reg_add_list_col = ["REGION_ODS_CODE","GROUP_CODE","ADD_LIST","ADD_LIST_SIZE"]
t6_nat_add_list_df = t6_nat_add_list_df[t6_nat_reg_add_list_col]

<h3>Region

In [ ]:
## Region data frame defined this forms the foundation (base) of the data frame to be created
t6_reg_add_list_base_col = ["REGION_ODS_CODE","GROUP_CODE","ADD_PAT_LIST"
                            ,"ADD_LIST_SIZE","INDICATOR_CODE"]
t6_reg_add_list_base_df = t6_reg_current_previous_df[t6_reg_add_list_base_col]

In [ ]:
## Count distinct number of indicators
t6_reg_add_list_ind_col =["REGION_ODS_CODE","INDICATOR_CODE","GROUP_CODE"]
t6_reg_add_list_ind_df = t6_reg_add_list_base_df[t6_reg_add_list_ind_col]

t6_reg_add_list_ind_df = t6_reg_add_list_ind_df.drop_duplicates()

t6_reg_add_list_ind_df = t6_reg_add_list_ind_df.groupby(by = ["REGION_ODS_CODE","GROUP_CODE"]
                                                        ,as_index=False
                                                        ).agg({"INDICATOR_CODE" : [pd.Series.count]})                                                              
t6_reg_add_list_ind_df.columns = t6_reg_add_list_ind_df.columns.droplevel(1) 

## Rename column 'INDICATOR_CODE' as 'COUNT_DISTINCT_IND_CODE' which contains counts
t6_reg_add_list_ind_df = t6_reg_add_list_ind_df.rename(columns={"INDICATOR_CODE":"COUNT_DISTINCT_IND_CODE"})

## Sum ADD_LIST_SIZE (additional list size)
t6_reg_add_list_df = t6_reg_add_list_base_df.drop(columns=["INDICATOR_CODE"])
t6_reg_add_list_sum_df = t6_reg_add_list_df.groupby(by = ["REGION_ODS_CODE","GROUP_CODE","ADD_PAT_LIST"]
                                                    ,as_index=False
                                                    )["ADD_LIST_SIZE"].sum()

In [ ]:
## Combine aggregated data frames
t6_reg_add_list_df = pd.merge(t6_reg_add_list_sum_df
                              ,t6_reg_add_list_ind_df
                              ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                              ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                              ,how ="left") 

## Define new column 'ADD_LIST' e.g. ADD_LIST_06OV
t6_reg_add_list_df["ADD_LIST"] = "ADD_LIST_"+t6_reg_add_list_df["ADD_PAT_LIST"]

## Drop unwanted columns
t6_reg_add_list_df = t6_reg_add_list_df.drop(columns=t6_nat_reg_list_col)

## Reorder columns ready for appending
t6_reg_add_list_df = t6_reg_add_list_df[t6_nat_reg_add_list_col]

<h3>Append national and region additional list size data frames

In [ ]:
## Append data frame and pivot
t6_nat_reg_add_list_df = t6_nat_add_list_df.append(t6_reg_add_list_df, ignore_index=True)

## Pivot t6_nat_reg_add_list_df from a long thin data frame to a short broad data frame with 
## a column for each additional list size with 'REGION_ODS_CODE, and 'GROUP_CODE' as the index
t6_add_list_df = pd.pivot_table(t6_nat_reg_add_list_df
                                ,values="ADD_LIST_SIZE"
                                ,index=["REGION_ODS_CODE","GROUP_CODE"]
                                ,columns= "ADD_LIST")
t6_add_list_df = t6_add_list_df.reset_index().rename_axis(None, axis=1)

<h3>Join Part 3 (Additional List df) to Part 2 (previous data)

In [ ]:
## Join additional list sizes to main national and region data frame
T6_master_df = pd.merge(t6_nat_reg_base_df
                        ,t6_add_list_df
                        ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                        ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                        ,how = 'left')

## Calculate the difference in the previous year's percentages to the curent years as percentage points
T6_master_df["PREV_CHANGE"] = T6_master_df["PREVALENCE"]-T6_master_df["PREV_PREVALENCE"]
T6_master_df["ACH_CHANGE"] = T6_master_df["ACH_PC"]-T6_master_df["PREV_ACHIEVEMENT_RATE"]
T6_master_df["PCA_CHANGE"] = T6_master_df["OVERALL_PCA_RATE"]-T6_master_df["PREV_PCA_RATE"]

T6_master_df = T6_master_df.drop_duplicates()

Create variable source data

In [ ]:
## Creates a list of 'REGION_ONS_CODE' including national level which is used to 
## create the variable 'nat_reg_count' used in 'Publication Outputs'
nat_reg_list = T6_master_df[["REGION_ONS_CODE"]]
nat_reg_list = nat_reg_list.drop_duplicates().reset_index(drop = True)

<h2>Create national and region data tables <br>and table titles

In [ ]:
"""   
Not all indicator group tables have the same data components. 
Therefore it is necessary to define the content of each indicator group data frame separately. 
The code block are in the indicator group order as they appear in the nat-reg-ach-prev-pca 
excel workbook (template 6)

"""

## Select data for AF group code
t6_af_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "AF"]

t6_af_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="AF")]

## Combines T6 Master table with previous years data
t6_af_df = pd.merge(t6_af_base_df
                    ,t6_af_nat_reg_current_piv_df
                    ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'AF' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_af_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES"
            ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES"
            ,"LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS"
            ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT"
            ,"PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS"
            ,"OVERALL_PCA_RATE","PCA_CHANGE","AF001_ACHIEVED_POINTS","AF001_ACH_RATE"
            ,"AF006_ACHIEVED_POINTS","AF006_ACHIEVEMENT_NUMERATOR","AF006_ACHIEVEMENT_DENOMINATOR"
            ,"AF006_UL_NET_ACH","AF006_PCA_COUNT","AF006_PCA_RATE","AF006_DEN_PLUS_PCA","AF006_PRI"
            ,"AF007_ACHIEVED_POINTS","AF007_ACHIEVEMENT_NUMERATOR","AF007_ACHIEVEMENT_DENOMINATOR"
            ,"AF007_UL_NET_ACH","AF007_PCA_COUNT","AF007_PCA_RATE","AF007_DEN_PLUS_PCA","AF007_PRI"]
t6_af_df = t6_af_df[t6_af_col]                     

In [ ]:
## Select data for BP group code
t6_bp_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "BP"]

t6_bp_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="BP")]

## Combines T6 Master table with previous years data
t6_bp_df = pd.merge(t6_bp_base_df
                    ,t6_bp_nat_reg_current_piv_df
                    ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'BP' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_bp_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
            ,"NUMBER_PRACTICES","ALT_LIST_SIZE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
            ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR"
            ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS" ,"OVERALL_PCA_RATE","PCA_CHANGE"
            ,"BP002_ACHIEVED_POINTS","BP002_ACHIEVEMENT_NUMERATOR","BP002_ACHIEVEMENT_DENOMINATOR"
            ,"BP002_UL_NET_ACH","BP002_PCA_COUNT","BP002_PCA_RATE","BP002_DEN_PLUS_PCA","BP002_PRI"]
t6_bp_df = t6_bp_df[t6_bp_col]               

In [ ]:
## Select data for CHD group code
t6_chd_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "CHD"]

t6_chd_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="CHD")]

## Combines T6 Master table with previous years data
t6_chd_df = pd.merge(t6_chd_base_df
                     ,t6_chd_nat_reg_current_piv_df
                     ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## Reorder columns for output data frame on tab 'CHD' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_chd_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES"
             ,"LIST_SIZE","ADD_LIST_79UN","ADD_LIST_80OV","REGISTER","PREVALENCE","PREV_CHANGE"
             ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
             ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
             ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
             ,"CHD001_ACHIEVED_POINTS","CHD001_ACH_RATE"
             ,"CHD005_ACHIEVED_POINTS","CHD005_ACHIEVEMENT_NUMERATOR","CHD005_ACHIEVEMENT_DENOMINATOR"
             ,"CHD005_UL_NET_ACH","CHD005_PCA_COUNT","CHD005_PCA_RATE","CHD005_DEN_PLUS_PCA","CHD005_PRI"
             ,"CHD008_ACHIEVED_POINTS","CHD008_ACHIEVEMENT_NUMERATOR","CHD008_ACHIEVEMENT_DENOMINATOR"
             ,"CHD008_UL_NET_ACH","CHD008_PCA_COUNT","CHD008_PCA_RATE","CHD008_DEN_PLUS_PCA","CHD008_PRI"
             ,"CHD009_ACHIEVED_POINTS","CHD009_ACHIEVEMENT_NUMERATOR","CHD009_ACHIEVEMENT_DENOMINATOR"
             ,"CHD009_UL_NET_ACH","CHD009_PCA_COUNT","CHD009_PCA_RATE","CHD009_DEN_PLUS_PCA","CHD009_PRI"]                                                                                                             
t6_chd_df = t6_chd_df[t6_chd_col]

In [ ]:
## Select data for HF group code
t6_hf_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "HF"]

t6_hf_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="HF")]

## Combines T6 Master table with previous years data
t6_hf_df = pd.merge(t6_hf_base_df
                    ,t6_hf_nat_reg_current_piv_df
                    ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'HF' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_hf_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
            ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES"
            ,"LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE"
            ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
            ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
            ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
            ,"HF001_ACHIEVED_POINTS","HF001_ACH_RATE"
            ,"HF003_ACHIEVED_POINTS","HF003_ACHIEVEMENT_NUMERATOR","HF003_ACHIEVEMENT_DENOMINATOR"
            ,"HF003_UL_NET_ACH","HF003_PCA_COUNT","HF003_PCA_RATE","HF003_DEN_PLUS_PCA","HF003_PRI"
            ,"HF005_ACHIEVED_POINTS","HF005_ACHIEVEMENT_NUMERATOR","HF005_ACHIEVEMENT_DENOMINATOR"
            ,"HF005_UL_NET_ACH","HF005_PCA_COUNT","HF005_PCA_RATE","HF005_DEN_PLUS_PCA","HF005_PRI"
            ,"HF006_ACHIEVED_POINTS","HF006_ACHIEVEMENT_NUMERATOR","HF006_ACHIEVEMENT_DENOMINATOR"
            ,"HF006_UL_NET_ACH","HF006_PCA_COUNT","HF006_PCA_RATE","HF006_DEN_PLUS_PCA","HF006_PRI"
            ,"HF007_ACHIEVED_POINTS","HF007_ACHIEVEMENT_NUMERATOR","HF007_ACHIEVEMENT_DENOMINATOR"
            ,"HF007_UL_NET_ACH","HF007_PCA_COUNT","HF007_PCA_RATE","HF007_DEN_PLUS_PCA","HF007_PRI"]
t6_hf_df = t6_hf_df[t6_hf_col]

In [ ]:
## Select data for HYP group code
t6_hyp_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "HYP"]

t6_hyp_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="HYP")]

## Combines T6 Master table with previous years data
t6_hyp_df = pd.merge(t6_hyp_base_df
                     ,t6_hyp_nat_reg_current_piv_df
                     ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## Reorder columns for output data frame on tab 'HYP' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_hyp_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES"
             ,"LIST_SIZE","ADD_LIST_79UN","ADD_LIST_80OV","REGISTER","PREVALENCE","PREV_CHANGE"
             ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
             ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
             ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
             ,"HYP001_ACHIEVED_POINTS","HYP001_ACH_RATE"
             ,"HYP003_ACHIEVED_POINTS","HYP003_ACHIEVEMENT_NUMERATOR","HYP003_ACHIEVEMENT_DENOMINATOR"
             ,"HYP003_UL_NET_ACH","HYP003_PCA_COUNT","HYP003_PCA_RATE","HYP003_DEN_PLUS_PCA","HYP003_PRI"
             ,"HYP007_ACHIEVED_POINTS","HYP007_ACHIEVEMENT_NUMERATOR","HYP007_ACHIEVEMENT_DENOMINATOR"
             ,"HYP007_UL_NET_ACH","HYP007_PCA_COUNT","HYP007_PCA_RATE","HYP007_DEN_PLUS_PCA","HYP007_PRI"]                     
t6_hyp_df = t6_hyp_df[t6_hyp_col]

In [ ]:
## Select data for PAD group code
t6_pad_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "PAD"]

t6_pad_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="PAD")]

## Combines T6 Master table with previous years data
t6_pad_df = pd.merge(t6_pad_base_df
                     ,t6_pad_nat_reg_current_piv_df
                     ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## Reorder columns for output data frame on tab 'PAD' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_pad_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES"
             ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE"
             ,"NUMBER_PRACTICES","LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE"
             ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
             ,"PAD001_ACHIEVED_POINTS","PAD001_ACH_RATE"]                      
t6_pad_df = t6_pad_df[t6_pad_col]

In [ ]:
## Select data for STIA group code
t6_stia_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "STIA"]

t6_stia_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="STIA")]

## Combines T6 Master table with previous years data
t6_stia_df = pd.merge(t6_stia_base_df
                      ,t6_stia_nat_reg_current_piv_df
                      ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                      ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                      ,how ="left")

## Reorder columns for output data frame on tab 'STIA' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_stia_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES"
              ,"LIST_SIZE","ADD_LIST_79UN","ADD_LIST_80OV","REGISTER","PREVALENCE","PREV_CHANGE"
              ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
              ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
              ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
              ,"STIA001_ACHIEVED_POINTS","STIA001_ACH_RATE"                         
              ,"STIA007_ACHIEVED_POINTS","STIA007_ACHIEVEMENT_NUMERATOR","STIA007_ACHIEVEMENT_DENOMINATOR"
              ,"STIA007_UL_NET_ACH","STIA007_PCA_COUNT","STIA007_PCA_RATE","STIA007_DEN_PLUS_PCA","STIA007_PRI"
              ,"STIA010_ACHIEVED_POINTS","STIA010_ACHIEVEMENT_NUMERATOR","STIA010_ACHIEVEMENT_DENOMINATOR"
              ,"STIA010_UL_NET_ACH","STIA010_PCA_COUNT","STIA010_PCA_RATE","STIA010_DEN_PLUS_PCA","STIA010_PRI"
              ,"STIA011_ACHIEVED_POINTS","STIA011_ACHIEVEMENT_NUMERATOR","STIA011_ACHIEVEMENT_DENOMINATOR"
              ,"STIA011_UL_NET_ACH","STIA011_PCA_COUNT","STIA011_PCA_RATE","STIA011_DEN_PLUS_PCA","STIA011_PRI"]                        
t6_stia_df = t6_stia_df[t6_stia_col]

In [ ]:
## Select data for AST group code
t6_ast_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "AST"]

t6_ast_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="AST")]

## Combines T6 Master table with previous years data
t6_ast_df = pd.merge(t6_ast_base_df
                     ,t6_ast_nat_reg_current_piv_df
                     ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## Reorder columns for output data frame on tab 'AST' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_ast_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES"
             ,"ALT_LIST_SIZE","ADD_LIST_06_19","REGISTER","PREVALENCE","PREV_CHANGE"
             ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
             ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
             ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
             ,"AST005_ACHIEVED_POINTS","AST005_ACH_RATE"
             ,"AST006_ACHIEVED_POINTS","AST006_ACHIEVEMENT_NUMERATOR","AST006_ACHIEVEMENT_DENOMINATOR"
             ,"AST006_UL_NET_ACH","AST006_PCA_COUNT","AST006_PCA_RATE","AST006_DEN_PLUS_PCA","AST006_PRI"
             ,"AST007_ACHIEVED_POINTS","AST007_ACHIEVEMENT_NUMERATOR","AST007_ACHIEVEMENT_DENOMINATOR"
             ,"AST007_UL_NET_ACH","AST007_PCA_COUNT","AST007_PCA_RATE","AST007_DEN_PLUS_PCA","AST007_PRI"
             ,"AST008_ACHIEVED_POINTS","AST008_ACHIEVEMENT_NUMERATOR","AST008_ACHIEVEMENT_DENOMINATOR"
             ,"AST008_UL_NET_ACH","AST008_PCA_COUNT","AST008_PCA_RATE","AST008_DEN_PLUS_PCA","AST008_PRI"]                                            
t6_ast_df = t6_ast_df[t6_ast_col]

In [ ]:
## Select data for COPD group code
t6_copd_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "COPD"]

t6_copd_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="COPD")]

## Combines T6 Master table with previous years data
t6_copd_df = pd.merge(t6_copd_base_df
                      ,t6_copd_nat_reg_current_piv_df
                      ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                      ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                      ,how ="left")

## Reorder columns for output data frame on tab 'COPD' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_copd_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE"
              ,"NUMBER_PRACTICES","LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE"
              ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
              ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
              ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
              ,"COPD009_ACHIEVED_POINTS","COPD009_ACH_RATE"
              ,"COPD008_ACHIEVED_POINTS","COPD008_ACHIEVEMENT_NUMERATOR","COPD008_ACHIEVEMENT_DENOMINATOR"
              ,"COPD008_UL_NET_ACH","COPD008_PCA_COUNT","COPD008_PCA_RATE","COPD008_DEN_PLUS_PCA","COPD008_PRI"
              ,"COPD010_ACHIEVED_POINTS","COPD010_ACHIEVEMENT_NUMERATOR","COPD010_ACHIEVEMENT_DENOMINATOR"
              ,"COPD010_UL_NET_ACH","COPD010_PCA_COUNT","COPD010_PCA_RATE","COPD010_DEN_PLUS_PCA","COPD010_PRI"]                                                                                                                                                                                                     
t6_copd_df = t6_copd_df[t6_copd_col]

In [ ]:
## Select data for OB group code
t6_ob_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "OB"]

t6_ob_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="OB")]

## Combines T6 Master table with previous years data
t6_ob_df = pd.merge(t6_ob_base_df
                    ,t6_ob_nat_reg_current_piv_df
                    ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'OB' in nat-reg-ach-prev-pca excel workbook (template 6) 
t6_ob_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE","REGISTER"
             ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE"
             ,"ACH_PC","ACH_CHANGE"
             ,"OB002_ACHIEVED_POINTS","OB002_ACH_RATE"]                                                                                                
t6_ob_df = t6_ob_df[t6_ob_col]

In [ ]:
## Select data for SMOK group code
t6_smok_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "SMOK"]

t6_smok_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="SMOK")]

## Combines T6 Master table with previous years data
t6_smok_df = pd.merge(t6_smok_base_df
                      ,t6_smok_nat_reg_current_piv_df
                      ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                      ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                      ,how ="left")

## Reorder columns for output data frame on tab 'SMOK' in nat-reg-ach-prev-pca excel workbook (template 6) 
t6_smok_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
               ,"NUMBER_PRACTICES","ALT_LIST_SIZE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
               ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"
               ,"TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
               ,"SMOK002_ACHIEVED_POINTS","SMOK002_ACHIEVEMENT_NUMERATOR","SMOK002_ACHIEVEMENT_DENOMINATOR"
               ,"SMOK002_UL_NET_ACH","SMOK002_PCA_COUNT","SMOK002_PCA_RATE","SMOK002_DEN_PLUS_PCA","SMOK002_PRI"
               ,"SMOK004_ACHIEVED_POINTS","SMOK004_ACHIEVEMENT_NUMERATOR","SMOK004_ACHIEVEMENT_DENOMINATOR"
               ,"SMOK004_UL_NET_ACH","SMOK004_PCA_COUNT","SMOK004_PCA_RATE","SMOK004_DEN_PLUS_PCA","SMOK004_PRI"
               ,"SMOK005_ACHIEVED_POINTS","SMOK005_ACHIEVEMENT_NUMERATOR","SMOK005_ACHIEVEMENT_DENOMINATOR"
               ,"SMOK005_UL_NET_ACH","SMOK005_PCA_COUNT","SMOK005_PCA_RATE","SMOK005_DEN_PLUS_PCA","SMOK005_PRI"]                                                                                                                                                                                                 
t6_smok_df = t6_smok_df[t6_smok_col]

In [ ]:
## Select data for CAN group code
t6_can_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "CAN"]

t6_can_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="CAN")]

## Combines T6 Master table with previous years data
t6_can_df = pd.merge(t6_can_base_df,t6_can_nat_reg_current_piv_df
                     ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## Reorder columns for output data frame on tab 'CAN' in nat-reg-ach-prev-pca excel workbook (template 6) 
t6_can_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES"
             ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE"
             ,"NUMBER_PRACTICES","LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE"
             ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
             ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
             ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
             ,"CAN001_ACHIEVED_POINTS","CAN001_ACH_RATE"
             ,"CAN004_ACHIEVED_POINTS","CAN004_ACHIEVEMENT_NUMERATOR","CAN004_ACHIEVEMENT_DENOMINATOR"
             ,"CAN004_UL_NET_ACH","CAN004_PCA_COUNT","CAN004_PCA_RATE","CAN004_DEN_PLUS_PCA","CAN004_PRI"
             ,"CAN005_ACHIEVED_POINTS","CAN005_ACHIEVEMENT_NUMERATOR","CAN005_ACHIEVEMENT_DENOMINATOR"
             ,"CAN005_UL_NET_ACH","CAN005_PCA_COUNT","CAN005_PCA_RATE","CAN005_DEN_PLUS_PCA","CAN005_PRI"]                                                                                                                                                          
t6_can_df = t6_can_df[t6_can_col]

In [ ]:
## Select data for CKD group code
t6_ckd_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "CKD"]

t6_ckd_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="CKD")]

## Combines T6 Master table with previous years data
t6_ckd_df = pd.merge(t6_ckd_base_df
                     ,t6_ckd_nat_reg_current_piv_df
                     ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## Reorder columns for output data frame on tab 'CKD' in nat-reg-ach-prev-pca excel workbook (template 6) 
t6_ckd_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES"
             ,"PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE"
             ,"NUMBER_PRACTICES","ALT_LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE"
             ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC"
             ,"ACH_CHANGE"
             ,"CKD005_ACHIEVED_POINTS","CKD005_ACH_RATE"]                                                                                        
t6_ckd_df = t6_ckd_df[t6_ckd_col]

In [ ]:
## Select data for DM group code
t6_dm_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "DM"]

t6_dm_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="DM")]

## Combines T6 Master table with previous years data
t6_dm_df = pd.merge(t6_dm_base_df
                    ,t6_dm_nat_reg_current_piv_df
                    ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'DM' in nat-reg-ach-prev-pca excel workbook (template 6) 
t6_dm_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES"
            ,"PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE"
            ,"NUMBER_PRACTICES","ALT_LIST_SIZE","ADD_LIST_40OV","REGISTER","PREVALENCE"
            ,"PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC"
            ,"ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
            ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
            ,"DM017_ACHIEVED_POINTS","DM017_ACH_RATE"
            ,"DM006_ACHIEVED_POINTS","DM006_ACHIEVEMENT_NUMERATOR","DM006_ACHIEVEMENT_DENOMINATOR"
            ,"DM006_UL_NET_ACH","DM006_PCA_COUNT","DM006_PCA_RATE","DM006_DEN_PLUS_PCA","DM006_PRI"
            ,"DM012_ACHIEVED_POINTS","DM012_ACHIEVEMENT_NUMERATOR","DM012_ACHIEVEMENT_DENOMINATOR"
            ,"DM012_UL_NET_ACH","DM012_PCA_COUNT","DM012_PCA_RATE","DM012_DEN_PLUS_PCA","DM012_PRI"
            ,"DM014_ACHIEVED_POINTS","DM014_ACHIEVEMENT_NUMERATOR","DM014_ACHIEVEMENT_DENOMINATOR"
            ,"DM014_UL_NET_ACH","DM014_PCA_COUNT","DM014_PCA_RATE","DM014_DEN_PLUS_PCA","DM014_PRI"
            ,"DM019_ACHIEVED_POINTS","DM019_ACHIEVEMENT_NUMERATOR","DM019_ACHIEVEMENT_DENOMINATOR"
            ,"DM019_UL_NET_ACH","DM019_PCA_COUNT","DM019_PCA_RATE","DM019_DEN_PLUS_PCA","DM019_PRI"
            ,"DM020_ACHIEVED_POINTS","DM020_ACHIEVEMENT_NUMERATOR","DM020_ACHIEVEMENT_DENOMINATOR"
            ,"DM020_UL_NET_ACH","DM020_PCA_COUNT","DM020_PCA_RATE","DM020_DEN_PLUS_PCA","DM020_PRI"
            ,"DM021_ACHIEVED_POINTS","DM021_ACHIEVEMENT_NUMERATOR","DM021_ACHIEVEMENT_DENOMINATOR"
            ,"DM021_UL_NET_ACH","DM021_PCA_COUNT","DM021_PCA_RATE","DM021_DEN_PLUS_PCA","DM021_PRI"
            ,"DM022_ACHIEVED_POINTS","DM022_ACHIEVEMENT_NUMERATOR","DM022_ACHIEVEMENT_DENOMINATOR"
            ,"DM022_UL_NET_ACH","DM022_PCA_COUNT","DM022_PCA_RATE","DM022_DEN_PLUS_PCA","DM022_PRI"
            ,"DM023_ACHIEVED_POINTS","DM023_ACHIEVEMENT_NUMERATOR","DM023_ACHIEVEMENT_DENOMINATOR"
            ,"DM023_UL_NET_ACH","DM023_PCA_COUNT","DM023_PCA_RATE","DM023_DEN_PLUS_PCA","DM023_PRI"]                                                                                                                                                                                                                                                                       
t6_dm_df = t6_dm_df[t6_dm_col]

In [ ]:
## Select data for NDH group code
t6_ndh_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "NDH"]

## As NDH had no achievement points allocated in the previous year, it had no achievement rate for comparison with this year
## Therefore the missing value in the columns 'Achievement (%)' and  'Year on year change' has been replaced with an 'z' for not applicable
t6_ndh_base_df["PREV_ACHIEVEMENT_RATE"] = t6_ndh_base_df["PREV_ACHIEVEMENT_RATE"].fillna("z")
t6_ndh_base_df["ACH_CHANGE"] = t6_ndh_base_df["ACH_CHANGE"].fillna("z")

t6_ndh_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="NDH")]

## Combines T6 Master table with previous years data
t6_ndh_df = pd.merge(t6_ndh_base_df
                     ,t6_ndh_nat_reg_current_piv_df
                     ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## Reorder columns for output data frame on tab 'NDH' in nat-reg-ach-prev-pca excel workbook (template 6) 
t6_ndh_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME", "PREV_NUMBER_PRACTICES"
             ,"PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE"
             ,"NUMBER_PRACTICES","ALT_LIST_SIZE","REGISTER","PREVALENCE"
             ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
             ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"
             ,"TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
             ,"NDH001_ACHIEVED_POINTS","NDH001_ACHIEVEMENT_NUMERATOR","NDH001_ACHIEVEMENT_DENOMINATOR"
             ,"NDH001_UL_NET_ACH","NDH001_PCA_COUNT","NDH001_PCA_RATE","NDH001_DEN_PLUS_PCA","NDH001_PRI"]                    
t6_ndh_df = t6_ndh_df[t6_ndh_col]

In [ ]:
## Select data for PC group code
t6_pc_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "PC"]

t6_pc_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="PC")]

## Combines T6 Master table with previous years data
t6_pc_df = pd.merge(t6_pc_base_df
                    ,t6_pc_nat_reg_current_piv_df
                    ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'PC' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_pc_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES"
            ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE"
            ,"NUMBER_PRACTICES","LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE"
            ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
            ,"PC001_ACHIEVED_POINTS","PC001_ACH_RATE"]
t6_pc_df = t6_pc_df[t6_pc_col]

In [ ]:
## Select data for DEM group code
t6_dem_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "DEM"]

t6_dem_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="DEM")]

## Combines T6 Master table with previous years data
t6_dem_df = pd.merge(t6_dem_base_df
                     ,t6_dem_nat_reg_current_piv_df
                     ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## Reorder columns for output data frame on tab 'DEM' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_dem_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES"
             ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE"
             ,"NUMBER_PRACTICES","LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE"
             ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
             ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS"
             ,"OVERALL_PCA_RATE","PCA_CHANGE"
             ,"DEM001_ACHIEVED_POINTS","DEM001_ACH_RATE"
             ,"DEM004_ACHIEVED_POINTS","DEM004_ACHIEVEMENT_NUMERATOR","DEM004_ACHIEVEMENT_DENOMINATOR"
             ,"DEM004_UL_NET_ACH","DEM004_PCA_COUNT","DEM004_PCA_RATE","DEM004_DEN_PLUS_PCA","DEM004_PRI"]                                          
t6_dem_df = t6_dem_df[t6_dem_col]

In [ ]:
## Select data for DEP group code
t6_dep_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "DEP"]

t6_dep_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="DEP")]

## Combines T6 Master table with previous years data
t6_dep_df = pd.merge(t6_dep_base_df
                     ,t6_dep_nat_reg_current_piv_df
                     ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## Reorder columns for output data frame on tab 'DEP' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_dep_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES"
             ,"PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE"
             ,"NUMBER_PRACTICES","ALT_LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE"
             ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
             ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS"
             ,"OVERALL_PCA_RATE","PCA_CHANGE"
             ,"DEP003_ACHIEVED_POINTS","DEP003_ACHIEVEMENT_NUMERATOR","DEP003_ACHIEVEMENT_DENOMINATOR"
             ,"DEP003_UL_NET_ACH","DEP003_PCA_COUNT","DEP003_PCA_RATE","DEP003_DEN_PLUS_PCA","DEP003_PRI"]                 
t6_dep_df = t6_dep_df[t6_dep_col]

In [ ]:
## Select data for EP group code
t6_ep_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "EP"]

t6_ep_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="EP")]

## Combines T6 Master table with previous years data
t6_ep_df = pd.merge(t6_ep_base_df
                    ,t6_ep_nat_reg_current_piv_df
                    ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'EP' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_ep_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES"
            ,"PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE"
            ,"NUMBER_PRACTICES","ALT_LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE"
            ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
            ,"EP001_ACHIEVED_POINTS","EP001_ACH_RATE"]                     
t6_ep_df = t6_ep_df[t6_ep_col]

In [ ]:
## Select data for LD group code
t6_ld_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "LD"]

t6_ld_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="LD")]

## Combines T6 Master table with previous years data
t6_ld_df = pd.merge(t6_ld_base_df
                    ,t6_ld_nat_reg_current_piv_df
                    ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'LD' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_ld_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES"
            ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE"
            ,"NUMBER_PRACTICES","LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE"
            ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
            ,"LD004_ACHIEVED_POINTS","LD004_ACH_RATE"]                    
t6_ld_df = t6_ld_df[t6_ld_col]

In [ ]:
## Select data for MH group code
t6_mh_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "MH"]

t6_mh_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="MH")]

## Combines T6 Master table with previous years data
t6_mh_df = pd.merge(t6_mh_base_df
                    ,t6_mh_nat_reg_current_piv_df
                    ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'MH' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_mh_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES"
             ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE"
             , "NUMBER_PRACTICES","LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE"
             , "PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
             ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
             ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
             ,"MH001_ACHIEVED_POINTS","MH001_ACH_RATE"
             ,"MH002_ACHIEVED_POINTS","MH002_ACHIEVEMENT_NUMERATOR","MH002_ACHIEVEMENT_DENOMINATOR"
             ,"MH002_UL_NET_ACH","MH002_PCA_COUNT","MH002_PCA_RATE","MH002_DEN_PLUS_PCA","MH002_PRI"
             ,"MH003_ACHIEVED_POINTS","MH003_ACHIEVEMENT_NUMERATOR","MH003_ACHIEVEMENT_DENOMINATOR"
             ,"MH003_UL_NET_ACH","MH003_PCA_COUNT","MH003_PCA_RATE","MH003_DEN_PLUS_PCA","MH003_PRI"
             ,"MH006_ACHIEVED_POINTS","MH006_ACHIEVEMENT_NUMERATOR","MH006_ACHIEVEMENT_DENOMINATOR"
             ,"MH006_UL_NET_ACH","MH006_PCA_COUNT","MH006_PCA_RATE","MH006_DEN_PLUS_PCA","MH006_PRI"
             ,"MH007_ACHIEVED_POINTS","MH007_ACHIEVEMENT_NUMERATOR", "MH007_ACHIEVEMENT_DENOMINATOR"
             ,"MH007_UL_NET_ACH","MH007_PCA_COUNT","MH007_PCA_RATE", "MH007_DEN_PLUS_PCA", "MH007_PRI"
             ,"MH011_ACHIEVED_POINTS","MH011_ACHIEVEMENT_NUMERATOR", "MH011_ACHIEVEMENT_DENOMINATOR"
             ,"MH011_UL_NET_ACH","MH011_PCA_COUNT","MH011_PCA_RATE", "MH011_DEN_PLUS_PCA","MH011_PRI"
             ,"MH012_ACHIEVED_POINTS","MH012_ACHIEVEMENT_NUMERATOR", "MH012_ACHIEVEMENT_DENOMINATOR"
             , "MH012_UL_NET_ACH","MH012_PCA_COUNT","MH012_PCA_RATE", "MH012_DEN_PLUS_PCA", "MH012_PRI"]                    
t6_mh_df = t6_mh_df[t6_mh_col]

In [ ]:
## Select data for OST group code
t6_ost_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "OST"]

t6_ost_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="OST")]

## Combines T6 Master table with previous years data
t6_ost_df = pd.merge(t6_ost_base_df
                     ,t6_ost_nat_reg_current_piv_df
                     ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## Reorder columns for output data frame on tab 'OST' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_ost_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES"
             ,"PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE"
             , "NUMBER_PRACTICES","ALT_LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE"
             ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
             ,"OST004_ACHIEVED_POINTS","OST004_ACH_RATE"]                                        
t6_ost_df = t6_ost_df[t6_ost_col]

In [ ]:
## Select data for RA group code
t6_ra_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "RA"]

t6_ra_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="RA")]

## Combines T6 Master table with previous years data
t6_ra_df = pd.merge(t6_ra_base_df
                    ,t6_ra_nat_reg_current_piv_df
                    ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'RA' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_ra_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES"
             ,"PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE"
             ,"NUMBER_PRACTICES","ALT_LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE"
             ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
             ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS"
             ,"OVERALL_PCA_RATE","PCA_CHANGE"
             ,"RA001_ACHIEVED_POINTS","RA001_ACH_RATE"
             ,"RA002_ACHIEVED_POINTS","RA002_ACHIEVEMENT_NUMERATOR","RA002_ACHIEVEMENT_DENOMINATOR"
             ,"RA002_UL_NET_ACH","RA002_PCA_COUNT","RA002_PCA_RATE","RA002_DEN_PLUS_PCA","RA002_PRI"]                 
t6_ra_df = t6_ra_df[t6_ra_col]

In [ ]:
## Select data for CS group code
t6_cs_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "CS"]

t6_cs_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="CS")]

## Combines T6 Master table with previous years data
t6_cs_df = pd.merge(t6_cs_base_df
                    ,t6_cs_nat_reg_current_piv_df
                    ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'CS' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_cs_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
            ,"NUMBER_PRACTICES","ADD_LIST_25_49_F","ADD_LIST_50_64_F"
            ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
            ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS"
            ,"OVERALL_PCA_RATE","PCA_CHANGE"                       
            ,"CS005_ACHIEVED_POINTS","CS005_ACHIEVEMENT_NUMERATOR","CS005_ACHIEVEMENT_DENOMINATOR"
            ,"CS005_UL_NET_ACH","CS005_PCA_COUNT","CS005_PCA_RATE","CS005_DEN_PLUS_PCA","CS005_PRI"
            ,"CS006_ACHIEVED_POINTS","CS006_ACHIEVEMENT_NUMERATOR","CS006_ACHIEVEMENT_DENOMINATOR"
            ,"CS006_UL_NET_ACH","CS006_PCA_COUNT","CS006_PCA_RATE","CS006_DEN_PLUS_PCA","CS006_PRI"]
t6_cs_df = t6_cs_df[t6_cs_col]                 

In [ ]:
## Select data for VI group code
t6_vi_base_df = T6_master_df[T6_master_df["GROUP_CODE"] == "VI"]

t6_vi_nat_reg_current_piv_df = t6_nat_reg_current_piv_df.loc[(t6_nat_reg_current_piv_df["GROUP_CODE"]=="VI")]

## Combines T6 Master table with previous years data
t6_vi_df = pd.merge(t6_vi_base_df
                    ,t6_vi_nat_reg_current_piv_df
                    ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'VI' in nat-reg-ach-prev-pca excel workbook (template 6)
t6_vi_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME","PREV_NUMBER_PRACTICES"
             ,"NUMBER_PRACTICES","ADD_LIST_UN01", "ADD_LIST_1_2", "ADD_LIST_4_5", "ADD_LIST_79_80"
             ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
             ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS"
             ,"OVERALL_PCA_RATE","PCA_CHANGE"
             ,"VI001_ACHIEVED_POINTS","VI001_ACHIEVEMENT_NUMERATOR","VI001_ACHIEVEMENT_DENOMINATOR"
             ,"VI001_UL_NET_ACH","VI001_PCA_COUNT","VI001_PCA_RATE","VI001_DEN_PLUS_PCA","VI001_PRI"
             ,"VI002_ACHIEVED_POINTS","VI002_ACHIEVEMENT_NUMERATOR","VI002_ACHIEVEMENT_DENOMINATOR"
             ,"VI002_UL_NET_ACH","VI002_PCA_COUNT","VI002_PCA_RATE","VI002_DEN_PLUS_PCA","VI002_PRI"
             ,"VI003_ACHIEVED_POINTS","VI003_ACHIEVEMENT_NUMERATOR","VI003_ACHIEVEMENT_DENOMINATOR"
             ,"VI003_UL_NET_ACH","VI003_PCA_COUNT","VI003_PCA_RATE","VI003_DEN_PLUS_PCA","VI003_PRI"
             ,"VI004_ACHIEVED_POINTS","VI004_ACHIEVEMENT_NUMERATOR","VI004_ACHIEVEMENT_DENOMINATOR"
             ,"VI004_UL_NET_ACH","VI004_PCA_COUNT","VI004_PCA_RATE","VI004_DEN_PLUS_PCA","VI004_PRI"]                      
t6_vi_df = t6_vi_df[t6_vi_col]

In [ ]:
## Join QI group codes with numeric key to the master data frame
## This means that QI codes can be reused year-on-year without having to change the body of the code
## QI data is only published for the current year as the indicators change and are therefore not comparable
t6_qi_master_df = pd.merge(QI_gr_codes_with_key
                           ,T6_master_df
                           ,left_on = ["GROUP_CODE"]
                           ,right_on = ["GROUP_CODE"]
                           ,how ="left")

In [ ]:
## Select numeric key for QIPDD group code
t6_qi1_base_df = t6_qi_master_df[t6_qi_master_df["GrC_KEY"] == "QI_1"]

## Select indicator specific data frame
t6_combined_nat_reg_all_qi1 = t6_nat_reg_current_piv_df

## Combines T6 Master table with indicator data
t6_qi1_df = pd.merge(t6_qi1_base_df
                     ,t6_combined_nat_reg_all_qi1
                     ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## Reorder columns for output data frame on tab 'QIPDD' in 
## nat-reg-ach-prev-pca excel workbook (template 6)
t6_qi1_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME"
             ,"NUMBER_PRACTICES","LIST_SIZE"
             ,"TOTAL_ACH_SCORE","ACH_PC"
             ,"QIPDD009_ACHIEVED_POINTS","QIPDD009_ACH_RATE"
             ,"QIPDD010_ACHIEVED_POINTS","QIPDD010_ACH_RATE"]                          
t6_qi1_df = t6_qi1_df[t6_qi1_col]

In [ ]:
## Select numeric key for QIOA group code
t6_qi2_base_df = t6_qi_master_df[t6_qi_master_df["GrC_KEY"] == "QI_2"]

## Select indicator specific data frame
t6_combined_nat_reg_all_qi2 = t6_nat_reg_current_piv_df

## Combines T6 Master table with indicator data
t6_qi2_df = pd.merge(t6_qi2_base_df
                     ,t6_combined_nat_reg_all_qi2
                     ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## Reorder columns for output data frame on tab 'QIOA' in 
## nat-reg-ach-prev-pca excel workbook (template 6)
t6_qi2_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME"
             ,"NUMBER_PRACTICES","LIST_SIZE"
             ,"TOTAL_ACH_SCORE","ACH_PC" 
             ,"QIOA011_ACHIEVED_POINTS","QIOA011_ACH_RATE"
             ,"QIOA012_ACHIEVED_POINTS","QIOA012_ACH_RATE"]                         
t6_qi2_df = t6_qi2_df[t6_qi2_col]

In [ ]:
## Select data for QIPDD_MI group code
t6_qi3_base_df = t6_qi_master_df[t6_qi_master_df["GrC_KEY"] == "QI_3"]

## Select indicator specific data frame
t6_combined_nat_reg_all_qi3 = t6_nat_reg_current_piv_df

## Combines T6 Master table with indicator data
t6_qi3_df = pd.merge(t6_qi3_base_df
                     ,t6_combined_nat_reg_all_qi3
                     ,left_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["REGION_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## This indicator has zero achievement points attributed to it 
## and cannot therefore calculate an achievement percentage
## The value is replaced with 'z' for data not applicable
t6_qi3_df["ACH_PC"] = t6_qi3_df["ACH_PC"].fillna("z")

## Reorder columns for output data frame on tab 'QIPDD_MI' in 
## nat-reg-ach-prev-pca excel workbook (template 6)
t6_qi3_col = ["REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME"
             ,"NUMBER_PRACTICES","ALT_LIST_SIZE"
             ,"TOTAL_ACH_SCORE","ACH_PC"
             ,"TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE"
             ,"QIPDDMI001_ACHIEVED_POINTS","QIPDDMI001_ACHIEVEMENT_NUMERATOR","QIPDDMI001_ACHIEVEMENT_DENOMINATOR"
             ,"QIPDDMI001_UL_NET_ACH","QIPDDMI001_PCA_COUNT","QIPDDMI001_PCA_RATE","QIPDDMI001_DEN_PLUS_PCA","QIPDDMI001_PRI"
             ,"QIPDDMI002_ACHIEVED_POINTS","QIPDDMI002_ACHIEVEMENT_NUMERATOR","QIPDDMI002_ACHIEVEMENT_DENOMINATOR"
             ,"QIPDDMI002_UL_NET_ACH","QIPDDMI002_PCA_COUNT","QIPDDMI002_PCA_RATE","QIPDDMI002_DEN_PLUS_PCA","QIPDDMI002_PRI"
             ,"QIPDDMI003_ACHIEVED_POINTS","QIPDDMI003_ACHIEVEMENT_NUMERATOR","QIPDDMI003_ACHIEVEMENT_DENOMINATOR"
             ,"QIPDDMI003_UL_NET_ACH","QIPDDMI003_PCA_COUNT","QIPDDMI003_PCA_RATE","QIPDDMI003_DEN_PLUS_PCA","QIPDDMI003_PRI"
             ,"QIPDDMI004_ACHIEVED_POINTS","QIPDDMI004_ACHIEVEMENT_NUMERATOR","QIPDDMI004_ACHIEVEMENT_DENOMINATOR"
             ,"QIPDDMI004_UL_NET_ACH","QIPDDMI004_PCA_COUNT","QIPDDMI004_PCA_RATE","QIPDDMI004_DEN_PLUS_PCA","QIPDDMI004_PRI"]                        
t6_qi3_df = t6_qi3_df[t6_qi3_col]


<h3>Create variables used in table titles, used by National and Region, <br>ICB, SUB_ICB and Practice tables

In [ ]:
""" 
Group excel table title variations

""" 

gr_pca_register_flag = ["GROUP_CODE","REGISTER_FLAG","PCAS"]
gr_table_variations = Indicator_Reference_Control_Table[gr_pca_register_flag].fillna(0)

gr_table_variations["TITLE_FLAG"] = gr_table_variations["REGISTER_FLAG"] + gr_table_variations["PCAS"]

gr_table_title_flag = gr_table_variations.groupby(by = ["GROUP_CODE"],as_index=False)["TITLE_FLAG"].max()

gr_table_titles = pd.merge(gr_table_variations
                              ,gr_table_title_flag
                              ,left_on = ["GROUP_CODE" ,"TITLE_FLAG"]
                              ,right_on = ["GROUP_CODE","TITLE_FLAG"]
                              ,how="inner"
                              ).drop_duplicates()

gr_table_titles.loc[:,"TITLE_TEXT"] =""

gr_table_titles["TITLE_TEXT"] = np.where((gr_table_titles["TITLE_FLAG"] == 2.0)
                                         ,"Prevalence, achievement and personalised care adjustments, "
                                         ,gr_table_titles["TITLE_TEXT"])

gr_table_titles["TITLE_TEXT"] = np.where((gr_table_titles["TITLE_FLAG"] == 1.0)&(gr_table_titles["REGISTER_FLAG"] == 1.0)
                                         ,"Prevalence and achievement, "
                                         ,gr_table_titles["TITLE_TEXT"])

gr_table_titles["TITLE_TEXT"] = np.where((gr_table_titles["TITLE_FLAG"] == 1.0)&(gr_table_titles["PCAS"] == 1.0)
                                         ,"Achievement and personalised care adjustments, "
                                         ,gr_table_titles["TITLE_TEXT"])

gr_table_titles["TITLE_TEXT"] = np.where((gr_table_titles["TITLE_FLAG"] == 0.0)
                                         ,"Achievement, "
                                         ,gr_table_titles["TITLE_TEXT"])

gr_table_titles = gr_table_titles.drop(columns = ["REGISTER_FLAG","PCAS","TITLE_FLAG"])

In [ ]:
## Reduced the number of Group Reference table columns - used by National and Region, 
## ICB (STP), Sub ICB (CCG) and Practice tables
gr_base_col = ["PUB_ORDER","GROUP_CODE","GROUP_DESCRIPTION","HIGHER_GROUP_CODE"
               ,"HIGHER_GROUP_DESCRIPTION"]
gr_base_df = gr_ref_table_df[gr_base_col]

gr_base_df = pd.merge(gr_base_df
                     ,grp_change_flag
                     ,left_on = ["GROUP_CODE"]
                     ,right_on = ["GROUP_CODE"]
                     ,how ="left")


gr_base_df = gr_base_df.drop_duplicates()
gr_base_df = gr_base_df.sort_values(by = ["PUB_ORDER", "GROUP_CODE"]
                                    ,ascending = [True, True]).reset_index(drop = True)
                                         
gr_message_df = pd.merge(gr_base_df
                         ,gr_table_titles
                         ,left_on = ["GROUP_CODE"]
                         ,right_on = ["GROUP_CODE"]
                         ,how ="left").reset_index()

In [ ]:
## Table Number - used by National, Region, ICB (STP), and Sub ICB (CCG) tables
gr_message_df = gr_message_df.rename(columns={"index":"TABLE_NO"})
gr_message_df["TABLE_NO"] = gr_message_df["TABLE_NO"]+1

gr_message_df = gr_message_df.drop(columns=["HIGHER_GROUP_CODE","INDICATOR_GROUP_CHANGE_NOTE"
                                            ,"NOTE_TYPE","MESSAGE"])

<h3>Create T6 National and Region Table titles

In [ ]:
## Combine elements that contribute to table title
gr_nat_reg_table_titles = gr_message_df
gr_nat_reg_table_titles["TABLE_NO"] = gr_nat_reg_table_titles["TABLE_NO"].astype(str)
gr_nat_reg_table_titles["FYEAR"] = Control_File.at[0, "FYEAR"]
gr_nat_reg_table_titles["HIGHER_GROUP_DESCRIPTION"] = gr_nat_reg_table_titles["HIGHER_GROUP_DESCRIPTION"].str.lower()
gr_nat_reg_table_titles["GROUP_DESCRIPTION"] = gr_nat_reg_table_titles["GROUP_DESCRIPTION"].str.lower()


gr_nat_reg_table_titles["TABLE_TITLE"] = "Table "+gr_nat_reg_table_titles["TABLE_NO"]\
                                            +": "+gr_nat_reg_table_titles["TITLE_TEXT"]\
                                                +gr_nat_reg_table_titles["HIGHER_GROUP_DESCRIPTION"] \
                                            +" group, "+gr_nat_reg_table_titles["GROUP_DESCRIPTION"]\
                                                +", "+gr_nat_reg_table_titles["FYEAR"]+", national and region level"

gr_nat_reg_table_titles = pd.merge(gr_nat_reg_table_titles
                                   ,QI_gr_codes_with_key
                                   ,left_on = ["GROUP_CODE"]
                                   ,right_on = ["GROUP_CODE"]
                                   ,how ="left")

gr_nat_reg_table_titles["GrC_KEY"] = gr_nat_reg_table_titles["GrC_KEY"].replace(np.nan, "")

gr_nat_reg_table_titles["GROUP_CODE"] = np.where(gr_nat_reg_table_titles["GrC_KEY"]== ""
                                                 ,gr_nat_reg_table_titles["GROUP_CODE"]
                                                 ,gr_nat_reg_table_titles["GrC_KEY"])

gr_nat_reg_table_titles = gr_nat_reg_table_titles.drop(columns=["FYEAR","HIGHER_GROUP_DESCRIPTION"
                                                                ,"GROUP_DESCRIPTION","TABLE_NO","GrC_KEY"])
nat_reg_table_titles = gr_nat_reg_table_titles

<b><h1>20 OUTPUTS - T8-PrevAchPCA_icb</h1></b><br><h2>Part 1 - QOF_MASTER for current Indicators<br>ICB (STP) data frame from QOF Master

In [ ]:
"""   
Column names with old nhs geography name STP are used and relaced by the new 
nhs geography name ICB when the data is output

"""

## Required indicator and achievement columns from QOF Master table
t8_stp_current_base_col = ["STP_ODS_CODE","GROUP_CODE","INDICATOR_CODE","ACHIEVED_POINTS","MAX_INDICATOR_POINTS"
                           ,"ACHIEVEMENT_NUMERATOR","ACHIEVEMENT_DENOMINATOR","PCA_COUNT"]
t8_stp_current_base_df = qof_master[t8_stp_current_base_col]

## Sum all non-index columns
t8_stp_current_sum_df = t8_stp_current_base_df.groupby(by = ["STP_ODS_CODE","GROUP_CODE","INDICATOR_CODE"]
                                                       ,as_index=False
                                                       ).agg({"ACHIEVED_POINTS" : [pd.Series.sum]
                                                             ,"MAX_INDICATOR_POINTS" : [pd.Series.sum]
                                                             ,"ACHIEVEMENT_NUMERATOR" : [pd.Series.sum]
                                                             ,"ACHIEVEMENT_DENOMINATOR" : [pd.Series.sum]
                                                             ,"PCA_COUNT" : [pd.Series.sum]})                                                                                                       
t8_stp_current_sum_df.columns = t8_stp_current_sum_df.columns.droplevel(1)

## Create all percentage columns
## Achievement Rate
t8_stp_current_sum_df["ACH_RATE"] = (t8_stp_current_sum_df["ACHIEVED_POINTS"]/
                                     t8_stp_current_sum_df["MAX_INDICATOR_POINTS"])*100

## Underlying Achievement net of PCAs
t8_stp_current_sum_df["UL_NET_ACH"] = (t8_stp_current_sum_df["ACHIEVEMENT_NUMERATOR"]/
                                       t8_stp_current_sum_df["ACHIEVEMENT_DENOMINATOR"])*100

## PCA Rate
t8_stp_current_sum_df["PCA_RATE"] = (t8_stp_current_sum_df["PCA_COUNT"]/
                                     (t8_stp_current_sum_df["ACHIEVEMENT_DENOMINATOR"]+
                                      t8_stp_current_sum_df["PCA_COUNT"]))*100

## Denominator plus PCAs
t8_stp_current_sum_df["DEN_PLUS_PCA"] = (t8_stp_current_sum_df["ACHIEVEMENT_DENOMINATOR"]+
                                         t8_stp_current_sum_df["PCA_COUNT"])

## Patients Receiving Intervention
t8_stp_current_sum_df["PRI"] = (t8_stp_current_sum_df["ACHIEVEMENT_NUMERATOR"]/
                                (t8_stp_current_sum_df["ACHIEVEMENT_DENOMINATOR"]+
                                 t8_stp_current_sum_df["PCA_COUNT"]))*100


## Drop unwanted column
t8_stp_current_sum_df = t8_stp_current_sum_df.drop(columns=["MAX_INDICATOR_POINTS"])

In [ ]:
## The short broad data frame is converted into a long thin data frame and the non-index columns are 
## stacked in a column called 'FIELD'
t8_stp_current_stacked_df = (t8_stp_current_sum_df.set_index(["STP_ODS_CODE","GROUP_CODE","INDICATOR_CODE"])
                                                  .stack()
                                                  .ffill(axis=0)
                                                  .bfill(axis=0, downcast='infer')
                                                  .reset_index()
                                                  .rename({"level_3":"FIELD", 0:"VALUE"}, axis=1))

## New column 'INDICATOR' is created by joining the 'INDICATOR_CODE' to 'FIELD' 
## with a hyphen e.g. AF001_ACHIEVED_POINTS
t8_stp_current_stacked_df["INDICATOR"] = (t8_stp_current_stacked_df["INDICATOR_CODE"]
                                          )+"_"+(t8_stp_current_stacked_df["FIELD"])

## Drop unwanted columns
t8_stp_current_stacked_col = ["STP_ODS_CODE","GROUP_CODE","INDICATOR","VALUE"]
t8_stp_current_stacked_df = t8_stp_current_stacked_df[t8_stp_current_stacked_col]

In [ ]:
## Pivot data frame from long thin data frame into short broad data frame
## With each column containing an individual elements of each indicator 
## e.g. 'AF001_ACHIEVED_POINTS', 'AF001_ACH_RATE'
t8_stp_current_piv_df = pd.pivot_table(t8_stp_current_stacked_df
                                       ,values="VALUE"                                                       
                                       ,index=["STP_ODS_CODE","GROUP_CODE"]
                                       ,columns= "INDICATOR")

t8_stp_current_piv_df = t8_stp_current_piv_df.reset_index().rename_axis(None, axis=1)

<h2>Part 2 - Join QOF Master to last years ICB (STP) data frame

In [ ]:
"""   
Creates a ICB (STP) level data frame that comprises data for the current year 
and previous year

"""

## Drop unwanted nhs geography columns from the QOF Master table
t8_stp_master_col = qof_master.drop(columns=["NAT_ONS_CODE","NAT_CODE","NAT_COUNTRY"
                                             ,"REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME"
                                             ,"CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME"
                                             ,"PCN_ODS_CODE","PCN_NAME","PRACTICE_NAME"
                                             ,"DOMAIN_CODE","SUB_DOMAIN_CODE"])
                                                   
## Drop unwanted column from the data frame that combines data from the previous year at ICB (STP) level
t8_stp_joined_df = previous_prevalence_stp_data_df.drop(columns=["HIGHER_GROUP_CODE"])

## The resulting data frame is used as the foundation for four other code blocks below
t8_stp_current_previous_df = pd.merge(t8_stp_master_col
                                      ,t8_stp_joined_df
                                      ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                                      ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                                      ,how = 'left') 

In [ ]:
## Select columns required for ICB (STP) group data frame for the previous year
## These columns are used as the index to which other columns are joined
t8_stp_previous_gr_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","GROUP_CODE"
                          ,"ALT_PAT_LIST","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
                          ,"PREV_ALT_PAT_LIST","PREV_PREVALENCE","PREV_REGISTER_SIZE"
                          ,"PREV_ACHIEVED_POINTS","PREV_TOTAL_INDICATOR_GROUP_POINTS"
                          ,"PREV_ACHIEVEMENT_RATE","PREV_PCA_COUNT","PREV_DENOMINATOR"
                          ,"PREV_PCA_RATE"]                                      
t8_stp_previous_gr_index_df = t8_stp_current_previous_df[t8_stp_previous_gr_col]

t8_stp_previous_gr_index_df = t8_stp_previous_gr_index_df.drop_duplicates()

In [ ]:
## Select columns required for ICB (STP) group data frame for the previous year
## Count distinct number of practices by indicator group
t8_stp_prac_gr_col = ["STP_ONS_CODE","PRACTICE_CODE","GROUP_CODE"]
t8_stp_prac_gr_df = t8_stp_current_previous_df[t8_stp_prac_gr_col]

t8_stp_prac_gr_df = t8_stp_prac_gr_df.drop_duplicates().reset_index(drop=True)

t8_stp_prac_gr_df = t8_stp_prac_gr_df.groupby(by = ["STP_ONS_CODE","GROUP_CODE"]
                                              ,as_index=False
                                              ).agg({"PRACTICE_CODE" : [pd.Series.count]})                                                    
t8_stp_prac_gr_df.columns = t8_stp_prac_gr_df.columns.droplevel(1)  

## Rename column 'PRACTICE_CODE' which contains a numeric count as 'NUMBER_PRACTICES'
t8_stp_prac_gr_df = t8_stp_prac_gr_df.rename(columns={"PRACTICE_CODE":"NUMBER_PRACTICES"})

In [ ]:
## Select columns required for ICB (STP) group data frame for the previous year
## Count distinct number of indicators by indicator group
t8_stp_ind_gr_col  = ["STP_ONS_CODE","INDICATOR_CODE","GROUP_CODE"]
t8_stp_ind_gr_df = t8_stp_current_previous_df[t8_stp_ind_gr_col]

t8_stp_ind_gr_df = t8_stp_ind_gr_df.drop_duplicates().reset_index(drop=True)

t8_stp_ind_gr_df = t8_stp_ind_gr_df.groupby(by = ["STP_ONS_CODE","GROUP_CODE"]
                                            ,as_index=False
                                            ).agg({"INDICATOR_CODE" : [pd.Series.count]})                                                  
t8_stp_ind_gr_df.columns = t8_stp_ind_gr_df.columns.droplevel(1)  

## Rename column 'PRACTICE_CODE' which contains a numeric count as 'NUMBER_PRACTICES'
t8_stp_ind_gr_df = t8_stp_ind_gr_df.rename(columns={"PRACTICE_CODE":"NUMBER_PRACTICES"})

In [ ]:
## Select columns required for ICB (STP) group data frame 
## Aggregate data in non-index columns
t8_stp_gr_current_aggregated_df = t8_stp_current_previous_df.groupby(by = ["STP_ONS_CODE","GROUP_CODE"]
                                                                     ,as_index=False
                                                                     ).agg({"LIST_SIZE" : [pd.Series.sum]
                                                                           ,"ALT_LIST_SIZE" : [pd.Series.sum]
                                                                           ,"REGISTER_SIZE" : [pd.Series.sum]
                                                                           ,"ACHIEVED_POINTS" : [pd.Series.sum]
                                                                           ,"PCA_COUNT" : [pd.Series.sum]
                                                                           ,"ACHIEVEMENT_DENOMINATOR" : [pd.Series.sum]
                                                                           ,"MAX_GROUP_POINTS" : [pd.Series.max]})                                                                          
t8_stp_gr_current_aggregated_df.columns = t8_stp_gr_current_aggregated_df.columns.droplevel(1)

## Rename columns 'LIST_SIZE' as 'LIST_SIZE_SUMMED' and 'ALT_LIST_SIZE' as 'ALT_LIST_SIZE_SUMMED' which contain total counts
t8_stp_gr_current_aggregated_df = t8_stp_gr_current_aggregated_df.rename(columns={"LIST_SIZE":"LIST_SIZE_SUMMED"
                                                                                  ,"ALT_LIST_SIZE":"ALT_LIST_SIZE_SUMMED"})

In [ ]:
## Join aggregated data to geographies and measures that are not aggregated
t8_stp_gr_df = pd.merge(t8_stp_previous_gr_index_df
                        ,t8_stp_prac_gr_df
                        ,left_on = ["STP_ONS_CODE","GROUP_CODE"]
                        ,right_on = ["STP_ONS_CODE","GROUP_CODE"]
                        ,how ="left")


t8_stp_gr_df = pd.merge(t8_stp_gr_df
                        ,t8_stp_ind_gr_df
                        ,left_on = ["STP_ONS_CODE","GROUP_CODE"]
                        ,right_on = ["STP_ONS_CODE","GROUP_CODE"]
                        ,how ="left")


t8_stp_gr_df = pd.merge(t8_stp_gr_df
                        ,t8_stp_gr_current_aggregated_df
                        ,left_on = ["STP_ONS_CODE","GROUP_CODE"]
                        ,right_on = ["STP_ONS_CODE","GROUP_CODE"]
                        ,how ="left")

In [ ]:
## Create finished t8 stp data set for T8 processing

## Create 'LIST_SIZE' column and drop aggregated 'LIST_SIZE_SUMMED' as it is no longer needed
t8_stp_gr_df["LIST_SIZE"] = t8_stp_gr_df["LIST_SIZE_SUMMED"]/t8_stp_gr_df["INDICATOR_CODE"]
t8_stp_gr_df = t8_stp_gr_df.drop(columns=["LIST_SIZE_SUMMED"])

## Create column 'ALT_LIST_SIZE' and leave values blank if there is no value in the corresponding 'ALT_PAT_LIST'
t8_stp_gr_df["ALT_LIST_SIZE"] = np.where(t8_stp_gr_df["ALT_PAT_LIST"] != None
                                         ,(t8_stp_gr_df["ALT_LIST_SIZE_SUMMED"]/t8_stp_gr_df["INDICATOR_CODE"])
                                         , "")


## Data columns that contain data totals with the prefix 'TOTAL'
t8_stp_gr_df = t8_stp_gr_df.rename(columns={"INDICATOR_CODE":"COUNT_DISTINCT_IND_CODE"
                                           ,"ACHIEVED_POINTS":"TOTAL_ACH_SCORE"
                                           ,"PCA_COUNT":"TOTAL_PCAS"
                                           ,"ACHIEVEMENT_DENOMINATOR":"TOTAL_DENOMINATORS"
                                           ,"REGISTER_SIZE":"REGISTER"})
                                                                                                 
## Select ordered ICB (STP) level columns to which calculated columns can be added
t8_stp_gr_col = ["GROUP_CODE","STP_ONS_CODE","STP_ODS_CODE","STP_NAME","NUMBER_PRACTICES"
                 ,"LIST_SIZE","ALT_LIST_SIZE","ALT_LIST_SIZE_SUMMED","COUNT_DISTINCT_IND_CODE"
                 ,"REGISTER","TOTAL_ACH_SCORE","MAX_GROUP_POINTS","TOTAL_PCAS"
                 ,"TOTAL_DENOMINATORS","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
                 ,"PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE"
                 ,"PREV_ACHIEVED_POINTS","PREV_TOTAL_INDICATOR_GROUP_POINTS"
                 ,"PREV_ACHIEVEMENT_RATE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"]                                          
t8_stp_base_df = t8_stp_gr_df[t8_stp_gr_col]

In [ ]:
"""   
Prevalence, Achievement Score, Achievement Percentage and Overall PCA rate
are calculated for ICB (STP) data and comprise part of the final published data frames
in icb-ach-prev-pca.xlsx

"""
## Change the data type of the column 'ALT_LIST_SIZE'used in the calculations
t8_stp_base_df["ALT_LIST_SIZE"] = t8_stp_base_df["ALT_LIST_SIZE"].astype(float)

## Calculate Prevalence
t8_stp_base_df["PREVALENCE"] = np.where(t8_stp_base_df["ALT_LIST_SIZE"] == 0
                                        ,(t8_stp_base_df["REGISTER"]/t8_stp_base_df["LIST_SIZE"])*100
                                        ,(t8_stp_base_df["REGISTER"]/t8_stp_base_df["ALT_LIST_SIZE"])*100)
                                                  
## Calculate Achievement Score
t8_stp_base_df["AVG_ACH_SCORE"] = t8_stp_base_df["TOTAL_ACH_SCORE"]/t8_stp_base_df["NUMBER_PRACTICES"]

## Calculate Achievement Percentage
t8_stp_base_df["ACH_PC"] = ((t8_stp_base_df["TOTAL_ACH_SCORE"]/t8_stp_base_df["NUMBER_PRACTICES"]
                             )/t8_stp_base_df["MAX_GROUP_POINTS"])*100

## Calculate Overall PCA rate
t8_stp_base_df["OVERALL_PCA_RATE"] = (t8_stp_base_df["TOTAL_PCAS"]/(t8_stp_base_df["TOTAL_DENOMINATORS"]\
                                                                    +t8_stp_base_df["TOTAL_PCAS"]))*100

## Reorder columns to include new values
t8_stp_base_col = ["GROUP_CODE","STP_ODS_CODE","STP_ONS_CODE","STP_NAME"
                   ,"NUMBER_PRACTICES","LIST_SIZE","ALT_LIST_SIZE","REGISTER"
                   ,"PREVALENCE", "TOTAL_ACH_SCORE","AVG_ACH_SCORE","ACH_PC","TOTAL_PCAS"
                   ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PREV_NUMBER_PRACTICES"
                   ,"PREV_PRACTICE_LIST_SIZE","PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE"
                   ,"PREV_PREVALENCE","PREV_ACHIEVED_POINTS"
                   ,"PREV_TOTAL_INDICATOR_GROUP_POINTS","PREV_ACHIEVEMENT_RATE"
                   ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"]                                                                                                                                                                                              
t8_stp_base_df = t8_stp_base_df[t8_stp_base_col]

<h2>Part 3 - Create ICB (STP) Additional List df

In [ ]:
"""   
Addional practice list sizes 'ADD_PAT_SIZE' with the accompanying description column 'ADD_PAT_LIST'
are created at ICB (STP) level by indicator group and are joined to the data frame created in Part 2

"""

## ICB (STP) data frame defined this forms the foundation (base) of the data frame to be created
t8_stp_add_list_col = ["STP_ODS_CODE","GROUP_CODE","ADD_PAT_LIST","ADD_LIST_SIZE","INDICATOR_CODE"]
t8_stp_add_list_base_df = t8_stp_current_previous_df[t8_stp_add_list_col]

In [ ]:
## Count distinct number of indicators
t8_stp_add_list_ind_col = ["STP_ODS_CODE","INDICATOR_CODE","GROUP_CODE"]
t8_stp_add_list_ind_df = t8_stp_add_list_base_df[t8_stp_add_list_ind_col]
t8_stp_add_list_ind_df = t8_stp_add_list_ind_df.drop_duplicates()

t8_stp_add_list_ind_df = t8_stp_add_list_ind_df.groupby(by = ["STP_ODS_CODE","GROUP_CODE"]
                                                        ,as_index=False
                                                          ).agg({"INDICATOR_CODE" : [pd.Series.count]})                                                                 
t8_stp_add_list_ind_df.columns = t8_stp_add_list_ind_df.columns.droplevel(1) 

## Rename column 'INDICATOR_CODE' as 'COUNT_DISTINCT_IND_CODE' which contains counts
t8_stp_add_list_ind_df = t8_stp_add_list_ind_df.rename(columns={"INDICATOR_CODE":"COUNT_DISTINCT_IND_CODE"})

## Sum ADD_LIST_SIZE (additional list size)
t8_stp_add_list_df = t8_stp_add_list_base_df.drop(columns=["INDICATOR_CODE"])
t8_stp_add_list_sum_df = t8_stp_add_list_df.groupby(by = ["STP_ODS_CODE","GROUP_CODE","ADD_PAT_LIST"]
                                                    ,as_index=False
                                                    )["ADD_LIST_SIZE"].sum()

In [ ]:
## Combine aggregated data frames
t8_stp_add_list_df = pd.merge(t8_stp_add_list_sum_df
                              ,t8_stp_add_list_ind_df
                              ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                              ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                              ,how ="left") 

## Define new column 'ADD_LIST' e.g. ADD_LIST_06OV
t8_stp_add_list_df["ADD_LIST"] = "ADD_LIST_"+t8_stp_add_list_df["ADD_PAT_LIST"]

## Drop unwanted columns
t8_stp_add_list_df = t8_stp_add_list_df.drop(columns=["ADD_PAT_LIST","COUNT_DISTINCT_IND_CODE"])

## Reorder columns
t6_stp_add_list_col = ["STP_ODS_CODE","GROUP_CODE","ADD_LIST","ADD_LIST_SIZE"]
t8_stp_add_list_df = t8_stp_add_list_df[t6_stp_add_list_col]

In [ ]:
## Pivot t8_stp_add_list_df from a long thin data frame to a short broad data frame with 
## a column for each additional list size with 'STP_ODS_CODE, and 'GROUP_CODE' as the index
t8_add_list_df = pd.pivot_table(t8_stp_add_list_df
                                ,values="ADD_LIST_SIZE"
                                ,index=["STP_ODS_CODE", "GROUP_CODE"]
                                ,columns= "ADD_LIST")

t8_add_list_df = t8_add_list_df.reset_index().rename_axis(None, axis=1)

<h3>Join Part 3 (Additional List df) to Part 2 (previous data)

In [ ]:
## Join additional list sizes to main stp data frame
T8_master_df = pd.merge(t8_stp_base_df
                        ,t8_add_list_df
                        ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                        ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                        ,how ="left")


## Calculate the difference in the previous year's percentages to the curent years as percentage points
T8_master_df["PREV_CHANGE"] = T8_master_df["PREVALENCE"]-T8_master_df["PREV_PREVALENCE"]
T8_master_df["ACH_CHANGE"] = T8_master_df["ACH_PC"]-T8_master_df["PREV_ACHIEVEMENT_RATE"]
T8_master_df["PCA_CHANGE"] = T8_master_df["OVERALL_PCA_RATE"]-T8_master_df["PREV_PCA_RATE"]

T8_master_df = T8_master_df.drop_duplicates()

Create variable source data

In [ ]:
## Creates a list of 'STP_ONS_CODE' which is use to 
## create the variable 'stpcount' in the 'Publication Outputs
stp_list = T8_master_df[["STP_ONS_CODE"]]
stp_list = stp_list.drop_duplicates().reset_index(drop = True)

<h2>Create ICB data (STP) tables<br> and table titles

In [ ]:
"""   
Not all indicator group tables have the same data components. 
Therefore it is necessary to define the content of each indicator group 
data frame separately. 

The code block are in the indicator group order as they appear in the 
icb-ach-prev-pca excel workbook (template 8)

"""

## Select data for AF group code
t8_af_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "AF"]

t8_af_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="AF")]


## Combines T8 Master table with previous years data
t8_af_df = pd.merge(t8_af_base_df
                    ,t8_af_stp_current_piv_df
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'AF' in icb-ach-prev-pca excel workbook (template 8)
t8_af_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES"
             ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE"
             ,"NUMBER_PRACTICES","LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE"
             ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC"
             ,"ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"
             ,"TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
             ,"AF001_ACHIEVED_POINTS","AF001_ACH_RATE","AF006_ACHIEVED_POINTS"
             ,"AF006_ACHIEVEMENT_NUMERATOR","AF006_ACHIEVEMENT_DENOMINATOR"
             ,"AF006_UL_NET_ACH","AF006_PCA_COUNT", "AF006_PCA_RATE"
             ,"AF006_DEN_PLUS_PCA","AF006_PRI","AF007_ACHIEVED_POINTS"
             ,"AF007_ACHIEVEMENT_NUMERATOR","AF007_ACHIEVEMENT_DENOMINATOR"
             ,"AF007_UL_NET_ACH","AF007_PCA_COUNT","AF007_PCA_RATE"
             ,"AF007_DEN_PLUS_PCA","AF007_PRI"]
t8_af_df = t8_af_df[t8_af_col]

t8_af_df = t8_af_df.sort_values(by = ["STP_NAME"],ascending = [True])

In [ ]:
## Select data for BP group code
t8_bp_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "BP"]

t8_bp_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="BP")]


## Combines T8 Master table with previous years data
t8_bp_df = pd.merge(t8_bp_base_df
                    ,t8_bp_stp_current_piv_df
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'BP' in icb-ach-prev-pca excel workbook (template 8)
t8_bp_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
             ,"NUMBER_PRACTICES","ALT_LIST_SIZE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
             ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR"
             ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
             ,"BP002_ACHIEVED_POINTS","BP002_ACHIEVEMENT_NUMERATOR","BP002_ACHIEVEMENT_DENOMINATOR"
             ,"BP002_UL_NET_ACH","BP002_PCA_COUNT","BP002_PCA_RATE","BP002_DEN_PLUS_PCA","BP002_PRI"]
t8_bp_df = t8_bp_df[t8_bp_col]

t8_bp_df = t8_bp_df.sort_values(by=["STP_NAME"],ascending = [True])

In [ ]:
## Select data for CHD group code
t8_chd_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "CHD"]

t8_chd_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="CHD")]


## Combines T8 Master table with previous years data
t8_chd_df = pd.merge(t8_chd_base_df
                    ,t8_chd_stp_current_piv_df
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'CHD' in icb-ach-prev-pca excel workbook (template 8)
t8_chd_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","ADD_LIST_79UN"
              ,"ADD_LIST_80OV","REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS"
              ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT"
              ,"PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE"
              ,"PCA_CHANGE", "CHD001_ACHIEVED_POINTS","CHD001_ACH_RATE"
              ,"CHD005_ACHIEVED_POINTS","CHD005_ACHIEVEMENT_NUMERATOR","CHD005_ACHIEVEMENT_DENOMINATOR"
              ,"CHD005_UL_NET_ACH","CHD005_PCA_COUNT","CHD005_PCA_RATE","CHD005_DEN_PLUS_PCA","CHD005_PRI"
              , "CHD008_ACHIEVED_POINTS","CHD008_ACHIEVEMENT_NUMERATOR","CHD008_ACHIEVEMENT_DENOMINATOR"
              ,"CHD008_UL_NET_ACH","CHD008_PCA_COUNT","CHD008_PCA_RATE","CHD008_DEN_PLUS_PCA","CHD008_PRI"
              , "CHD009_ACHIEVED_POINTS","CHD009_ACHIEVEMENT_NUMERATOR","CHD009_ACHIEVEMENT_DENOMINATOR"
              ,"CHD009_UL_NET_ACH","CHD009_PCA_COUNT","CHD009_PCA_RATE","CHD009_DEN_PLUS_PCA","CHD009_PRI"]
t8_chd_df = t8_chd_df[t8_chd_col]

t8_chd_df = t8_chd_df.sort_values(by=["STP_NAME"],ascending = [True])

In [ ]:
## Select data for HF group code
t8_hf_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "HF"]

t8_hf_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="HF")]


## Combines T8 Master table with previous years data
t8_hf_df = pd.merge(t8_hf_base_df
                    ,t8_hf_stp_current_piv_df
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'HF' in icb-ach-prev-pca excel workbook (template 8)
t8_hf_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","REGISTER","PREVALENCE"
             ,"PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC"
             ,"ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
             ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE","HF001_ACHIEVED_POINTS","HF001_ACH_RATE"
             ,"HF003_ACHIEVED_POINTS","HF003_ACHIEVEMENT_NUMERATOR","HF003_ACHIEVEMENT_DENOMINATOR"
             ,"HF003_UL_NET_ACH","HF003_PCA_COUNT","HF003_PCA_RATE","HF003_DEN_PLUS_PCA","HF003_PRI"
             ,"HF005_ACHIEVED_POINTS","HF005_ACHIEVEMENT_NUMERATOR","HF005_ACHIEVEMENT_DENOMINATOR"
             ,"HF005_UL_NET_ACH","HF005_PCA_COUNT","HF005_PCA_RATE","HF005_DEN_PLUS_PCA","HF005_PRI"
             ,"HF006_ACHIEVED_POINTS","HF006_ACHIEVEMENT_NUMERATOR","HF006_ACHIEVEMENT_DENOMINATOR"
             ,"HF006_UL_NET_ACH","HF006_PCA_COUNT","HF006_PCA_RATE","HF006_DEN_PLUS_PCA","HF006_PRI"
             ,"HF007_ACHIEVED_POINTS","HF007_ACHIEVEMENT_NUMERATOR","HF007_ACHIEVEMENT_DENOMINATOR"
             ,"HF007_UL_NET_ACH","HF007_PCA_COUNT","HF007_PCA_RATE","HF007_DEN_PLUS_PCA","HF007_PRI"]
t8_hf_df = t8_hf_df[t8_hf_col]

t8_hf_df = t8_hf_df.sort_values(by=["STP_NAME"],ascending = [True])

In [ ]:
## Select data for HYP group code
t8_hyp_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "HYP"]

t8_hyp_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="HYP")]


## Combines T8 Master table with previous years data
t8_hyp_df = pd.merge(t8_hyp_base_df
                    ,t8_hyp_stp_current_piv_df
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'HYP' in icb-ach-prev-pca excel workbook (template 8)
t8_hyp_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","ADD_LIST_79UN"
              ,"ADD_LIST_80OV","REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS"
              ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT"
              ,"PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE"
              ,"PCA_CHANGE","HYP001_ACHIEVED_POINTS","HYP001_ACH_RATE"
              ,"HYP003_ACHIEVED_POINTS","HYP003_ACHIEVEMENT_NUMERATOR","HYP003_ACHIEVEMENT_DENOMINATOR"
              ,"HYP003_UL_NET_ACH","HYP003_PCA_COUNT","HYP003_PCA_RATE","HYP003_DEN_PLUS_PCA","HYP003_PRI"
              ,"HYP007_ACHIEVED_POINTS","HYP007_ACHIEVEMENT_NUMERATOR","HYP007_ACHIEVEMENT_DENOMINATOR"
              ,"HYP007_UL_NET_ACH","HYP007_PCA_COUNT","HYP007_PCA_RATE","HYP007_DEN_PLUS_PCA","HYP007_PRI"]                
t8_hyp_df = t8_hyp_df[t8_hyp_col]

t8_hyp_df = t8_hyp_df.sort_values(by=["STP_NAME"],ascending = [True])

In [ ]:
## Select data for PAD group code
t8_pad_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "PAD"]

t8_pad_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="PAD")]


## Combines T8 Master table with previous years data
t8_pad_df = pd.merge(t8_pad_base_df
                    ,t8_pad_stp_current_piv_df
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'PAD' in icb-ach-prev-pca excel workbook (template 8)
t8_pad_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","REGISTER"
              ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
              ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PAD001_ACHIEVED_POINTS","PAD001_ACH_RATE"]                     
t8_pad_df = t8_pad_df[t8_pad_col]                     

t8_pad_df = t8_pad_df.sort_values(by=["STP_NAME"],ascending = [True])

In [ ]:
## Select data for STIA group code
t8_stia_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "STIA"]

t8_stia_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="STIA")]


## Combines T8 Master table with previous years data
t8_stia_df = pd.merge(t8_stia_base_df
                    ,t8_stia_stp_current_piv_df
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'STIA' in icb-ach-prev-pca excel workbook (template 8)
t8_stia_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
               ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","ADD_LIST_79UN"
               ,"ADD_LIST_80OV","REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS"
               ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT"
               ,"PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE"
               ,"PCA_CHANGE","STIA001_ACHIEVED_POINTS","STIA001_ACH_RATE"
               ,"STIA007_ACHIEVED_POINTS","STIA007_ACHIEVEMENT_NUMERATOR","STIA007_ACHIEVEMENT_DENOMINATOR"
               ,"STIA007_UL_NET_ACH","STIA007_PCA_COUNT","STIA007_PCA_RATE","STIA007_DEN_PLUS_PCA","STIA007_PRI"
               ,"STIA010_ACHIEVED_POINTS","STIA010_ACHIEVEMENT_NUMERATOR","STIA010_ACHIEVEMENT_DENOMINATOR"
               ,"STIA010_UL_NET_ACH","STIA010_PCA_COUNT","STIA010_PCA_RATE","STIA010_DEN_PLUS_PCA","STIA010_PRI"
               ,"STIA011_ACHIEVED_POINTS","STIA011_ACHIEVEMENT_NUMERATOR","STIA011_ACHIEVEMENT_DENOMINATOR"
               ,"STIA011_UL_NET_ACH","STIA011_PCA_COUNT","STIA011_PCA_RATE","STIA011_DEN_PLUS_PCA","STIA011_PRI"]                                               
t8_stia_df = t8_stia_df[t8_stia_col]                       

t8_stia_df = t8_stia_df.sort_values(by=["STP_NAME"],ascending = [True])

In [ ]:
## Select data for AST group code
t8_ast_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "AST"]

t8_ast_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="AST")]


## Combines T8 Master table with previous years data
t8_ast_df = pd.merge(t8_ast_base_df
                    ,t8_ast_stp_current_piv_df
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'AST' in icb-ach-prev-pca excel workbook (template 8)
t8_ast_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE"
              ,"ADD_LIST_06_19","REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS"
              ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT"
              ,"PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE"
              ,"PCA_CHANGE","AST005_ACHIEVED_POINTS","AST005_ACH_RATE"
              ,"AST006_ACHIEVED_POINTS","AST006_ACHIEVEMENT_NUMERATOR","AST006_ACHIEVEMENT_DENOMINATOR"
              ,"AST006_UL_NET_ACH","AST006_PCA_COUNT","AST006_PCA_RATE","AST006_DEN_PLUS_PCA","AST006_PRI"
              ,"AST007_ACHIEVED_POINTS","AST007_ACHIEVEMENT_NUMERATOR","AST007_ACHIEVEMENT_DENOMINATOR"
              ,"AST007_UL_NET_ACH","AST007_PCA_COUNT","AST007_PCA_RATE","AST007_DEN_PLUS_PCA","AST007_PRI"
              ,"AST008_ACHIEVED_POINTS","AST008_ACHIEVEMENT_NUMERATOR","AST008_ACHIEVEMENT_DENOMINATOR"
              ,"AST008_UL_NET_ACH","AST008_PCA_COUNT","AST008_PCA_RATE","AST008_DEN_PLUS_PCA","AST008_PRI"]                   
t8_ast_df = t8_ast_df[t8_ast_col]                       
                      
t8_ast_df = t8_ast_df.sort_values(by=["STP_NAME"],ascending = [True])

In [ ]:
## Select data for COPD group code
t8_copd_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "COPD"]

t8_copd_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="COPD")]


## Combines T8 Master table with previous years data
t8_copd_df = pd.merge(t8_copd_base_df
                    ,t8_copd_stp_current_piv_df
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'COPD' in icb-ach-prev-pca excel workbook (template 8)
t8_copd_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
               ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","REGISTER"
               ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
               ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR"
               ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
               ,"COPD009_ACHIEVED_POINTS","COPD009_ACH_RATE"
               ,"COPD008_ACHIEVED_POINTS","COPD008_ACHIEVEMENT_NUMERATOR","COPD008_ACHIEVEMENT_DENOMINATOR"
               ,"COPD008_UL_NET_ACH","COPD008_PCA_COUNT","COPD008_PCA_RATE","COPD008_DEN_PLUS_PCA","COPD008_PRI"
               ,"COPD010_ACHIEVED_POINTS","COPD010_ACHIEVEMENT_NUMERATOR","COPD010_ACHIEVEMENT_DENOMINATOR"
               ,"COPD010_UL_NET_ACH","COPD010_PCA_COUNT","COPD010_PCA_RATE","COPD010_DEN_PLUS_PCA","COPD010_PRI"]
t8_copd_df = t8_copd_df[t8_copd_col]                         

t8_copd_df = t8_copd_df.sort_values(by=["STP_NAME"],ascending = [True])

In [ ]:
## Select data for OB group code
t8_ob_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "OB"]

t8_ob_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="OB")]


## Combines T8 Master table with previous years data
t8_ob_df = pd.merge(t8_ob_base_df
                    ,t8_ob_stp_current_piv_df
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'OB' in icb-ach-prev-pca excel workbook (template 8)
t8_ob_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE","REGISTER"
             ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
             ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","OB002_ACHIEVED_POINTS","OB002_ACH_RATE"]                    
t8_ob_df = t8_ob_df[t8_ob_col]

t8_ob_df = t8_ob_df.sort_values(by=["STP_NAME"],ascending = [True])                                                           

In [ ]:
## Select data for SMOK group code
t8_smok_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "SMOK"]

t8_smok_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="SMOK")]


## Combines T8 Master table with previous years data
t8_smok_df = pd.merge(t8_smok_base_df
                      ,t8_smok_stp_current_piv_df    
                      ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                      ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                      ,how ="left")

## Reorder columns for output data frame on tab 'SMOK' in icb-ach-prev-pca excel workbook (template 8)
t8_smok_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
               ,"NUMBER_PRACTICES","ALT_LIST_SIZE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
               ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE", "PREV_PCA_COUNT","PREV_DENOMINATOR"
               ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
               ,"SMOK002_ACHIEVED_POINTS","SMOK002_ACHIEVEMENT_NUMERATOR","SMOK002_ACHIEVEMENT_DENOMINATOR"
               ,"SMOK002_UL_NET_ACH","SMOK002_PCA_COUNT","SMOK002_PCA_RATE","SMOK002_DEN_PLUS_PCA","SMOK002_PRI"
               , "SMOK004_ACHIEVED_POINTS","SMOK004_ACHIEVEMENT_NUMERATOR","SMOK004_ACHIEVEMENT_DENOMINATOR"
               ,"SMOK004_UL_NET_ACH","SMOK004_PCA_COUNT","SMOK004_PCA_RATE","SMOK004_DEN_PLUS_PCA","SMOK004_PRI"
               ,"SMOK005_ACHIEVED_POINTS","SMOK005_ACHIEVEMENT_NUMERATOR","SMOK005_ACHIEVEMENT_DENOMINATOR"
               ,"SMOK005_UL_NET_ACH","SMOK005_PCA_COUNT","SMOK005_PCA_RATE","SMOK005_DEN_PLUS_PCA","SMOK005_PRI"]
t8_smok_df = t8_smok_df[t8_smok_col]

t8_smok_df = t8_smok_df.sort_values(by=["STP_NAME"],ascending = [True])

In [ ]:
## Select data for CAN group code
t8_can_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "CAN"]

t8_can_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="CAN")]


## Combines T8 Master table with previous years data
t8_can_df = pd.merge(t8_can_base_df
                      ,t8_can_stp_current_piv_df    
                      ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                      ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                      ,how ="left")

## Reorder columns for output data frame on tab 'CAN' in icb-ach-prev-pca excel workbook (template 8)
t8_can_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","REGISTER"
              ,"PREVALENCE","PREV_CHANGE", "PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
              ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE", "PREV_PCA_COUNT","PREV_DENOMINATOR"
              ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
              , "CAN001_ACHIEVED_POINTS","CAN001_ACH_RATE"
              , "CAN004_ACHIEVED_POINTS","CAN004_ACHIEVEMENT_NUMERATOR","CAN004_ACHIEVEMENT_DENOMINATOR"
              ,"CAN004_UL_NET_ACH","CAN004_PCA_COUNT","CAN004_PCA_RATE","CAN004_DEN_PLUS_PCA","CAN004_PRI"
              ,"CAN005_ACHIEVED_POINTS","CAN005_ACHIEVEMENT_NUMERATOR","CAN005_ACHIEVEMENT_DENOMINATOR"
              ,"CAN005_UL_NET_ACH","CAN005_PCA_COUNT","CAN005_PCA_RATE","CAN005_DEN_PLUS_PCA","CAN005_PRI"]
t8_can_df = t8_can_df[t8_can_col]
 
t8_can_df = t8_can_df.sort_values(by=["STP_NAME"],ascending = [True])

In [ ]:
## Select data for CKD group code
t8_ckd_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "CKD"]

t8_ckd_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="CKD")]


## Combines T8 Master table with previous years data
t8_ckd_df = pd.merge(t8_ckd_base_df
                      ,t8_ckd_stp_current_piv_df    
                      ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                      ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                      ,how ="left")

## Reorder columns for output data frame on tab 'CKD' in icb-ach-prev-pca excel workbook (template 8)
t8_ckd_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE"
              ,"REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS"
              ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
              , "CKD005_ACHIEVED_POINTS","CKD005_ACH_RATE"]
t8_ckd_df = t8_ckd_df[t8_ckd_col]
                      
t8_ckd_df = t8_ckd_df.sort_values(by=["STP_NAME"],ascending = [True])                    

In [ ]:
## Select data for DM group code
t8_dm_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "DM"]

t8_dm_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="DM")]


## Combines T8 Master table with previous years data
t8_dm_df = pd.merge(t8_dm_base_df
                    ,t8_dm_stp_current_piv_df    
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'DM' in icb-ach-prev-pca excel workbook (template 8)
t8_dm_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE"
             ,"ADD_LIST_40OV","REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS"
             ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT"
             ,"PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS"
             ,"OVERALL_PCA_RATE","PCA_CHANGE","DM017_ACHIEVED_POINTS","DM017_ACH_RATE"
             ,"DM006_ACHIEVED_POINTS","DM006_ACHIEVEMENT_NUMERATOR","DM006_ACHIEVEMENT_DENOMINATOR"
             ,"DM006_UL_NET_ACH","DM006_PCA_COUNT","DM006_PCA_RATE","DM006_DEN_PLUS_PCA","DM006_PRI"
             ,"DM012_ACHIEVED_POINTS","DM012_ACHIEVEMENT_NUMERATOR","DM012_ACHIEVEMENT_DENOMINATOR"
             ,"DM012_UL_NET_ACH","DM012_PCA_COUNT","DM012_PCA_RATE","DM012_DEN_PLUS_PCA","DM012_PRI"
             ,"DM014_ACHIEVED_POINTS","DM014_ACHIEVEMENT_NUMERATOR","DM014_ACHIEVEMENT_DENOMINATOR"
             ,"DM014_UL_NET_ACH","DM014_PCA_COUNT","DM014_PCA_RATE","DM014_DEN_PLUS_PCA","DM014_PRI"
             ,"DM019_ACHIEVED_POINTS","DM019_ACHIEVEMENT_NUMERATOR","DM019_ACHIEVEMENT_DENOMINATOR"
             ,"DM019_UL_NET_ACH","DM019_PCA_COUNT","DM019_PCA_RATE","DM019_DEN_PLUS_PCA","DM019_PRI"
             ,"DM020_ACHIEVED_POINTS","DM020_ACHIEVEMENT_NUMERATOR","DM020_ACHIEVEMENT_DENOMINATOR"
             ,"DM020_UL_NET_ACH","DM020_PCA_COUNT","DM020_PCA_RATE","DM020_DEN_PLUS_PCA","DM020_PRI"
             ,"DM021_ACHIEVED_POINTS","DM021_ACHIEVEMENT_NUMERATOR","DM021_ACHIEVEMENT_DENOMINATOR"
             ,"DM021_UL_NET_ACH","DM021_PCA_COUNT","DM021_PCA_RATE","DM021_DEN_PLUS_PCA","DM021_PRI"
             ,"DM022_ACHIEVED_POINTS","DM022_ACHIEVEMENT_NUMERATOR","DM022_ACHIEVEMENT_DENOMINATOR"
             ,"DM022_UL_NET_ACH","DM022_PCA_COUNT","DM022_PCA_RATE","DM022_DEN_PLUS_PCA","DM022_PRI"
             ,"DM023_ACHIEVED_POINTS","DM023_ACHIEVEMENT_NUMERATOR","DM023_ACHIEVEMENT_DENOMINATOR"
             ,"DM023_UL_NET_ACH","DM023_PCA_COUNT","DM023_PCA_RATE","DM023_DEN_PLUS_PCA","DM023_PRI"]
t8_dm_df = t8_dm_df[t8_dm_col]                     

t8_dm_df = t8_dm_df.sort_values(by=["STP_NAME"],ascending = [True])                

In [ ]:
## Select data for PC group code
t8_pc_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "PC"]

t8_pc_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="PC")]


## Combines T8 Master table with previous years data
t8_pc_df = pd.merge(t8_pc_base_df
                    ,t8_pc_stp_current_piv_df    
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'PC' in icb-ach-prev-pca excel workbook (template 8)
t8_pc_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","REGISTER"
             ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
             ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PC001_ACHIEVED_POINTS","PC001_ACH_RATE"]           
t8_pc_df = t8_pc_df[t8_pc_col]
                    
t8_pc_df = t8_pc_df.sort_values(by=["STP_NAME"],ascending = [True])         

In [ ]:
## Select data for DEM group code
t8_dem_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "DEM"]

t8_dem_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="DEM")]


## Combines T8 Master table with previous years data
t8_dem_df = pd.merge(t8_dem_base_df
                    ,t8_dem_stp_current_piv_df    
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'DEM' in icb-ach-prev-pca excel workbook (template 8)
t8_dem_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","REGISTER"
              ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
              ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
              ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
              ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE","DEM001_ACHIEVED_POINTS"
              ,"DEM001_ACH_RATE","DEM004_ACHIEVED_POINTS","DEM004_ACHIEVEMENT_NUMERATOR"
              ,"DEM004_ACHIEVEMENT_DENOMINATOR","DEM004_UL_NET_ACH","DEM004_PCA_COUNT"
              ,"DEM004_PCA_RATE","DEM004_DEN_PLUS_PCA","DEM004_PRI"]
t8_dem_df = t8_dem_df[t8_dem_col]
                      
t8_dem_df = t8_dem_df.sort_values(by=["STP_NAME"],ascending = [True])

In [ ]:
## Select data for DEP group code
t8_dep_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "DEP"]

t8_dep_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="DEP")]


## Combines T8 Master table with previous years data
t8_dep_df = pd.merge(t8_dep_base_df
                    ,t8_dep_stp_current_piv_df    
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'DEP' in icb-ach-prev-pca excel workbook (template 8)
t8_dep_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE","REGISTER"
              ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
              ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE", "PREV_PCA_COUNT","PREV_DENOMINATOR"
              ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
              ,"DEP003_ACHIEVED_POINTS","DEP003_ACHIEVEMENT_NUMERATOR","DEP003_ACHIEVEMENT_DENOMINATOR"
              ,"DEP003_UL_NET_ACH","DEP003_PCA_COUNT","DEP003_PCA_RATE","DEP003_DEN_PLUS_PCA","DEP003_PRI"]
t8_dep_df = t8_dep_df[t8_dep_col]
                      
t8_dep_df = t8_dep_df.sort_values(by=["STP_NAME"],ascending = [True])

In [ ]:
## Select data for EP group code
t8_ep_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "EP"]

t8_ep_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="EP")]


## Combines T8 Master table with previous years data
t8_ep_df = pd.merge(t8_ep_base_df
                    ,t8_ep_stp_current_piv_df    
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'EP' in icb-ach-prev-pca excel workbook (template 8)
t8_ep_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE","REGISTER"
             ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
             ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","EP001_ACHIEVED_POINTS","EP001_ACH_RATE"]
t8_ep_df = t8_ep_df[t8_ep_col]
                     
t8_ep_df = t8_ep_df.sort_values(by=["STP_NAME"],ascending = [True])

In [ ]:
## Select data for LD group code
t8_ld_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "LD"]

t8_ld_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="LD")]


## Combines T8 Master table with previous years data
t8_ld_df = pd.merge(t8_ld_base_df
                    ,t8_ld_stp_current_piv_df    
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'LD' in icb-ach-prev-pca excel workbook (template 8)
t8_ld_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","REGISTER"
             ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
             ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","LD004_ACHIEVED_POINTS","LD004_ACH_RATE"]
t8_ld_df = t8_ld_df[t8_ld_col]
                     
t8_ld_df = t8_ld_df.sort_values(by=["STP_NAME"],ascending = [True])               

In [ ]:
## Select data for MH group code
t8_mh_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "MH"]

t8_mh_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="MH")]


## Combines T8 Master table with previous years data
t8_mh_df = pd.merge(t8_mh_base_df
                    ,t8_mh_stp_current_piv_df    
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'MH' in icb-ach-prev-pca excel workbook (template 8)
t8_mh_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","REGISTER"
             ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
             ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR"
             ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
             ,"MH001_ACHIEVED_POINTS","MH001_ACH_RATE"
             ,"MH002_ACHIEVED_POINTS","MH002_ACHIEVEMENT_NUMERATOR","MH002_ACHIEVEMENT_DENOMINATOR"
             ,"MH002_UL_NET_ACH","MH002_PCA_COUNT","MH002_PCA_RATE","MH002_DEN_PLUS_PCA","MH002_PRI"
             ,"MH003_ACHIEVED_POINTS","MH003_ACHIEVEMENT_NUMERATOR","MH003_ACHIEVEMENT_DENOMINATOR"
             ,"MH003_UL_NET_ACH","MH003_PCA_COUNT","MH003_PCA_RATE","MH003_DEN_PLUS_PCA","MH003_PRI"
             ,"MH006_ACHIEVED_POINTS","MH006_ACHIEVEMENT_NUMERATOR","MH006_ACHIEVEMENT_DENOMINATOR"
             ,"MH006_UL_NET_ACH","MH006_PCA_COUNT","MH006_PCA_RATE","MH006_DEN_PLUS_PCA","MH006_PRI"
             ,"MH007_ACHIEVED_POINTS","MH007_ACHIEVEMENT_NUMERATOR","MH007_ACHIEVEMENT_DENOMINATOR"
             ,"MH007_UL_NET_ACH","MH007_PCA_COUNT","MH007_PCA_RATE","MH007_DEN_PLUS_PCA","MH007_PRI"
             ,"MH011_ACHIEVED_POINTS","MH011_ACHIEVEMENT_NUMERATOR","MH011_ACHIEVEMENT_DENOMINATOR"
             ,"MH011_UL_NET_ACH","MH011_PCA_COUNT","MH011_PCA_RATE","MH011_DEN_PLUS_PCA","MH011_PRI"
             ,"MH012_ACHIEVED_POINTS","MH012_ACHIEVEMENT_NUMERATOR","MH012_ACHIEVEMENT_DENOMINATOR"
             ,"MH012_UL_NET_ACH","MH012_PCA_COUNT","MH012_PCA_RATE","MH012_DEN_PLUS_PCA","MH012_PRI"]
t8_mh_df = t8_mh_df[t8_mh_col]
                     
t8_mh_df = t8_mh_df.sort_values(by=["STP_NAME"],ascending = [True])

In [ ]:
## Select data for OST group code
t8_ost_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "OST"]

t8_ost_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="OST")]


## Combines T8 Master table with previous years data
t8_ost_df = pd.merge(t8_ost_base_df
                    ,t8_ost_stp_current_piv_df    
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'OST' in icb-ach-prev-pca excel workbook (template 8)
t8_ost_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE","REGISTER"
              ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
              ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
              ,"OST004_ACHIEVED_POINTS","OST004_ACH_RATE"]       
                        
t8_ost_df = t8_ost_df[t8_ost_col]
                       
t8_ost_df = t8_ost_df.sort_values(by=["STP_NAME"],ascending = [True])

In [ ]:
## Select data for RA group code
t8_ra_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "RA"]

t8_ra_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="RA")]


## Combines T8 Master table with previous years data
t8_ra_df = pd.merge(t8_ra_base_df
                    ,t8_ra_stp_current_piv_df    
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'RA' in icb-ach-prev-pca excel workbook (template 8)
t8_ra_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE","REGISTER"
             ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
             ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR"
             ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
             ,"RA001_ACHIEVED_POINTS","RA001_ACH_RATE"
             ,"RA002_ACHIEVED_POINTS","RA002_ACHIEVEMENT_NUMERATOR","RA002_ACHIEVEMENT_DENOMINATOR"
             ,"RA002_UL_NET_ACH","RA002_PCA_COUNT","RA002_PCA_RATE","RA002_DEN_PLUS_PCA","RA002_PRI"]
t8_ra_df = t8_ra_df[t8_ra_col]
                     
t8_ra_df = t8_ra_df.sort_values(by=["STP_NAME"],ascending = [True])

In [ ]:
## Select data for NDH group code
t8_ndh_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "NDH"]

## As NDH had no achievement points allocated in the previous year and therefore had no achievement rate for comparison with this years
## The missing value in the columns 'Achievement (%)' and  'Year on year change' has been replaced with an 'z' for not applicable
t8_ndh_base_df["PREV_ACHIEVEMENT_RATE"] = t8_ndh_base_df["PREV_ACHIEVEMENT_RATE"].fillna("z")
t8_ndh_base_df["ACH_CHANGE"] = t8_ndh_base_df["ACH_CHANGE"].fillna("z")

t8_ndh_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="NDH")]

## Combines T8 Master table with previous years data
t8_ndh_df = pd.merge(t8_ndh_base_df
                    ,t8_ndh_stp_current_piv_df    
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'NDH' in icb-ach-prev-pca excel workbook (template 8)
t8_ndh_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE"
              ,"REGISTER","PREVALENCE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
              ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR"
              ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
              , "NDH001_ACHIEVED_POINTS","NDH001_ACHIEVEMENT_NUMERATOR","NDH001_ACHIEVEMENT_DENOMINATOR"
              ,"NDH001_UL_NET_ACH","NDH001_PCA_COUNT","NDH001_PCA_RATE","NDH001_DEN_PLUS_PCA","NDH001_PRI"]
t8_ndh_df = t8_ndh_df[t8_ndh_col]
                     
t8_ndh_df = t8_ndh_df.sort_values(by=["STP_NAME"],ascending = [True])                                 

In [ ]:
## Select data for CS group code
t8_cs_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "CS"]

t8_cs_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="CS")]


## Combines T8 Master table with previous years data
t8_cs_df = pd.merge(t8_cs_base_df
                    ,t8_cs_stp_current_piv_df    
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'CS' in icb-ach-prev-pca excel workbook (template 8)
t8_cs_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
             ,"NUMBER_PRACTICES","ADD_LIST_25_49_F","ADD_LIST_50_64_F","PREV_ACHIEVED_POINTS"
             ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT"
             ,"PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS"
             ,"OVERALL_PCA_RATE","PCA_CHANGE","CS005_ACHIEVED_POINTS"
             ,"CS005_ACHIEVEMENT_NUMERATOR","CS005_ACHIEVEMENT_DENOMINATOR"
             ,"CS005_UL_NET_ACH","CS005_PCA_COUNT","CS005_PCA_RATE","CS005_DEN_PLUS_PCA","CS005_PRI"
             ,"CS006_ACHIEVED_POINTS","CS006_ACHIEVEMENT_NUMERATOR","CS006_ACHIEVEMENT_DENOMINATOR"
             ,"CS006_UL_NET_ACH","CS006_PCA_COUNT","CS006_PCA_RATE","CS006_DEN_PLUS_PCA","CS006_PRI"]
t8_cs_df = t8_cs_df[t8_cs_col]                    

t8_cs_df = t8_cs_df.sort_values(by=["STP_NAME"],ascending = [True])          

In [ ]:
## Select data for VI group code
t8_vi_base_df = T8_master_df[T8_master_df["GROUP_CODE"] == "VI"]

t8_vi_stp_current_piv_df = t8_stp_current_piv_df.loc[(t8_stp_current_piv_df["GROUP_CODE"]=="VI")]


## Combines T8 Master table with previous years data
t8_vi_df = pd.merge(t8_vi_base_df
                    ,t8_vi_stp_current_piv_df    
                    ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'VI' in icb-ach-prev-pca excel workbook (template 8)
t8_vi_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","PREV_NUMBER_PRACTICES","NUMBER_PRACTICES"
             ,"ADD_LIST_UN01", "ADD_LIST_1_2", "ADD_LIST_4_5", "ADD_LIST_79_80","PREV_ACHIEVED_POINTS"
             ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT"
             ,"PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE"
             ,"PCA_CHANGE","VI001_ACHIEVED_POINTS","VI001_ACHIEVEMENT_NUMERATOR","VI001_ACHIEVEMENT_DENOMINATOR"
             ,"VI001_UL_NET_ACH","VI001_PCA_COUNT","VI001_PCA_RATE","VI001_DEN_PLUS_PCA","VI001_PRI"
             ,"VI002_ACHIEVED_POINTS","VI002_ACHIEVEMENT_NUMERATOR","VI002_ACHIEVEMENT_DENOMINATOR"
             ,"VI002_UL_NET_ACH","VI002_PCA_COUNT","VI002_PCA_RATE","VI002_DEN_PLUS_PCA","VI002_PRI"
             ,"VI003_ACHIEVED_POINTS","VI003_ACHIEVEMENT_NUMERATOR","VI003_ACHIEVEMENT_DENOMINATOR"
             ,"VI003_UL_NET_ACH","VI003_PCA_COUNT","VI003_PCA_RATE","VI003_DEN_PLUS_PCA","VI003_PRI"
             ,"VI004_ACHIEVED_POINTS","VI004_ACHIEVEMENT_NUMERATOR","VI004_ACHIEVEMENT_DENOMINATOR"
             ,"VI004_UL_NET_ACH","VI004_PCA_COUNT","VI004_PCA_RATE","VI004_DEN_PLUS_PCA","VI004_PRI"]
t8_vi_df = t8_vi_df[t8_vi_col]

t8_vi_df = t8_vi_df.sort_values(by=["STP_NAME"],ascending = [True])                   

In [ ]:
## Join QI group codes with numeric key to the master data frame
## This means that QI codes can be reused year-on-year without having to change the body of the code
## QI data is only published for the current year as the indicators change and are therefore not comparable
T8_master_df = pd.merge(QI_gr_codes_with_key
                        ,T8_master_df
                        ,left_on = ["GROUP_CODE"]
                        ,right_on = ["GROUP_CODE"]
                        ,how ="left")

In [ ]:
## Select numeric key for QIPDD group code
t8_qi1_base_df = T8_master_df[T8_master_df["GrC_KEY"] == "QI_1"]

## Select indicator specific data frame
t8_combined_stp_all_qi1 = t8_stp_current_piv_df

## Combines T8 Master table with indicator data
t8_qi1_df = pd.merge(t8_qi1_base_df
                     ,t8_combined_stp_all_qi1
                     ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## Reorder columns for output data frame on tab 'QIPDD' in icb-ach-prev-pca excel workbook (template 8)
t8_qi1_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME"
              ,"NUMBER_PRACTICES","LIST_SIZE"
              ,"TOTAL_ACH_SCORE","ACH_PC"
              ,"QIPDD009_ACHIEVED_POINTS","QIPDD009_ACH_RATE"
              ,"QIPDD010_ACHIEVED_POINTS","QIPDD010_ACH_RATE"]
t8_qi1_df = t8_qi1_df[t8_qi1_col]

t8_qi1_df = t8_qi1_df.sort_values(by=["STP_NAME"],ascending = [True])

In [ ]:
## Select numeric key for QIOA group code
t8_qi2_base_df = T8_master_df[T8_master_df["GrC_KEY"] == "QI_2"]

## Select indicator specific data frame
t8_combined_stp_all_qi2 = t8_stp_current_piv_df

## Combines T8 Master table with indicator data
t8_qi2_df = pd.merge(t8_qi2_base_df
                     ,t8_combined_stp_all_qi2
                     ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## Reorder columns for output data frame on tab 'QIOA' in icb-ach-prev-pca excel workbook (template 8)
t8_qi2_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME"
              ,"NUMBER_PRACTICES","LIST_SIZE"
              ,"TOTAL_ACH_SCORE","ACH_PC"
              ,"QIOA011_ACHIEVED_POINTS","QIOA011_ACH_RATE"
              ,"QIOA012_ACHIEVED_POINTS","QIOA012_ACH_RATE"]
t8_qi2_df = t8_qi2_df[t8_qi2_col]
                         
t8_qi2_df = t8_qi2_df.sort_values(by=["STP_NAME"],ascending = [True])                  

In [ ]:
## Select numeric key for QIPDD_MI group code
t8_qi3_base_df = T8_master_df[T8_master_df["GrC_KEY"] == "QI_3"]

## Select indicator specific data frame
t8_combined_stp_all_qi3 = t8_stp_current_piv_df

## Combines T8 Master table with indicator data
t8_qi3_df = pd.merge(t8_qi3_base_df
                     ,t8_combined_stp_all_qi3
                     ,left_on = ["STP_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["STP_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## This indicator has zero achievement points attributed to it and therefore an achievement percentage cannot be calculated
## The value is replaced with 'z' for data not applicable
t8_qi3_df["ACH_PC"] = t8_qi3_df["ACH_PC"].fillna("z")

## Reorder columns for output data frame on tab 'QIPDD_MI' in icb-ach-prev-pca excel workbook (template 8)
t8_qi3_col = ["STP_ODS_CODE","STP_ONS_CODE","STP_NAME","NUMBER_PRACTICES","ALT_LIST_SIZE"
              ,"TOTAL_ACH_SCORE","ACH_PC","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE"
              ,"QIPDDMI001_ACHIEVED_POINTS","QIPDDMI001_ACHIEVEMENT_NUMERATOR","QIPDDMI001_ACHIEVEMENT_DENOMINATOR"
              ,"QIPDDMI001_UL_NET_ACH","QIPDDMI001_PCA_COUNT","QIPDDMI001_PCA_RATE","QIPDDMI001_DEN_PLUS_PCA","QIPDDMI001_PRI"
              ,"QIPDDMI002_ACHIEVED_POINTS","QIPDDMI002_ACHIEVEMENT_NUMERATOR","QIPDDMI002_ACHIEVEMENT_DENOMINATOR"
              ,"QIPDDMI002_UL_NET_ACH","QIPDDMI002_PCA_COUNT","QIPDDMI002_PCA_RATE","QIPDDMI002_DEN_PLUS_PCA","QIPDDMI002_PRI"
              ,"QIPDDMI003_ACHIEVED_POINTS","QIPDDMI003_ACHIEVEMENT_NUMERATOR","QIPDDMI003_ACHIEVEMENT_DENOMINATOR"
              ,"QIPDDMI003_UL_NET_ACH","QIPDDMI003_PCA_COUNT","QIPDDMI003_PCA_RATE","QIPDDMI003_DEN_PLUS_PCA","QIPDDMI003_PRI"
              ,"QIPDDMI004_ACHIEVED_POINTS","QIPDDMI004_ACHIEVEMENT_NUMERATOR","QIPDDMI004_ACHIEVEMENT_DENOMINATOR"
              ,"QIPDDMI004_UL_NET_ACH","QIPDDMI004_PCA_COUNT","QIPDDMI004_PCA_RATE","QIPDDMI004_DEN_PLUS_PCA","QIPDDMI004_PRI"]
t8_qi3_df = t8_qi3_df[t8_qi3_col] 

t8_qi3_df = t8_qi3_df.sort_values(by=["STP_NAME"],ascending = [True]) 

<h3>Create T8 ICB (STP) Table titles

In [ ]:
## Combine elements that contribute to table title
gr_stp_table_titles = gr_message_df
gr_stp_table_titles["TABLE_NO"] = gr_stp_table_titles["TABLE_NO"].astype(str)
gr_stp_table_titles["FYEAR"] = Control_File.at[0, "FYEAR"]
gr_stp_table_titles["HIGHER_GROUP_DESCRIPTION"] = gr_stp_table_titles["HIGHER_GROUP_DESCRIPTION"].str.lower()
gr_stp_table_titles["GROUP_DESCRIPTION"] = gr_stp_table_titles["GROUP_DESCRIPTION"].str.lower()

gr_stp_table_titles["TABLE_TITLE"] = "Table "+gr_stp_table_titles["TABLE_NO"]+": Prevalence, achievement and personalised care adjustments, " \
                                                                    +gr_stp_table_titles["HIGHER_GROUP_DESCRIPTION"] \
                                                                    +" group, "+gr_stp_table_titles["GROUP_DESCRIPTION"]+", " \
                                                                    +gr_stp_table_titles["FYEAR"]+", ICB level"

gr_stp_table_titles = pd.merge(gr_stp_table_titles,QI_gr_codes_with_key, on = ["GROUP_CODE"],how = 'left')
gr_stp_table_titles["GrC_KEY"] = gr_stp_table_titles["GrC_KEY"].replace(np.nan, "")
gr_stp_table_titles["GROUP_CODE"] = np.where(gr_stp_table_titles["GrC_KEY"]== "", gr_stp_table_titles["GROUP_CODE"], gr_stp_table_titles["GrC_KEY"])

gr_stp_table_titles = gr_stp_table_titles.drop(columns=["FYEAR","HIGHER_GROUP_DESCRIPTION","GROUP_DESCRIPTION","TABLE_NO","GrC_KEY"])
stp_table_titles = gr_stp_table_titles

<b><h1>22 OUTPUTS - T9-PrevAchPCA_sicbl</h1></b><br><h2>Part 1 - QOF_MASTER for current Indicators<br>Sub ICB (CCG) data frame from QOF Master

In [ ]:
"""   
Column names with old nhs geography name CCG are used and relaced by the new 
nhs geography name Sub ICB Loc when the data is output

"""

## Required indicator and achievement columns from QOF Master table
t9_ccg_current_base_col = ["CCG_ODS_CODE","GROUP_CODE","INDICATOR_CODE","ACHIEVED_POINTS"
                           ,"MAX_INDICATOR_POINTS","ACHIEVEMENT_NUMERATOR"
                           ,"ACHIEVEMENT_DENOMINATOR","PCA_COUNT"]                          
t9_ccg_current_base_df = qof_master[t9_ccg_current_base_col]

## Sum all non-index columns
t9_ccg_current_sum_df = t9_ccg_current_base_df.groupby(by = ["CCG_ODS_CODE","GROUP_CODE","INDICATOR_CODE"]
                                                       ,as_index=False
                                                        ).agg({"ACHIEVED_POINTS" : [pd.Series.sum]
                                                               ,"MAX_INDICATOR_POINTS" : [pd.Series.sum]
                                                               ,"ACHIEVEMENT_NUMERATOR" : [pd.Series.sum]
                                                               ,"ACHIEVEMENT_DENOMINATOR" : [pd.Series.sum]
                                                               ,"PCA_COUNT" : [pd.Series.sum]})                                                                                                                                                         
t9_ccg_current_sum_df.columns = t9_ccg_current_sum_df.columns.droplevel(1)
                                                                                                          
                                                                                                       
## Create all percentage columns
## Achievement Rate
t9_ccg_current_sum_df["ACH_RATE"] = (t9_ccg_current_sum_df["ACHIEVED_POINTS"]/t9_ccg_current_sum_df["MAX_INDICATOR_POINTS"])*100

## Underlying Achievement net of PCAs
t9_ccg_current_sum_df["UL_NET_ACH"] = (t9_ccg_current_sum_df["ACHIEVEMENT_NUMERATOR"]/t9_ccg_current_sum_df["ACHIEVEMENT_DENOMINATOR"])*100

## PCA Rate
t9_ccg_current_sum_df["PCA_RATE"] = (t9_ccg_current_sum_df["PCA_COUNT"]/(t9_ccg_current_sum_df["ACHIEVEMENT_DENOMINATOR"]\
                                                                         +t9_ccg_current_sum_df["PCA_COUNT"]))*100

## Denominator plus PCAs
t9_ccg_current_sum_df["DEN_PLUS_PCA"] = (t9_ccg_current_sum_df["ACHIEVEMENT_DENOMINATOR"]+t9_ccg_current_sum_df["PCA_COUNT"])

## Patients Receiving Intervention
t9_ccg_current_sum_df["PRI"] = (t9_ccg_current_sum_df["ACHIEVEMENT_NUMERATOR"]/(t9_ccg_current_sum_df["ACHIEVEMENT_DENOMINATOR"]\
                                                                                +t9_ccg_current_sum_df["PCA_COUNT"]))*100

## Drop unwanted column
t9_ccg_current_sum_df = t9_ccg_current_sum_df.drop(columns=["MAX_INDICATOR_POINTS"])                                                                                                          

In [ ]:
## The short broad data frame is converted into a long thin data frame and the non-index columns 
## are stacked in a column called 'FIELD'
t9_ccg_ind_pre_stacked = (t9_ccg_current_sum_df.set_index(["CCG_ODS_CODE","GROUP_CODE","INDICATOR_CODE"])
                                               .stack()
                                               .ffill(axis=0)
                                               .bfill(axis=0, downcast='infer')
                                               .reset_index()
                                               .rename({"level_3":"FIELD", 0:"VALUE"}, axis=1))

## New column 'INDICATOR' is created by joining the 'INDICATOR_CODE' to 'FIELD' 
## with a hyphen and drop unwanted columns e.g. AF001_ACHIEVED_POINTS
t9_ccg_ind_pre_stacked["INDICATOR"] = (t9_ccg_ind_pre_stacked["INDICATOR_CODE"]
                                       )+"_"+(t9_ccg_ind_pre_stacked["FIELD"])

## Drop unwanted columns
t9_combined_ccg_all_col = ["CCG_ODS_CODE","GROUP_CODE","INDICATOR","VALUE"]
t9_combined_ccg_all = t9_ccg_ind_pre_stacked[t9_combined_ccg_all_col]

In [ ]:
## Pivot data frame from long thin data frame into short broad data frame
## With each column containing an individual elements of each indicator 
## e.g. 'AF001_ACHIEVED_POINTS', 'AF001_ACH_RATE'
t9_ccg_current_piv_df = pd.pivot_table(t9_combined_ccg_all
                                         ,values="VALUE"
                                         ,index=["CCG_ODS_CODE","GROUP_CODE"]
                                         ,columns= "INDICATOR")

t9_ccg_current_piv_df = t9_ccg_current_piv_df.reset_index().rename_axis(None, axis=1)

<h2>Part 2 - Join QOF Master to last years Sub ICB (CCG) data frame

In [ ]:
"""   
Creates a Sub ICB (CCG) level data frame that comprises data for the current year 
and previous year

"""

## Drop unwanted nhs geography columns from the QOF Master table
t9_ccg_qof_master = qof_master.drop(columns=["NAT_ONS_CODE","NAT_CODE","NAT_COUNTRY"
                                             ,"REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME"
                                             ,"STP_ODS_CODE","STP_ONS_CODE","STP_NAME"
                                             ,"PCN_ODS_CODE","PCN_NAME","PRACTICE_NAME"
                                             ,"DOMAIN_CODE","SUB_DOMAIN_CODE"])
                                                   

## Drop unwanted column from the data frame that combines data from the previous year at Sub ICB (CCG) level
t9_prev_out_ccg_joined = previous_prevalence_ccg_data_df.drop(columns=["HIGHER_GROUP_CODE"])

## The resulting data frame is used as the foundation for four other code blocks below
t9_ccg_current_previous_df = pd.merge(t9_ccg_qof_master
                                      ,t9_prev_out_ccg_joined
                                      ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                                      ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                                      ,how ="left") 

In [ ]:
## Select columns required for Sub ICB (CCG) group data frame for the previous year
## These columns are used as the index to which other columns are joined
t9_ccg_previous_gr_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","GROUP_CODE"
                          ,"ALT_PAT_LIST","PREV_NUMBER_PRACTICES"
                          ,"PREV_PRACTICE_LIST_SIZE","PREV_ALT_PAT_LIST"
                          ,"PREV_PREVALENCE","PREV_REGISTER_SIZE","PREV_ACHIEVED_POINTS"
                          ,"PREV_TOTAL_INDICATOR_GROUP_POINTS","PREV_ACHIEVEMENT_RATE"
                          ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"]
t9_ccg_previous_gr_index_df = t9_ccg_current_previous_df[t9_ccg_previous_gr_col]

t9_ccg_previous_gr_index_df = t9_ccg_previous_gr_index_df.drop_duplicates().reset_index(drop=True)                                  

In [ ]:
## Select columns required for Sub ICB (CCG) group data frame for the previous year
## Count distinct number of practices by indicator group
t9_ccg_prac_gr_col = ["CCG_ONS_CODE","PRACTICE_CODE","GROUP_CODE"]
t9_ccg_prac_gr_df = t9_ccg_current_previous_df[t9_ccg_prac_gr_col]
t9_ccg_prac_gr_df = t9_ccg_prac_gr_df.drop_duplicates().reset_index(drop=True)

t9_ccg_prac_gr_df = t9_ccg_prac_gr_df.groupby(by = ["CCG_ONS_CODE","GROUP_CODE"]
                                              ,as_index=False
                                              ).agg({"PRACTICE_CODE" : [pd.Series.count]})                                                     
t9_ccg_prac_gr_df.columns = t9_ccg_prac_gr_df.columns.droplevel(1)  

## Rename column 'PRACTICE_CODE' which contains a numeric count as 'NUMBER_PRACTICES'
t9_ccg_prac_gr_df = t9_ccg_prac_gr_df.rename(columns={"PRACTICE_CODE":"NUMBER_PRACTICES"})

In [ ]:
## Select columns required for Sub ICB (CCG) group data frame for the previous year
## Count distinct number of indicators by indicator group
t9_ccg_ind_gr_col = ["CCG_ONS_CODE","INDICATOR_CODE","GROUP_CODE"]
t9_ccg_ind_gr_df = t9_ccg_current_previous_df[t9_ccg_ind_gr_col]

t9_ccg_ind_gr_df = t9_ccg_ind_gr_df.drop_duplicates().reset_index(drop=True)

t9_ccg_ind_gr_df = t9_ccg_ind_gr_df.groupby(by = ["CCG_ONS_CODE","GROUP_CODE"]
                                            ,as_index=False
                                            ).agg({"INDICATOR_CODE" : [pd.Series.count]})
t9_ccg_ind_gr_df.columns = t9_ccg_ind_gr_df.columns.droplevel(1)  

## Rename column 'PRACTICE_CODE' which contains a numeric count as 'NUMBER_PRACTICES'
t9_ccg_ind_gr_df = t9_ccg_ind_gr_df.rename(columns={"PRACTICE_CODE":"NUMBER_PRACTICES"})

In [ ]:
## Select columns required for Sub ICB (CCG) group data frame 
## Aggregate data in non index columns
t9_ccg_gr_current_aggregated_df = t9_ccg_current_previous_df.groupby(["CCG_ONS_CODE","GROUP_CODE"]).agg({"LIST_SIZE" : [pd.Series.sum]
                                                                                                         ,"ALT_LIST_SIZE" : [pd.Series.sum]
                                                                                                         ,"REGISTER_SIZE" : [pd.Series.sum]
                                                                                                         ,"ACHIEVED_POINTS" : [pd.Series.sum]
                                                                                                         ,"PCA_COUNT" : [pd.Series.sum]
                                                                                                         ,"ACHIEVEMENT_DENOMINATOR" : [pd.Series.sum]
                                                                                                         ,"MAX_GROUP_POINTS" : [pd.Series.max]})                                                                            
t9_ccg_gr_current_aggregated_df.columns = t9_ccg_gr_current_aggregated_df.columns.droplevel(1)

## Rename columns 'LIST_SIZE' as 'LIST_SIZE_SUMMED' and 'ALT_LIST_SIZE' as 'ALT_LIST_SIZE_SUMMED' which contain total counts
t9_ccg_gr_current_aggregated_df = t9_ccg_gr_current_aggregated_df.rename(columns={"LIST_SIZE":"LIST_SIZE_SUMMED"
                                                                                  ,"ALT_LIST_SIZE":"ALT_LIST_SIZE_SUMMED"})

In [ ]:
## Join aggregated data to geographies and measures that are not aggregated
t9_ccg_gr_df = pd.merge(t9_ccg_previous_gr_index_df
                        ,t9_ccg_prac_gr_df
                        ,left_on = ["CCG_ONS_CODE","GROUP_CODE"]
                        ,right_on = ["CCG_ONS_CODE","GROUP_CODE"]
                        ,how ="left")

t9_ccg_gr_df = pd.merge(t9_ccg_gr_df
                        ,t9_ccg_ind_gr_df
                        ,left_on = ["CCG_ONS_CODE","GROUP_CODE"]
                        ,right_on = ["CCG_ONS_CODE","GROUP_CODE"]
                        ,how ="left")

t9_ccg_gr_df = pd.merge(t9_ccg_gr_df
                        ,t9_ccg_gr_current_aggregated_df
                        ,left_on = ["CCG_ONS_CODE","GROUP_CODE"]
                        ,right_on = ["CCG_ONS_CODE","GROUP_CODE"]
                        ,how ="left")

In [ ]:
#Create finished Sub ICB (CCG) data set for T9 processing

## Create 'LIST_SIZE' column and drop aggregated 'LIST_SIZE_SUMMED' as it is no longer needed
t9_ccg_gr_df["LIST_SIZE"] = t9_ccg_gr_df["LIST_SIZE_SUMMED"]/t9_ccg_gr_df["INDICATOR_CODE"]
t9_ccg_gr_df = t9_ccg_gr_df.drop(columns=["LIST_SIZE_SUMMED"])

## Create column 'ALT_LIST_SIZE' and leave values blank if there is no value in the corresponding 'ALT_PAT_LIST'
t9_ccg_gr_df["ALT_LIST_SIZE"] = np.where(t9_ccg_gr_df["ALT_PAT_LIST"] != None
                                         ,(t9_ccg_gr_df["ALT_LIST_SIZE_SUMMED"]/t9_ccg_gr_df["INDICATOR_CODE"])
                                         ,"")

## Data columns that contain data totals with the prefix 'TOTAL'
t9_ccg_gr_df = t9_ccg_gr_df.rename(columns={"INDICATOR_CODE":"COUNT_DISTINCT_IND_CODE"
                                            ,"ACHIEVED_POINTS":"TOTAL_ACH_SCORE"
                                            ,"PCA_COUNT":"TOTAL_PCAS"
                                            ,"ACHIEVEMENT_DENOMINATOR":"TOTAL_DENOMINATORS"
                                            ,"REGISTER_SIZE":"REGISTER"})
                                                                                                          
## Select ordered Sub ICB (CCG) level columns to which calculated columns can be added
t9_ccg_gr_col = ["GROUP_CODE","CCG_ONS_CODE","CCG_ODS_CODE","CCG_NAME","NUMBER_PRACTICES","LIST_SIZE"
                               ,"ALT_LIST_SIZE","ALT_LIST_SIZE_SUMMED","COUNT_DISTINCT_IND_CODE","REGISTER"
                               ,"TOTAL_ACH_SCORE","MAX_GROUP_POINTS","TOTAL_PCAS","TOTAL_DENOMINATORS"
                               ,"PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE","PREV_ALT_PAT_LIST"
                               ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","PREV_ACHIEVED_POINTS"
                               ,"PREV_TOTAL_INDICATOR_GROUP_POINTS","PREV_ACHIEVEMENT_RATE"
                               ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"]
t9_ccg_base_df = t9_ccg_gr_df[t9_ccg_gr_col]                                              
                                               

In [ ]:
"""   
Prevalence, Achievement Score, Achievement Percentage and Overall PCA rate
are calculated for Sub ICB (CCG) data and comprise part of the final published data frames
in sicbl-ach-prev-pca.xlsx

"""

## Change the data type of the column 'ALT_LIST_SIZE'used in the calcualtions
t9_ccg_base_df["ALT_LIST_SIZE"] = t9_ccg_base_df["ALT_LIST_SIZE"].astype(float)

## Calculate Prevalence
t9_ccg_base_df["PREVALENCE"] = np.where(t9_ccg_base_df["ALT_LIST_SIZE"] == 0
                                        ,(t9_ccg_base_df["REGISTER"]/t9_ccg_base_df["LIST_SIZE"])*100
                                        ,(t9_ccg_base_df["REGISTER"]/t9_ccg_base_df["ALT_LIST_SIZE"])*100)
                                                 
## Calculate Achievement Score
t9_ccg_base_df["AVG_ACH_SCORE"] = t9_ccg_base_df["TOTAL_ACH_SCORE"]/t9_ccg_base_df["NUMBER_PRACTICES"]

## Calculate Achievement Percentage
t9_ccg_base_df["ACH_PC"] = ((t9_ccg_base_df["TOTAL_ACH_SCORE"]/
                             t9_ccg_base_df["NUMBER_PRACTICES"])/t9_ccg_base_df["MAX_GROUP_POINTS"])*100

## Calculate Overall PCA rate
t9_ccg_base_df["OVERALL_PCA_RATE"] = (t9_ccg_base_df["TOTAL_PCAS"]/
                                      (t9_ccg_base_df["TOTAL_DENOMINATORS"]+t9_ccg_base_df["TOTAL_PCAS"]))*100

## Reorder columns to include new values
t9_ccg_base_col = ["GROUP_CODE","CCG_ONS_CODE","CCG_ODS_CODE","CCG_NAME","NUMBER_PRACTICES"
                   ,"LIST_SIZE","ALT_LIST_SIZE","REGISTER","PREVALENCE"
                   ,"TOTAL_ACH_SCORE","AVG_ACH_SCORE","ACH_PC","TOTAL_PCAS"
                   ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PREV_NUMBER_PRACTICES"
                   ,"PREV_PRACTICE_LIST_SIZE","PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE"
                   ,"PREV_PREVALENCE","PREV_ACHIEVED_POINTS","PREV_TOTAL_INDICATOR_GROUP_POINTS"
                   ,"PREV_ACHIEVEMENT_RATE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"]
t9_ccg_base_df = t9_ccg_base_df[t9_ccg_base_col]                                                 

<h2>Part 3 - Create SUB ICB (CCG) Additional List df

In [ ]:
"""   
Addional practice list sizes 'ADD_PAT_SIZE' with the accompanying description column 'ADD_PAT_LIST'
are created at SUB ICB (CCG) level by indicator group and are joined to the data frame created in Part 2

"""

## Sub ICB (CCG) data frame defined this forms the foundation (base) of the data frame to be created
t9_ccg_add_list_col = ["CCG_ODS_CODE","GROUP_CODE","ADD_PAT_LIST"
                       ,"ADD_LIST_SIZE","INDICATOR_CODE"]
t9_ccg_add_list_base_df = t9_ccg_current_previous_df[t9_ccg_add_list_col]

In [ ]:
## Count distinct number of indicators
t9_ccg_add_list_ind_col = ["CCG_ODS_CODE","INDICATOR_CODE","GROUP_CODE"]
t9_ccg_add_list_ind_df = t9_ccg_add_list_base_df[t9_ccg_add_list_ind_col]
t9_ccg_add_list_ind_df = t9_ccg_add_list_ind_df.drop_duplicates()

t9_ccg_add_list_ind_df = t9_ccg_add_list_ind_df.groupby(by = ["CCG_ODS_CODE","GROUP_CODE"]
                                                        ,as_index=False
                                                        ).agg({"INDICATOR_CODE" : [pd.Series.count]})                                                              
t9_ccg_add_list_ind_df.columns = t9_ccg_add_list_ind_df.columns.droplevel(1) 

## Rename column 'INDICATOR_CODE' as 'COUNT_DISTINCT_IND_CODE' which contains counts
t9_ccg_add_list_ind_df = t9_ccg_add_list_ind_df.rename(columns={"INDICATOR_CODE":"COUNT_DISTINCT_IND_CODE"})

## Drop unwanted column
t9_ccg_add_list_df = t9_ccg_add_list_base_df.drop(columns=["INDICATOR_CODE"])

## Sum ADD_LIST_SIZE (additional list size)
t9_ccg_add_list_sum_df = t9_ccg_add_list_df.groupby(by = ["CCG_ODS_CODE","GROUP_CODE","ADD_PAT_LIST"]
                                                    ,as_index=False
                                                    )["ADD_LIST_SIZE"].sum()

In [ ]:
## Combine aggregated data frames
t9_ccg_add_list_df = pd.merge(t9_ccg_add_list_sum_df
                              ,t9_ccg_add_list_ind_df
                              ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                              ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                              ,how ="left") 

## Define new column 'ADD_LIST' e.g. ADD_LIST_06OV
t9_ccg_add_list_df["ADD_LIST"] = "ADD_LIST_"+t9_ccg_add_list_df["ADD_PAT_LIST"]

## Drop unwanted columns
t9_ccg_add_list_df = t9_ccg_add_list_df.drop(columns=["ADD_PAT_LIST","COUNT_DISTINCT_IND_CODE"])

## Reorder columns
t9_ccg_add_list_col = ["CCG_ODS_CODE","GROUP_CODE","ADD_LIST","ADD_LIST_SIZE"]
t9_ccg_add_list_df = t9_ccg_add_list_df[t9_ccg_add_list_col]

In [ ]:
## Pivot t9_ccg_add_list_df from a long thin data frame to a short broad data frame with 
## a column for each additional list size with 'CCG_ODS_CODE, and 'GROUP_CODE' as the index
t9_add_list_df = pd.pivot_table(t9_ccg_add_list_df
                                ,values="ADD_LIST_SIZE"
                                ,index=["CCG_ODS_CODE", "GROUP_CODE"]
                                ,columns= "ADD_LIST")

t9_add_list_df = t9_add_list_df.reset_index().rename_axis(None, axis=1)

<h3>Join Part 3 (Additional List df) to Part 2 (previous data)

In [ ]:
## Join additional list sizes to main Sub ICB (CCG) data frame
T9_master_df = pd.merge(t9_ccg_base_df
                        ,t9_add_list_df
                        ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                        ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                        ,how ="left")

## Calculate the difference in the previous year's percentages to the curent years as percentage points
T9_master_df["PREV_CHANGE"] = T9_master_df["PREVALENCE"]-T9_master_df["PREV_PREVALENCE"]
T9_master_df["ACH_CHANGE"] = T9_master_df["ACH_PC"]-T9_master_df["PREV_ACHIEVEMENT_RATE"]
T9_master_df["PCA_CHANGE"] = T9_master_df["OVERALL_PCA_RATE"]-T9_master_df["PREV_PCA_RATE"]

T9_master_df = T9_master_df.drop_duplicates()

Create variable source data

In [ ]:
## Creates a list of 'CCG_ONS_CODE' which is use to 
## create the variable 'ccgcount' in the 'Publication Outputs
ccg_list = T9_master_df[["CCG_ONS_CODE"]]
ccg_list = ccg_list.drop_duplicates().reset_index(drop = True)

<h2>Create SUB ICB (CCG) data tables<br> and table titles

In [ ]:
"""   
Not all indicator group tables have the same data components. 
Therefore it is necessary to define the content of each indicator group data frame separately. 
The code block are in the indicator group order as they appear in the sicbl-ach-prev-pca excel workbook (template 8)

"""

## Select data for AF group code
t9_af_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "AF"]

t9_af_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="AF")]


## Combines T9 Master table with previous years data
t9_af_df = pd.merge(t9_af_base_df
                    ,t9_af_ccg_current_piv_df
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'AF' in sicbl-ach-prev-pca excel workbook (template 9)
t9_af_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","REGISTER"
             ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
             ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR"
             ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
             ,"AF001_ACHIEVED_POINTS","AF001_ACH_RATE","AF006_ACHIEVED_POINTS"
             ,"AF006_ACHIEVEMENT_NUMERATOR","AF006_ACHIEVEMENT_DENOMINATOR","AF006_UL_NET_ACH"
             ,"AF006_PCA_COUNT","AF006_PCA_RATE","AF006_DEN_PLUS_PCA","AF006_PRI","AF007_ACHIEVED_POINTS"
             ,"AF007_ACHIEVEMENT_NUMERATOR","AF007_ACHIEVEMENT_DENOMINATOR","AF007_UL_NET_ACH"
             ,"AF007_PCA_COUNT","AF007_PCA_RATE","AF007_DEN_PLUS_PCA","AF007_PRI"]
t9_af_df = t9_af_df[t9_af_col]

t9_af_df = t9_af_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)          

In [ ]:
## Select data for BP group code
t9_bp_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "BP"]

t9_bp_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="BP")]


## Combines T9 Master table with previous years data
t9_bp_df = pd.merge(t9_bp_base_df
                    ,t9_bp_ccg_current_piv_df
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'BP' in sicbl-ach-prev-pca excel workbook (template 9)
t9_bp_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
             ,"NUMBER_PRACTICES","ALT_LIST_SIZE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
             ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR"
             ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
             ,"BP002_ACHIEVED_POINTS","BP002_ACHIEVEMENT_NUMERATOR","BP002_ACHIEVEMENT_DENOMINATOR"
             ,"BP002_UL_NET_ACH","BP002_PCA_COUNT","BP002_PCA_RATE","BP002_DEN_PLUS_PCA","BP002_PRI"]
t9_bp_df = t9_bp_df[t9_bp_col]

t9_bp_df = t9_bp_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)   

In [ ]:
## Select data for CHD group code
t9_chd_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "CHD"]

t9_chd_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="CHD")]


## Combines T9 Master table with previous years data
t9_chd_df = pd.merge(t9_chd_base_df
                    ,t9_chd_ccg_current_piv_df
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'CHD' in sicbl-ach-prev-pca excel workbook (template 9)
t9_chd_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE", "NUMBER_PRACTICES","LIST_SIZE","ADD_LIST_79UN"
              ,"ADD_LIST_80OV","REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS"
              ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT"
              ,"PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE"
              ,"PCA_CHANGE","CHD001_ACHIEVED_POINTS","CHD001_ACH_RATE"
              ,"CHD005_ACHIEVED_POINTS","CHD005_ACHIEVEMENT_NUMERATOR","CHD005_ACHIEVEMENT_DENOMINATOR"
              ,"CHD005_UL_NET_ACH","CHD005_PCA_COUNT","CHD005_PCA_RATE","CHD005_DEN_PLUS_PCA","CHD005_PRI"
              ,"CHD008_ACHIEVED_POINTS","CHD008_ACHIEVEMENT_NUMERATOR","CHD008_ACHIEVEMENT_DENOMINATOR"
              ,"CHD008_UL_NET_ACH","CHD008_PCA_COUNT","CHD008_PCA_RATE","CHD008_DEN_PLUS_PCA","CHD008_PRI"
              ,"CHD009_ACHIEVED_POINTS","CHD009_ACHIEVEMENT_NUMERATOR","CHD009_ACHIEVEMENT_DENOMINATOR"
              ,"CHD009_UL_NET_ACH","CHD009_PCA_COUNT","CHD009_PCA_RATE","CHD009_DEN_PLUS_PCA","CHD009_PRI"]
t9_chd_df = t9_chd_df[t9_chd_col]
                      
t9_chd_df = t9_chd_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)   

In [ ]:
## Select data for HF group code
t9_hf_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "HF"]

t9_hf_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="HF")]


## Combines T9 Master table with previous years data
t9_hf_df = pd.merge(t9_hf_base_df
                    ,t9_hf_ccg_current_piv_df
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'HF' in sicbl-ach-prev-pca excel workbook (template 9)
t9_hf_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","REGISTER"
             ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE"
             ,"ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
             ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE","HF001_ACHIEVED_POINTS","HF001_ACH_RATE"
             ,"HF003_ACHIEVED_POINTS","HF003_ACHIEVEMENT_NUMERATOR","HF003_ACHIEVEMENT_DENOMINATOR"
             ,"HF003_UL_NET_ACH","HF003_PCA_COUNT","HF003_PCA_RATE","HF003_DEN_PLUS_PCA","HF003_PRI"
             ,"HF005_ACHIEVED_POINTS","HF005_ACHIEVEMENT_NUMERATOR","HF005_ACHIEVEMENT_DENOMINATOR"
             ,"HF005_UL_NET_ACH","HF005_PCA_COUNT","HF005_PCA_RATE","HF005_DEN_PLUS_PCA","HF005_PRI"
             ,"HF006_ACHIEVED_POINTS","HF006_ACHIEVEMENT_NUMERATOR","HF006_ACHIEVEMENT_DENOMINATOR"
             ,"HF006_UL_NET_ACH","HF006_PCA_COUNT","HF006_PCA_RATE","HF006_DEN_PLUS_PCA","HF006_PRI"
             ,"HF007_ACHIEVED_POINTS","HF007_ACHIEVEMENT_NUMERATOR","HF007_ACHIEVEMENT_DENOMINATOR"
             ,"HF007_UL_NET_ACH","HF007_PCA_COUNT","HF007_PCA_RATE","HF007_DEN_PLUS_PCA","HF007_PRI"]
t9_hf_df = t9_hf_df[t9_hf_col]

t9_hf_df = t9_hf_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)          

In [ ]:
## Select data for HYP group code
t9_hyp_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "HYP"]

t9_hyp_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="HYP")]


## Combines T9 Master table with previous years data
t9_hyp_df = pd.merge(t9_hyp_base_df
                    ,t9_hyp_ccg_current_piv_df
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'HYP' in sicbl-ach-prev-pca excel workbook (template 9)
t9_hyp_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","ADD_LIST_79UN"
              ,"ADD_LIST_80OV","REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS"
              ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT"
              ,"PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE"
              ,"PCA_CHANGE","HYP001_ACHIEVED_POINTS","HYP001_ACH_RATE","HYP003_ACHIEVED_POINTS"
              ,"HYP003_ACHIEVEMENT_NUMERATOR","HYP003_ACHIEVEMENT_DENOMINATOR","HYP003_UL_NET_ACH"
              ,"HYP003_PCA_COUNT","HYP003_PCA_RATE","HYP003_DEN_PLUS_PCA","HYP003_PRI"
              ,"HYP007_ACHIEVED_POINTS","HYP007_ACHIEVEMENT_NUMERATOR","HYP007_ACHIEVEMENT_DENOMINATOR"
              ,"HYP007_UL_NET_ACH","HYP007_PCA_COUNT","HYP007_PCA_RATE","HYP007_DEN_PLUS_PCA","HYP007_PRI"]
t9_hyp_df = t9_hyp_df[t9_hyp_col]

t9_hyp_df = t9_hyp_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)                                    

In [ ]:
## Select data for PAD group code
t9_pad_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "PAD"]

t9_pad_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="PAD")]


## Combines T9 Master table with previous years data
t9_pad_df = pd.merge(t9_pad_base_df
                    ,t9_pad_ccg_current_piv_df
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'PAD' in sicbl-ach-prev-pca excel workbook (template 9)
t9_pad_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","REGISTER"
              ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE"
              ,"ACH_PC","ACH_CHANGE","PAD001_ACHIEVED_POINTS","PAD001_ACH_RATE"]
t9_pad_df = t9_pad_df[t9_pad_col]
                       
t9_pad_df = t9_pad_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)                    

In [ ]:
## Select data for STIA group code
t9_stia_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "STIA"]

t9_stia_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="STIA")]


## Combines T9 Master table with previous years data
t9_stia_df = pd.merge(t9_stia_base_df
                      ,t9_stia_ccg_current_piv_df
                      ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                      ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                      ,how ="left")

## Reorder and sort columns for output data frame on tab 'STIA' in sicbl-ach-prev-pca excel workbook (template 9)
t9_stia_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
               ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","ADD_LIST_79UN"
               ,"ADD_LIST_80OV","REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS"
               ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT"
               ,"PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE"
               ,"PCA_CHANGE","STIA001_ACHIEVED_POINTS","STIA001_ACH_RATE"
               ,"STIA007_ACHIEVED_POINTS","STIA007_ACHIEVEMENT_NUMERATOR","STIA007_ACHIEVEMENT_DENOMINATOR"
               ,"STIA007_UL_NET_ACH","STIA007_PCA_COUNT","STIA007_PCA_RATE","STIA007_DEN_PLUS_PCA","STIA007_PRI"
               ,"STIA010_ACHIEVED_POINTS","STIA010_ACHIEVEMENT_NUMERATOR","STIA010_ACHIEVEMENT_DENOMINATOR"
               ,"STIA010_UL_NET_ACH","STIA010_PCA_COUNT","STIA010_PCA_RATE","STIA010_DEN_PLUS_PCA","STIA010_PRI"
               ,"STIA011_ACHIEVED_POINTS","STIA011_ACHIEVEMENT_NUMERATOR","STIA011_ACHIEVEMENT_DENOMINATOR"
               ,"STIA011_UL_NET_ACH","STIA011_PCA_COUNT","STIA011_PCA_RATE","STIA011_DEN_PLUS_PCA","STIA011_PRI"]
t9_stia_df = t9_stia_df[t9_stia_col]

t9_stia_df = t9_stia_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)                                           

In [ ]:
## Select data for AST group code
t9_ast_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "AST"]

t9_ast_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="AST")]


## Combines T9 Master table with previous years data
t9_ast_df = pd.merge(t9_ast_base_df
                      ,t9_ast_ccg_current_piv_df
                      ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                      ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                      ,how ="left")

## Reorder and sort columns for output data frame on tab 'AST' in sicbl-ach-prev-pca excel workbook (template 9)
t9_ast_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE"
              ,"PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE","ADD_LIST_06_19","REGISTER","PREVALENCE"
              ,"PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
              ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS"
              ,"OVERALL_PCA_RATE","PCA_CHANGE","AST005_ACHIEVED_POINTS","AST005_ACH_RATE"
              ,"AST006_ACHIEVED_POINTS","AST006_ACHIEVEMENT_NUMERATOR","AST006_ACHIEVEMENT_DENOMINATOR"
              ,"AST006_UL_NET_ACH","AST006_PCA_COUNT","AST006_PCA_RATE","AST006_DEN_PLUS_PCA","AST006_PRI"
              ,"AST007_ACHIEVED_POINTS","AST007_ACHIEVEMENT_NUMERATOR","AST007_ACHIEVEMENT_DENOMINATOR"
              ,"AST007_UL_NET_ACH","AST007_PCA_COUNT","AST007_PCA_RATE","AST007_DEN_PLUS_PCA","AST007_PRI"
              ,"AST008_ACHIEVED_POINTS","AST008_ACHIEVEMENT_NUMERATOR","AST008_ACHIEVEMENT_DENOMINATOR"
              ,"AST008_UL_NET_ACH","AST008_PCA_COUNT","AST008_PCA_RATE","AST008_DEN_PLUS_PCA","AST008_PRI"]
t9_ast_df = t9_ast_df[t9_ast_col]                                        

t9_ast_df = t9_ast_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)             

In [ ]:
## Select data for COPD group code
t9_copd_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "COPD"]

t9_copd_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="COPD")]


## Combines T9 Master table with previous years data
t9_copd_df = pd.merge(t9_copd_base_df
                      ,t9_copd_ccg_current_piv_df
                      ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                      ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                      ,how ="left")

## Reorder and sort columns for output data frame on tab 'COPD' in sicbl-ach-prev-pca excel workbook (template 9)
t9_copd_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
               ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","REGISTER"
               ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
               ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR"
               ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE"
               ,"PCA_CHANGE","COPD009_ACHIEVED_POINTS","COPD009_ACH_RATE"
               ,"COPD008_ACHIEVED_POINTS","COPD008_ACHIEVEMENT_NUMERATOR","COPD008_ACHIEVEMENT_DENOMINATOR"
               ,"COPD008_UL_NET_ACH","COPD008_PCA_COUNT","COPD008_PCA_RATE","COPD008_DEN_PLUS_PCA","COPD008_PRI"
               ,"COPD010_ACHIEVED_POINTS","COPD010_ACHIEVEMENT_NUMERATOR","COPD010_ACHIEVEMENT_DENOMINATOR"
               ,"COPD010_UL_NET_ACH","COPD010_PCA_COUNT","COPD010_PCA_RATE","COPD010_DEN_PLUS_PCA","COPD010_PRI"]
t9_copd_df = t9_copd_df[t9_copd_col]
                        
t9_copd_df = t9_copd_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)                      

In [ ]:
## Select data for OB group code
t9_ob_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "OB"]

t9_ob_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="OB")]


## Combines T9 Master table with previous years data
t9_ob_df = pd.merge(t9_ob_base_df
                    ,t9_ob_ccg_current_piv_df
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'OB' in sicbl-ach-prev-pca excel workbook (template 9)
t9_ob_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE"
             ,"REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS"
             ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
             ,"OB002_ACHIEVED_POINTS","OB002_ACH_RATE"]
t9_ob_df = t9_ob_df[t9_ob_col]

t9_ob_df = t9_ob_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)
                    

In [ ]:
## Select data for SMOK group code
t9_smok_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "SMOK"]

t9_smok_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="SMOK")]


## Combines T9 Master table with previous years data
t9_smok_df = pd.merge(t9_smok_base_df
                      ,t9_smok_ccg_current_piv_df
                      ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                      ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                      ,how ="left")

## Reorder and sort columns for output data frame on tab 'SMOK' in sicbl-ach-prev-pca excel workbook (template 9)
t9_smok_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
               ,"NUMBER_PRACTICES","ALT_LIST_SIZE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
               ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR"
               ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
               ,"SMOK002_ACHIEVED_POINTS","SMOK002_ACHIEVEMENT_NUMERATOR","SMOK002_ACHIEVEMENT_DENOMINATOR"
               ,"SMOK002_UL_NET_ACH","SMOK002_PCA_COUNT","SMOK002_PCA_RATE","SMOK002_DEN_PLUS_PCA","SMOK002_PRI"
               ,"SMOK004_ACHIEVED_POINTS","SMOK004_ACHIEVEMENT_NUMERATOR","SMOK004_ACHIEVEMENT_DENOMINATOR"
               ,"SMOK004_UL_NET_ACH","SMOK004_PCA_COUNT","SMOK004_PCA_RATE","SMOK004_DEN_PLUS_PCA","SMOK004_PRI"
               ,"SMOK005_ACHIEVED_POINTS","SMOK005_ACHIEVEMENT_NUMERATOR","SMOK005_ACHIEVEMENT_DENOMINATOR"
               ,"SMOK005_UL_NET_ACH","SMOK005_PCA_COUNT","SMOK005_PCA_RATE","SMOK005_DEN_PLUS_PCA","SMOK005_PRI"]
t9_smok_df = t9_smok_df[t9_smok_col]

t9_smok_df = t9_smok_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)                   

In [ ]:
## Select data for CAN group code
t9_can_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "CAN"]

t9_can_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="CAN")]


## Combines T9 Master table with previous years data
t9_can_df = pd.merge(t9_can_base_df
                     ,t9_can_ccg_current_piv_df
                     ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## Reorder and sort columns for output data frame on tab 'CAN' in sicbl-ach-prev-pca excel workbook (template 9)
t9_can_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","REGISTER","PREVALENCE"
              ,"PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
              ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS"
              ,"OVERALL_PCA_RATE","PCA_CHANGE","CAN001_ACHIEVED_POINTS","CAN001_ACH_RATE"
              ,"CAN004_ACHIEVED_POINTS","CAN004_ACHIEVEMENT_NUMERATOR","CAN004_ACHIEVEMENT_DENOMINATOR"
              ,"CAN004_UL_NET_ACH","CAN004_PCA_COUNT","CAN004_PCA_RATE","CAN004_DEN_PLUS_PCA","CAN004_PRI"
              ,"CAN005_ACHIEVED_POINTS","CAN005_ACHIEVEMENT_NUMERATOR","CAN005_ACHIEVEMENT_DENOMINATOR"
              ,"CAN005_UL_NET_ACH","CAN005_PCA_COUNT","CAN005_PCA_RATE","CAN005_DEN_PLUS_PCA","CAN005_PRI"]
t9_can_df = t9_can_df[t9_can_col] 

t9_can_df = t9_can_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)         

In [ ]:
## Select data for CKD group code
t9_ckd_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "CKD"]

t9_ckd_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="CKD")]


## Combines T9 Master table with previous years data
t9_ckd_df = pd.merge(t9_ckd_base_df
                     ,t9_ckd_ccg_current_piv_df
                     ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## Reorder and sort columns for output data frame on tab 'CKD' in sicbl-ach-prev-pca excel workbook (template 9)
t9_ckd_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE"
              ,"REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
              ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","CKD005_ACHIEVED_POINTS","CKD005_ACH_RATE"]
t9_ckd_df = t9_ckd_df[t9_ckd_col]

t9_ckd_df = t9_ckd_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)   

In [ ]:
## Select data for DM group code
t9_dm_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "DM"]

t9_dm_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="DM")]


## Combines T9 Master table with previous years data
t9_dm_df = pd.merge(t9_dm_base_df
                    ,t9_dm_ccg_current_piv_df
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'DM' in sicbl-ach-prev-pca excel workbook (template 9)
t9_dm_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE"
             ,"ADD_LIST_40OV","REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS"
             ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT"
             ,"PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE"
             ,"PCA_CHANGE","DM017_ACHIEVED_POINTS","DM017_ACH_RATE"
             ,"DM006_ACHIEVED_POINTS","DM006_ACHIEVEMENT_NUMERATOR","DM006_ACHIEVEMENT_DENOMINATOR"
             ,"DM006_UL_NET_ACH","DM006_PCA_COUNT","DM006_PCA_RATE","DM006_DEN_PLUS_PCA","DM006_PRI"
             ,"DM012_ACHIEVED_POINTS","DM012_ACHIEVEMENT_NUMERATOR","DM012_ACHIEVEMENT_DENOMINATOR"
             ,"DM012_UL_NET_ACH","DM012_PCA_COUNT","DM012_PCA_RATE","DM012_DEN_PLUS_PCA","DM012_PRI"
             ,"DM014_ACHIEVED_POINTS","DM014_ACHIEVEMENT_NUMERATOR","DM014_ACHIEVEMENT_DENOMINATOR"
             ,"DM014_UL_NET_ACH","DM014_PCA_COUNT","DM014_PCA_RATE","DM014_DEN_PLUS_PCA","DM014_PRI"
             ,"DM019_ACHIEVED_POINTS","DM019_ACHIEVEMENT_NUMERATOR","DM019_ACHIEVEMENT_DENOMINATOR"
             ,"DM019_UL_NET_ACH","DM019_PCA_COUNT","DM019_PCA_RATE","DM019_DEN_PLUS_PCA","DM019_PRI"
             ,"DM020_ACHIEVED_POINTS","DM020_ACHIEVEMENT_NUMERATOR","DM020_ACHIEVEMENT_DENOMINATOR"
             ,"DM020_UL_NET_ACH","DM020_PCA_COUNT","DM020_PCA_RATE","DM020_DEN_PLUS_PCA","DM020_PRI"
             ,"DM021_ACHIEVED_POINTS","DM021_ACHIEVEMENT_NUMERATOR","DM021_ACHIEVEMENT_DENOMINATOR"
             ,"DM021_UL_NET_ACH","DM021_PCA_COUNT","DM021_PCA_RATE","DM021_DEN_PLUS_PCA","DM021_PRI"
             ,"DM022_ACHIEVED_POINTS","DM022_ACHIEVEMENT_NUMERATOR","DM022_ACHIEVEMENT_DENOMINATOR"
             ,"DM022_UL_NET_ACH","DM022_PCA_COUNT","DM022_PCA_RATE","DM022_DEN_PLUS_PCA","DM022_PRI"
             ,"DM023_ACHIEVED_POINTS","DM023_ACHIEVEMENT_NUMERATOR","DM023_ACHIEVEMENT_DENOMINATOR"
             ,"DM023_UL_NET_ACH","DM023_PCA_COUNT","DM023_PCA_RATE","DM023_DEN_PLUS_PCA","DM023_PRI"]
t9_dm_df = t9_dm_df[t9_dm_col]

t9_dm_df = t9_dm_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)   

In [ ]:
## Select data for PC group code
t9_pc_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "PC"]

t9_pc_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="PC")]


## Combines T9 Master table with previous years data
t9_pc_df = pd.merge(t9_pc_base_df
                    ,t9_pc_ccg_current_piv_df
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'PC' in sicbl-ach-prev-pca excel workbook (template 9)
t9_pc_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","REGISTER"
             ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
             ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE", "PC001_ACHIEVED_POINTS","PC001_ACH_RATE"]
t9_pc_df = t9_pc_df[t9_pc_col]

t9_pc_df = t9_pc_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)                

In [ ]:
## Select data for DEM group code
t9_dem_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "DEM"]

t9_dem_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="DEM")]


## Combines T9 Master table with previous years data
t9_dem_df = pd.merge(t9_dem_base_df
                    ,t9_dem_ccg_current_piv_df
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'DEM' in sicbl-ach-prev-pca excel workbook (template 9)
t9_dem_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE", "NUMBER_PRACTICES","LIST_SIZE","REGISTER","PREVALENCE"
              ,"PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC"
              ,"ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
              ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE","DEM001_ACHIEVED_POINTS","DEM001_ACH_RATE"
              ,"DEM004_ACHIEVED_POINTS","DEM004_ACHIEVEMENT_NUMERATOR","DEM004_ACHIEVEMENT_DENOMINATOR"
              ,"DEM004_UL_NET_ACH","DEM004_PCA_COUNT","DEM004_PCA_RATE","DEM004_DEN_PLUS_PCA","DEM004_PRI"]
t9_dem_df = t9_dem_df[t9_dem_col]

t9_dem_df = t9_dem_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)                 

In [ ]:
## Select data for DEP group code
t9_dep_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "DEP"]

t9_dep_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="DEP")]


## Combines T9 Master table with previous years data
t9_dep_df = pd.merge(t9_dep_base_df
                    ,t9_dep_ccg_current_piv_df
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'DEP' in sicbl-ach-prev-pca excel workbook (template 9)
t9_dep_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE","REGISTER"
              ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE"
              ,"ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
              ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE","DEP003_ACHIEVED_POINTS"
              ,"DEP003_ACHIEVEMENT_NUMERATOR","DEP003_ACHIEVEMENT_DENOMINATOR","DEP003_UL_NET_ACH"
              ,"DEP003_PCA_COUNT","DEP003_PCA_RATE","DEP003_DEN_PLUS_PCA","DEP003_PRI"]
t9_dep_df = t9_dep_df[t9_dep_col]

t9_dep_df = t9_dep_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)

In [ ]:
## Select data for EP group code
t9_ep_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "EP"]

t9_ep_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="EP")]


## Combines T9 Master table with previous years data
t9_ep_df = pd.merge(t9_ep_base_df
                    ,t9_ep_ccg_current_piv_df
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'EP' in sicbl-ach-prev-pca excel workbook (template 9)
t9_ep_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE","REGISTER"
             ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
             ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","EP001_ACHIEVED_POINTS","EP001_ACH_RATE"]
t9_ep_df = t9_ep_df[t9_ep_col]

t9_ep_df = t9_ep_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)

In [ ]:
## Select data for LD group code
t9_ld_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "LD"]

t9_ld_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="LD")]


## Combines T9 Master table with previous years data
t9_ld_df = pd.merge(t9_ld_base_df
                    ,t9_ld_ccg_current_piv_df
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'LD' in sicbl-ach-prev-pca excel workbook (template 9)
t9_ld_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","REGISTER"
             ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
             ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","LD004_ACHIEVED_POINTS","LD004_ACH_RATE"]
t9_ld_df = t9_ld_df[t9_ld_col]

t9_ld_df = t9_ld_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)

In [ ]:
## Select data for MH group code
t9_mh_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "MH"]

t9_mh_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="MH")]


## Combines T9 Master table with previous years data
t9_mh_df = pd.merge(t9_mh_base_df
                    ,t9_mh_ccg_current_piv_df
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'MH' in sicbl-ach-prev-pca excel workbook (template 9)
t9_mh_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_PRACTICE_LIST_SIZE"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","LIST_SIZE","REGISTER","PREVALENCE"
             ,"PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC"
             ,"ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS"
             ,"OVERALL_PCA_RATE","PCA_CHANGE","MH001_ACHIEVED_POINTS","MH001_ACH_RATE"
             ,"MH002_ACHIEVED_POINTS","MH002_ACHIEVEMENT_NUMERATOR","MH002_ACHIEVEMENT_DENOMINATOR"
             ,"MH002_UL_NET_ACH","MH002_PCA_COUNT","MH002_PCA_RATE","MH002_DEN_PLUS_PCA","MH002_PRI"
             ,"MH003_ACHIEVED_POINTS","MH003_ACHIEVEMENT_NUMERATOR","MH003_ACHIEVEMENT_DENOMINATOR"
             ,"MH003_UL_NET_ACH","MH003_PCA_COUNT","MH003_PCA_RATE","MH003_DEN_PLUS_PCA","MH003_PRI"
             ,"MH006_ACHIEVED_POINTS","MH006_ACHIEVEMENT_NUMERATOR","MH006_ACHIEVEMENT_DENOMINATOR"
             ,"MH006_UL_NET_ACH","MH006_PCA_COUNT","MH006_PCA_RATE","MH006_DEN_PLUS_PCA","MH006_PRI"                     
             ,"MH007_ACHIEVED_POINTS","MH007_ACHIEVEMENT_NUMERATOR","MH007_ACHIEVEMENT_DENOMINATOR"
             ,"MH007_UL_NET_ACH","MH007_PCA_COUNT","MH007_PCA_RATE","MH007_DEN_PLUS_PCA","MH007_PRI"                     
             ,"MH011_ACHIEVED_POINTS","MH011_ACHIEVEMENT_NUMERATOR","MH011_ACHIEVEMENT_DENOMINATOR"
             ,"MH011_UL_NET_ACH","MH011_PCA_COUNT","MH011_PCA_RATE","MH011_DEN_PLUS_PCA","MH011_PRI"                     
             , "MH012_ACHIEVED_POINTS","MH012_ACHIEVEMENT_NUMERATOR","MH012_ACHIEVEMENT_DENOMINATOR"
             ,"MH012_UL_NET_ACH","MH012_PCA_COUNT","MH012_PCA_RATE","MH012_DEN_PLUS_PCA","MH012_PRI"]
t9_mh_df = t9_mh_df[t9_mh_col]

t9_mh_df = t9_mh_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)
                     

In [ ]:
## Select data for OST group code
t9_ost_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "OST"]

t9_ost_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="OST")]


## Combines T9 Master table with previous years data
t9_ost_df = pd.merge(t9_ost_base_df
                    ,t9_ost_ccg_current_piv_df
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'OST' in sicbl-ach-prev-pca excel workbook (template 9)
t9_ost_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE"
              ,"REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
              ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","OST004_ACHIEVED_POINTS","OST004_ACH_RATE"]
t9_ost_df = t9_ost_df[t9_ost_col]

t9_ost_df = t9_ost_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)            

In [ ]:
## Select data for RA group code
t9_ra_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "RA"]

t9_ra_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="RA")]


## Combines T9 Master table with previous years data
t9_ra_df = pd.merge(t9_ra_base_df
                    ,t9_ra_ccg_current_piv_df
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder and sort columns for output data frame on tab 'RA' in sicbl-ach-prev-pca excel workbook (template 9)
t9_ra_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
             ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE","REGISTER"
             ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
             ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR"
             ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
             ,"RA001_ACHIEVED_POINTS","RA001_ACH_RATE"
             ,"RA002_ACHIEVED_POINTS","RA002_ACHIEVEMENT_NUMERATOR","RA002_ACHIEVEMENT_DENOMINATOR"
             ,"RA002_UL_NET_ACH","RA002_PCA_COUNT","RA002_PCA_RATE","RA002_DEN_PLUS_PCA","RA002_PRI"]
t9_ra_df = t9_ra_df[t9_ra_col]
                     
t9_ra_df = t9_ra_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)

In [ ]:
## Select data for NDH group code
t9_ndh_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "NDH"]

## As NDH had no achievement points allocated in the previous year, it had no achievement rate for comparison with this year
## Therefore the missing value in the columns 'Achievement (%)' and  'Year on year change' has been replaced with an 'z' for not applicable
t9_ndh_base_df["PREV_ACHIEVEMENT_RATE"] = t9_ndh_base_df["PREV_ACHIEVEMENT_RATE"].fillna("z")
t9_ndh_base_df["ACH_CHANGE"] = t9_ndh_base_df["ACH_CHANGE"].fillna("z")

t9_ndh_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="NDH")]


## Combines T8 Master table with previous years data
t9_ndh_df = pd.merge(t9_ndh_base_df
                    ,t9_ndh_ccg_current_piv_df    
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'NDH' in sicbl-ach-prev-pca excel workbook (template 9)
t9_ndh_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
              ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","NUMBER_PRACTICES","ALT_LIST_SIZE","REGISTER"
              ,"PREVALENCE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC"
              ,"ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
              ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE","NDH001_ACHIEVED_POINTS"
              ,"NDH001_ACHIEVEMENT_NUMERATOR","NDH001_ACHIEVEMENT_DENOMINATOR","NDH001_UL_NET_ACH"
              ,"NDH001_PCA_COUNT","NDH001_PCA_RATE","NDH001_DEN_PLUS_PCA","NDH001_PRI"]
t9_ndh_df = t9_ndh_df[t9_ndh_col]

t9_ndh_df = t9_ndh_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)                                                  

In [ ]:
## Select data for CS group code
t9_cs_base_df= T9_master_df[T9_master_df["GROUP_CODE"] == "CS"]

t9_cs_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="CS")]


## Combines T8 Master table with previous years data
t9_cs_df = pd.merge(t9_cs_base_df
                    ,t9_cs_ccg_current_piv_df    
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'CS' in sicbl-ach-prev-pca excel workbook (template 9)
t9_cs_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","PREV_ALT_PAT_LIST"
             ,"NUMBER_PRACTICES","ADD_LIST_25_49_F","ADD_LIST_50_64_F","PREV_ACHIEVED_POINTS"
             ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT"
             ,"PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS"
             ,"OVERALL_PCA_RATE","PCA_CHANGE","CS005_ACHIEVED_POINTS","CS005_ACHIEVEMENT_NUMERATOR"
             ,"CS005_ACHIEVEMENT_DENOMINATOR","CS005_UL_NET_ACH","CS005_PCA_COUNT","CS005_PCA_RATE"
             ,"CS005_DEN_PLUS_PCA","CS005_PRI","CS006_ACHIEVED_POINTS","CS006_ACHIEVEMENT_NUMERATOR"
             ,"CS006_ACHIEVEMENT_DENOMINATOR","CS006_UL_NET_ACH","CS006_PCA_COUNT","CS006_PCA_RATE"
             ,"CS006_DEN_PLUS_PCA","CS006_PRI"]
t9_cs_df = t9_cs_df[t9_cs_col]

t9_cs_df = t9_cs_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True) 

In [ ]:
## Select data for VI group code
t9_vi_base_df = T9_master_df[T9_master_df["GROUP_CODE"] == "VI"]

t9_vi_ccg_current_piv_df = t9_ccg_current_piv_df.loc[(t9_ccg_current_piv_df["GROUP_CODE"]=="VI")]


## Combines T8 Master table with previous years data
t9_vi_df = pd.merge(t9_vi_base_df
                    ,t9_vi_ccg_current_piv_df    
                    ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                    ,how ="left")

## Reorder columns for output data frame on tab 'VI' in sicbl-ach-prev-pca excel workbook (template 9)
t9_vi_col = ["CCG_ONS_CODE","CCG_ODS_CODE","CCG_NAME","PREV_NUMBER_PRACTICES","NUMBER_PRACTICES"
             ,"ADD_LIST_UN01", "ADD_LIST_1_2", "ADD_LIST_4_5", "ADD_LIST_79_80"
             ,"PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
             ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS"
             ,"OVERALL_PCA_RATE","PCA_CHANGE"                   
             ,"VI001_ACHIEVED_POINTS","VI001_ACHIEVEMENT_NUMERATOR","VI001_ACHIEVEMENT_DENOMINATOR"
             ,"VI001_UL_NET_ACH","VI001_PCA_COUNT","VI001_PCA_RATE","VI001_DEN_PLUS_PCA","VI001_PRI"
             ,"VI002_ACHIEVED_POINTS","VI002_ACHIEVEMENT_NUMERATOR","VI002_ACHIEVEMENT_DENOMINATOR"
             ,"VI002_UL_NET_ACH","VI002_PCA_COUNT","VI002_PCA_RATE","VI002_DEN_PLUS_PCA","VI002_PRI"
             ,"VI003_ACHIEVED_POINTS","VI003_ACHIEVEMENT_NUMERATOR","VI003_ACHIEVEMENT_DENOMINATOR"
             ,"VI003_UL_NET_ACH","VI003_PCA_COUNT","VI003_PCA_RATE","VI003_DEN_PLUS_PCA","VI003_PRI"
             ,"VI004_ACHIEVED_POINTS","VI004_ACHIEVEMENT_NUMERATOR","VI004_ACHIEVEMENT_DENOMINATOR"
             ,"VI004_UL_NET_ACH","VI004_PCA_COUNT","VI004_PCA_RATE","VI004_DEN_PLUS_PCA","VI004_PRI"]
t9_vi_df = t9_vi_df[t9_vi_col]

t9_vi_df = t9_vi_df.sort_values(by=["CCG_NAME"],ascending = [True])                     

In [ ]:
## Join QI group codes with numeric key to the master data frame
## This means that QI codes can be reused year-on-year without having to change the body of the code
## QI data is only published for the current year as the indicators change and are therefore not comparable
t9_master_df = pd.merge(QI_gr_codes_with_key
                        ,T9_master_df
                        ,left_on = ["GROUP_CODE"]
                        ,right_on = ["GROUP_CODE"]
                        ,how ="left")

In [ ]:
## Select numeric key for QIPDD group code
t9_qi1_base_df = t9_master_df[t9_master_df["GrC_KEY"] == "QI_1"]

## Select indicator specific data frame
t9_combined_ccg_all_qi1 = t9_ccg_current_piv_df

## Combines T9 Master table with indicator data
t9_qi1_df = pd.merge(t9_qi1_base_df
                     ,t9_combined_ccg_all_qi1
                     ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

## Reorder columns for output data frame on tab 'QIPDD' in sicbl-ach-prev-pca excel workbook (template 9)
t9_qi1_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","NUMBER_PRACTICES"
              ,"LIST_SIZE","TOTAL_ACH_SCORE","ACH_PC"
              ,"QIPDD009_ACHIEVED_POINTS","QIPDD009_ACH_RATE"
              ,"QIPDD010_ACHIEVED_POINTS","QIPDD010_ACH_RATE"]
                           
t9_qi1_df = t9_qi1_df[t9_qi1_col]

t9_qi1_df = t9_qi1_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)

In [ ]:
## Select numeric key for QIOA group code
t9_qi2_base_df = t9_master_df[t9_master_df["GrC_KEY"] == "QI_2"]

## Select indicator specific data frame
t9_combined_ccg_all_qi2 = t9_ccg_current_piv_df

## Combines T9 Master table with indicator data
t9_qi2_df = pd.merge(t9_qi2_base_df
                     ,t9_combined_ccg_all_qi2
                     ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                     ,how ="left")

# Reorder columns for output data frame on tab 'QIOA' in sicbl-ach-prev-pca excel workbook (template 9)
t9_qi2_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","NUMBER_PRACTICES","LIST_SIZE"
              ,"TOTAL_ACH_SCORE","ACH_PC"
              ,"QIOA011_ACHIEVED_POINTS","QIOA011_ACH_RATE"
              ,"QIOA012_ACHIEVED_POINTS","QIOA012_ACH_RATE"]
t9_qi2_df = t9_qi2_df[t9_qi2_col]
                         
t9_qi2_df = t9_qi2_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)       

In [ ]:
## Select numeric key for QIPDD_MI group code
t9_qi3_base_df = t9_master_df[t9_master_df["GrC_KEY"] == "QI_3"]

## Select indicator specific data frame
t9_combined_ccg_all_qi3 = t9_ccg_current_piv_df

## Combines T9 Master table with indicator data
t9_qi3_df = pd.merge(t9_qi3_base_df
                     ,t9_combined_ccg_all_qi3
                     ,left_on = ["CCG_ODS_CODE","GROUP_CODE"]
                     ,right_on = ["CCG_ODS_CODE","GROUP_CODE"]
                     ,how = 'left')

## This indicator has zero achievement points attributed to it and therefore an achievement percentage cannot be calculated
## The value is replaced with 'z' for data not applicable
t9_qi3_df["ACH_PC"] = t9_qi3_df["ACH_PC"].fillna("z")

## Reorder columns for output data frame on tab 'QIPDD_MI' in sicbl-ach-prev-pca excel workbook (template 9)
t9_qi3_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","NUMBER_PRACTICES","ALT_LIST_SIZE"
              ,"TOTAL_ACH_SCORE","ACH_PC","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE"
              ,"QIPDDMI001_ACHIEVED_POINTS","QIPDDMI001_ACHIEVEMENT_NUMERATOR","QIPDDMI001_ACHIEVEMENT_DENOMINATOR"
              ,"QIPDDMI001_UL_NET_ACH","QIPDDMI001_PCA_COUNT","QIPDDMI001_PCA_RATE","QIPDDMI001_DEN_PLUS_PCA","QIPDDMI001_PRI"
              ,"QIPDDMI002_ACHIEVED_POINTS","QIPDDMI002_ACHIEVEMENT_NUMERATOR","QIPDDMI002_ACHIEVEMENT_DENOMINATOR"
              ,"QIPDDMI002_UL_NET_ACH","QIPDDMI002_PCA_COUNT","QIPDDMI002_PCA_RATE","QIPDDMI002_DEN_PLUS_PCA","QIPDDMI002_PRI"
              ,"QIPDDMI003_ACHIEVED_POINTS","QIPDDMI003_ACHIEVEMENT_NUMERATOR","QIPDDMI003_ACHIEVEMENT_DENOMINATOR"
              ,"QIPDDMI003_UL_NET_ACH","QIPDDMI003_PCA_COUNT","QIPDDMI003_PCA_RATE","QIPDDMI003_DEN_PLUS_PCA","QIPDDMI003_PRI"
              ,"QIPDDMI004_ACHIEVED_POINTS","QIPDDMI004_ACHIEVEMENT_NUMERATOR","QIPDDMI004_ACHIEVEMENT_DENOMINATOR"
              ,"QIPDDMI004_UL_NET_ACH","QIPDDMI004_PCA_COUNT","QIPDDMI004_PCA_RATE","QIPDDMI004_DEN_PLUS_PCA","QIPDDMI004_PRI"]                         
t9_qi3_df = t9_qi3_df[t9_qi3_col]

t9_qi3_df = t9_qi3_df.sort_values(by=["CCG_NAME"],ascending = [True]).reset_index(drop = True)          

<h3>Create T9 SUB ICB (CCG) Table titles

In [ ]:
## Combine elements that contribute to table title
gr_ccg_table_titles = gr_message_df
gr_ccg_table_titles["TABLE_NO"] = gr_ccg_table_titles["TABLE_NO"].astype(str)
gr_ccg_table_titles["FYEAR"] = Control_File.at[0, "FYEAR"]
gr_ccg_table_titles["HIGHER_GROUP_DESCRIPTION"] = gr_ccg_table_titles["HIGHER_GROUP_DESCRIPTION"].str.lower()
gr_ccg_table_titles["GROUP_DESCRIPTION"] = gr_ccg_table_titles["GROUP_DESCRIPTION"].str.lower()

gr_ccg_table_titles["TABLE_TITLE"] = "Table "+gr_ccg_table_titles["TABLE_NO"]+": Prevalence, achievement and personalised care adjustments, "+gr_ccg_table_titles["HIGHER_GROUP_DESCRIPTION"] \
                                +" group, "+gr_ccg_table_titles["GROUP_DESCRIPTION"]+", "+gr_ccg_table_titles["FYEAR"]+", sub ICB loc level"


gr_ccg_table_titles = pd.merge(gr_ccg_table_titles
                               ,QI_gr_codes_with_key
                               ,left_on = ["GROUP_CODE"]
                               ,right_on = ["GROUP_CODE"]
                               ,how ="left")


gr_ccg_table_titles["GrC_KEY"] = gr_ccg_table_titles["GrC_KEY"].replace(np.nan, "")
gr_ccg_table_titles["GROUP_CODE"] = np.where(gr_ccg_table_titles["GrC_KEY"]== ""
                                             ,gr_ccg_table_titles["GROUP_CODE"]
                                             ,gr_ccg_table_titles["GrC_KEY"])

gr_ccg_table_titles = gr_ccg_table_titles.drop(columns=["FYEAR","HIGHER_GROUP_DESCRIPTION","GROUP_DESCRIPTION","TABLE_NO","GrC_KEY"])
ccg_table_titles = gr_ccg_table_titles

<b><h1>23 OUTPUTS - T10-11-12-13-14-15-16-17-Prac-PrevAchPCA</h1></b><br><h2>Part 1 - QOF_MASTER for current indicators<br>Practice data frame from QOF Master

In [ ]:
"""   
The practice level data is processed as one data frame before being split into clinical groups 
for publication in separate excel workbooks 

"""

## Required indicator and achievement columns from QOF Master table
t10_17_prac_current_base_col = ["PRACTICE_CODE","GROUP_CODE","INDICATOR_CODE","ACHIEVED_POINTS"
                                ,"MAX_INDICATOR_POINTS","ACHIEVEMENT_NUMERATOR"
                                ,"ACHIEVEMENT_DENOMINATOR","PCA_COUNT"]
t10_17_prac_current_base_df = qof_master[t10_17_prac_current_base_col]

t10_17_prac_current_sum_df = t10_17_prac_current_base_df.groupby(by = ["PRACTICE_CODE","GROUP_CODE","INDICATOR_CODE"]
                                                                 ,as_index=False
                                                                 ).agg({"ACHIEVED_POINTS" : [pd.Series.sum]
                                                                        ,"MAX_INDICATOR_POINTS" : [pd.Series.sum]
                                                                        ,"ACHIEVEMENT_NUMERATOR" : [pd.Series.sum]
                                                                        ,"ACHIEVEMENT_DENOMINATOR" : [pd.Series.sum]
                                                                        ,"PCA_COUNT" : [pd.Series.sum]})                                                                            
t10_17_prac_current_sum_df.columns = t10_17_prac_current_sum_df.columns.droplevel(1)


## Create all percentage columns
## Achievement Rate
t10_17_prac_current_sum_df["ACH_RATE"] = (t10_17_prac_current_sum_df["ACHIEVED_POINTS"]/
                                          t10_17_prac_current_sum_df["MAX_INDICATOR_POINTS"])*100

## Underlying Achievement net of PCAs
t10_17_prac_current_sum_df["UL_NET_ACH"] = (t10_17_prac_current_sum_df["ACHIEVEMENT_NUMERATOR"]/
                                            t10_17_prac_current_sum_df["ACHIEVEMENT_DENOMINATOR"])*100

## PCA Rate
t10_17_prac_current_sum_df["PCA_RATE"] = (t10_17_prac_current_sum_df["PCA_COUNT"]/(
    t10_17_prac_current_sum_df["ACHIEVEMENT_DENOMINATOR"]+t10_17_prac_current_sum_df["PCA_COUNT"]))*100

## Denominator plus PCAs
t10_17_prac_current_sum_df["DEN_PLUS_PCA"] = (t10_17_prac_current_sum_df["ACHIEVEMENT_DENOMINATOR"]+
                                              t10_17_prac_current_sum_df["PCA_COUNT"])

## Patients Receiving Intervention
t10_17_prac_current_sum_df["PRI"] = (t10_17_prac_current_sum_df["ACHIEVEMENT_NUMERATOR"]/
                                     (t10_17_prac_current_sum_df["ACHIEVEMENT_DENOMINATOR"]+
                                      t10_17_prac_current_sum_df["PCA_COUNT"]))*100

## Drop unwanted column
t10_17_prac_current_sum_df = t10_17_prac_current_sum_df.drop(columns=["MAX_INDICATOR_POINTS"])


In [ ]:
## The short broad data frame is converted into a long thin data frame and the non-index 
## columns are stacked in a column called 'FIELD'
t10_17_prac_current_stacked_df = (t10_17_prac_current_sum_df.set_index(["PRACTICE_CODE","GROUP_CODE","INDICATOR_CODE"])
                                                                       .stack()
                                                                       .ffill(axis=0)
                                                                       .bfill(axis=0, downcast='infer')
                                                                       .reset_index()
                                                                       .rename({"level_3":"FIELD", 0:"VALUE"}, axis=1))
                                                            
## New column 'INDICATOR' is created by joining the 'INDICATOR_CODE' to 'FIELD' 
## with a hyphen and drop unwanted columns
t10_17_prac_current_stacked_df["INDICATOR"] = (t10_17_prac_current_stacked_df["INDICATOR_CODE"]
                                               )+"_"+(t10_17_prac_current_stacked_df["FIELD"])

## Drop unwanted columns                                               
t10_17_prac_current_stacked_df = t10_17_prac_current_stacked_df[["PRACTICE_CODE","GROUP_CODE","INDICATOR","VALUE"]]

In [ ]:
## Pivot data frame from long thin data frame into short broad data frame where the new columns 
## consist of the newly created 'INDICATOR' column
t10_17_prac_current_piv_df = pd.pivot_table(t10_17_prac_current_stacked_df
                                              ,values="VALUE"
                                              ,index=["PRACTICE_CODE","GROUP_CODE"]
                                              ,columns= "INDICATOR")

t10_17_prac_current_piv_df = t10_17_prac_current_piv_df.reset_index().rename_axis(None, axis=1)

<h2>Part 2 - Join QOF Master to last years Practice data frame

In [ ]:
"""   
Creates a practice level data frame that comprises data for the current year 
and previous year

"""

## Drop unwanted nhs geography columns from the QOF Master table
t10_17_prac_master_col = qof_master.drop(columns=["NAT_ONS_CODE","NAT_CODE","NAT_COUNTRY"
                                                  ,"REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME"
                                                  ,"STP_ODS_CODE","STP_ONS_CODE","STP_NAME"
                                                  ,"DOMAIN_CODE","SUB_DOMAIN_CODE"])

## Drop unwanted column from the data frame that combines data from the previous year
t10_17_prac_joined_df = previous_prevalence_prac_data_df.drop(columns=["HIGHER_GROUP_CODE"])

## The resulting data frame is used as the foundation for three other code blocks below
t10_17_prac_current_previous_df = pd.merge(t10_17_prac_master_col
                                            ,t10_17_prac_joined_df
                                            ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                                            ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                                            ,how ="left") 

In [ ]:
## Select columns required for practice group data frame for the previous year
## These columns are used as the index to which other columns are joined
t10_17_prac_previous_gr_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE"
                               ,"PRACTICE_NAME","GROUP_CODE","ALT_PAT_LIST","PREV_PRACTICE_LIST_SIZE"
                               ,"PREV_ALT_PAT_LIST","PREV_PREVALENCE","PREV_REGISTER_SIZE","PREV_ACHIEVED_POINTS"
                               ,"PREV_TOTAL_INDICATOR_GROUP_POINTS", "PREV_ACHIEVEMENT_RATE","PREV_PCA_COUNT"
                               ,"PREV_DENOMINATOR","PREV_PCA_RATE"]
t10_17_prac_previous_gr_index_df = t10_17_prac_current_previous_df[t10_17_prac_previous_gr_col]

t10_17_prac_previous_gr_index_df = t10_17_prac_previous_gr_index_df.drop_duplicates().reset_index(drop=True)                                              

In [ ]:
## Select columns required for practice group data frame for the previous year
## Count distinct number of practices by indicator group
t10_17_prac_ind_gr_col = ["PRACTICE_CODE","INDICATOR_CODE","GROUP_CODE"]
t10_17_prac_ind_gr_df = t10_17_prac_current_previous_df[t10_17_prac_ind_gr_col]

t10_17_prac_ind_gr_df = t10_17_prac_ind_gr_df.drop_duplicates().reset_index(drop=True)

t10_17_prac_ind_gr_df = t10_17_prac_ind_gr_df.groupby(by = ["PRACTICE_CODE","GROUP_CODE"]
                                                      ,as_index=False
                                                      ).agg({"INDICATOR_CODE" : [pd.Series.count]})                                                             
t10_17_prac_ind_gr_df.columns = t10_17_prac_ind_gr_df.columns.droplevel(1)  

In [ ]:
## Select columns required for practice group data frame 
## Aggregate data in non-index columns
t10_17_prac_gr_aggregated_df = t10_17_prac_current_previous_df.groupby(by = ["PRACTICE_CODE","GROUP_CODE"]
                                                                       ,as_index=False
                                                                       ).agg({"LIST_SIZE" : [pd.Series.sum]
                                                                              ,"ALT_LIST_SIZE" : [pd.Series.sum]
                                                                              ,"REGISTER_SIZE" : [pd.Series.sum]
                                                                              ,"ACHIEVED_POINTS" : [pd.Series.sum]
                                                                              ,"PCA_COUNT" : [pd.Series.sum]
                                                                              ,"ACHIEVEMENT_DENOMINATOR" : [pd.Series.sum]
                                                                              ,"MAX_GROUP_POINTS" : [pd.Series.max]})                                                                                                                                                            
t10_17_prac_gr_aggregated_df.columns = t10_17_prac_gr_aggregated_df.columns.droplevel(1)

## Rename columns 'LIST_SIZE' as 'LIST_SIZE_SUMMED' and 'ALT_LIST_SIZE' as 'ALT_LIST_SIZE_SUMMED' which contain total counts
t10_17_prac_gr_aggregated_df = t10_17_prac_gr_aggregated_df.rename(columns={"LIST_SIZE":"LIST_SIZE_SUMMED"
                                                                            ,"ALT_LIST_SIZE":"ALT_LIST_SIZE_SUMMED"})

In [ ]:
## Join aggregated data to geographies and measures that are not aggregated
t10_17_prac_gr_df = pd.merge(t10_17_prac_previous_gr_index_df
                             ,t10_17_prac_ind_gr_df
                             ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                             ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                             ,how ="left")

t10_17_prac_gr_df = pd.merge(t10_17_prac_gr_df
                             ,t10_17_prac_gr_aggregated_df
                             ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                             ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                             ,how ="left")

In [ ]:
## Create finished practice data frame for T10-17 processing

## Create 'LIST_SIZE' column and drop aggregated 'LIST_SIZE_SUMMED' as it is no longer needed
t10_17_prac_gr_df["LIST_SIZE"] = t10_17_prac_gr_df["LIST_SIZE_SUMMED"]/t10_17_prac_gr_df["INDICATOR_CODE"]
t10_17_prac_gr_df = t10_17_prac_gr_df.drop(columns=["LIST_SIZE_SUMMED"])

## Create column 'ALT_LIST_SIZE' and leave values blank if there is no value in the corresponding 'ALT_PAT_LIST'
t10_17_prac_gr_df["ALT_LIST_SIZE"] = np.where(t10_17_prac_gr_df["ALT_PAT_LIST"] != None
                                              ,(t10_17_prac_gr_df["ALT_LIST_SIZE_SUMMED"]/t10_17_prac_gr_df["INDICATOR_CODE"])
                                              , "")

## Data columns that contain data totals with the prefix 'TOTAL'
t10_17_prac_gr_df = t10_17_prac_gr_df.rename(columns={"INDICATOR_CODE":"COUNT_DISTINCT_IND_CODE"
                                                      ,"ACHIEVED_POINTS":"TOTAL_ACH_SCORE","PCA_COUNT":"TOTAL_PCAS"
                                                      ,"ACHIEVEMENT_DENOMINATOR":"TOTAL_DENOMINATORS"
                                                      ,"REGISTER_SIZE":"REGISTER"})
                                                                        
## Select practice level columns ordered to which calculated columns can be added
t10_17_prac_gr_col = ["GROUP_CODE","CCG_ONS_CODE","CCG_ODS_CODE","CCG_NAME"
                      ,"PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                      ,"LIST_SIZE","ALT_LIST_SIZE","ALT_LIST_SIZE_SUMMED"
                      ,"COUNT_DISTINCT_IND_CODE","REGISTER","TOTAL_ACH_SCORE"
                      ,"MAX_GROUP_POINTS","TOTAL_PCAS","TOTAL_DENOMINATORS"
                      ,"PREV_PRACTICE_LIST_SIZE","PREV_ALT_PAT_LIST"
                      ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","PREV_ACHIEVED_POINTS",
                      "PREV_TOTAL_INDICATOR_GROUP_POINTS","PREV_ACHIEVEMENT_RATE",
                      "PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"]
t10_17_prac_base_df = t10_17_prac_gr_df[t10_17_prac_gr_col]

In [ ]:
"""   
Prevalence, Achievement Score, Achievement Percentage and Overall PCA rate are
calculated for practice data and comprise part of the final published data frames in
prev-ach-pca-cv-prac.xlsx
prev-ach-pca-resp-prac.xlsx
prev-ach-pca-ls-prac.xlsx
prev-ach-pca-hd-prac.xlsx
prev-ach-pca-neu-prac.xlsx
prev-ach-pca-ms-prac.xlsx
prev-ach-pca-fer-obs-gyn-prac.xlsx
prev-ach-qual-imp-prac.xlsx
prev-ach-pca-vi-prac.xlsx

"""
## Change the data type of the column 'ALT_LIST_SIZE'used in the calculations
t10_17_prac_base_df["ALT_LIST_SIZE"] = t10_17_prac_base_df["ALT_LIST_SIZE"].astype(float)

## Calculate Prevalence
t10_17_prac_base_df["PREVALENCE"] = np.where(t10_17_prac_base_df["ALT_LIST_SIZE"] == 0
                                             , (t10_17_prac_base_df["REGISTER"]/t10_17_prac_base_df["LIST_SIZE"])*100
                                             ,(t10_17_prac_base_df["REGISTER"]/t10_17_prac_base_df["ALT_LIST_SIZE"])*100)
                                                   
## Calculate Achievement Percentage
t10_17_prac_base_df["ACH_PC"] = (t10_17_prac_base_df["TOTAL_ACH_SCORE"]/t10_17_prac_base_df["MAX_GROUP_POINTS"])*100

## Calculate Overall PCA rate
t10_17_prac_base_df["OVERALL_PCA_RATE"] = (t10_17_prac_base_df["TOTAL_PCAS"]/
                                           (t10_17_prac_base_df["TOTAL_DENOMINATORS"]+
                                            t10_17_prac_base_df["TOTAL_PCAS"]))*100

## Reorder columns to include new values
t10_17_prac_base_col = ["GROUP_CODE","CCG_ONS_CODE","CCG_ODS_CODE","CCG_NAME"
                        ,"PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                        ,"LIST_SIZE","ALT_LIST_SIZE","REGISTER","PREVALENCE"
                        ,"TOTAL_ACH_SCORE","ACH_PC","TOTAL_PCAS","TOTAL_DENOMINATORS"
                        ,"OVERALL_PCA_RATE","PREV_PRACTICE_LIST_SIZE","PREV_ALT_PAT_LIST"
                        ,"PREV_REGISTER_SIZE","PREV_PREVALENCE","PREV_ACHIEVED_POINTS"
                        ,"PREV_TOTAL_INDICATOR_GROUP_POINTS","PREV_ACHIEVEMENT_RATE"
                        ,"PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"]
t10_17_prac_base_df = t10_17_prac_base_df[t10_17_prac_base_col]                                                   

<h2>Part 3 - Create Practice Additional List df

In [ ]:
"""   
Addional practice list sizes 'ADD_PAT_SIZE' with the accompanying description column 'ADD_PAT_LIST'
are created at practice level by indicator group and are joined to the data frame created in Part 2

"""

## Practice data frame defined this forms the foundation (base) of the data frame to be created
t10_17_prac_add_list_base_col = ["PRACTICE_CODE","GROUP_CODE","ADD_PAT_LIST"
                                 ,"ADD_LIST_SIZE","INDICATOR_CODE"]
t10_17_prac_add_list_base_df = t10_17_prac_current_previous_df[t10_17_prac_add_list_base_col]

In [ ]:
## Count distinct number of indicators
t10_17_prac_add_list_ind_col = ["PRACTICE_CODE","INDICATOR_CODE","GROUP_CODE"]
t10_17_prac_add_list_ind_df = t10_17_prac_add_list_base_df[t10_17_prac_add_list_ind_col]
t10_17_prac_add_list_ind_df = t10_17_prac_add_list_ind_df.drop_duplicates()

t10_17_prac_add_list_ind_df = t10_17_prac_add_list_ind_df.groupby(by = ["PRACTICE_CODE","GROUP_CODE"]
                                                                  ,as_index=False
                                                                  ).agg({"INDICATOR_CODE" : [pd.Series.count]})                                                                        
t10_17_prac_add_list_ind_df.columns = t10_17_prac_add_list_ind_df.columns.droplevel(1) 

t10_17_prac_add_list_ind_df = t10_17_prac_add_list_ind_df.rename(columns={"INDICATOR_CODE":"COUNT_DISTINCT_IND_CODE"})


## Sum ADD_LIST_SIZE (additional list size)
t10_17_prac_add_list_df = t10_17_prac_add_list_base_df.drop(columns=["INDICATOR_CODE"])
t10_17_prac_add_list_sum_df = t10_17_prac_add_list_df.groupby(by = ["PRACTICE_CODE","GROUP_CODE","ADD_PAT_LIST"]
                                                              ,as_index=False
                                                              )["ADD_LIST_SIZE"].sum()

In [ ]:
## Combine aggregated data frames
t10_17_prac_add_list_df = pd.merge(t10_17_prac_add_list_sum_df
                                   ,t10_17_prac_add_list_ind_df
                                   ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                                   ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                                   ,how ="left") 

## Define new column 'ADD_LIST' e.g. ADD_LIST_06OV
t10_17_prac_add_list_df["ADD_LIST"] = "ADD_LIST_"+t10_17_prac_add_list_df["ADD_PAT_LIST"]

## Drop unwanted columns
t10_17_prac_add_list_df = t10_17_prac_add_list_df.drop(columns=["ADD_PAT_LIST","COUNT_DISTINCT_IND_CODE"])

## Reorder columns
t10_17_prac_add_list_df = t10_17_prac_add_list_df[["PRACTICE_CODE","GROUP_CODE","ADD_LIST","ADD_LIST_SIZE"]]

In [ ]:
## Pivot t10_17_prac_add_list_df from a long thin data frame to a short broad data frame with 
## a column for each additional list size with 'PRACTICE_CODE, and 'GROUP_CODE' as the index
t10_17_add_list_df = pd.pivot_table(t10_17_prac_add_list_df
                                    ,values="ADD_LIST_SIZE"
                                    ,index=["PRACTICE_CODE", "GROUP_CODE"]
                                    ,columns= "ADD_LIST")

t10_17_add_list_df = t10_17_add_list_df.reset_index().rename_axis(None, axis=1)

<h3>Join Part 3 (Additional List df) to Part 2 (previous data)

In [ ]:
t10_17_master_df = pd.merge(t10_17_prac_base_df
                            ,t10_17_add_list_df
                            ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                            ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                            ,how ="left")

## Calculate the difference in the previous year's percentages to the curent years as percentage points
t10_17_master_df["PREV_CHANGE"] = t10_17_master_df["PREVALENCE"]-t10_17_master_df["PREV_PREVALENCE"]
t10_17_master_df["ACH_CHANGE"] = t10_17_master_df["ACH_PC"]-t10_17_master_df["PREV_ACHIEVEMENT_RATE"]
t10_17_master_df["PCA_CHANGE"] = t10_17_master_df["OVERALL_PCA_RATE"]-t10_17_master_df["PREV_PCA_RATE"]

## Remove duplicates and replace 'nan' values with a blank
t10_17_master_df = t10_17_master_df.drop_duplicates()
t10_17_master_df = t10_17_master_df.replace(np.nan, '', regex=True)

In [ ]:
## Convert columns that contain numbers as strings to numeric values
t10_17_master_df["LIST_SIZE"] = (t10_17_master_df["LIST_SIZE"]).astype(int)
t10_17_master_df["ALT_LIST_SIZE"] = (t10_17_master_df["ALT_LIST_SIZE"]).astype(int)
t10_17_master_df["REGISTER"] = (t10_17_master_df["REGISTER"]).astype(int)
t10_17_master_df["PREVALENCE"] = t10_17_master_df["PREVALENCE"]
t10_17_master_df["TOTAL_ACH_SCORE"] = t10_17_master_df["TOTAL_ACH_SCORE"]
t10_17_master_df["ACH_PC"] = pd.to_numeric(t10_17_master_df["ACH_PC"],errors='coerce')
t10_17_master_df["TOTAL_PCAS"] = (t10_17_master_df["TOTAL_PCAS"]).astype(int)
t10_17_master_df["TOTAL_DENOMINATORS"] = (t10_17_master_df["TOTAL_DENOMINATORS"]).astype(int)
t10_17_master_df["OVERALL_PCA_RATE"] = pd.to_numeric(t10_17_master_df["OVERALL_PCA_RATE"],errors='coerce')
t10_17_master_df["PREV_PRACTICE_LIST_SIZE"] = pd.to_numeric(t10_17_master_df["PREV_PRACTICE_LIST_SIZE"],errors='coerce')
t10_17_master_df["PREV_ALT_PAT_LIST"] = pd.to_numeric(t10_17_master_df["PREV_ALT_PAT_LIST"],errors='coerce')
t10_17_master_df["PREV_REGISTER_SIZE"] = pd.to_numeric(t10_17_master_df["PREV_REGISTER_SIZE"],errors='coerce')
t10_17_master_df["PREV_PREVALENCE"] = pd.to_numeric(t10_17_master_df["PREV_PREVALENCE"],errors='coerce')
t10_17_master_df["PREV_ACHIEVED_POINTS"] = pd.to_numeric(t10_17_master_df["PREV_ACHIEVED_POINTS"],errors='coerce')
t10_17_master_df["PREV_TOTAL_INDICATOR_GROUP_POINTS"] = pd.to_numeric(t10_17_master_df["PREV_TOTAL_INDICATOR_GROUP_POINTS"],errors='coerce')
t10_17_master_df["PREV_ACHIEVEMENT_RATE"] = pd.to_numeric(t10_17_master_df["PREV_ACHIEVEMENT_RATE"],errors='coerce')
t10_17_master_df["PREV_PCA_COUNT"] = pd.to_numeric(t10_17_master_df["PREV_PCA_COUNT"],errors='coerce')
t10_17_master_df["PREV_DENOMINATOR"] = pd.to_numeric(t10_17_master_df["PREV_DENOMINATOR"],errors='coerce')
t10_17_master_df["PREV_PCA_RATE"] = pd.to_numeric(t10_17_master_df["PREV_PCA_RATE"],errors='coerce')
t10_17_master_df["ADD_LIST_06_19"] = pd.to_numeric(t10_17_master_df["ADD_LIST_06_19"],errors='coerce')
t10_17_master_df["ADD_LIST_25_49_F"] = pd.to_numeric(t10_17_master_df["ADD_LIST_25_49_F"],errors='coerce')
t10_17_master_df["ADD_LIST_40OV"] = pd.to_numeric(t10_17_master_df["ADD_LIST_40OV"],errors='coerce')
t10_17_master_df["ADD_LIST_50_64_F"] = pd.to_numeric(t10_17_master_df["ADD_LIST_50_64_F"],errors='coerce')
t10_17_master_df["ADD_LIST_79UN"] = pd.to_numeric(t10_17_master_df["ADD_LIST_79UN"],errors='coerce')
t10_17_master_df["ADD_LIST_80OV"] = pd.to_numeric(t10_17_master_df["ADD_LIST_80OV"],errors='coerce')
t10_17_master_df["PREV_CHANGE"] = pd.to_numeric(t10_17_master_df["PREV_CHANGE"],errors='coerce')
t10_17_master_df["ACH_CHANGE"] = pd.to_numeric(t10_17_master_df["ACH_CHANGE"],errors='coerce')
t10_17_master_df["PCA_CHANGE"] = pd.to_numeric(t10_17_master_df["PCA_CHANGE"],errors='coerce')


<h2>Create Practice data tables <br>and table titles

In [ ]:
## Select data for AF group code
t10_17_af_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "AF"]

t10_17_af_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="AF")]

## Combines t10_17 Master table with previous years data
t10_17_af_df = pd.merge(t10_17_af_base_df
                        ,t10_17_af_prac_current_piv_df
                        ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,how ="left")

## Reorder and sort columns for output data frame on tab 'AF' in prac-ach-prev-pca-cv excel workbook (template 10)
t10_17_af_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                 ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE","LIST_SIZE","REGISTER"
                 ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE"
                 ,"ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
                 ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
                 ,"AF001_ACHIEVED_POINTS","AF001_ACH_RATE"
                 ,"AF006_ACHIEVED_POINTS","AF006_ACHIEVEMENT_NUMERATOR","AF006_ACHIEVEMENT_DENOMINATOR"
                 ,"AF006_UL_NET_ACH","AF006_PCA_COUNT","AF006_PCA_RATE","AF006_DEN_PLUS_PCA","AF006_PRI"
                 ,"AF007_ACHIEVED_POINTS","AF007_ACHIEVEMENT_NUMERATOR","AF007_ACHIEVEMENT_DENOMINATOR"
                 ,"AF007_UL_NET_ACH","AF007_PCA_COUNT","AF007_PCA_RATE","AF007_DEN_PLUS_PCA","AF007_PRI"]
t10_17_af_df = t10_17_af_df[t10_17_af_col]
                            
t10_17_af_df = t10_17_af_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])

In [ ]:
## Select data for BP group code
t10_17_bp_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "BP"]

t10_17_bp_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="BP")]

## Combines t10_17 Master table with previous years data
t10_17_bp_df = pd.merge(t10_17_bp_base_df
                        ,t10_17_bp_prac_current_piv_df
                        ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,how ="left")

## Reorder and sort columns for output data frame on tab 'BP' in prac-ach-prev-pca-cv excel workbook (template 10)
t10_17_bp_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                 ,"PREV_ALT_PAT_LIST","ALT_LIST_SIZE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE"
                 ,"ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
                 ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
                 ,"BP002_ACHIEVED_POINTS","BP002_ACHIEVEMENT_NUMERATOR","BP002_ACHIEVEMENT_DENOMINATOR"
                 ,"BP002_UL_NET_ACH","BP002_PCA_COUNT","BP002_PCA_RATE","BP002_DEN_PLUS_PCA","BP002_PRI"]
t10_17_bp_df = t10_17_bp_df[t10_17_bp_col]
                            
t10_17_bp_df = t10_17_bp_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])                     

In [ ]:
## Select data for CHD group code
t10_17_chd_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "CHD"]

t10_17_chd_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="CHD")]

## Combines t10_17 Master table with previous years data
t10_17_chd_df = pd.merge(t10_17_chd_base_df
                        ,t10_17_chd_prac_current_piv_df
                        ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,how ="left")

## Reorder and sort columns for output data frame on tab 'CHD' in prac-ach-prev-pca-cv excel workbook (template 10)
t10_17_chd_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                  ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE","LIST_SIZE","ADD_LIST_79UN"
                  ,"ADD_LIST_80OV","REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS"
                  ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR",
                  "PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
                  ,"CHD001_ACHIEVED_POINTS","CHD001_ACH_RATE"
                  ,"CHD005_ACHIEVED_POINTS","CHD005_ACHIEVEMENT_NUMERATOR","CHD005_ACHIEVEMENT_DENOMINATOR"
                  ,"CHD005_UL_NET_ACH","CHD005_PCA_COUNT","CHD005_PCA_RATE","CHD005_DEN_PLUS_PCA","CHD005_PRI"
                  ,"CHD008_ACHIEVED_POINTS","CHD008_ACHIEVEMENT_NUMERATOR","CHD008_ACHIEVEMENT_DENOMINATOR"
                  ,"CHD008_UL_NET_ACH","CHD008_PCA_COUNT","CHD008_PCA_RATE","CHD008_DEN_PLUS_PCA","CHD008_PRI"
                  ,"CHD009_ACHIEVED_POINTS","CHD009_ACHIEVEMENT_NUMERATOR","CHD009_ACHIEVEMENT_DENOMINATOR"
                  ,"CHD009_UL_NET_ACH","CHD009_PCA_COUNT","CHD009_PCA_RATE","CHD009_DEN_PLUS_PCA","CHD009_PRI"]
t10_17_chd_df = t10_17_chd_df[t10_17_chd_col]
                              
t10_17_chd_df = t10_17_chd_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])

In [ ]:
## Select data for HF group code
t10_17_hf_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "HF"]

t10_17_hf_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="HF")]

## Combines t10_17 Master table with previous years data
t10_17_hf_df = pd.merge(t10_17_hf_base_df
                        ,t10_17_hf_prac_current_piv_df
                        ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,how ="left")

## Reorder and sort columns for output data frame on tab 'HF' in prac-ach-prev-pca-cv excel workbook (template 10)
t10_17_hf_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                 ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE","LIST_SIZE","REGISTER"
                 ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE"
                 ,"ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
                 ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
                 ,"HF001_ACHIEVED_POINTS","HF001_ACH_RATE"
                 ,"HF003_ACHIEVED_POINTS","HF003_ACHIEVEMENT_NUMERATOR","HF003_ACHIEVEMENT_DENOMINATOR"
                 ,"HF003_UL_NET_ACH","HF003_PCA_COUNT","HF003_PCA_RATE","HF003_DEN_PLUS_PCA","HF003_PRI"
                 ,"HF005_ACHIEVED_POINTS","HF005_ACHIEVEMENT_NUMERATOR","HF005_ACHIEVEMENT_DENOMINATOR"
                 ,"HF005_UL_NET_ACH","HF005_PCA_COUNT","HF005_PCA_RATE","HF005_DEN_PLUS_PCA","HF005_PRI"
                 ,"HF006_ACHIEVED_POINTS","HF006_ACHIEVEMENT_NUMERATOR","HF006_ACHIEVEMENT_DENOMINATOR"
                 ,"HF006_UL_NET_ACH","HF006_PCA_COUNT","HF006_PCA_RATE","HF006_DEN_PLUS_PCA","HF006_PRI"
                 ,"HF007_ACHIEVED_POINTS","HF007_ACHIEVEMENT_NUMERATOR","HF007_ACHIEVEMENT_DENOMINATOR"
                 ,"HF007_UL_NET_ACH","HF007_PCA_COUNT","HF007_PCA_RATE","HF007_DEN_PLUS_PCA","HF007_PRI"]
t10_17_hf_df = t10_17_hf_df[t10_17_hf_col]
                             
t10_17_hf_df = t10_17_hf_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])                        

In [ ]:
## Select data for HYP group code
t10_17_hyp_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "HYP"]

t10_17_hyp_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="HYP")]

## Combines t10_17 Master table with previous years data
t10_17_hyp_df = pd.merge(t10_17_hyp_base_df
                        ,t10_17_hyp_prac_current_piv_df
                        ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,how ="left")

## Reorder and sort columns for output data frame on tab 'HYP' in prac-ach-prev-pca-cv excel workbook (template 10)
t10_17_hyp_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                  ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE","LIST_SIZE","ADD_LIST_79UN"
                  ,"ADD_LIST_80OV","REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
                  ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"
                  ,"TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
                  ,"HYP001_ACHIEVED_POINTS","HYP001_ACH_RATE"
                  ,"HYP003_ACHIEVED_POINTS","HYP003_ACHIEVEMENT_NUMERATOR","HYP003_ACHIEVEMENT_DENOMINATOR"
                  ,"HYP003_UL_NET_ACH","HYP003_PCA_COUNT","HYP003_PCA_RATE","HYP003_DEN_PLUS_PCA","HYP003_PRI"
                  ,"HYP007_ACHIEVED_POINTS","HYP007_ACHIEVEMENT_NUMERATOR","HYP007_ACHIEVEMENT_DENOMINATOR"
                  ,"HYP007_UL_NET_ACH","HYP007_PCA_COUNT","HYP007_PCA_RATE","HYP007_DEN_PLUS_PCA","HYP007_PRI"]
t10_17_hyp_df = t10_17_hyp_df[t10_17_hyp_col]
                              
t10_17_hyp_df = t10_17_hyp_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])      

In [ ]:
## Select data for PAD group code
t10_17_pad_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "PAD"]

t10_17_pad_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="PAD")]

## Combines t10_17 Master table with previous years data
t10_17_pad_df = pd.merge(t10_17_pad_base_df
                        ,t10_17_pad_prac_current_piv_df
                        ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,how ="left")

## Reorder and sort columns for output data frame on tab 'PAD' in prac-ach-prev-pca-cv excel workbook (template 10)
t10_17_pad_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE"
                  ,"PRACTICE_NAME","PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE"
                  ,"LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS"
                  ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
                  ,"PAD001_ACHIEVED_POINTS","PAD001_ACH_RATE"]
t10_17_pad_df = t10_17_pad_df[t10_17_pad_col]
                               
t10_17_pad_df = t10_17_pad_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])                           

In [ ]:
## Select data for STIA group code
t10_17_stia_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "STIA"]

t10_17_stia_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="STIA")]

## Combines t10_17 Master table with previous years data
t10_17_stia_df = pd.merge(t10_17_stia_base_df
                          ,t10_17_stia_prac_current_piv_df
                          ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,how ="left")

## Reorder and sort columns for output data frame on tab 'STIA' in prac-ach-prev-pca-cv excel workbook (template 10)
t10_17_stia_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                   ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE","LIST_SIZE","ADD_LIST_79UN"
                   ,"ADD_LIST_80OV","REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
                   ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"
                   ,"TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
                   ,"STIA001_ACHIEVED_POINTS","STIA001_ACH_RATE"
                   ,"STIA007_ACHIEVED_POINTS","STIA007_ACHIEVEMENT_NUMERATOR","STIA007_ACHIEVEMENT_DENOMINATOR"
                   ,"STIA007_UL_NET_ACH","STIA007_PCA_COUNT","STIA007_PCA_RATE","STIA007_DEN_PLUS_PCA","STIA007_PRI"
                   ,"STIA010_ACHIEVED_POINTS","STIA010_ACHIEVEMENT_NUMERATOR","STIA010_ACHIEVEMENT_DENOMINATOR"
                   ,"STIA010_UL_NET_ACH","STIA010_PCA_COUNT","STIA010_PCA_RATE","STIA010_DEN_PLUS_PCA","STIA010_PRI"
                   ,"STIA011_ACHIEVED_POINTS","STIA011_ACHIEVEMENT_NUMERATOR","STIA011_ACHIEVEMENT_DENOMINATOR"
                   ,"STIA011_UL_NET_ACH","STIA011_PCA_COUNT","STIA011_PCA_RATE","STIA011_DEN_PLUS_PCA","STIA011_PRI"]
t10_17_stia_df = t10_17_stia_df[t10_17_stia_col]
                                
t10_17_stia_df = t10_17_stia_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])                                               

In [ ]:
## Select data for AST group code
t10_17_ast_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "AST"]

t10_17_ast_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="AST")]

## Combines t10_17 Master table with previous years data
t10_17_ast_df = pd.merge(t10_17_ast_base_df
                          ,t10_17_ast_prac_current_piv_df
                          ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,how ="left")

## Reorder and sort columns for output data frame on tab 'AST' in prac-ach-prev-pca-resp excel workbook (template 11)
t10_17_ast_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                  ,"PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE","ALT_LIST_SIZE","ADD_LIST_06_19"
                  ,"REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
                  ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"
                  ,"TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
                  ,"AST005_ACHIEVED_POINTS","AST005_ACH_RATE"
                  ,"AST006_ACHIEVED_POINTS","AST006_ACHIEVEMENT_NUMERATOR","AST006_ACHIEVEMENT_DENOMINATOR"
                  ,"AST006_UL_NET_ACH","AST006_PCA_COUNT","AST006_PCA_RATE","AST006_DEN_PLUS_PCA","AST006_PRI"
                  ,"AST007_ACHIEVED_POINTS","AST007_ACHIEVEMENT_NUMERATOR","AST007_ACHIEVEMENT_DENOMINATOR"
                  ,"AST007_UL_NET_ACH","AST007_PCA_COUNT","AST007_PCA_RATE","AST007_DEN_PLUS_PCA","AST007_PRI"
                  ,"AST008_ACHIEVED_POINTS","AST008_ACHIEVEMENT_NUMERATOR","AST008_ACHIEVEMENT_DENOMINATOR"
                  ,"AST008_UL_NET_ACH","AST008_PCA_COUNT","AST008_PCA_RATE","AST008_DEN_PLUS_PCA","AST008_PRI"]
t10_17_ast_df = t10_17_ast_df[t10_17_ast_col]                       
                              
t10_17_ast_df = t10_17_ast_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])                          

In [ ]:
## Select data for COPD group code
t10_17_copd_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "COPD"]

t10_17_copd_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="COPD")]


## Combines t10_17 Master table with previous years data
t10_17_copd_df = pd.merge(t10_17_copd_base_df
                          ,t10_17_copd_prac_current_piv_df
                          ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,how ="left")

## Reorder and sort columns for output data frame on tab 'COPD' in prac-ach-prev-pca-resp excel workbook (template 11)
t10_17_copd_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                   ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE","LIST_SIZE","REGISTER"
                   ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE"
                   ,"ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
                   ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
                   ,"COPD009_ACHIEVED_POINTS","COPD009_ACH_RATE"
                   ,"COPD008_ACHIEVED_POINTS","COPD008_ACHIEVEMENT_NUMERATOR","COPD008_ACHIEVEMENT_DENOMINATOR"
                   ,"COPD008_UL_NET_ACH","COPD008_PCA_COUNT","COPD008_PCA_RATE","COPD008_DEN_PLUS_PCA","COPD008_PRI"
                   ,"COPD010_ACHIEVED_POINTS","COPD010_ACHIEVEMENT_NUMERATOR","COPD010_ACHIEVEMENT_DENOMINATOR"
                   ,"COPD010_UL_NET_ACH","COPD010_PCA_COUNT","COPD010_PCA_RATE","COPD010_DEN_PLUS_PCA","COPD010_PRI"]
t10_17_copd_df = t10_17_copd_df[t10_17_copd_col]
                                
t10_17_copd_df = t10_17_copd_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])                          

In [ ]:
## Select data for OB group code
t10_17_ob_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "OB"]

t10_17_ob_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="OB")]

## Combines t10_17 Master table with previous years data
t10_17_ob_df = pd.merge(t10_17_ob_base_df
                          ,t10_17_ob_prac_current_piv_df
                          ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,how ="left")

## Reorder and sort columns for output data frame on tab 'OB' in prac-ach-prev-pca-ls excel workbook (template 12)
t10_17_ob_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                 ,"PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE","ALT_LIST_SIZE","REGISTER","PREVALENCE"
                 ,"PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
                 ,"OB002_ACHIEVED_POINTS","OB002_ACH_RATE"]
t10_17_ob_df = t10_17_ob_df[t10_17_ob_col]
                            
t10_17_ob_df = t10_17_ob_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])                   

In [ ]:
## Select data for SMOK group code
t10_17_smok_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "SMOK"]

t10_17_smok_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="SMOK")]

## Combines t10_17 Master table with previous years data
t10_17_smok_df = pd.merge(t10_17_smok_base_df
                          ,t10_17_smok_prac_current_piv_df
                          ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,how ="left")

## Reorder and sort columns for output data frame on tab 'SMOK' in prac-ach-prev-pca-ls excel workbook (template 12)
t10_17_smok_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                   ,"PREV_ALT_PAT_LIST","ALT_LIST_SIZE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
                   ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE"
                   ,"TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
                   ,"SMOK002_ACHIEVED_POINTS","SMOK002_ACHIEVEMENT_NUMERATOR","SMOK002_ACHIEVEMENT_DENOMINATOR"
                   ,"SMOK002_UL_NET_ACH","SMOK002_PCA_COUNT","SMOK002_PCA_RATE","SMOK002_DEN_PLUS_PCA","SMOK002_PRI"
                   ,"SMOK004_ACHIEVED_POINTS","SMOK004_ACHIEVEMENT_NUMERATOR","SMOK004_ACHIEVEMENT_DENOMINATOR"
                   ,"SMOK004_UL_NET_ACH","SMOK004_PCA_COUNT","SMOK004_PCA_RATE","SMOK004_DEN_PLUS_PCA","SMOK004_PRI"
                   ,"SMOK005_ACHIEVED_POINTS","SMOK005_ACHIEVEMENT_NUMERATOR","SMOK005_ACHIEVEMENT_DENOMINATOR"
                   ,"SMOK005_UL_NET_ACH","SMOK005_PCA_COUNT","SMOK005_PCA_RATE","SMOK005_DEN_PLUS_PCA","SMOK005_PRI"]
t10_17_smok_df = t10_17_smok_df[t10_17_smok_col]
                                
t10_17_smok_df = t10_17_smok_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])                         

In [ ]:
## Select data for CAN group code
t10_17_can_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "CAN"]

t10_17_can_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="CAN")]

## Combines t10_17 Master table with previous years data
t10_17_can_df = pd.merge(t10_17_can_base_df
                          ,t10_17_can_prac_current_piv_df
                          ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,how ="left")

## Reorder and sort columns for output data frame on tab 'CAN' in prac-ach-prev-pca-hd excel workbook (template 13)
t10_17_can_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                  ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE","LIST_SIZE","REGISTER"
                  ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE"
                  ,"ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
                  ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
                  ,"CAN001_ACHIEVED_POINTS","CAN001_ACH_RATE"
                  ,"CAN004_ACHIEVED_POINTS","CAN004_ACHIEVEMENT_NUMERATOR","CAN004_ACHIEVEMENT_DENOMINATOR"
                  ,"CAN004_UL_NET_ACH","CAN004_PCA_COUNT","CAN004_PCA_RATE","CAN004_DEN_PLUS_PCA","CAN004_PRI"
                  ,"CAN005_ACHIEVED_POINTS","CAN005_ACHIEVEMENT_NUMERATOR","CAN005_ACHIEVEMENT_DENOMINATOR"
                  ,"CAN005_UL_NET_ACH","CAN005_PCA_COUNT","CAN005_PCA_RATE","CAN005_DEN_PLUS_PCA","CAN005_PRI"]
t10_17_can_df = t10_17_can_df[t10_17_can_col]                           
                                                                         
t10_17_can_df = t10_17_can_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])

In [ ]:
## Select data for CKD group code
t10_17_ckd_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "CKD"]

t10_17_ckd_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="CKD")]


## Combines t10_17 Master table with previous years data
t10_17_ckd_df = pd.merge(t10_17_ckd_base_df
                          ,t10_17_ckd_prac_current_piv_df
                          ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,how ="left")

## Reorder and sort columns for output data frame on tab 'CKD' in prac-ach-prev-pca-hd excel workbook (template 13)
t10_17_ckd_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE"
                  ,"PRACTICE_NAME","PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE"
                  ,"ALT_LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS"
                  ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
                  ,"CKD005_ACHIEVED_POINTS","CKD005_ACH_RATE"]
t10_17_ckd_df = t10_17_ckd_df[t10_17_ckd_col]
                              
t10_17_ckd_df = t10_17_ckd_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])

In [ ]:
## Select data for DM group code
t10_17_dm_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "DM"]

t10_17_dm_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="DM")]

## Combines t10_17 Master table with previous years data
t10_17_dm_df = pd.merge(t10_17_dm_base_df
                        ,t10_17_dm_prac_current_piv_df
                        ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,how ="left")

## Reorder and sort columns for output data frame on tab 'DM' in prac-ach-prev-pca-hd excel workbook (template 13)
t10_17_dm_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                 ,"PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE","ALT_LIST_SIZE","ADD_LIST_40OV"
                 ,"REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
                 ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR"
                 ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
                 ,"DM017_ACHIEVED_POINTS","DM017_ACH_RATE"                        
                 ,"DM006_ACHIEVED_POINTS","DM006_ACHIEVEMENT_NUMERATOR","DM006_ACHIEVEMENT_DENOMINATOR"
                 ,"DM006_UL_NET_ACH","DM006_PCA_COUNT","DM006_PCA_RATE","DM006_DEN_PLUS_PCA","DM006_PRI"
                 ,"DM012_ACHIEVED_POINTS","DM012_ACHIEVEMENT_NUMERATOR","DM012_ACHIEVEMENT_DENOMINATOR"
                 ,"DM012_UL_NET_ACH","DM012_PCA_COUNT","DM012_PCA_RATE","DM012_DEN_PLUS_PCA","DM012_PRI"
                 ,"DM014_ACHIEVED_POINTS","DM014_ACHIEVEMENT_NUMERATOR","DM014_ACHIEVEMENT_DENOMINATOR"
                 ,"DM014_UL_NET_ACH","DM014_PCA_COUNT","DM014_PCA_RATE","DM014_DEN_PLUS_PCA","DM014_PRI"
                 ,"DM019_ACHIEVED_POINTS","DM019_ACHIEVEMENT_NUMERATOR","DM019_ACHIEVEMENT_DENOMINATOR"
                 ,"DM019_UL_NET_ACH","DM019_PCA_COUNT","DM019_PCA_RATE","DM019_DEN_PLUS_PCA","DM019_PRI"
                 ,"DM020_ACHIEVED_POINTS","DM020_ACHIEVEMENT_NUMERATOR","DM020_ACHIEVEMENT_DENOMINATOR"
                 ,"DM020_UL_NET_ACH","DM020_PCA_COUNT","DM020_PCA_RATE","DM020_DEN_PLUS_PCA","DM020_PRI"
                 ,"DM021_ACHIEVED_POINTS","DM021_ACHIEVEMENT_NUMERATOR","DM021_ACHIEVEMENT_DENOMINATOR"
                 ,"DM021_UL_NET_ACH","DM021_PCA_COUNT","DM021_PCA_RATE","DM021_DEN_PLUS_PCA","DM021_PRI"
                 ,"DM022_ACHIEVED_POINTS","DM022_ACHIEVEMENT_NUMERATOR","DM022_ACHIEVEMENT_DENOMINATOR"
                 ,"DM022_UL_NET_ACH","DM022_PCA_COUNT","DM022_PCA_RATE","DM022_DEN_PLUS_PCA","DM022_PRI"
                 ,"DM023_ACHIEVED_POINTS","DM023_ACHIEVEMENT_NUMERATOR","DM023_ACHIEVEMENT_DENOMINATOR"
                 ,"DM023_UL_NET_ACH","DM023_PCA_COUNT","DM023_PCA_RATE","DM023_DEN_PLUS_PCA","DM023_PRI"]
t10_17_dm_df = t10_17_dm_df[t10_17_dm_col]                     
                             
t10_17_dm_df = t10_17_dm_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])                 

In [ ]:
## Select data for PC group code
t10_17_pc_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "PC"]

t10_17_pc_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="PC")]

## Combines t10_17 Master table with previous years data
t10_17_pc_df = pd.merge(t10_17_pc_base_df
                          ,t10_17_pc_prac_current_piv_df
                          ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,how ="left")

## Reorder and sort columns for output data frame on tab 'PC' in prac-ach-prev-pca-hd excel workbook (template 13)
t10_17_pc_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE"
                 ,"PRACTICE_NAME","PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE"
                 ,"LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS"
                 ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
                 ,"PC001_ACHIEVED_POINTS","PC001_ACH_RATE"]
t10_17_pc_df = t10_17_pc_df[t10_17_pc_col]
                            
t10_17_pc_df = t10_17_pc_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])      

In [ ]:
## Select data for DEM group code
t10_17_dem_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "DEM"]

t10_17_dem_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="DEM")]

## Combines t10_17 Master table with previous years data
t10_17_dem_df = pd.merge(t10_17_dem_base_df
                          ,t10_17_dem_prac_current_piv_df
                          ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,how ="left")

## Reorder and sort columns for output data frame on tab 'DEM' in prac-ach-prev-pca-neu excel workbook (template 14)
t10_17_dem_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                  ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE", "LIST_SIZE","REGISTER"
                  ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
                  ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR"
                  ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
                  ,"DEM001_ACHIEVED_POINTS","DEM001_ACH_RATE"
                  ,"DEM004_ACHIEVED_POINTS","DEM004_ACHIEVEMENT_NUMERATOR","DEM004_ACHIEVEMENT_DENOMINATOR","DEM004_UL_NET_ACH","DEM004_PCA_COUNT","DEM004_PCA_RATE","DEM004_DEN_PLUS_PCA","DEM004_PRI"]
t10_17_dem_df = t10_17_dem_df[t10_17_dem_col]
                              
t10_17_dem_df = t10_17_dem_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])                    

In [ ]:
## Select data for DEP group code
t10_17_dep_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "DEP"]

t10_17_dep_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="DEP")]

## Combines t10_17 Master table with previous years data
t10_17_dep_df = pd.merge(t10_17_dep_base_df
                          ,t10_17_dep_prac_current_piv_df
                          ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                          ,how ="left")

## Reorder and sort columns for output data frame on tab 'DEP' in prac-ach-prev-pca-neu excel workbook (template 14)
t10_17_dep_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                  ,"PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE","ALT_LIST_SIZE","REGISTER"
                  ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE"
                  ,"ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
                  ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
                  ,"DEP003_ACHIEVED_POINTS","DEP003_ACHIEVEMENT_NUMERATOR","DEP003_ACHIEVEMENT_DENOMINATOR"
                  ,"DEP003_UL_NET_ACH","DEP003_PCA_COUNT","DEP003_PCA_RATE","DEP003_DEN_PLUS_PCA","DEP003_PRI"]
t10_17_dep_df = t10_17_dep_df[t10_17_dep_col]
                              
t10_17_dep_df = t10_17_dep_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])               

In [ ]:
## Select data for EP group code
t10_17_ep_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "EP"]

t10_17_ep_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="EP")]

## Combines t10_17 Master table with previous years data
t10_17_ep_df = pd.merge(t10_17_ep_base_df
                        ,t10_17_ep_prac_current_piv_df
                        ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,how ="left")

## Reorder and sort columns for output data frame on tab 'EP' in prac-ach-prev-pca-neu excel workbook (template 14)
t10_17_ep_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                 ,"PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE","ALT_LIST_SIZE","REGISTER"
                 ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
                 ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
                 ,"EP001_ACHIEVED_POINTS","EP001_ACH_RATE"]
t10_17_ep_df = t10_17_ep_df[t10_17_ep_col]
                             
t10_17_ep_df = t10_17_ep_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])

In [ ]:
## Select data for LD group code
t10_17_ld_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "LD"]

t10_17_ld_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="LD")]

## Combines t10_17 Master table with previous years data
t10_17_ld_df = pd.merge(t10_17_ld_base_df
                        ,t10_17_ld_prac_current_piv_df
                        ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,how ="left")

## Reorder and sort columns for output data frame on tab 'LD' in prac-ach-prev-pca-neu excel workbook (template 14)
t10_17_ld_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                 ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE","LIST_SIZE","REGISTER"
                 ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
                 ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
                 ,"LD004_ACHIEVED_POINTS","LD004_ACH_RATE"]
t10_17_ld_df = t10_17_ld_df[t10_17_ld_col]
                             
t10_17_ld_df = t10_17_ld_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])                    

In [ ]:
## Select data for MH group code
t10_17_mh_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "MH"]

t10_17_mh_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="MH")]


## Combines t10_17 Master table with previous years data
t10_17_mh_df = pd.merge(t10_17_mh_base_df
                        ,t10_17_mh_prac_current_piv_df
                        ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,how ="left")

## Reorder and sort columns for output data frame on tab 'MH' in prac-ach-prev-pca-neu excel workbook (template 14)
t10_17_mh_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                 ,"PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE","LIST_SIZE","REGISTER"
                 ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE"
                 ,"ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
                 ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
                 ,"MH001_ACHIEVED_POINTS","MH001_ACH_RATE"                         
                 ,"MH002_ACHIEVED_POINTS","MH002_ACHIEVEMENT_NUMERATOR","MH002_ACHIEVEMENT_DENOMINATOR"
                 ,"MH002_UL_NET_ACH","MH002_PCA_COUNT","MH002_PCA_RATE","MH002_DEN_PLUS_PCA","MH002_PRI"
                 ,"MH003_ACHIEVED_POINTS","MH003_ACHIEVEMENT_NUMERATOR","MH003_ACHIEVEMENT_DENOMINATOR"
                 ,"MH003_UL_NET_ACH","MH003_PCA_COUNT","MH003_PCA_RATE","MH003_DEN_PLUS_PCA","MH003_PRI"
                 ,"MH006_ACHIEVED_POINTS","MH006_ACHIEVEMENT_NUMERATOR","MH006_ACHIEVEMENT_DENOMINATOR"
                 ,"MH006_UL_NET_ACH","MH006_PCA_COUNT","MH006_PCA_RATE","MH006_DEN_PLUS_PCA","MH006_PRI"
                 ,"MH007_ACHIEVED_POINTS","MH007_ACHIEVEMENT_NUMERATOR","MH007_ACHIEVEMENT_DENOMINATOR"
                 ,"MH007_UL_NET_ACH","MH007_PCA_COUNT","MH007_PCA_RATE","MH007_DEN_PLUS_PCA","MH007_PRI"
                 ,"MH011_ACHIEVED_POINTS","MH011_ACHIEVEMENT_NUMERATOR","MH011_ACHIEVEMENT_DENOMINATOR"
                 ,"MH011_UL_NET_ACH","MH011_PCA_COUNT","MH011_PCA_RATE","MH011_DEN_PLUS_PCA","MH011_PRI"
                 ,"MH012_ACHIEVED_POINTS","MH012_ACHIEVEMENT_NUMERATOR","MH012_ACHIEVEMENT_DENOMINATOR"
                 ,"MH012_UL_NET_ACH","MH012_PCA_COUNT","MH012_PCA_RATE","MH012_DEN_PLUS_PCA","MH012_PRI"]
t10_17_mh_df = t10_17_mh_df[t10_17_mh_col]
                             
t10_17_mh_df = t10_17_mh_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])                   

In [ ]:
## Select data for OST group code
t10_17_ost_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "OST"]

t10_17_ost_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="OST")]


## Combines t10_17 Master table with previous years data
t10_17_ost_df = pd.merge(t10_17_ost_base_df
                        ,t10_17_ost_prac_current_piv_df
                        ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,how ="left")

## Reorder and sort columns for output data frame on tab 'OST' in prac-ach-prev-pca-ms excel workbook (template 15)
t10_17_ost_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                  ,"PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE","ALT_LIST_SIZE","REGISTER"
                  ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
                  ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE"
                  ,"OST004_ACHIEVED_POINTS","OST004_ACH_RATE"]
t10_17_ost_df = t10_17_ost_df[t10_17_ost_col]
                               
t10_17_ost_df = t10_17_ost_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])                            

In [ ]:
## Select data for RA group code
t10_17_ra_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "RA"]

t10_17_ra_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="RA")]


## Combines t10_17 Master table with previous years data
t10_17_ra_df = pd.merge(t10_17_ra_base_df
                        ,t10_17_ra_prac_current_piv_df
                        ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,how ="left")

## Reorder and sort columns for output data frame on tab 'RA' in prac-ach-prev-pca-ms excel workbook (template 15)
t10_17_ra_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                 ,"PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE","ALT_LIST_SIZE","REGISTER"
                 ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE"
                 ,"ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
                 ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
                 ,"RA001_ACHIEVED_POINTS","RA001_ACH_RATE"
                 ,"RA002_ACHIEVED_POINTS","RA002_ACHIEVEMENT_NUMERATOR","RA002_ACHIEVEMENT_DENOMINATOR"
                 ,"RA002_UL_NET_ACH","RA002_PCA_COUNT","RA002_PCA_RATE","RA002_DEN_PLUS_PCA","RA002_PRI"]
t10_17_ra_df = t10_17_ra_df[t10_17_ra_col]
                             
t10_17_ra_df = t10_17_ra_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])                 

In [ ]:
## Select data for NDH group code
t10_17_ndh_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "NDH"]

## As NDH had no achievement points allocated in the previous year, it had no achievement rate for comparison with this year
## Therefore the missing value in the columns 'Achievement (%)' and  'Year on year change' has been replaced with an 'z' for not applicable
t10_17_ndh_base_df["PREV_ACHIEVEMENT_RATE"] = t10_17_ndh_base_df["PREV_ACHIEVEMENT_RATE"].fillna("z")
t10_17_ndh_base_df["ACH_CHANGE"] = t10_17_ndh_base_df["ACH_CHANGE"].fillna("z")

t10_17_ndh_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="NDH")]


## Combines T8 Master table with previous years data
t10_17_ndh_df = pd.merge(t10_17_ndh_base_df
                         ,t10_17_ndh_prac_current_piv_df    
                         ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                         ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                         ,how ="left")

## Reorder columns for output data frame on tab 'NDH' in prac-ach-prev-pca-hd excel workbook (template 13)
t10_17_ndh_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                  ,"PREV_ALT_PAT_LIST","PREV_REGISTER_SIZE","PREV_PREVALENCE","ALT_LIST_SIZE", "REGISTER"
                  ,"PREVALENCE","PREV_CHANGE","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE"
                  ,"ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR","PREV_PCA_RATE","TOTAL_PCAS"
                  ,"TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"                  
                  ,"NDH001_ACHIEVED_POINTS","NDH001_ACHIEVEMENT_NUMERATOR","NDH001_ACHIEVEMENT_DENOMINATOR"
                  ,"NDH001_UL_NET_ACH","NDH001_PCA_COUNT","NDH001_PCA_RATE","NDH001_DEN_PLUS_PCA","NDH001_PRI"]
t10_17_ndh_df = t10_17_ndh_df[t10_17_ndh_col]
                             
t10_17_ndh_df = t10_17_ndh_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])

In [ ]:
## Select data for CS group code
t10_17_cs_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "CS"]

t10_17_cs_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="CS")]


## Combines t10_17 Master table with previous years data
t10_17_cs_df = pd.merge(t10_17_cs_base_df
                        ,t10_17_cs_prac_current_piv_df
                        ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,how ="left")

## Reorder and sort columns for output data frame on tab 'CS' in prac-ach-prev-pca-fer-obs-gyn excel workbook (template 16)
t10_17_cs_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                 ,"PREV_ALT_PAT_LIST","ADD_LIST_25_49_F","ADD_LIST_50_64_F","PREV_ACHIEVED_POINTS","PREV_ACHIEVEMENT_RATE"
                 ,"TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE", "PREV_PCA_COUNT","PREV_DENOMINATOR"
                 ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"
                 ,"CS005_ACHIEVED_POINTS","CS005_ACHIEVEMENT_NUMERATOR","CS005_ACHIEVEMENT_DENOMINATOR"
                 ,"CS005_UL_NET_ACH","CS005_PCA_COUNT","CS005_PCA_RATE","CS005_DEN_PLUS_PCA","CS005_PRI"
                 ,"CS006_ACHIEVED_POINTS","CS006_ACHIEVEMENT_NUMERATOR","CS006_ACHIEVEMENT_DENOMINATOR"
                 ,"CS006_UL_NET_ACH","CS006_PCA_COUNT","CS006_PCA_RATE","CS006_DEN_PLUS_PCA","CS006_PRI"]
t10_17_cs_df = t10_17_cs_df[t10_17_cs_col]
                             
t10_17_cs_df = t10_17_cs_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])   

In [ ]:
## Select data for VI group code
t10_17_vi_base_df = t10_17_master_df[t10_17_master_df["GROUP_CODE"] == "VI"]

t10_17_vi_prac_current_piv_df = t10_17_prac_current_piv_df.loc[(t10_17_prac_current_piv_df["GROUP_CODE"]=="VI")]


## Combines t10_17 Master table with previous years data
t10_17_vi_df = pd.merge(t10_17_vi_base_df
                        ,t10_17_vi_prac_current_piv_df
                        ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                        ,how = "left")

## Reorder and sort columns for output data frame on tab 'VI' in prac-ach-prev-pca-vi excel workbook (template 19)
t10_17_vi_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                 ,"ADD_LIST_UN01", "ADD_LIST_1_2", "ADD_LIST_4_5", "ADD_LIST_79_80","PREV_ACHIEVED_POINTS"
                 ,"PREV_ACHIEVEMENT_RATE","TOTAL_ACH_SCORE","ACH_PC","ACH_CHANGE","PREV_PCA_COUNT","PREV_DENOMINATOR"
                 ,"PREV_PCA_RATE","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE","PCA_CHANGE"                   
                 ,"VI001_ACHIEVED_POINTS","VI001_ACHIEVEMENT_NUMERATOR","VI001_ACHIEVEMENT_DENOMINATOR"
                 ,"VI001_UL_NET_ACH","VI001_PCA_COUNT","VI001_PCA_RATE","VI001_DEN_PLUS_PCA","VI001_PRI"
                 ,"VI002_ACHIEVED_POINTS","VI002_ACHIEVEMENT_NUMERATOR","VI002_ACHIEVEMENT_DENOMINATOR"
                 ,"VI002_UL_NET_ACH","VI002_PCA_COUNT","VI002_PCA_RATE","VI002_DEN_PLUS_PCA","VI002_PRI"
                 ,"VI003_ACHIEVED_POINTS","VI003_ACHIEVEMENT_NUMERATOR","VI003_ACHIEVEMENT_DENOMINATOR"
                 ,"VI003_UL_NET_ACH","VI003_PCA_COUNT","VI003_PCA_RATE","VI003_DEN_PLUS_PCA","VI003_PRI"
                 ,"VI004_ACHIEVED_POINTS","VI004_ACHIEVEMENT_NUMERATOR","VI004_ACHIEVEMENT_DENOMINATOR"
                 ,"VI004_UL_NET_ACH","VI004_PCA_COUNT","VI004_PCA_RATE","VI004_DEN_PLUS_PCA","VI004_PRI"]
t10_17_vi_df = t10_17_vi_df[t10_17_vi_col]

t10_17_vi_df = t10_17_vi_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])                  

In [ ]:
## Join QI group codes with numeric key to the master data frame
## This means that QI codes can be reused year-on-year without having to change the body of the code
## QI data is only published for the current year as the indicators change and are therefore not comparable
t10_17_master_df = pd.merge(QI_gr_codes_with_key
                            ,t10_17_master_df
                            ,left_on = ["GROUP_CODE"]
                            ,right_on = ["GROUP_CODE"]
                            ,how = "left")

In [ ]:
## Select numeric key for QIPDD group code
t10_17_qi1_base_df = t10_17_master_df[t10_17_master_df["GrC_KEY"] == "QI_1"]

## Select indicator specific data frame
t10_17_combined_all_qi1_df = t10_17_prac_current_piv_df

## Combines t10_17 Master table with indicator data
t10_17_qi1_df = pd.merge(t10_17_qi1_base_df
                         ,t10_17_combined_all_qi1_df
                         ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                         ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                         ,how = "left")

## Reorder columns for output data frame on tab 'QIPDD' in prac-ach-prev-pca-qual-imp excel workbook (template 17)
t10_17_qi1_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                  ,"LIST_SIZE","TOTAL_ACH_SCORE","ACH_PC"
                  ,"QIPDD009_ACHIEVED_POINTS","QIPDD009_ACH_RATE"
                  ,"QIPDD010_ACHIEVED_POINTS","QIPDD010_ACH_RATE"]
t10_17_qi1_df = t10_17_qi1_df[t10_17_qi1_col]
                                   
t10_17_qi1_df = t10_17_qi1_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])                        

In [ ]:
## Select numeric key for QIOA group code
t10_17_qi2_base_df = t10_17_master_df[t10_17_master_df["GrC_KEY"] == "QI_2"]

## Select indicator specific data frame
t10_17_combined_all_qi2_df = t10_17_prac_current_piv_df

## Combines t10_17 Master table with indicator data
t10_17_qi2_df = pd.merge(t10_17_qi2_base_df
                         ,t10_17_combined_all_qi2_df
                         ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                         ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                         ,how = "left")

## Reorder columns for output data frame on tab 'QIOA' in prac-ach-prev-pca-qual-imp excel workbook (template 17)
t10_17_qi2_col =["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                 ,"LIST_SIZE","TOTAL_ACH_SCORE","ACH_PC"
                 ,"QIOA011_ACHIEVED_POINTS","QIOA011_ACH_RATE"
                 ,"QIOA012_ACHIEVED_POINTS","QIOA012_ACH_RATE"]
t10_17_qi2_df = t10_17_qi2_df[t10_17_qi2_col]
                                 
t10_17_qi2_df = t10_17_qi2_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])                    

In [ ]:
## Select numeric key for QIPDD_MI group code
t10_17_qi3_base_df = t10_17_master_df[t10_17_master_df["GrC_KEY"] == "QI_3"]

## Select indicator specific data frame
t10_17_combined_all_qi3_df = t10_17_prac_current_piv_df

## Combines T8 Master table with indicator data
t10_17_qi3_df = pd.merge(t10_17_qi3_base_df
                         ,t10_17_combined_all_qi3_df
                         ,left_on = ["PRACTICE_CODE","GROUP_CODE"]
                         ,right_on = ["PRACTICE_CODE","GROUP_CODE"]
                         ,how = "left")

## This indicator has zero achievement points attributed to it and cannot therefore calculate an achievement percentage
## The value is replaced with 'z' for data not applicable
t10_17_qi3_df["ACH_PC"] = t10_17_qi3_df["ACH_PC"].fillna("z")

## Reorder columns for output data frame on tab 'QIPDD_MI' in prac-ach-prev-pca-qual-imp excel workbook (template 17)
t10_17_qi3_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE","PRACTICE_NAME"
                  ,"ALT_LIST_SIZE","TOTAL_ACH_SCORE","ACH_PC","TOTAL_PCAS","TOTAL_DENOMINATORS","OVERALL_PCA_RATE"
                  ,"QIPDDMI001_ACHIEVED_POINTS","QIPDDMI001_ACHIEVEMENT_NUMERATOR","QIPDDMI001_ACHIEVEMENT_DENOMINATOR"
                  ,"QIPDDMI001_UL_NET_ACH","QIPDDMI001_PCA_COUNT","QIPDDMI001_PCA_RATE","QIPDDMI001_DEN_PLUS_PCA","QIPDDMI001_PRI"
                  ,"QIPDDMI002_ACHIEVED_POINTS","QIPDDMI002_ACHIEVEMENT_NUMERATOR","QIPDDMI002_ACHIEVEMENT_DENOMINATOR"
                  ,"QIPDDMI002_UL_NET_ACH","QIPDDMI002_PCA_COUNT","QIPDDMI002_PCA_RATE","QIPDDMI002_DEN_PLUS_PCA","QIPDDMI002_PRI"
                  ,"QIPDDMI003_ACHIEVED_POINTS","QIPDDMI003_ACHIEVEMENT_NUMERATOR","QIPDDMI003_ACHIEVEMENT_DENOMINATOR"
                  ,"QIPDDMI003_UL_NET_ACH","QIPDDMI003_PCA_COUNT","QIPDDMI003_PCA_RATE","QIPDDMI003_DEN_PLUS_PCA","QIPDDMI003_PRI"
                  ,"QIPDDMI004_ACHIEVED_POINTS","QIPDDMI004_ACHIEVEMENT_NUMERATOR","QIPDDMI004_ACHIEVEMENT_DENOMINATOR"
                  ,"QIPDDMI004_UL_NET_ACH","QIPDDMI004_PCA_COUNT","QIPDDMI004_PCA_RATE","QIPDDMI004_DEN_PLUS_PCA","QIPDDMI004_PRI"]
t10_17_qi3_df = t10_17_qi3_df[t10_17_qi3_col]
                         
t10_17_qi3_df = t10_17_qi3_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])       

<h3>Create LVSD data frame only used in practice data

In [ ]:
## Drop unwanted columns from the master data frame
t10_17_prac_lvsd_base_df = qof_master.drop(columns=["NAT_ONS_CODE","NAT_CODE","NAT_COUNTRY"
                                                    ,"REGION_ODS_CODE","REGION_ONS_CODE","REGION_NAME"
                                                    ,"STP_ODS_CODE","STP_ONS_CODE","STP_NAME"
                                                    ,"DOMAIN_CODE","SUB_DOMAIN_CODE"])

## Filter for 'HF003' indicator which equates to LVSD                                               
t10_17_prac_lvsd_fil_df = t10_17_prac_lvsd_base_df.loc[(t10_17_prac_lvsd_base_df["INDICATOR_CODE"]=="HF003")]


## Sum all non-index indicators
t10_17_prac_lvsd_summed_df = t10_17_prac_lvsd_fil_df.groupby(by = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME"
                                                                   ,"PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE"
                                                                   ,"PRACTICE_NAME","INDICATOR_CODE"]
                                                                   ,as_index=False).agg({"LIST_SIZE" : [pd.Series.sum]
                                                                                         ,"PCA_COUNT" : [pd.Series.sum]
                                                                                         ,"ACHIEVEMENT_DENOMINATOR" : [pd.Series.sum]})                                                                                                                                                                                                                                                                    
t10_17_prac_lvsd_summed_df.columns = t10_17_prac_lvsd_summed_df.columns.droplevel(1)

## Rename columns 'PCA_COUNT' as 'TOTAL_PCAS' and 'ACHIEVEMENT_DENOMINATOR' as 'TOTAL_DENOMINATOR' which contain total counts
t10_17_prac_lvsd_summed_df = t10_17_prac_lvsd_summed_df.rename(columns={"PCA_COUNT":"TOTAL_PCAS"
                                                                        ,"ACHIEVEMENT_DENOMINATOR":"TOTAL_DENOMINATOR"})

In [ ]:
## Drop unwanted columns from previous years LVSD data
t10_17_prac_previous_lvsd_df = LVSD_data.drop(columns=["HIGHER_GROUP_CODE"])

## Merge LVSD data for the current and previous years
t10_17_prac_lvsd_df = pd.merge(t10_17_prac_lvsd_summed_df
                                ,t10_17_prac_previous_lvsd_df
                                ,left_on = ["PRACTICE_CODE"]
                                ,right_on = ["PRACTICE_CODE"]
                                ,how = "left")

In [ ]:
## Calculate 'REGISTER', 'PREVALENCE' and 'PREV_CHANGE' from existing data 
t10_17_prac_lvsd_df["REGISTER"] = t10_17_prac_lvsd_df["TOTAL_PCAS"]+t10_17_prac_lvsd_df["TOTAL_DENOMINATOR"]

t10_17_prac_lvsd_df["PREVALENCE"] = ((t10_17_prac_lvsd_df["TOTAL_PCAS"]+
                                      t10_17_prac_lvsd_df["TOTAL_DENOMINATOR"])/
                                      t10_17_prac_lvsd_df["LIST_SIZE"])*100

t10_17_prac_lvsd_df["PREV_CHANGE"] = (((t10_17_prac_lvsd_df["TOTAL_PCAS"]+
                                        t10_17_prac_lvsd_df["TOTAL_DENOMINATOR"])/
                                        t10_17_prac_lvsd_df["LIST_SIZE"])*100)-t10_17_prac_lvsd_df["PREV_PREVALENCE"]

## Drop unwanted columns
t10_17_prac_lvsd = t10_17_prac_lvsd_df.drop(columns=["INDICATOR_CODE","TOTAL_PCAS","TOTAL_DENOMINATOR"])

## Reorder columns for output data frame on tab 'LVSD' in prac-ach-prev-pca-cv excel workbook (template 10)
t10_17_lvsd_col = ["CCG_ODS_CODE","CCG_ONS_CODE","CCG_NAME","PCN_ODS_CODE","PCN_NAME","PRACTICE_CODE"
                   ,"PRACTICE_NAME","PREV_PRACTICE_LIST_SIZE","PREV_REGISTER_SIZE","PREV_PREVALENCE"
                   ,"LIST_SIZE","REGISTER","PREVALENCE","PREV_CHANGE"]
t10_17_lvsd_df = t10_17_prac_lvsd_df[t10_17_lvsd_col]

t10_17_lvsd_df = t10_17_lvsd_df.sort_values(by=["CCG_ODS_CODE", "PRACTICE_NAME"],ascending = [True, True])
                                

<h3>Create T10 to T19 Practice Table titles

In [ ]:
## Reduced number of Group Reference table columns - used by Practice tables
prac_gr_table_col = ["PUB_ORDER","GROUP_CODE","GROUP_DESCRIPTION"
                     ,"HIGHER_GROUP_CODE","HIGHER_GROUP_DESCRIPTION"]
prac_gr_table_df = gr_ref_table_df[prac_gr_table_col]

## Adds messages that appear under the table title to group data frame that consists of columns
## 'PUB_ORDER', 'GROUP_CODE', 'GROUP_DESCRIPTION', 'HIGHER_GROUP_CODE', 'HIGHER_GROUP_DESCRIPTION'
prac_gr_message_df = pd.merge(prac_gr_table_df
                              ,grp_change_flag
                              ,left_on = ["GROUP_CODE"]
                              ,right_on = ["GROUP_CODE"]
                              ,how = 'left')


## Remove duplicates and unwanted columns
prac_gr_message_df = prac_gr_message_df.drop_duplicates()
prac_gr_message_df = prac_gr_message_df.drop(columns=["INDICATOR_GROUP_CHANGE_NOTE"
                                                      ,"NOTE_TYPE","MESSAGE"])


## Join indicator group data frame to indicator group message data frame
gr_prac_table_titles = pd.merge(prac_gr_message_df,gr_table_titles, on = ["GROUP_CODE"],how = 'left')

In [ ]:
## Create row for LVSD values as LVSD only appears at practice level
## from indicator HF003
new_row = {"PUB_ORDER":1
           ,"GROUP_CODE":"LVSD"
           ,"GROUP_DESCRIPTION":" heart failure due to left ventricular systolic dysfunction"
           ,"HIGHER_GROUP_CODE":"CV"
           ,"HIGHER_GROUP_DESCRIPTION":"Cardiovascular"
           ,"TITLE_TEXT":"Prevalence, "}

## Append row to the data frame
gr_prac_table_titles = gr_prac_table_titles.append(new_row, ignore_index=True)

## Sort values
gr_prac_table_titles = gr_prac_table_titles.sort_values(by=["PUB_ORDER", "GROUP_CODE"]
                                                        ,ascending = [True, True]).reset_index(drop = True)

In [ ]:
## Combine elements that contribute to table titles
gr_prac_table_titles["FYEAR"] = Control_File.at[0, "FYEAR"]
gr_prac_table_titles["HIGHER_GROUP_DESCRIPTION"] = gr_prac_table_titles["HIGHER_GROUP_DESCRIPTION"].str.lower()
gr_prac_table_titles["GROUP_DESCRIPTION"] = gr_prac_table_titles["GROUP_DESCRIPTION"].str.lower()

gr_prac_table_titles["TABLE_TITLE"] = ": "+gr_prac_table_titles["TITLE_TEXT"]+gr_prac_table_titles["HIGHER_GROUP_DESCRIPTION"] \
                                +" group, "+gr_prac_table_titles["GROUP_DESCRIPTION"]+", "+gr_prac_table_titles["FYEAR"]+", GP practice level"

gr_prac_table_titles = pd.merge(gr_prac_table_titles,QI_gr_codes_with_key, on = ["GROUP_CODE"],how = 'left')
gr_prac_table_titles["GrC_KEY"] = gr_prac_table_titles["GrC_KEY"].replace(np.nan, "")

gr_prac_table_titles["GROUP_CODE"] = np.where(gr_prac_table_titles["GrC_KEY"]== ""
                                              , gr_prac_table_titles["GROUP_CODE"]
                                              , gr_prac_table_titles["GrC_KEY"])

gr_prac_table_titles = gr_prac_table_titles.drop(columns=["PUB_ORDER","FYEAR","HIGHER_GROUP_DESCRIPTION"
                                                          ,"GROUP_DESCRIPTION","GrC_KEY"])

<h3>Create practice table titles for higher groups


In [ ]:
"""   
As tables for each higher group need to start from table 1, data frame indexes are important
in determining the table numbers

"""

## Table titles for higher group code CV
## Filter data frame for higher group code and reset index and drop old index, this is needed to help determin the table numbers
prac_table_titles_cv = gr_prac_table_titles[gr_prac_table_titles["HIGHER_GROUP_CODE"] == "CV"].reset_index(drop=True)

## The index is reset a second time as a new column called 'index' and rename as 'TABLE_NO'
prac_table_titles_cv = prac_table_titles_cv.reset_index()
prac_table_titles_cv = prac_table_titles_cv.rename(columns={"index":"TABLE_NO"})

## 'TABLE_NO' numbering starts at 0. To make this work as a table number 1 is added to the 'TABLE_NO' value 
## and it is converted to a string data type to enable it to be joined to the remaining elements in the table title
prac_table_titles_cv["TABLE_NO"] = (prac_table_titles_cv["TABLE_NO"]+1).astype(str)

## Combine all elements of the table title
prac_table_titles_cv["TABLE_TITLE"] = "Table "+prac_table_titles_cv["TABLE_NO"]+prac_table_titles_cv["TABLE_TITLE"]

## Drop unwanted columns. The resulting data frame compries columns
## 'GROUP_CODE', 'TITLE_TEXT'
prac_table_titles_cv = prac_table_titles_cv.drop(columns=["TABLE_NO","HIGHER_GROUP_CODE","TITLE_TEXT"])

In [ ]:
## Table titles for higher group code RESP
## Filter data frame for higher group code and reset index and drop old index, this is needed to help determin the table numbers
prac_table_titles_resp = gr_prac_table_titles[gr_prac_table_titles["HIGHER_GROUP_CODE"] == "RESP"].reset_index(drop=True)

## The index is reset a second time as a new column called 'index' and rename as 'TABLE_NO'
prac_table_titles_resp = prac_table_titles_resp.reset_index()
prac_table_titles_resp = prac_table_titles_resp.rename(columns={"index":"TABLE_NO"})

## 'TABLE_NO' numbering starts at 0. To make this work as a table number 1 is added to the 'TABLE_NO' value 
## and it is converted to a string data type to enable it to be joined to the remaining elements in the table title
prac_table_titles_resp["TABLE_NO"] = (prac_table_titles_resp["TABLE_NO"]+1).astype(str)

## Combine all elements of the table title
prac_table_titles_resp["TABLE_TITLE"] = "Table "+prac_table_titles_resp["TABLE_NO"]+prac_table_titles_resp["TABLE_TITLE"]

## Drop unwanted columns. The resulting data frame compries columns
## 'GROUP_CODE', 'TITLE_TEXT'
prac_table_titles_resp = prac_table_titles_resp.drop(columns=["TABLE_NO","HIGHER_GROUP_CODE","TITLE_TEXT"])

In [ ]:
## Table titles for higher group code HD
## Filter data frame for higher group code and reset index and drop old index, this is needed to help determin the table numbers
prac_table_titles_hd = gr_prac_table_titles[gr_prac_table_titles["HIGHER_GROUP_CODE"] == "HD"].reset_index(drop=True)

## The index is reset a second time as a new column called 'index' and rename as 'TABLE_NO'
prac_table_titles_hd = prac_table_titles_hd.reset_index()
prac_table_titles_hd = prac_table_titles_hd.rename(columns={"index":"TABLE_NO"})

## 'TABLE_NO' numbering starts at 0. To make this work as a table number 1 is added to the 'TABLE_NO' value 
## and it is converted to a string data type to enable it to be joined to the remaining elements in the table title
prac_table_titles_hd["TABLE_NO"] = (prac_table_titles_hd["TABLE_NO"]+1).astype(str)

## Combine all elements of the table title
prac_table_titles_hd["TABLE_TITLE"] = "Table "+prac_table_titles_hd["TABLE_NO"]+prac_table_titles_hd["TABLE_TITLE"]

## Drop unwanted columns. The resulting data frame compries columns
## 'GROUP_CODE', 'TITLE_TEXT'
prac_table_titles_hd = prac_table_titles_hd.drop(columns=["TABLE_NO","HIGHER_GROUP_CODE","TITLE_TEXT"])

In [ ]:
## Table titles for higher group code GYN
## Filter data frame for higher group code and reset index and drop old index, this is needed to help determin the table numbers
prac_table_titles_gyn = gr_prac_table_titles[gr_prac_table_titles["HIGHER_GROUP_CODE"] == "GYN"].reset_index(drop=True)

## The index is reset a second time as a new column called 'index' and rename as 'TABLE_NO'
prac_table_titles_gyn = prac_table_titles_gyn.reset_index()
prac_table_titles_gyn = prac_table_titles_gyn.rename(columns={"index":"TABLE_NO"})

## 'TABLE_NO' numbering starts at 0. To make this work as a table number 1 is added to the 'TABLE_NO' value 
## and it is converted to a string data type to enable it to be joined to the remaining elements in the table title
prac_table_titles_gyn["TABLE_NO"] = (prac_table_titles_gyn["TABLE_NO"]+1).astype(str)

## Combine all elements of the table title
prac_table_titles_gyn["TABLE_TITLE"] = "Table "+prac_table_titles_gyn["TABLE_NO"]+prac_table_titles_gyn["TABLE_TITLE"]

## Drop unwanted columns. The resulting data frame compries columns
## 'GROUP_CODE', 'TITLE_TEXT'
prac_table_titles_gyn = prac_table_titles_gyn.drop(columns=["TABLE_NO","HIGHER_GROUP_CODE","TITLE_TEXT"])

In [ ]:
## Table titles for higher group code NEU
## Filter data frame for higher group code and reset index and drop old index, this is needed to help determin the table numbers
prac_table_titles_neu = gr_prac_table_titles[gr_prac_table_titles["HIGHER_GROUP_CODE"] == "NEU"].reset_index(drop=True)

## The index is reset a second time as a new column called 'index' and rename as 'TABLE_NO'
prac_table_titles_neu = prac_table_titles_neu.reset_index()
prac_table_titles_neu = prac_table_titles_neu.rename(columns={"index":"TABLE_NO"})

## 'TABLE_NO' numbering starts at 0. To make this work as a table number 1 is added to the 'TABLE_NO' value 
## and it is converted to a string data type to enable it to be joined to the remaining elements in the table title
prac_table_titles_neu["TABLE_NO"] = (prac_table_titles_neu["TABLE_NO"]+1).astype(str)

## Combine all elements of the table title
prac_table_titles_neu["TABLE_TITLE"] = "Table "+prac_table_titles_neu["TABLE_NO"]+prac_table_titles_neu["TABLE_TITLE"]

## Drop unwanted columns. The resulting data frame compries columns
## 'GROUP_CODE', 'TITLE_TEXT'
prac_table_titles_neu = prac_table_titles_neu.drop(columns=["TABLE_NO","HIGHER_GROUP_CODE","TITLE_TEXT"])

In [ ]:
## Table titles for higher group code NDH
## Filter data frame for higher group code and reset index and drop old index, this is needed to help determin the table numbers
prac_table_titles_ndh = gr_prac_table_titles[gr_prac_table_titles["HIGHER_GROUP_CODE"] == "NDH"].reset_index(drop=True)

## The index is reset a second time as a new column called 'index' and rename as 'TABLE_NO'
prac_table_titles_ndh = prac_table_titles_ndh.reset_index()
prac_table_titles_ndh = prac_table_titles_ndh.rename(columns={"index":"TABLE_NO"})

## 'TABLE_NO' numbering starts at 0. To make this work as a table number 1 is added to the 'TABLE_NO' value 
## and it is converted to a string data type to enable it to be joined to the remaining elements in the table title
prac_table_titles_ndh["TABLE_NO"] = (prac_table_titles_ndh["TABLE_NO"]+1).astype(str)

## Combine all elements of the table title
prac_table_titles_ndh["TABLE_TITLE"] = "Table "+prac_table_titles_ndh["TABLE_NO"]+prac_table_titles_ndh["TABLE_TITLE"]

## Drop unwanted columns. The resulting data frame compries columns
## 'GROUP_CODE', 'TITLE_TEXT'
prac_table_titles_ndh = prac_table_titles_ndh.drop(columns=["TABLE_NO","HIGHER_GROUP_CODE","TITLE_TEXT"])


In [ ]:
## Table titles for higher group code LS
## Filter data frame for higher group code and reset index and drop old index, this is needed to help determin the table numbers
prac_table_titles_ls = gr_prac_table_titles[gr_prac_table_titles["HIGHER_GROUP_CODE"] == "LS"].reset_index(drop=True)

## The index is reset a second time as a new column called 'index' and rename as 'TABLE_NO'
prac_table_titles_ls = prac_table_titles_ls.reset_index()
prac_table_titles_ls = prac_table_titles_ls.rename(columns={"index":"TABLE_NO"})

## 'TABLE_NO' numbering starts at 0. To make this work as a table number 1 is added to the 'TABLE_NO' value 
## and it is converted to a string data type to enable it to be joined to the remaining elements in the table title
prac_table_titles_ls["TABLE_NO"] = (prac_table_titles_ls["TABLE_NO"]+1).astype(str)

## Combine all elements of the table title
prac_table_titles_ls["TABLE_TITLE"] = "Table "+prac_table_titles_ls["TABLE_NO"]+prac_table_titles_ls["TABLE_TITLE"]

## Drop unwanted columns. The resulting data frame compries columns
## 'GROUP_CODE', 'TITLE_TEXT'
prac_table_titles_ls = prac_table_titles_ls.drop(columns=["TABLE_NO","HIGHER_GROUP_CODE","TITLE_TEXT"])

In [ ]:
## Table titles for higher group code MS
## Filter data frame for higher group code and reset index and drop old index, this is needed to help determin the table numbers
prac_table_titles_ms = gr_prac_table_titles[gr_prac_table_titles["HIGHER_GROUP_CODE"] == "MS"].reset_index(drop=True)

## The index is reset a second time as a new column called 'index' and rename as 'TABLE_NO'
prac_table_titles_ms = prac_table_titles_ms.reset_index()
prac_table_titles_ms = prac_table_titles_ms.rename(columns={"index":"TABLE_NO"})

## 'TABLE_NO' numbering starts at 0. To make this work as a table number 1 is added to the 'TABLE_NO' value 
## and it is converted to a string data type to enable it to be joined to the remaining elements in the table title
prac_table_titles_ms["TABLE_NO"] = (prac_table_titles_ms["TABLE_NO"]+1).astype(str)

## Combine all elements of the table title
prac_table_titles_ms["TABLE_TITLE"] = "Table "+prac_table_titles_ms["TABLE_NO"]+prac_table_titles_ms["TABLE_TITLE"]

## Drop unwanted columns. The resulting data frame compries columns
## 'GROUP_CODE', 'TITLE_TEXT'
prac_table_titles_ms = prac_table_titles_ms.drop(columns=["TABLE_NO","HIGHER_GROUP_CODE","TITLE_TEXT"])

In [ ]:
## Table titles for higher group code VI
## Filter data frame for higher group code and reset index and drop old index, this is needed to help determin the table numbers
prac_table_titles_vi = gr_prac_table_titles[gr_prac_table_titles["HIGHER_GROUP_CODE"] == "VI"].reset_index(drop=True)

## The index is reset a second time as a new column called 'index' and rename as 'TABLE_NO'
prac_table_titles_vi = prac_table_titles_vi.reset_index()
prac_table_titles_vi = prac_table_titles_vi.rename(columns={"index":"TABLE_NO"})

## 'TABLE_NO' numbering starts at 0. To make this work as a table number 1 is added to the 'TABLE_NO' value 
## and it is converted to a string data type to enable it to be joined to the remaining elements in the table title
prac_table_titles_vi["TABLE_NO"] = (prac_table_titles_vi["TABLE_NO"]+1).astype(str)

## Combine all elements of the table title
prac_table_titles_vi["TABLE_TITLE"] = "Table "+prac_table_titles_vi["TABLE_NO"]+prac_table_titles_vi["TABLE_TITLE"]

## Drop unwanted columns. The resulting data frame compries columns
## 'GROUP_CODE', 'TITLE_TEXT'
prac_table_titles_vi = prac_table_titles_vi.drop(columns=["TABLE_NO","HIGHER_GROUP_CODE","TITLE_TEXT"])


In [ ]:
## Table titles for higher group code QI
## Filter data frame for higher group code and reset index and drop old index, this is needed to help determin the table numbers
prac_table_titles_qi = gr_prac_table_titles[gr_prac_table_titles["HIGHER_GROUP_CODE"] == "QI"]

# As QI indicators have a numerical pseudonym these are resorted by numerical pseudonym
prac_table_titles_qi = prac_table_titles_qi.sort_values(by=["GROUP_CODE"],ascending = [True]).reset_index(drop = True)

## The index is reset a second time as a new column called 'index' and rename as 'TABLE_NO'
prac_table_titles_qi = prac_table_titles_qi.reset_index()
prac_table_titles_qi = prac_table_titles_qi.rename(columns={"index":"TABLE_NO"})

## 'TABLE_NO' numbering starts at 0. To make this work as a table number 1 is added to the 'TABLE_NO' value 
## and it is converted to a string data type to enable it to be joined to the remaining elements in the table title
prac_table_titles_qi["TABLE_NO"] = (prac_table_titles_qi["TABLE_NO"]+1).astype(str)

## Combine all elements of the table title
prac_table_titles_qi["TABLE_TITLE"] = "Table "+prac_table_titles_qi["TABLE_NO"]+prac_table_titles_qi["TABLE_TITLE"]

## Drop unwanted columns. The resulting data frame compries columns
## 'GROUP_CODE', 'TITLE_TEXT'
prac_table_titles_qi = prac_table_titles_qi.drop(columns=["TABLE_NO","HIGHER_GROUP_CODE","TITLE_TEXT"])

## 6. Populate Excel templates
[Return to contents](#Table-of-Contents)

In [ ]:
""" 
Calculate group point value by indicator group and sub domain code
Used in 6. Populate Excel Templates

"""
gr_points_by_sub_domain_cols = ["SUB_DOMAIN_CODE","GROUP_CODE","INDICATOR_POINT_VALUE"]
gr_points_by_sub_domain = Indicator_Reference_Control_Table[gr_points_by_sub_domain_cols]

gr_points_by_sub_domain = gr_points_by_sub_domain.groupby(by = ["SUB_DOMAIN_CODE","GROUP_CODE"]
                                                          ,as_index=False
                                                          ).sum().rename(columns={"INDICATOR_POINT_VALUE":"GROUP_POINT_VALUE"})


<b><h1>Publication Outputs---------------------------------------------</h1></b>

#### Define cell references for formatting in excel tables

In [ ]:
"""   
Calculate cell reference dependant on the last row of national and region data for 
Copyright statement
Source statement
Position of the line underneath the table

"""

nat_reg_count = len(nat_reg_list.index)
nat_reg_count_pub = "A"+str(nat_reg_count + 16)
nat_reg_source_ref = "A"+str(nat_reg_count + 13)
nat_reg_source_count = nat_reg_count + 13

## Prints the ouputs for easy reference
print(nat_reg_count_pub)
print(nat_reg_source_ref)
print(nat_reg_source_count)

In [ ]:
"""   
Calculate cell reference dependant on the last row of ICB (STP) data for 
Copyright statement
Source statement
Position of the line underneath the table

"""

stp_count = len(stp_list.index)
stp_count_pub = "A"+str(stp_count + 16)
stp_source_ref = "A"+str(stp_count + 13)
stp_source_count = stp_count + 13

## Prints the ouputs for easy reference
print(stp_count_pub)
print(stp_source_ref)
print(stp_source_count)

In [ ]:
"""   
Calculate cell reference dependant on the last row of Sub ICB (CCG) data for 
Copyright statement
Source statement
Position of the line underneath the table

"""

ccg_count = len(ccg_list.index)
ccg_count_pub = "A"+str(ccg_count + 16)
ccg_source_ref = "A"+str(ccg_count + 13)
ccg_source_count = ccg_count + 13

## Prints the ouputs for easy reference
print(ccg_count_pub)
print(ccg_source_ref)
print(ccg_source_count)

In [ ]:
"""   
Calculate cell reference dependant on the last row of practice data for 
Copyright statement
Source statement
Position of the line underneath the table 

"""

prac_count = len(final_prac_list.index)
prac_count_pub = "A"+str(prac_count + 16)
source_count_ref = "A"+str(prac_count + 13)
source_count_pub = prac_count + 13

## Prints the ouputs for easy reference
print(prac_count_pub)
print(source_count_ref)
print(source_count_pub)


#### Create messages that appear under titles in excel tables by indicator group

In [ ]:
## Get change note number by indicator group
message_1 = Indicator_Reference_Control_Table[["GROUP_CODE","INDICATOR_GROUP_CHANGE_NOTE"]]

## Get message by change note number
message_2 = Indicator_Change_Message_Table[["INDICATOR_GROUP_CHANGE_NOTE","MESSAGE"]]

## Merge change note number by indicator group with message by change note number
message_table = pd.merge(message_1
                         ,message_2
                         ,left_on = ["INDICATOR_GROUP_CHANGE_NOTE"]
                         ,right_on = ["INDICATOR_GROUP_CHANGE_NOTE"]
                         ,how = 'left')

## Drop unwanted column and remove duplicates and replace 'nan' values with a blank
message_table = message_table.drop(columns=["INDICATOR_GROUP_CHANGE_NOTE"])
message_table = message_table.drop_duplicates().reset_index(drop = True)
message_table = message_table.fillna('')

## Add QI indicators numerical pseudonym as column 'GrC_KEY' and in this new column replace 'nan' values with a blank 
message_table = pd.merge(message_table
                         ,QI_gr_codes_with_key
                         ,left_on = ["GROUP_CODE"]
                         ,right_on = ["GROUP_CODE"]
                         ,how = "left")

message_table["GrC_KEY"] = message_table["GrC_KEY"].replace(np.nan, "")

## Replace QI group code with numerical pseudonyms in 'GROUP_CODE' column and drop unwanted 'GrC_KEY' coumn
message_table["GROUP_CODE"] = np.where(message_table["GrC_KEY"]== ""
                                       , message_table["GROUP_CODE"]
                                       , message_table["GrC_KEY"])

message_table = message_table.drop(columns=["GrC_KEY"])

#### Create indicator group specific message variables 

In [ ]:
## Filter messages under table titles on excel tables by indicator group
message_table_af = message_table.loc[message_table["GROUP_CODE"]=="AF"]
message_af = message_table_af.at[0,"MESSAGE"]

message_table_ast = message_table.loc[message_table["GROUP_CODE"]=="AST"].reset_index(drop=True)
message_ast = message_table_ast.at[0,"MESSAGE"]

message_table_can = message_table.loc[message_table["GROUP_CODE"]=="CAN"].reset_index(drop=True)
message_can = message_table_can.at[0,"MESSAGE"]

message_table_chd = message_table.loc[message_table["GROUP_CODE"]=="CHD"].reset_index(drop=True)
message_chd = message_table_chd.at[0,"MESSAGE"]

message_table_ckd = message_table.loc[message_table["GROUP_CODE"]=="CKD"].reset_index(drop=True)
message_ckd = message_table_ckd.at[0,"MESSAGE"]

message_table_copd = message_table.loc[message_table["GROUP_CODE"]=="COPD"].reset_index(drop=True)
message_copd = message_table_copd.at[0,"MESSAGE"]

message_table_dem = message_table.loc[message_table["GROUP_CODE"]=="DEM"].reset_index(drop=True)
message_dem = message_table_dem.at[0,"MESSAGE"]

message_table_dep = message_table.loc[message_table["GROUP_CODE"]=="DEP"].reset_index(drop=True)
message_dep = message_table_dep.at[0,"MESSAGE"]

message_table_dm = message_table.loc[message_table["GROUP_CODE"]=="DM"].reset_index(drop=True)
message_dm = message_table_dm.at[0,"MESSAGE"]

message_table_ep = message_table.loc[message_table["GROUP_CODE"]=="EP"].reset_index(drop=True)
message_ep = message_table_ep.at[0,"MESSAGE"]

message_table_hf = message_table.loc[message_table["GROUP_CODE"]=="HF"].reset_index(drop=True)
message_hf = message_table_hf.at[0,"MESSAGE"]

message_table_hyp = message_table.loc[message_table["GROUP_CODE"]=="HYP"].reset_index(drop=True)
message_hyp = message_table_hyp.at[0,"MESSAGE"]

message_table_ld = message_table.loc[message_table["GROUP_CODE"]=="LD"].reset_index(drop=True)
message_ld = message_table_ld.at[0,"MESSAGE"]

message_table_mh = message_table.loc[message_table["GROUP_CODE"]=="MH"].reset_index(drop=True)
message_mh = message_table_mh.at[0,"MESSAGE"]

message_table_ndh = message_table.loc[message_table["GROUP_CODE"]=="NDH"].reset_index(drop=True)
message_ndh = message_table_ndh.at[0,"MESSAGE"]

message_table_ost = message_table.loc[message_table["GROUP_CODE"]=="OST"].reset_index(drop=True)
message_ost = message_table_ost.at[0,"MESSAGE"]

message_table_pad = message_table.loc[message_table["GROUP_CODE"]=="PAD"].reset_index(drop=True)
message_pad = message_table_pad.at[0,"MESSAGE"]

message_table_pc = message_table.loc[message_table["GROUP_CODE"]=="PC"].reset_index(drop=True)
message_pc = message_table_pc.at[0,"MESSAGE"]

message_table_ra = message_table.loc[message_table["GROUP_CODE"]=="RA"].reset_index(drop=True)
message_ra = message_table_ra.at[0,"MESSAGE"]

message_table_stia = message_table.loc[message_table["GROUP_CODE"]=="STIA"].reset_index(drop=True)
message_stia = message_table_stia.at[0,"MESSAGE"]

message_table_bp = message_table.loc[message_table["GROUP_CODE"]=="BP"].reset_index(drop=True)
message_bp = message_table_bp.at[0,"MESSAGE"]

message_table_ob = message_table.loc[message_table["GROUP_CODE"]=="OB"].reset_index(drop=True)
message_ob = message_table_ob.at[0,"MESSAGE"]

message_table_smok = message_table.loc[message_table["GROUP_CODE"]=="SMOK"].reset_index(drop=True)
message_smok = message_table_smok.at[0,"MESSAGE"]

message_table_cs = message_table.loc[message_table["GROUP_CODE"]=="CS"].reset_index(drop=True)
message_cs = message_table_cs.at[0,"MESSAGE"]

message_table_vi = message_table.loc[message_table["GROUP_CODE"]=="VI"].reset_index(drop=True)
message_vi = message_table_vi.at[0,"MESSAGE"]

message_table_qi1 = message_table.loc[message_table["GROUP_CODE"]=="QI_1"].reset_index(drop=True)
message_qi1 = message_table_qi1.at[0,"MESSAGE"]

message_table_qi2 = message_table.loc[message_table["GROUP_CODE"]=="QI_2"].reset_index(drop=True)
message_qi2 = message_table_qi2.at[0,"MESSAGE"]

message_table_qi3 = message_table.loc[message_table["GROUP_CODE"]=="QI_3"].reset_index(drop=True)
message_qi3 = message_table_qi3.at[0,"MESSAGE"]

In [ ]:
## QI indicator group codes are replace with the more generalise 
## codes of 'QI_1, 'QI_2', @QI_3' this allows for codes to change 
## every year witout impacting the majority of the code
Group_Points_df = pd.merge(gr_points_by_sub_domain
                           ,QI_gr_codes_with_key
                           ,left_on = ["GROUP_CODE"]
                           ,right_on = ["GROUP_CODE"]
                           ,how = 'left')

Group_Points_df = Group_Points_df.fillna('')

Group_Points_df["GROUP_CODE"] = np.where(Group_Points_df["GrC_KEY"]== ""
                                         ,Group_Points_df["GROUP_CODE"]
                                         ,Group_Points_df["GrC_KEY"])

Group_Points_df = Group_Points_df.drop(columns=["GrC_KEY"])

In [ ]:
## Filter group points in table headers
group_points_cl = Group_Points_df.loc[Group_Points_df["SUB_DOMAIN_CODE"]=="CL"]
group_points_cl = group_points_cl.drop(columns=["SUB_DOMAIN_CODE"])

group_points_cl_af = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="AF"].reset_index(drop=True)
group_points_af = "(max "+str(group_points_cl_af.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_ast = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="AST"].reset_index(drop=True)
group_points_ast = "(max "+str(group_points_cl_ast.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_can = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="CAN"].reset_index(drop=True)
group_points_can = "(max "+str(group_points_cl_can.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_chd = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="CHD"].reset_index(drop=True)
group_points_chd = "(max "+str(group_points_cl_chd.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_ckd = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="CKD"].reset_index(drop=True)
group_points_ckd = "(max "+str(group_points_cl_ckd.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_copd = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="COPD"].reset_index(drop=True)
group_points_copd = "(max "+str(group_points_cl_copd.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_dem = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="DEM"].reset_index(drop=True)
group_points_dem = "(max "+str(group_points_cl_dem.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_dep = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="DEP"].reset_index(drop=True)
group_points_dep = "(max "+str(group_points_cl_dep.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_dm = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="DM"].reset_index(drop=True)
group_points_dm = "(max "+str(group_points_cl_dm.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_ep = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="EP"].reset_index(drop=True)
group_points_ep = "(max "+str(group_points_cl_ep.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_hf = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="HF"].reset_index(drop=True)
group_points_hf = "(max "+str(group_points_cl_hf.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_hyp = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="HYP"].reset_index(drop=True)
group_points_hyp = "(max "+str(group_points_cl_hyp.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_ld = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="LD"].reset_index(drop=True)
group_points_ld = "(max "+str(group_points_cl_ld.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_mh = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="MH"].reset_index(drop=True)
group_points_mh = "(max "+str(group_points_cl_mh.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_ndh = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="NDH"].reset_index(drop=True)
group_points_ndh = "(max "+str(group_points_cl_ndh.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_ost = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="OST"].reset_index(drop=True)
group_points_ost = "(max "+str(group_points_cl_ost.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_pad = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="PAD"].reset_index(drop=True)
group_points_pad = "(max "+str(group_points_cl_pad.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_pc = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="PC"].reset_index(drop=True)
group_points_pc = "(max "+str(group_points_cl_pc.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_ra = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="RA"].reset_index(drop=True)
group_points_ra = "(max "+str(group_points_cl_ra.at[0,"GROUP_POINT_VALUE"])+")"

group_points_cl_stia = group_points_cl.loc[group_points_cl["GROUP_CODE"]=="STIA"].reset_index(drop=True)
group_points_stia = "(max "+str(group_points_cl_stia.at[0,"GROUP_POINT_VALUE"])+")"


group_points_ph = Group_Points_df.loc[Group_Points_df["SUB_DOMAIN_CODE"]=="PH"]
group_points_ph = group_points_ph.drop(columns=["SUB_DOMAIN_CODE"])

group_points_ph_bp = group_points_ph.loc[group_points_ph["GROUP_CODE"]=="BP"].reset_index(drop=True)
group_points_bp = "(max "+str(group_points_ph_bp.at[0,"GROUP_POINT_VALUE"])+")"

group_points_ph_ob = group_points_ph.loc[group_points_ph["GROUP_CODE"]=="OB"].reset_index(drop=True)
group_points_ob = "(max "+str(group_points_ph_ob.at[0,"GROUP_POINT_VALUE"])+")"

group_points_ph_smok = group_points_ph.loc[group_points_ph["GROUP_CODE"]=="SMOK"].reset_index(drop=True)
group_points_smok = "(max "+str(group_points_ph_smok.at[0,"GROUP_POINT_VALUE"])+")"


group_points_phas = Group_Points_df.loc[Group_Points_df["SUB_DOMAIN_CODE"]=="PHAS"]
group_points_phas = group_points_phas.drop(columns=["SUB_DOMAIN_CODE"])

group_points_phas_cs = group_points_phas.loc[group_points_phas["GROUP_CODE"]=="CS"].reset_index(drop=True)
group_points_cs = "(max "+str(group_points_phas_cs.at[0,"GROUP_POINT_VALUE"])+")"


group_points_PHVI = Group_Points_df.loc[Group_Points_df["SUB_DOMAIN_CODE"]=="PHVI"]
group_points_PHVI = group_points_PHVI.drop(columns=["SUB_DOMAIN_CODE"])

group_points_PHVI_vi = group_points_PHVI.loc[group_points_PHVI["GROUP_CODE"]=="VI"].reset_index(drop=True)
group_points_vi = "(max "+str(group_points_PHVI_vi.at[0,"GROUP_POINT_VALUE"])+")"


group_points_qi = Group_Points_df.loc[(Group_Points_df["SUB_DOMAIN_CODE"]=="QI")|(Group_Points_df["SUB_DOMAIN_CODE"]=="QUALI")]
group_points_qi = group_points_qi.drop(columns=["SUB_DOMAIN_CODE"]).reset_index(drop=True)

group_points_qi_1 = group_points_qi.loc[group_points_qi["GROUP_CODE"]=="QI_1"].reset_index(drop=True)
group_points_qi1 = "(max "+str(group_points_qi_1.at[0,"GROUP_POINT_VALUE"])+")"

group_points_qi_2 = group_points_qi.loc[group_points_qi["GROUP_CODE"]=="QI_2"].reset_index(drop=True)
group_points_qi2 = "(max "+str(group_points_qi_2.at[0,"GROUP_POINT_VALUE"])+")"

group_points_qi_3 = group_points_qi.loc[group_points_qi["GROUP_CODE"]=="QI_3"].reset_index(drop=True)
group_points_qi3 = "(max "+str(group_points_qi_3.at[0,"GROUP_POINT_VALUE"])+")"

In [ ]:
"""
Variable previous_gr_points calculated in 'Previous Years Data'

"""

## Populate group points in table headers for previous year
previous_gr_points_cl_af = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="AF"]
previous_gr_points_af = "(max "+str(previous_gr_points_cl_af.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_ast = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="AST"].reset_index(drop=True)
previous_gr_points_ast = "(max "+str(previous_gr_points_cl_ast.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_can = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="CAN"].reset_index(drop=True)
previous_gr_points_can = "(max "+str(previous_gr_points_cl_can.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_chd = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="CHD"].reset_index(drop=True)
previous_gr_points_chd = "(max "+str(previous_gr_points_cl_chd.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_ckd = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="CKD"].reset_index(drop=True)
previous_gr_points_ckd = "(max "+str(previous_gr_points_cl_ckd.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_copd = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="COPD"].reset_index(drop=True)
previous_gr_points_copd = "(max "+str(previous_gr_points_cl_copd.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_dem = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="DEM"].reset_index(drop=True)
previous_gr_points_dem = "(max "+str(previous_gr_points_cl_dem.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_dep = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="DEP"].reset_index(drop=True)
previous_gr_points_dep = "(max "+str(previous_gr_points_cl_dep.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_dm = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="DM"].reset_index(drop=True)
previous_gr_points_dm = "(max "+str(previous_gr_points_cl_dm.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_ep = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="EP"].reset_index(drop=True)
previous_gr_points_ep = "(max "+str(previous_gr_points_cl_ep.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_hf = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="HF"].reset_index(drop=True)
previous_gr_points_hf = "(max "+str(previous_gr_points_cl_hf.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_hyp = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="HYP"].reset_index(drop=True)
previous_gr_points_hyp = "(max "+str(previous_gr_points_cl_hyp.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_ld = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="LD"].reset_index(drop=True)
previous_gr_points_ld = "(max "+str(previous_gr_points_cl_ld.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_mh = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="MH"].reset_index(drop=True)
previous_gr_points_mh = "(max "+str(previous_gr_points_cl_mh.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_ndh = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="NDH"].reset_index(drop=True)
previous_gr_points_ndh = "(max "+str(previous_gr_points_cl_ndh.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_ost = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="OST"].reset_index(drop=True)
previous_gr_points_ost = "(max "+str(previous_gr_points_cl_ost.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_pad = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="PAD"].reset_index(drop=True)
previous_gr_points_pad = "(max "+str(previous_gr_points_cl_pad.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_pc = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="PC"].reset_index(drop=True)
previous_gr_points_pc = "(max "+str(previous_gr_points_cl_pc.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_ra = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="RA"].reset_index(drop=True)
previous_gr_points_ra = "(max "+str(previous_gr_points_cl_ra.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_cl_stia = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="STIA"].reset_index(drop=True)
previous_gr_points_stia = "(max "+str(previous_gr_points_cl_stia.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_ph_bp = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="BP"].reset_index(drop=True)
previous_gr_points_bp = "(max "+str(previous_gr_points_ph_bp.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_ph_ob = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="OB"].reset_index(drop=True)
previous_gr_points_ob = "(max "+str(previous_gr_points_ph_ob.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_ph_smok = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="SMOK"].reset_index(drop=True)
previous_gr_points_smok = "(max "+str(previous_gr_points_ph_smok.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_phas_cs = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="CS"].reset_index(drop=True)
previous_gr_points_cs = "(max "+str(previous_gr_points_phas_cs.at[0,"GROUP_POINT_VALUE"])+")"

previous_gr_points_ph_vi = previous_gr_points.loc[previous_gr_points["GROUP_CODE"]=="VI"].reset_index(drop=True)
previous_gr_points_vi = "(max "+str(previous_gr_points_ph_vi.at[0,"GROUP_POINT_VALUE"])+")"

In [ ]:
## Populate indicator points in table headers
## AF
ind_points_af001 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="AF001"].reset_index(drop=True)
ind_points_af001 = "(max "+str(ind_points_af001.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_af006 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="AF006"].reset_index(drop=True)
ind_points_af006 = "(max "+str(ind_points_af006.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_af007 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="AF007"].reset_index(drop=True)
ind_points_af007 = "(max "+str(ind_points_af007.at[0,"INDICATOR_POINT_VALUE"])+")"

## BP
ind_points_bp002 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="BP002"].reset_index(drop=True)
ind_points_bp002 = "(max "+str(ind_points_bp002.at[0,"INDICATOR_POINT_VALUE"])+")"

## CHD
ind_points_chd001 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="CHD001"].reset_index(drop=True)
ind_points_chd001 = "(max "+str(ind_points_chd001.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_chd005 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="CHD005"].reset_index(drop=True)
ind_points_chd005 = "(max "+str(ind_points_chd005.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_chd008 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="CHD008"].reset_index(drop=True)
ind_points_chd008 = "(max "+str(ind_points_chd008.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_chd009 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="CHD009"].reset_index(drop=True)
ind_points_chd009 = "(max "+str(ind_points_chd009.at[0,"INDICATOR_POINT_VALUE"])+")"

## HF
ind_points_hf001 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="HF001"].reset_index(drop=True)
ind_points_hf001 = "(max "+str(ind_points_hf001.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_hf003 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="HF003"].reset_index(drop=True)
ind_points_hf003 = "(max "+str(ind_points_hf003.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_hf005 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="HF005"].reset_index(drop=True)
ind_points_hf005 = "(max "+str(ind_points_hf005.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_hf006 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="HF006"].reset_index(drop=True)
ind_points_hf006 = "(max "+str(ind_points_hf006.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_hf007 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="HF007"].reset_index(drop=True)
ind_points_hf007 = "(max "+str(ind_points_hf007.at[0,"INDICATOR_POINT_VALUE"])+")"

## HYP
ind_points_hyp001 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="HYP001"].reset_index(drop=True)
ind_points_hyp001 = "(max "+str(ind_points_hyp001.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_hyp003 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="HYP003"].reset_index(drop=True)
ind_points_hyp003 = "(max "+str(ind_points_hyp003.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_hyp007 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="HYP007"].reset_index(drop=True)
ind_points_hyp007 = "(max "+str(ind_points_hyp007.at[0,"INDICATOR_POINT_VALUE"])+")"

## PAD
ind_points_pad001 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="PAD001"].reset_index(drop=True)
ind_points_pad001 = "(max "+str(ind_points_pad001.at[0,"INDICATOR_POINT_VALUE"])+")"

## STIA
ind_points_stia001 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="STIA001"].reset_index(drop=True)
ind_points_stia001 = "(max "+str(ind_points_stia001.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_stia007 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="STIA007"].reset_index(drop=True)
ind_points_stia007 = "(max "+str(ind_points_stia007.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_stia010 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="STIA010"].reset_index(drop=True)
ind_points_stia010 = "(max "+str(ind_points_stia010.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_stia011 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="STIA011"].reset_index(drop=True)
ind_points_stia011 = "(max "+str(ind_points_stia011.at[0,"INDICATOR_POINT_VALUE"])+")"

## AST
ind_points_ast005 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="AST005"].reset_index(drop=True)
ind_points_ast005 = "(max "+str(ind_points_ast005.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_ast006 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="AST006"].reset_index(drop=True)
ind_points_ast006 = "(max "+str(ind_points_ast006.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_ast007 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="AST007"].reset_index(drop=True)
ind_points_ast007 = "(max "+str(ind_points_ast007.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_ast008 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="AST008"].reset_index(drop=True)
ind_points_ast008 = "(max "+str(ind_points_ast008.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_copd008 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="COPD008"].reset_index(drop=True)
ind_points_copd008 = "(max "+str(ind_points_copd008.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_copd009 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="COPD009"].reset_index(drop=True)
ind_points_copd009 = "(max "+str(ind_points_copd009.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_copd010 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="COPD010"].reset_index(drop=True)
ind_points_copd010 = "(max "+str(ind_points_copd010.at[0,"INDICATOR_POINT_VALUE"])+")"

## OB
ind_points_ob002 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="OB002"].reset_index(drop=True)
ind_points_ob002 = "(max "+str(ind_points_ob002.at[0,"INDICATOR_POINT_VALUE"])+")"

## SMOK
ind_points_smok002 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="SMOK002"].reset_index(drop=True)
ind_points_smok002 = "(max "+str(ind_points_smok002.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_smok004 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="SMOK004"].reset_index(drop=True)
ind_points_smok004 = "(max "+str(ind_points_smok004.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_smok005 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="SMOK005"].reset_index(drop=True)
ind_points_smok005 = "(max "+str(ind_points_smok005.at[0,"INDICATOR_POINT_VALUE"])+")"

## CAN
ind_points_can001 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="CAN001"].reset_index(drop=True)
ind_points_can001 = "(max "+str(ind_points_can001.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_can004 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="CAN004"].reset_index(drop=True)
ind_points_can004 = "(max "+str(ind_points_can004.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_can005 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="CAN005"].reset_index(drop=True)
ind_points_can005 = "(max "+str(ind_points_can005.at[0,"INDICATOR_POINT_VALUE"])+")"

## CKD
ind_points_ckd005 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="CKD005"].reset_index(drop=True)
ind_points_ckd005 = "(max "+str(ind_points_ckd005.at[0,"INDICATOR_POINT_VALUE"])+")"

## DM
ind_points_dm017 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="DM017"].reset_index(drop=True)
ind_points_dm017 = "(max "+str(ind_points_dm017.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_dm006 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="DM006"].reset_index(drop=True)
ind_points_dm006 = "(max "+str(ind_points_dm006.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_dm012 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="DM012"].reset_index(drop=True)
ind_points_dm012 = "(max "+str(ind_points_dm012.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_dm014 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="DM014"].reset_index(drop=True)
ind_points_dm014 = "(max "+str(ind_points_dm014.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_dm019 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="DM019"].reset_index(drop=True)
ind_points_dm019 = "(max "+str(ind_points_dm019.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_dm020 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="DM020"].reset_index(drop=True)
ind_points_dm020 = "(max "+str(ind_points_dm020.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_dm021 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="DM021"].reset_index(drop=True)
ind_points_dm021 = "(max "+str(ind_points_dm021.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_dm022 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="DM022"].reset_index(drop=True)
ind_points_dm022 = "(max "+str(ind_points_dm022.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_dm023 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="DM023"].reset_index(drop=True)
ind_points_dm023 = "(max "+str(ind_points_dm023.at[0,"INDICATOR_POINT_VALUE"])+")"

## PC
ind_points_pc001 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="PC001"].reset_index(drop=True)
ind_points_pc001 = "(max "+str(ind_points_pc001.at[0,"INDICATOR_POINT_VALUE"])+")"

## DEM
ind_points_dem001 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="DEM001"].reset_index(drop=True)
ind_points_dem001 = "(max "+str(ind_points_dem001.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_dem004 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="DEM004"].reset_index(drop=True)
ind_points_dem004 = "(max "+str(ind_points_dem004.at[0,"INDICATOR_POINT_VALUE"])+")"

## DEP
ind_points_dep003 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="DEP003"].reset_index(drop=True)
ind_points_dep003 = "(max "+str(ind_points_dep003.at[0,"INDICATOR_POINT_VALUE"])+")"

## EP
ind_points_ep001 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="EP001"].reset_index(drop=True)
ind_points_ep001 = "(max "+str(ind_points_ep001.at[0,"INDICATOR_POINT_VALUE"])+")"

## LD
ind_points_ld004 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="LD004"].reset_index(drop=True)
ind_points_ld004 = "(max "+str(ind_points_ld004.at[0,"INDICATOR_POINT_VALUE"])+")"

## MH
ind_points_mh001 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="MH001"].reset_index(drop=True)
ind_points_mh001 = "(max "+str(ind_points_mh001.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_mh002 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="MH002"].reset_index(drop=True)
ind_points_mh002 = "(max "+str(ind_points_mh002.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_mh003 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="MH003"].reset_index(drop=True)
ind_points_mh003 = "(max "+str(ind_points_mh003.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_mh006 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="MH006"].reset_index(drop=True)
ind_points_mh006 = "(max "+str(ind_points_mh006.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_mh007 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="MH007"].reset_index(drop=True)
ind_points_mh007 = "(max "+str(ind_points_mh007.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_mh011 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="MH011"].reset_index(drop=True)
ind_points_mh011 = "(max "+str(ind_points_mh011.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_mh012 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="MH012"].reset_index(drop=True)
ind_points_mh012 = "(max "+str(ind_points_mh012.at[0,"INDICATOR_POINT_VALUE"])+")"

## OST
ind_points_ost004 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="OST004"].reset_index(drop=True)
ind_points_ost004 = "(max "+str(ind_points_ost004.at[0,"INDICATOR_POINT_VALUE"])+")"

## RA
ind_points_ra001 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="RA001"].reset_index(drop=True)
ind_points_ra001 = "(max "+str(ind_points_ra001.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_ra002 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="RA002"].reset_index(drop=True)
ind_points_ra002 = "(max "+str(ind_points_ra002.at[0,"INDICATOR_POINT_VALUE"])+")"

## NDH
ind_points_ndh001 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="NDH001"].reset_index(drop=True)
ind_points_ndh001 = "(max "+str(ind_points_ndh001.at[0,"INDICATOR_POINT_VALUE"])+")"

## CS
ind_points_cs005 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="CS005"].reset_index(drop=True)
ind_points_cs005 = "(max "+str(ind_points_cs005.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_cs006 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="CS006"].reset_index(drop=True)
ind_points_cs006 = "(max "+str(ind_points_cs006.at[0,"INDICATOR_POINT_VALUE"])+")"

## VI
ind_points_vi001 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="VI001"].reset_index(drop=True)
ind_points_vi001 = "(max "+str(ind_points_vi001.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_vi002 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="VI002"].reset_index(drop=True)
ind_points_vi002 = "(max "+str(ind_points_vi002.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_vi003 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="VI003"].reset_index(drop=True)
ind_points_vi003 = "(max "+str(ind_points_vi003.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_vi004 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="VI004"].reset_index(drop=True)
ind_points_vi004 = "(max "+str(ind_points_vi004.at[0,"INDICATOR_POINT_VALUE"])+")"

## QIPDD
ind_points_qipdd009 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="QIPDD009"].reset_index(drop=True)
ind_points_qipdd009 = "(max "+str(ind_points_qipdd009.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_qipdd010 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="QIPDD010"].reset_index(drop=True)
ind_points_qipdd010 = "(max "+str(ind_points_qipdd010.at[0,"INDICATOR_POINT_VALUE"])+")"

## QIOA
ind_points_qioa011 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="QIOA011"].reset_index(drop=True)
ind_points_qioa011 = "(max "+str(ind_points_qioa011.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_qioa012 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="QIOA012"].reset_index(drop=True)
ind_points_qioa012 = "(max "+str(ind_points_qioa012.at[0,"INDICATOR_POINT_VALUE"])+")"

## QIPDDMI
ind_points_qipddmi001 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="QIPDDMI001"].reset_index(drop=True)
ind_points_qipddmi001 = "(max "+str(ind_points_qipddmi001.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_qipddmi002 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="QIPDDMI002"].reset_index(drop=True)
ind_points_qipddmi002 = "(max "+str(ind_points_qipddmi002.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_qipddmi003 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="QIPDDMI003"].reset_index(drop=True)
ind_points_qipddmi003 = "(max "+str(ind_points_qipddmi003.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_qipddmi004 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="QIPDDMI004"].reset_index(drop=True)
ind_points_qipddmi004 = "(max "+str(ind_points_qipddmi004.at[0,"INDICATOR_POINT_VALUE"])+")"

## PDDMI
ind_points_pddmi001 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="QIPDDMI001"].reset_index(drop=True)
ind_points_pddmi001 = "(max "+str(ind_points_pddmi001.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_pddmi002 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="QIPDDMI002"].reset_index(drop=True)
ind_points_pddmi002 = "(max "+str(ind_points_pddmi002.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_pddmi003 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="QIPDDMI003"].reset_index(drop=True)
ind_points_pddmi003 = "(max "+str(ind_points_pddmi003.at[0,"INDICATOR_POINT_VALUE"])+")"

ind_points_pddmi004 = Indicator_Reference_Control_Table.loc[Indicator_Reference_Control_Table["INDICATOR_CODE"]=="QIPDDMI004"].reset_index(drop=True)
ind_points_pddmi004 = "(max "+str(ind_points_pddmi004.at[0,"INDICATOR_POINT_VALUE"])+")"

#### Define the font types used in the data outputs this can also be done in excel <br>but it is easier here for the copyright statement which may move depending on the sheet content

In [ ]:
## Copyright font
C_FONT = Font(name='Arial',
                size=10,
                bold=False,
                italic=False,
                vertAlign=None,
                underline='none',
                strike=False,
                color='000000')

In [ ]:
## Publication name on Title sheet
NAME_FONT = Font(name='Arial',
                size=27,
                bold=True,
                italic=False,
                vertAlign=None,
                underline='none',
                strike=False,
                color='005EB8')

In [ ]:
## Publication date font on Title sheet
DATE_FONT = Font(name='Arial',
                size=11,
                bold=False,
                italic=False,
                vertAlign=None,
                underline='none',
                strike=False,
                color='000000')

In [ ]:
## Message font on group sheets
MESSAGE_FONT = Font(name='Arial',
                size=10,
                bold=False,
                italic=True,
                vertAlign=None,
                underline='none',
                strike=False,
                color='000000')

In [ ]:
## Publication link on the title page of each excel workbook
LINK_FONT = Font(name='Arial',
                size=11,
                bold=False,
                italic=False,
                vertAlign=None,
                underline='single',
                strike=False,
                color='003087')

In [ ]:
## Source text (Source: Data source, NHS England) at the bottom of each excel table
SOURCE_FONT = Font(name='Arial',
                size=9,
                bold=True,
                italic=False,
                vertAlign=None,
                underline='none',
                strike=False,
                color='000000')

In [ ]:
## Format for the line at the bottom of each excel table
T_BORDER = Border(left=Side(border_style='thin',color='FFFFFF'),
                    right=Side(border_style='thin',color='FFFFFF'),
                    top=Side(border_style='thin',color='FFFFFF'),
                    bottom=Side(border_style='thin',color='BFBFBF'))

## Template 3 Achievement Excel table
[Return to contents](#Table-of-Contents)

In [ ]:
## Open excel workbook template to be populated
## TEMPLATE-3-qof-YYYY-prac-dom-ach.xlsx 
wb = openpyxl.load_workbook(filename = templates +"\TEMPLATE-3-qof-YYYY-prac-dom-ach.xlsx")

#### Name the worksheets in the TEMPLATE workbook to be used

In [ ]:
## Name the worksheets in the TEMPLATE workbook to be used
T3_TITLE = wb["Title sheet"]
T3_OVA = wb["Overall domain achievement"]
T3_CL = wb["Clinical domain"]
T3_PH = wb["Public health domain"]
T3_PHAS = wb["Public health AS domain"]
T3_PHVI = wb["Public health VI domain"]
T3_QI = wb["Quality improvement domain"]

#### Populate Publication title, Date and Link on contents page and format

In [ ]:
T3_TITLE["A7"] = QOF_title
T3_TITLE["A7"].font = NAME_FONT
T3_TITLE["B10"] = Publication_date
T3_TITLE["B10"].font = DATE_FONT
T3_TITLE["B11"].hyperlink = URL
T3_TITLE["B11"].style = "Hyperlink"
T3_TITLE["B11"].font = LINK_FONT
T3_TITLE["A14"] = Notes
T3_TITLE["A14"].font = DATE_FONT
T3_TITLE["A32"] = RS_name
T3_TITLE["A32"].font = DATE_FONT

#### Populate copyright for worksheets and format

In [ ]:
T3_TITLE["A36"] = Copyright
T3_TITLE["A36"].font = DATE_FONT

T3_OVA[prac_count_pub] = Copyright
T3_OVA[prac_count_pub].font = C_FONT

T3_CL[prac_count_pub] = Copyright
T3_CL[prac_count_pub].font = C_FONT

T3_PH[prac_count_pub] = Copyright
T3_PH[prac_count_pub].font = C_FONT

T3_PHAS[prac_count_pub] = Copyright
T3_PHAS[prac_count_pub].font = C_FONT

T3_PHVI[prac_count_pub] = Copyright
T3_PHVI[prac_count_pub].font = C_FONT

T3_QI[prac_count_pub] = Copyright
T3_QI[prac_count_pub].font = C_FONT

In [ ]:
## Add Source text (Source: Data source, NHS England) to worksheets and format
T3_OVA[source_count_ref] = Source
T3_OVA[source_count_ref].font = SOURCE_FONT

T3_CL[source_count_ref] = Source
T3_CL[source_count_ref].font = SOURCE_FONT

T3_PH[source_count_ref] = Source
T3_PH[source_count_ref].font = SOURCE_FONT

T3_PHAS[source_count_ref] = Source
T3_PHAS[source_count_ref].font = SOURCE_FONT

T3_PHVI[source_count_ref] = Source
T3_PHVI[source_count_ref].font = SOURCE_FONT

T3_QI[source_count_ref] = Source
T3_QI[source_count_ref].font = SOURCE_FONT

In [ ]:
## Add line at the bottom of the tables
## max row and col (range)
for row in T3_OVA.iter_cols(min_row=int(source_count_pub)
                            ,max_col=36
                            ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

for row in T3_CL.iter_cols(min_row=int(source_count_pub)
                           ,max_col=30
                           ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

for row in T3_PH.iter_cols(min_row=int(source_count_pub)
                           ,max_col=13
                           ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

for row in T3_PHAS.iter_cols(min_row=int(source_count_pub)
                             ,max_col=11
                             ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

for row in T3_PHVI.iter_cols(min_row=int(source_count_pub)
                             ,max_col=11
                             ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

for row in T3_QI.iter_cols(min_row=int(source_count_pub)
                           ,max_col=13
                           ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

In [ ]:
## Add table titles to worksheets and format
T3_OVA_TITLE = ach_table_titles_df.loc[(ach_table_titles_df["TABLE_NO"]==1)].reset_index(drop=True)
T3_OVA["A9"] = T3_OVA_TITLE.at[0, "TABLE_TITLE"]

T3_CL_TITLE = ach_table_titles_df.loc[(ach_table_titles_df["TABLE_NO"]==2)].reset_index(drop=True)
T3_CL["A9"] = T3_CL_TITLE.at[0, "TABLE_TITLE"]

T3_PH_TITLE = ach_table_titles_df.loc[(ach_table_titles_df["TABLE_NO"]==3)].reset_index(drop=True)
T3_PH["A9"] = T3_PH_TITLE.at[0, "TABLE_TITLE"]

T3_PHAS_TITLE = ach_table_titles_df.loc[(ach_table_titles_df["TABLE_NO"]==4)].reset_index(drop=True)
T3_PHAS["A9"] = T3_PHAS_TITLE.at[0, "TABLE_TITLE"]

T3_PHVI_TITLE = ach_table_titles_df.loc[(ach_table_titles_df["TABLE_NO"]==5)].reset_index(drop=True)
T3_PHVI["A9"] = T3_PHVI_TITLE.at[0, "TABLE_TITLE"]

T3_QI_TITLE = ach_table_titles_df.loc[(ach_table_titles_df["TABLE_NO"]==6)].reset_index(drop=True)
T3_QI["A9"] = T3_QI_TITLE.at[0, "TABLE_TITLE"]

In [ ]:
## Add current and previous financial years to 'Overall domain achievement'
T3_OVA["H11"] = Previous_year
T3_OVA["I11"] = Fyear
T3_OVA["J11"] = Previous_year
T3_OVA["N11"] = Fyear
T3_OVA["S11"] = Previous_year
T3_OVA["U11"] = Fyear
T3_OVA["W11"] = Previous_year
T3_OVA["Y11"] = Fyear
T3_OVA["AA11"] = Previous_year
T3_OVA["AC11"] = Fyear
T3_OVA["AE11"] = Previous_year
T3_OVA["AG11"] = Fyear
T3_OVA["AI11"] = Fyear


In [ ]:
## Add achievement score table column headers
T3_OVA["J12"] = "Achievement score ("+previous_max_points+" max)"
T3_OVA["K12"] = "QOF points total / "+previous_max_points+" available (%)"
T3_OVA["N12"] = "Achievement score ("+max_points+" max)"
T3_OVA["O12"] = "QOF points total / "+max_points+" available (%)"

T3_OVA["S12"] = "Achievement score (max "+str(previous_cl_ach_points.at[0,"DOMAIN_POINT_VALUE"])+")"
T3_OVA["U12"] = "Achievement score (max "+str(cl_ach_points.at[0,"DOMAIN_POINT_VALUE"])+")"

T3_OVA["W12"] = "Achievement score (max "+str(previous_ph_ach_points.at[0,"DOMAIN_POINT_VALUE"])+")"
T3_OVA["Y12"] = "Achievement score (max "+str(ph_ach_points.at[0,"DOMAIN_POINT_VALUE"])+")" 

T3_OVA["AA12"] = "Achievement score (max "+str(previous_phas_ach_points.at[0,"DOMAIN_POINT_VALUE"])+")"
T3_OVA["AC12"] = "Achievement score (max "+str(phas_ach_points.at[0,"DOMAIN_POINT_VALUE"])+")"

T3_OVA["AE12"] = "Achievement score (max "+str(previous_phvi_ach_points.at[0,"DOMAIN_POINT_VALUE"])+")"
T3_OVA["AG12"] = "Achievement score (max "+str(phvi_ach_points.at[0,"DOMAIN_POINT_VALUE"])+")"

T3_OVA["AI12"] = "Achievement score (max "+str(qi_ach_points.at[0,"DOMAIN_POINT_VALUE"])+")"

In [ ]:
## Populate group points in table headers
T3_CL["I12"] = "Total achievement score "+group_points_af
T3_CL["J12"] = "Total achievement score "+group_points_ast
T3_CL["K12"] = "Total achievement score "+group_points_can
T3_CL["L12"] = "Total achievement score "+group_points_chd
T3_CL["M12"] = "Total achievement score "+group_points_ckd
T3_CL["N12"] = "Total achievement score "+group_points_copd
T3_CL["O12"] = "Total achievement score "+group_points_dem
T3_CL["P12"] = "Total achievement score "+group_points_dep
T3_CL["Q12"] = "Total achievement score "+group_points_dm
T3_CL["R12"] = "Total achievement score "+group_points_ep
T3_CL["S12"] = "Total achievement score "+group_points_hf
T3_CL["T12"] = "Total achievement score "+group_points_hyp
T3_CL["U12"] = "Total achievement score "+group_points_ld
T3_CL["V12"] = "Total achievement score "+group_points_mh
T3_CL["W12"] = "Total achievement score "+group_points_ndh
T3_CL["X12"] = "Total achievement score "+group_points_ost
T3_CL["Y12"] = "Total achievement score "+group_points_pad
T3_CL["Z12"] = "Total achievement score "+group_points_pc
T3_CL["AA12"] = "Total achievement score "+group_points_ra
T3_CL["AB12"] = "Total achievement score "+group_points_stia
T3_CL["AC12"] = "Total achievement score (max "+str(cl_ach_points.at[0,"DOMAIN_POINT_VALUE"])+")" 

T3_PH["I12"] = "Total achievement score "+group_points_bp
T3_PH["J12"] = "Total achievement score "+group_points_ob
T3_PH["K12"] = "Total achievement score "+group_points_smok
T3_PH["L12"] = "Total achievement score (max "+str(ph_ach_points.at[0,"DOMAIN_POINT_VALUE"])+")"

T3_PHAS["I12"] = "Total achievement score "+group_points_cs
T3_PHAS["J12"] = "Total achievement score (max "+str(phas_ach_points.at[0,"DOMAIN_POINT_VALUE"])+")"

T3_PHVI["I12"] = "Total achievement score "+group_points_vi
T3_PHVI["J12"] = "Total achievement score (max "+str(phvi_ach_points.at[0,"DOMAIN_POINT_VALUE"])+")"

T3_QI["I12"] = "Total achievement score "+group_points_qi1
T3_QI["J12"] = "Total achievement score "+group_points_qi2
T3_QI["K12"] = "Total achievement score "+group_points_qi3
T3_QI["L12"] = "Total achievement score (max "+str(qi_ach_points.at[0,"DOMAIN_POINT_VALUE"])+")"

<h3>Save Achievement workbook

In [ ]:
## Save workbook
if test_run is True:
    wb.save(f"test_folder\\qof-"+ short_year +"-prac-dom-ach.xlsx")
elif test_run is False:
    wb.save(outputs+"\qof-"+ short_year +"-prac-dom-ach.xlsx")

<h3>Write data to saved excel workbook qof-prac-dom-ach

In [ ]:
## Write data to saved workbook
if test_run is True:
    writer = pd.ExcelWriter(f"test_folder\\qof-"+ short_year +"-prac-dom-ach.xlsx"
                            , engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

elif test_run is False:
    writer = pd.ExcelWriter(outputs+"\qof-"+ short_year +"-prac-dom-ach.xlsx"
                            , engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

<h4>Populate worksheets with data</h4><h7>Writes data to saved workbook template by sheet name - startrow includes the header, consequently the data starts at row 13 startrow = 12

In [ ]:
ach_all_overall_df.to_excel(writer,sheet_name='Overall domain achievement',index=False,header = False,startrow=12)
ach_all_cl_df.to_excel(writer,sheet_name='Clinical domain',index=False,header = False,startrow=12)
ach_all_ph_df.to_excel(writer,sheet_name='Public health domain',index=False,header = False,startrow=12)
ach_all_phas_df.to_excel(writer,sheet_name='Public health AS domain',index=False,header = False,startrow=12)
ach_all_vi_df.to_excel(writer,sheet_name='Public health VI domain',index=False,header = False,startrow=12)
ach_all_qi_df.to_excel(writer,sheet_name='Quality improvement domain',index=False,header = False,startrow=12)                       

In [ ]:
## Saves data written to excel workbook
writer.save()

## Template 4 PCA Excel table
[Return to contents](#Table-of-Contents)

In [ ]:
## Open excel workbook template to be populated
## TEMPLATE-4-qof-YYYY-prac-dom-pca.xlsx
wb = openpyxl.load_workbook(filename = templates +"\TEMPLATE-4-qof-YYYY-prac-dom-pca.xlsx")

#### Name the worksheets in the TEMPLATE workbook to be used

In [ ]:
## Add the pseudonyms for worksheets in the TEMPLATE workbook to be used
T4_TITLE = wb["Title sheet"]
T4_OVA = wb["Overall domain PCAs"]
T4_CL = wb["Clinical domain"]
T4_PH = wb["Public health domain"]
T4_PHAS = wb["Public health AS domain"]
T4_PHVI = wb["Public health VI domain"]
T4_QI = wb["Quality improvement domain"]

#### Populate Publication title, Date and Link on contents page and format

In [ ]:
T4_TITLE["A7"] = QOF_title
T4_TITLE["A7"].font = NAME_FONT
T4_TITLE["B10"] = Publication_date
T4_TITLE["B10"].font = DATE_FONT
T4_TITLE["B11"].hyperlink = URL
T4_TITLE["B11"].style = "Hyperlink"
T4_TITLE["B11"].font = LINK_FONT
T4_TITLE["A14"] = Notes
T4_TITLE["A14"].font = DATE_FONT
T4_TITLE["A32"] = RS_name
T4_TITLE["A32"].font = DATE_FONT

#### Populate copyright for worksheets and format

In [ ]:
T4_TITLE["A36"] = Copyright
T4_TITLE["A36"].font = DATE_FONT

T4_OVA[prac_count_pub] = Copyright
T4_OVA[prac_count_pub].font = C_FONT

T4_CL[prac_count_pub] = Copyright
T4_CL[prac_count_pub].font = C_FONT

T4_PH[prac_count_pub] = Copyright
T4_PH[prac_count_pub].font = C_FONT

T4_PHAS[prac_count_pub] = Copyright
T4_PHAS[prac_count_pub].font = C_FONT

T4_PHVI[prac_count_pub] = Copyright
T4_PHVI[prac_count_pub].font = C_FONT

T4_QI[prac_count_pub] = Copyright
T4_QI[prac_count_pub].font = C_FONT

In [ ]:
## Add Source text (Source: Data source, NHS England) to worksheets and format
T4_OVA[source_count_ref] = Source
T4_OVA[source_count_ref].font = SOURCE_FONT

T4_CL[source_count_ref] = Source
T4_CL[source_count_ref].font = SOURCE_FONT

T4_PH[source_count_ref] = Source
T4_PH[source_count_ref].font = SOURCE_FONT

T4_PHAS[source_count_ref] = Source
T4_PHAS[source_count_ref].font = SOURCE_FONT

T4_PHVI[source_count_ref] = Source
T4_PHVI[source_count_ref].font = SOURCE_FONT

T4_QI[source_count_ref] = Source
T4_QI[source_count_ref].font = SOURCE_FONT

In [ ]:
## Add line at the bottom of the table
## max row and col (range)
for row in T4_OVA.iter_cols(min_row=int(source_count_pub)
                            ,max_col=47
                            ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

for row in T4_CL.iter_cols(min_row=int(source_count_pub)
                           ,max_col=39
                           ,max_row=int(source_count_pub)):
   for cell in row:
       cell.border = T_BORDER

for row in T4_PH.iter_cols(min_row=int(source_count_pub)
                           ,max_col=15
                           ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

for row in T4_PHAS.iter_cols(min_row=int(source_count_pub)
                             ,max_col=13
                             ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

for row in T4_PHVI.iter_cols(min_row=int(source_count_pub)
                             ,max_col=13
                             ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

for row in T4_QI.iter_cols(min_row=int(source_count_pub)
                           ,max_col=13
                           ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

In [ ]:
## Add table titles to worksheets and format
T4_OVA_TITLE = pca_table_titles_df.loc[(pca_table_titles_df["TABLE_NO"]==1)].reset_index(drop=True)
T4_OVA["A9"] = T4_OVA_TITLE.at[0, "TABLE_TITLE"]

T4_CL_TITLE = pca_table_titles_df.loc[(pca_table_titles_df["TABLE_NO"]==2)].reset_index(drop=True)
T4_CL["A9"] = T4_CL_TITLE.at[0, "TABLE_TITLE"]

T4_PH_TITLE = pca_table_titles_df.loc[(pca_table_titles_df["TABLE_NO"]==3)].reset_index(drop=True)
T4_PH["A9"] = T4_PH_TITLE.at[0, "TABLE_TITLE"]

T4_PHAS_TITLE = pca_table_titles_df.loc[(pca_table_titles_df["TABLE_NO"]==4)].reset_index(drop=True)
T4_PHAS["A9"] = T4_PHAS_TITLE.at[0, "TABLE_TITLE"]

T4_PHVI_TITLE = pca_table_titles_df.loc[(pca_table_titles_df["TABLE_NO"]==5)].reset_index(drop=True)
T4_PHVI["A9"] = T4_PHVI_TITLE.at[0, "TABLE_TITLE"]

T4_QI_TITLE = pca_table_titles_df.loc[(pca_table_titles_df["TABLE_NO"]==6)].reset_index(drop=True)
T4_QI["A9"] = T4_QI_TITLE.at[0, "TABLE_TITLE"]

In [ ]:
## Add current and previous financial years to 'Overall domain achievement'
T4_OVA["H11"] = Previous_year
T4_OVA["I11"] = Fyear
T4_OVA["J11"] = Previous_year
T4_OVA["M11"] = Fyear
T4_OVA["Q11"] = Previous_year
T4_OVA["T11"] = Fyear
T4_OVA["X11"] = Previous_year
T4_OVA["AA11"] = Fyear
T4_OVA["AE11"] = Previous_year
T4_OVA["AH11"] = Fyear
T4_OVA["AL11"] = Previous_year
T4_OVA["AO11"] = Fyear
T4_OVA["AS11"] = Fyear

<h3>Save PCA workbook

In [ ]:
## Save workbook
if test_run is True:
    wb.save(f"test_folder\\qof-"+ short_year +"-prac-dom-pca.xlsx")
elif test_run is False:
    wb.save(outputs+"\qof-" + short_year +"-prac-dom-pca.xlsx")

<h3>Write data to saved excel workbook qof-prac-dom-pca

In [ ]:
## Write data to saved workbook
if test_run is True:
    writer = pd.ExcelWriter(f"test_folder\\qof-"+ short_year +"-prac-dom-pca.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

elif test_run is False:
    writer = pd.ExcelWriter(outputs+"\qof-" + short_year +"-prac-dom-pca.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

<h4>Populate worksheets with data</h4><h7>Writes data to saved workbook template by sheet name - startrow includes the header, consequently the data starts at row 13 startrow = 12

In [ ]:
overall_pca_df.to_excel(writer, sheet_name='Overall domain PCAs',index=False, header = False,startrow=12)
pca_cl_df.to_excel(writer, sheet_name='Clinical domain',index=False, header = False,startrow=12)
pca_ph_df.to_excel(writer, sheet_name='Public health domain',index=False, header = False,startrow=12)
pca_phas_df.to_excel(writer, sheet_name='Public health AS domain',index=False, header = False,startrow=12)
pca_phvi_df.to_excel(writer, sheet_name='Public health VI domain',index=False, header = False,startrow=12)
pca_qi_df.to_excel(writer, sheet_name='Quality improvement domain',index=False, header = False,startrow=12)

In [ ]:
## Saves data written to excel workbook
writer.save()

 ## Template 6 National and Regional Excel table
[Return to contents](#Table-of-Contents)

In [ ]:
## Open excel workbook template to be populated
## TEMPLATE-6-qof-YYYY-nat-reg-ach-prev-pca.xlsx
wb = openpyxl.load_workbook(filename = templates +"\TEMPLATE-6-qof-YYYY-nat-reg-ach-prev-pca.xlsx")

#### Name the worksheets in the TEMPLATE workbook to be used

In [ ]:
## Add the pseudonyms for worksheets in the TEMPLATE workbook to be used
T6_TITLE = wb["Title sheet"]
T6_AF = wb["AF"]
T6_BP = wb["BP"]
T6_CHD = wb["CHD"]
T6_HF = wb["HF"]
T6_HYP = wb["HYP"]
T6_PAD = wb["PAD"]
T6_STIA = wb["STIA"]
T6_AST = wb["AST"]
T6_COPD = wb["COPD"]
T6_OB = wb["OB"]
T6_SMOK = wb["SMOK"]
T6_CAN = wb["CAN"]
T6_CKD = wb["CKD"]
T6_DM = wb["DM"]
T6_PC = wb["PC"]
T6_DEM = wb["DEM"]
T6_DEP = wb["DEP"]
T6_EP = wb["EP"]
T6_LD = wb["LD"]
T6_MH = wb["MH"]
T6_OST = wb["OST"]
T6_RA = wb["RA"]
T6_NDH = wb["NDH"]
T6_CS = wb["CS"]
T6_QI_1 = wb["QIPDD"]
T6_QI_2 = wb["QIOA"]
T6_QI_3 = wb["QIPDDMI"]
T6_VI = wb["VI"]

#### Populate Publication title, Date and Link on contents page and format

In [ ]:
T6_TITLE["A7"] = QOF_title
T6_TITLE["A7"].font = NAME_FONT
T6_TITLE["B10"] = Publication_date
T6_TITLE["B10"].font = DATE_FONT
T6_TITLE["B11"].hyperlink = URL
T6_TITLE["B11"].style = "Hyperlink"
T6_TITLE["B11"].font = LINK_FONT
T6_TITLE["A68"] = RS_name
T6_TITLE["A68"].font = DATE_FONT

#### Populate copyright for worksheets and format

In [ ]:
T6_TITLE["A72"] = Copyright
T6_TITLE["A72"].font = DATE_FONT

T6_AF[nat_reg_count_pub] = Copyright
T6_AF[nat_reg_count_pub].font = C_FONT

T6_BP[nat_reg_count_pub] = Copyright
T6_BP[nat_reg_count_pub].font = C_FONT

T6_CHD[nat_reg_count_pub] = Copyright
T6_CHD[nat_reg_count_pub].font = C_FONT

T6_HF[nat_reg_count_pub] = Copyright
T6_HF[nat_reg_count_pub].font = C_FONT

T6_HYP[nat_reg_count_pub] = Copyright
T6_HYP[nat_reg_count_pub].font = C_FONT

T6_PAD[nat_reg_count_pub] = Copyright
T6_PAD[nat_reg_count_pub].font = C_FONT

T6_STIA[nat_reg_count_pub] = Copyright
T6_STIA[nat_reg_count_pub].font = C_FONT

T6_AST[nat_reg_count_pub] = Copyright
T6_AST[nat_reg_count_pub].font = C_FONT

T6_COPD[nat_reg_count_pub] = Copyright
T6_COPD[nat_reg_count_pub].font = C_FONT

T6_OB[nat_reg_count_pub] = Copyright
T6_OB[nat_reg_count_pub].font = C_FONT

T6_SMOK[nat_reg_count_pub] = Copyright
T6_SMOK[nat_reg_count_pub].font = C_FONT

T6_CAN[nat_reg_count_pub] = Copyright
T6_CAN[nat_reg_count_pub].font = C_FONT

T6_CKD[nat_reg_count_pub] = Copyright
T6_CKD[nat_reg_count_pub].font = C_FONT

T6_DM[nat_reg_count_pub] = Copyright
T6_DM[nat_reg_count_pub].font = C_FONT

T6_PC[nat_reg_count_pub] = Copyright
T6_PC[nat_reg_count_pub].font = C_FONT

T6_DEM[nat_reg_count_pub] = Copyright
T6_DEM[nat_reg_count_pub].font = C_FONT

T6_DEP[nat_reg_count_pub] = Copyright
T6_DEP[nat_reg_count_pub].font = C_FONT

T6_EP[nat_reg_count_pub] = Copyright
T6_EP[nat_reg_count_pub].font = C_FONT

T6_LD[nat_reg_count_pub] = Copyright
T6_LD[nat_reg_count_pub].font = C_FONT

T6_MH[nat_reg_count_pub] = Copyright
T6_MH[nat_reg_count_pub].font = C_FONT

T6_OST[nat_reg_count_pub] = Copyright
T6_OST[nat_reg_count_pub].font = C_FONT

T6_RA[nat_reg_count_pub] = Copyright
T6_RA[nat_reg_count_pub].font = C_FONT

T6_NDH[nat_reg_count_pub] = Copyright 
T6_NDH[nat_reg_count_pub].font = C_FONT

T6_CS[nat_reg_count_pub] = Copyright
T6_CS[nat_reg_count_pub].font = C_FONT

T6_QI_1[nat_reg_count_pub] = Copyright
T6_QI_1[nat_reg_count_pub].font = C_FONT

T6_QI_2[nat_reg_count_pub] = Copyright
T6_QI_2[nat_reg_count_pub].font = C_FONT

T6_QI_3[nat_reg_count_pub] = Copyright
T6_QI_3[nat_reg_count_pub].font = C_FONT

T6_VI[nat_reg_count_pub] = Copyright
T6_VI[nat_reg_count_pub].font = C_FONT

In [ ]:
## Add Source text (Source: Data source, NHS England) to worksheets and format
T6_AF[nat_reg_source_ref] = Source
T6_AF[nat_reg_source_ref].font = SOURCE_FONT
T6_BP[nat_reg_source_ref] = Source
T6_BP[nat_reg_source_ref].font = SOURCE_FONT
T6_CHD[nat_reg_source_ref] = Source
T6_CHD[nat_reg_source_ref].font = SOURCE_FONT
T6_HF[nat_reg_source_ref] = Source
T6_HF[nat_reg_source_ref].font = SOURCE_FONT
T6_HYP[nat_reg_source_ref] = Source
T6_HYP[nat_reg_source_ref].font = SOURCE_FONT
T6_PAD[nat_reg_source_ref] = Source
T6_PAD[nat_reg_source_ref].font = SOURCE_FONT 
T6_STIA[nat_reg_source_ref] = Source
T6_STIA[nat_reg_source_ref].font = SOURCE_FONT 

T6_AST[nat_reg_source_ref] = Source
T6_AST[nat_reg_source_ref].font = SOURCE_FONT
T6_COPD[nat_reg_source_ref] = Source
T6_COPD[nat_reg_source_ref].font = SOURCE_FONT

T6_OB[nat_reg_source_ref] = Source
T6_OB[nat_reg_source_ref].font = SOURCE_FONT
T6_SMOK[nat_reg_source_ref] = Source
T6_SMOK[nat_reg_source_ref].font = SOURCE_FONT

T6_CAN[nat_reg_source_ref] = Source
T6_CAN[nat_reg_source_ref].font = SOURCE_FONT
T6_CKD[nat_reg_source_ref] = Source
T6_CKD[nat_reg_source_ref].font = SOURCE_FONT 
T6_DM[nat_reg_source_ref] = Source
T6_DM[nat_reg_source_ref].font = SOURCE_FONT 
T6_NDH[nat_reg_source_ref] = Source 
T6_NDH[nat_reg_source_ref].font = SOURCE_FONT
T6_PC[nat_reg_source_ref] = Source
T6_PC[nat_reg_source_ref].font = SOURCE_FONT 

T6_DEM[nat_reg_source_ref] = Source
T6_DEM[nat_reg_source_ref].font = SOURCE_FONT 
T6_DEP[nat_reg_source_ref] = Source
T6_DEP[nat_reg_source_ref].font = SOURCE_FONT
T6_EP[nat_reg_source_ref] = Source
T6_EP[nat_reg_source_ref].font = SOURCE_FONT
T6_LD[nat_reg_source_ref] = Source
T6_LD[nat_reg_source_ref].font = SOURCE_FONT
T6_MH[nat_reg_source_ref] = Source
T6_MH[nat_reg_source_ref].font = SOURCE_FONT

T6_OST[nat_reg_source_ref] = Source
T6_OST[nat_reg_source_ref].font = SOURCE_FONT 
T6_RA[nat_reg_source_ref] = Source
T6_RA[nat_reg_source_ref].font = SOURCE_FONT
 
T6_CS[nat_reg_source_ref] = Source
T6_CS[nat_reg_source_ref].font = SOURCE_FONT

T6_QI_1[nat_reg_source_ref] = Source
T6_QI_1[nat_reg_source_ref].font = SOURCE_FONT
T6_QI_2[nat_reg_source_ref] = Source
T6_QI_2[nat_reg_source_ref].font = SOURCE_FONT
T6_QI_3[nat_reg_source_ref] = Source
T6_QI_3[nat_reg_source_ref].font = SOURCE_FONT

T6_VI[nat_reg_source_ref] = Source
T6_VI[nat_reg_source_ref].font = SOURCE_FONT

In [ ]:
## Add line at the bottom of the table
## max row and col (range)
for row in T6_AF.iter_cols(min_row=int(nat_reg_source_count)
                           ,max_col=42
                           ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER

for row in T6_BP.iter_cols(min_row=int(nat_reg_source_count)
                           ,max_col=27
                           ,max_row=int(nat_reg_source_count)):
   for cell in row:
       cell.border = T_BORDER

for row in T6_CHD.iter_cols(min_row=int(nat_reg_source_count)
                            ,max_col=52
                            ,max_row=int(nat_reg_source_count)):
   for cell in row:
       cell.border = T_BORDER

for row in T6_HF.iter_cols(min_row=int(nat_reg_source_count)
                           ,max_col=58
                           ,max_row=int(nat_reg_source_count)):
   for cell in row:
       cell.border = T_BORDER

for row in T6_HYP.iter_cols(min_row=int(nat_reg_source_count)
                            ,max_col=44
                            ,max_row=int(nat_reg_source_count)):
   for cell in row:
       cell.border = T_BORDER

for row in T6_PAD.iter_cols(min_row=int(nat_reg_source_count)
                            ,max_col=19
                            ,max_row=int(nat_reg_source_count)):
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_STIA.iter_cols(min_row=int(nat_reg_source_count)
                             ,max_col=52
                             ,max_row=int(nat_reg_source_count)):
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_AST.iter_cols(min_row=int(nat_reg_source_count)
                            ,max_col=51
                            ,max_row=int(nat_reg_source_count)):
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_COPD.iter_cols(min_row=int(nat_reg_source_count)
                             ,max_col=42
                             ,max_row=int(nat_reg_source_count)):
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_OB.iter_cols(min_row=int(nat_reg_source_count)
                           ,max_col=19
                           ,max_row=int(nat_reg_source_count)):
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_SMOK.iter_cols(min_row=int(nat_reg_source_count)
                             ,max_col=43
                             ,max_row=int(nat_reg_source_count)):
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_CAN.iter_cols(min_row=int(nat_reg_source_count)
                            ,max_col=42
                            ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_CKD.iter_cols(min_row=int(nat_reg_source_count)
                            ,max_col=19
                            ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER

for row in T6_DM.iter_cols(min_row=int(nat_reg_source_count)
                           ,max_col=91
                           ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_NDH.iter_cols(min_row=int(nat_reg_source_count)
                            ,max_col=31
                            ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_PC.iter_cols(min_row=int(nat_reg_source_count)
                           ,max_col=19
                           ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_DEM.iter_cols(min_row=int(nat_reg_source_count)
                            ,max_col=34
                            ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_DEP.iter_cols(min_row=int(nat_reg_source_count)
                            ,max_col=32
                            ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_EP.iter_cols(min_row=int(nat_reg_source_count)
                           ,max_col=19
                           ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_LD.iter_cols(min_row=int(nat_reg_source_count)
                           ,max_col=19
                           ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_MH.iter_cols(min_row=int(nat_reg_source_count)
                           ,max_col=74
                           ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_OST.iter_cols(min_row=int(nat_reg_source_count)
                            ,max_col=19
                            ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_RA.iter_cols(min_row=int(nat_reg_source_count)
                           ,max_col=34
                           ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_CS.iter_cols(min_row=int(nat_reg_source_count)
                           ,max_col=36
                           ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_VI.iter_cols(min_row=int(nat_reg_source_count)
                           ,max_col=53
                           ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_QI_1.iter_cols(min_row=int(nat_reg_source_count)
                             ,max_col=11
                             ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_QI_2.iter_cols(min_row=int(nat_reg_source_count)
                             ,max_col=11
                             ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T6_QI_3.iter_cols(min_row=int(nat_reg_source_count)
                             ,max_col=42
                             ,max_row=int(nat_reg_source_count)): 
   for cell in row:
       cell.border = T_BORDER

#### Populate table titles and message for worksheets and format

In [ ]:
## National and Region tables titles
T6_AF_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="AF"].reset_index(drop = True)
T6_AF["A8"] = T6_AF_TITLE.at[0, "TABLE_TITLE"]
T6_AF["A9"] = message_af
T6_AF["A9"].font = MESSAGE_FONT

T6_BP_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="BP"].reset_index(drop = True)
T6_BP["A8"] = T6_BP_TITLE.at[0, "TABLE_TITLE"]
T6_BP["A9"] = message_bp
T6_BP["A9"].font = MESSAGE_FONT

T6_CHD_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="CHD"].reset_index(drop = True)
T6_CHD["A8"] = T6_CHD_TITLE.at[0, "TABLE_TITLE"]
T6_CHD["A9"] = message_chd
T6_CHD["A9"].font = MESSAGE_FONT

T6_HF_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="HF"].reset_index(drop = True)
T6_HF["A8"] = T6_HF_TITLE.at[0, "TABLE_TITLE"]
T6_HF["A9"] = message_hf
T6_HF["A9"].font = MESSAGE_FONT

T6_HYP_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="HYP"].reset_index(drop = True)
T6_HYP["A8"] = T6_HYP_TITLE.at[0, "TABLE_TITLE"]
T6_HYP["A9"] = message_hyp
T6_HYP["A9"].font = MESSAGE_FONT

T6_PAD_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="PAD"].reset_index(drop = True)
T6_PAD["A8"] = T6_PAD_TITLE.at[0, "TABLE_TITLE"]
T6_PAD["A9"] = message_pad
T6_PAD["A9"].font = MESSAGE_FONT

T6_STIA_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="STIA"].reset_index(drop = True)
T6_STIA["A8"] = T6_STIA_TITLE.at[0, "TABLE_TITLE"]
T6_STIA["A9"] = message_stia
T6_STIA["A9"].font = MESSAGE_FONT

T6_AST_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="AST"].reset_index(drop = True)
T6_AST["A8"] = T6_AST_TITLE.at[0, "TABLE_TITLE"]
T6_AST["A9"] = message_ast
T6_AST["A9"].font = MESSAGE_FONT

T6_COPD_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="COPD"].reset_index(drop = True)
T6_COPD["A8"] = T6_COPD_TITLE.at[0, "TABLE_TITLE"]
T6_COPD["A9"] = message_copd
T6_COPD["A9"].font = MESSAGE_FONT

T6_OB_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="OB"].reset_index(drop = True)
T6_OB["A8"] = T6_OB_TITLE.at[0, "TABLE_TITLE"]
T6_OB["A9"] = message_ob
T6_OB["A9"].font = MESSAGE_FONT

T6_SMOK_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="SMOK"].reset_index(drop = True)
T6_SMOK["A8"] = T6_SMOK_TITLE.at[0, "TABLE_TITLE"]
T6_SMOK["A9"] = message_smok
T6_SMOK["A9"].font = MESSAGE_FONT

T6_CAN_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="CAN"].reset_index(drop = True)
T6_CAN["A8"] = T6_CAN_TITLE.at[0, "TABLE_TITLE"]
T6_CAN["A9"] = message_can
T6_CAN["A9"].font = MESSAGE_FONT

T6_CKD_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="CKD"].reset_index(drop = True)
T6_CKD["A8"] = T6_CKD_TITLE.at[0, "TABLE_TITLE"]
T6_CKD["A9"] = message_ckd
T6_CKD["A9"].font = MESSAGE_FONT

T6_DM_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="DM"].reset_index(drop = True)
T6_DM["A8"] = T6_DM_TITLE.at[0, "TABLE_TITLE"]
T6_DM["A9"] = message_dm
T6_DM["A9"].font = MESSAGE_FONT

T6_PC_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="PC"].reset_index(drop = True)
T6_PC["A8"] = T6_PC_TITLE.at[0, "TABLE_TITLE"]
T6_PC["A9"] = message_pc
T6_PC["A9"].font = MESSAGE_FONT

T6_DEM_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="DEM"].reset_index(drop = True)
T6_DEM["A8"] = T6_DEM_TITLE.at[0, "TABLE_TITLE"]
T6_DEM["A9"] = message_dem
T6_DEM["A9"].font = MESSAGE_FONT

T6_DEP_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="DEP"].reset_index(drop = True)
T6_DEP["A8"] = T6_DEP_TITLE.at[0, "TABLE_TITLE"]
T6_DEP["A9"] = message_dep
T6_DEP["A9"].font = MESSAGE_FONT

T6_EP_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="EP"].reset_index(drop = True)
T6_EP["A8"] = T6_EP_TITLE.at[0, "TABLE_TITLE"]
T6_EP["A9"] = message_ep
T6_EP["A9"].font = MESSAGE_FONT

T6_LD_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="LD"].reset_index(drop = True)
T6_LD["A8"] = T6_LD_TITLE.at[0, "TABLE_TITLE"]
T6_LD["A9"] = message_ld
T6_LD["A9"].font = MESSAGE_FONT

T6_MH_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="MH"].reset_index(drop = True)
T6_MH["A8"] = T6_MH_TITLE.at[0, "TABLE_TITLE"]
T6_MH["A9"] = message_mh
T6_MH["A9"].font = MESSAGE_FONT

T6_OST_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="OST"].reset_index(drop = True)
T6_OST["A8"] = T6_OST_TITLE.at[0, "TABLE_TITLE"]
T6_OST["A9"] = message_ost
T6_OST["A9"].font = MESSAGE_FONT

T6_RA_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="RA"].reset_index(drop = True)
T6_RA["A8"] = T6_RA_TITLE.at[0, "TABLE_TITLE"]
T6_RA["A9"] = message_ra
T6_RA["A9"].font = MESSAGE_FONT

T6_NDH_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="NDH"].reset_index(drop = True)
T6_NDH["A8"] = T6_NDH_TITLE.at[0, "TABLE_TITLE"]
T6_NDH["A9"] = message_ndh
T6_NDH["A9"].font = MESSAGE_FONT

T6_CS_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="CS"].reset_index(drop = True)
T6_CS["A8"] = T6_CS_TITLE.at[0, "TABLE_TITLE"]
T6_CS["A9"] = message_cs
T6_CS["A9"].font = MESSAGE_FONT

T6_QI_1_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="QI_1"].reset_index(drop = True)
T6_QI_1["A8"] = T6_QI_1_TITLE.at[0, "TABLE_TITLE"]
T6_QI_1["A9"] = message_qi1
T6_QI_1["A9"].font = MESSAGE_FONT

T6_QI_2_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="QI_2"].reset_index(drop = True)
T6_QI_2["A8"] = T6_QI_2_TITLE.at[0, "TABLE_TITLE"]
T6_QI_2["A9"] = message_qi2
T6_QI_2["A9"].font = MESSAGE_FONT

T6_QI_3_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="QI_3"].reset_index(drop = True)
T6_QI_3["A8"] = T6_QI_3_TITLE.at[0, "TABLE_TITLE"]
T6_QI_3["A9"] = message_qi3
T6_QI_3["A9"].font = MESSAGE_FONT

T6_VI_TITLE = gr_nat_reg_table_titles.loc[gr_nat_reg_table_titles["GROUP_CODE"]=="VI"].reset_index(drop = True)
T6_VI["A8"] = T6_VI_TITLE.at[0, "TABLE_TITLE"]
T6_VI["A9"] = message_vi
T6_VI["A9"].font = MESSAGE_FONT

#### Populate Fyears in indicator group tables

In [ ]:
## Populate current and previous financial years for all indicator group tables
T6_AF["D11"] = Previous_year
T6_AF["H11"] = Fyear
T6_AF["M11"] = Previous_year
T6_AF["O11"] = Fyear
T6_AF["R11"] = Previous_year
T6_AF["U11"] = Fyear

T6_BP["D11"] = Previous_year
T6_BP["F11"] = Fyear
T6_BP["H11"] = Previous_year
T6_BP["J11"] = Fyear
T6_BP["M11"] = Previous_year
T6_BP["P11"] = Fyear

T6_CHD["D11"] = Previous_year
T6_CHD["H11"] = Fyear
T6_CHD["O11"] = Previous_year
T6_CHD["Q11"] = Fyear
T6_CHD["T11"] = Previous_year
T6_CHD["W11"] = Fyear

T6_HF["D11"] = Previous_year
T6_HF["H11"] = Fyear
T6_HF["M11"] = Previous_year
T6_HF["O11"] = Fyear
T6_HF["R11"] = Previous_year
T6_HF["U11"] = Fyear

T6_HYP["D11"] = Previous_year
T6_HYP["H11"] = Fyear
T6_HYP["O11"] = Previous_year
T6_HYP["Q11"] = Fyear
T6_HYP["T11"] = Previous_year
T6_HYP["W11"] = Fyear

T6_PAD["D11"] = Previous_year
T6_PAD["H11"] = Fyear
T6_PAD["M11"] = Previous_year
T6_PAD["O11"] = Fyear

T6_STIA["D11"] = Previous_year
T6_STIA["H11"] = Fyear
T6_STIA["O11"] = Previous_year
T6_STIA["Q11"] = Fyear
T6_STIA["T11"] = Previous_year
T6_STIA["W11"] = Fyear

T6_AST["D11"] = Previous_year
T6_AST["H11"] = Fyear
T6_AST["N11"] = Previous_year
T6_AST["P11"] = Fyear
T6_AST["S11"] = Previous_year
T6_AST["V11"] = Fyear

T6_COPD["D11"] = Previous_year
T6_COPD["H11"] = Fyear
T6_COPD["M11"] = Previous_year
T6_COPD["O11"] = Fyear
T6_COPD["R11"] = Previous_year
T6_COPD["U11"] = Fyear

T6_OB["D11"] = Previous_year
T6_OB["H11"] = Fyear
T6_OB["M11"] = Previous_year
T6_OB["O11"] = Fyear

T6_SMOK["D11"] = Previous_year
T6_SMOK["F11"] = Fyear
T6_SMOK["H11"] = Previous_year
T6_SMOK["J11"] = Fyear
T6_SMOK["M11"] = Previous_year
T6_SMOK["P11"] = Fyear

T6_CAN["D11"] = Previous_year
T6_CAN["H11"] = Fyear
T6_CAN["M11"] = Previous_year
T6_CAN["O11"] = Fyear
T6_CAN["R11"] = Previous_year
T6_CAN["U11"] = Fyear

T6_CKD["D11"] = Previous_year
T6_CKD["H11"] = Fyear
T6_CKD["M11"] = Previous_year
T6_CKD["O11"] = Fyear

T6_DM["D11"] = Previous_year
T6_DM["H11"] = Fyear
T6_DM["N11"] = Previous_year
T6_DM["P11"] = Fyear
T6_DM["S11"] = Previous_year
T6_DM["V11"] = Fyear

T6_PC["D11"] = Previous_year
T6_PC["H11"] = Fyear
T6_PC["M11"] = Previous_year
T6_PC["O11"] = Fyear

T6_DEM["D11"] = Previous_year
T6_DEM["H11"] = Fyear
T6_DEM["M11"] = Previous_year
T6_DEM["O11"] = Fyear
T6_DEM["R11"] = Previous_year
T6_DEM["U11"] = Fyear

T6_DEP["D11"] = Previous_year
T6_DEP["H11"] = Fyear
T6_DEP["M11"] = Previous_year
T6_DEP["O11"] = Fyear
T6_DEP["R11"] = Previous_year
T6_DEP["U11"] = Fyear

T6_EP["D11"] = Previous_year
T6_EP["H11"] = Fyear
T6_EP["M11"] = Previous_year
T6_EP["O11"] = Fyear

T6_LD["D11"] = Previous_year
T6_LD["H11"] = Fyear
T6_LD["M11"] = Previous_year
T6_LD["O11"] = Fyear

T6_MH["D11"] = Previous_year
T6_MH["H11"] = Fyear
T6_MH["M11"] = Previous_year
T6_MH["O11"] = Fyear
T6_MH["R11"] = Previous_year
T6_MH["U11"] = Fyear

T6_OST["D11"] = Previous_year
T6_OST["H11"] = Fyear
T6_OST["M11"] = Previous_year
T6_OST["O11"] = Fyear

T6_RA["D11"] = Previous_year
T6_RA["H11"] = Fyear
T6_RA["M11"] = Previous_year
T6_RA["O11"] = Fyear
T6_RA["R11"] = Previous_year
T6_RA["U11"] = Fyear

T6_NDH["D11"] = Previous_year
T6_NDH["H11"] = Fyear
T6_NDH["L11"] = Previous_year
T6_NDH["N11"] = Fyear
T6_NDH["Q11"] = Previous_year
T6_NDH["T11"] = Fyear

T6_CS["D11"] = Previous_year
T6_CS["F11"] = Fyear
T6_CS["I11"] = Previous_year
T6_CS["K11"] = Fyear
T6_CS["N11"] = Previous_year
T6_CS["Q11"] = Fyear

T6_QI_1["D11"] = Fyear
T6_QI_1["F11"] = Fyear

T6_QI_2["D11"] = Fyear
T6_QI_2["F11"] = Fyear

T6_QI_3["D11"] = Fyear
T6_QI_3["F11"] = Fyear
T6_QI_3["H11"] = Fyear

T6_VI["D11"] = Previous_year
T6_VI["E11"] = Fyear
T6_VI["J11"] = Previous_year
T6_VI["L11"] = Fyear
T6_VI["O11"] = Previous_year
T6_VI["R11"] = Fyear

#### Populate group points in table headers

In [ ]:
## Populate indicator group points for the current year in table headers
T6_AF["O12"] = "Total Achievement Score "+group_points_af
T6_BP["J12"] = "Total Achievement Score "+group_points_bp
T6_CHD["Q12"] ="Total Achievement Score "+ group_points_chd
T6_HF["O12"] = "Total Achievement Score "+group_points_hf
T6_HYP["Q12"] = "Total Achievement Score "+group_points_hyp
T6_PAD["O12"] = "Total Achievement Score "+group_points_pad
T6_STIA["Q12"] = "Total Achievement Score "+group_points_stia
T6_AST["P12"] = "Total Achievement Score "+group_points_ast
T6_COPD["O12"] = "Total Achievement Score "+group_points_copd
T6_OB["O12"] = "Total Achievement Score "+group_points_ob
T6_SMOK["J12"] = "Total Achievement Score "+group_points_smok
T6_CAN["O12"] = "Total Achievement Score "+group_points_can
T6_CKD["O12"] = "Total Achievement Score "+group_points_ckd
T6_DM["P12"] = "Total Achievement Score "+group_points_dm
T6_PC["O12"] = "Total Achievement Score "+group_points_pc
T6_DEM["O12"] = "Total Achievement Score "+group_points_dem
T6_DEP["O12"] = "Total Achievement Score "+group_points_dep
T6_EP["O12"] = "Total Achievement Score "+group_points_ep
T6_LD["O12"] = "Total Achievement Score "+group_points_ld
T6_MH["O12"] = "Total Achievement Score "+group_points_mh
T6_OST["O12"] = "Total Achievement Score "+group_points_ost
T6_RA["O12"] = "Total Achievement Score "+group_points_ra
T6_NDH["N12"] = "Total Achievement Score "+group_points_ndh
T6_CS["K12"] = "Total Achievement Score "+group_points_cs
T6_QI_1["F12"] = "Total Achievement Score "+group_points_qi1
T6_QI_2["F12"] = "Total Achievement Score "+group_points_qi2
T6_QI_3["F12"] = "Total Achievement Score "+group_points_qi3
T6_VI["L12"] = "Total Achievement Score "+group_points_vi

## Populate indicator group points for the previous year in table headers
T6_AF["M12"] = "Total Achievement Score "+previous_gr_points_af
T6_BP["H12"] = "Total Achievement Score "+previous_gr_points_bp
T6_CHD["O12"] = "Total Achievement Score "+previous_gr_points_chd
T6_HF["M12"] = "Total Achievement Score "+previous_gr_points_hf
T6_HYP["O12"] = "Total Achievement Score "+previous_gr_points_hyp
T6_PAD["M12"] = "Total Achievement Score "+previous_gr_points_pad
T6_STIA["O12"] = "Total Achievement Score "+previous_gr_points_stia
T6_AST["N12"] = "Total Achievement Score "+previous_gr_points_ast
T6_COPD["M12"] = "Total Achievement Score "+previous_gr_points_copd
T6_OB["M12"] = "Total Achievement Score "+previous_gr_points_ob
T6_SMOK["H12"] = "Total Achievement Score "+previous_gr_points_smok
T6_CAN["M12"] = "Total Achievement Score "+previous_gr_points_can
T6_CKD["M12"] = "Total Achievement Score "+previous_gr_points_ckd
T6_DM["N12"] = "Total Achievement Score "+previous_gr_points_dm
T6_PC["M12"] = "Total Achievement Score "+previous_gr_points_pc
T6_DEM["M12"] = "Total Achievement Score "+previous_gr_points_dem
T6_DEP["M12"] = "Total Achievement Score "+previous_gr_points_dep
T6_EP["M12"] = "Total Achievement Score "+previous_gr_points_ep
T6_LD["M12"] = "Total Achievement Score "+previous_gr_points_ld
T6_MH["M12"] = "Total Achievement Score "+previous_gr_points_mh
T6_OST["M12"] = "Total Achievement Score "+previous_gr_points_ost
T6_RA["M12"] = "Total Achievement Score "+previous_gr_points_ra
T6_NDH["L12"] = "Total Achievement Score "+previous_gr_points_ndh
T6_CS["I12"] = "Total Achievement Score "+previous_gr_points_cs
T6_VI["J12"] = "Total Achievement Score "+previous_gr_points_vi

#### Populate indicator points in table headers

In [ ]:
## Populate indicator points for the current year in table headers
T6_AF["Y12"] = "Achievement Score "+ind_points_af001
T6_AF["AA12"] = "Achievement Score "+ind_points_af006
T6_AF["AI12"] = "Achievement Score "+ind_points_af007

T6_BP["T12"] = "Achievement Score "+ind_points_bp002

T6_CHD["AA12"] = "Achievement Score "+ind_points_chd001
T6_CHD["AC12"] = "Achievement Score "+ind_points_chd005
T6_CHD["AK12"] = "Achievement Score "+ind_points_chd008
T6_CHD["AS12"] = "Achievement Score "+ind_points_chd009

T6_HF["Y12"] = "Achievement Score "+ind_points_hf001
T6_HF["AA12"] = "Achievement Score "+ind_points_hf003
T6_HF["AI12"] = "Achievement Score "+ind_points_hf005
T6_HF["AQ12"] = "Achievement Score "+ind_points_hf006
T6_HF["AY12"] = "Achievement Score "+ind_points_hf007

T6_HYP["AA12"] = "Achievement Score "+ind_points_hyp001
T6_HYP["AC12"] = "Achievement Score "+ind_points_hyp003
T6_HYP["AK12"] = "Achievement Score "+ind_points_hyp007

T6_PAD["R12"] = "Achievement Score "+ind_points_pad001

T6_STIA["AA12"] = "Achievement Score "+ind_points_stia001
T6_STIA["AC12"] = "Achievement Score "+ind_points_stia007
T6_STIA["AK12"] = "Achievement Score "+ind_points_stia010
T6_STIA["AS12"] = "Achievement Score "+ind_points_stia011

T6_AST["Z12"] = "Achievement Score "+ind_points_ast005
T6_AST["AB12"] = "Achievement Score "+ind_points_ast006
T6_AST["AJ12"] = "Achievement Score "+ind_points_ast007
T6_AST["AR12"] = "Achievement Score "+ind_points_ast008

T6_COPD["Y12"] = "Achievement Score "+ind_points_copd009
T6_COPD["AA12"] = "Achievement Score "+ind_points_copd008
T6_COPD["AI12"] = "Achievement Score "+ind_points_copd010

T6_OB["R12"] = "Achievement Score "+ind_points_ob002

T6_SMOK["T12"] = "Achievement Score "+ind_points_smok002
T6_SMOK["AB12"] = "Achievement Score "+ind_points_smok004
T6_SMOK["AJ12"] = "Achievement Score "+ind_points_smok005

T6_CAN["Y12"] = "Achievement Score "+ind_points_can001

T6_CAN["AA12"] = "Achievement Score "+ind_points_can004
T6_CAN["AI12"] = "Achievement Score "+ind_points_can005

T6_CKD["R12"] = "Achievement Score "+ind_points_ckd005

T6_DM["Z12"] = "Achievement Score "+ind_points_dm017
T6_DM["AB12"] = "Achievement Score "+ind_points_dm006
T6_DM["AJ12"] = "Achievement Score "+ind_points_dm012
T6_DM["AR12"] = "Achievement Score "+ind_points_dm014
T6_DM["AZ12"] = "Achievement Score "+ind_points_dm019
T6_DM["BH12"] = "Achievement Score "+ind_points_dm020
T6_DM["BP12"] = "Achievement Score "+ind_points_dm021
T6_DM["BX12"] = "Achievement Score "+ind_points_dm022
T6_DM["CF12"] = "Achievement Score "+ind_points_dm023

T6_PC["R12"] = "Achievement Score "+ind_points_pc001

T6_DEM["Y12"] = "Achievement Score "+ind_points_dem001
T6_DEM["AA12"] = "Achievement Score "+ind_points_dem004

T6_DEP["Y12"] = "Achievement Score "+ind_points_dep003

T6_EP["R12"] = "Achievement Score "+ind_points_ep001

T6_LD["R12"] = "Achievement Score "+ind_points_ld004

T6_MH["Y12"] = "Achievement Score "+ind_points_mh001
T6_MH["AA12"] = "Achievement Score "+ind_points_mh002
T6_MH["AI12"] = "Achievement Score "+ind_points_mh003
T6_MH["AQ12"] = "Achievement Score "+ind_points_mh006
T6_MH["AY12"] = "Achievement Score "+ind_points_mh007
T6_MH["BG12"] = "Achievement Score "+ind_points_mh011
T6_MH["BO12"] = "Achievement Score "+ind_points_mh012

T6_OST["R12"] = "Achievement Score "+ind_points_ost004

T6_RA["Y12"] = "Achievement Score "+ind_points_ra001
T6_RA["AA12"] = "Achievement Score "+ind_points_ra002

T6_NDH["X12"] = "Achievement Score "+ind_points_ndh001

T6_CS["U12"] = "Achievement Score "+ind_points_cs005
T6_CS["AC12"] = "Achievement Score "+ind_points_cs006

T6_QI_1["H12"] = "Achievement Score "+ind_points_qipdd009
T6_QI_1["J12"] = "Achievement Score "+ind_points_qipdd010

T6_QI_2["H12"] = "Achievement Score "+ind_points_qioa011
T6_QI_2["J12"] = "Achievement Score "+ind_points_qioa012

T6_QI_3["K12"] = "Achievement Score "+ind_points_qipddmi001
T6_QI_3["S12"] = "Achievement Score "+ind_points_qipddmi002
T6_QI_3["AA12"] = "Achievement Score "+ind_points_qipddmi003
T6_QI_3["AI12"] = "Achievement Score "+ind_points_qipddmi004

T6_VI["V12"] = "Achievement Score "+ind_points_vi001
T6_VI["AD12"] = "Achievement Score "+ind_points_vi002
T6_VI["AL12"] = "Achievement Score "+ind_points_vi003
T6_VI["AT12"] = "Achievement Score "+ind_points_vi004

<h2>Save Workbook

In [ ]:
## Save workbook
if test_run is True:
    wb.save(f"test_folder\\qof-"+ short_year +"-nat-reg-ach-prev-pca.xlsx")
elif test_run is False:
    wb.save(outputs+"\qof-"+ short_year +"-nat-reg-ach-prev-pca.xlsx")

<h2>Write data to saved excel workbook

In [ ]:
## Write data to saved workbook
if test_run is True:
    writer = pd.ExcelWriter(f"test_folder\\qof-"+ short_year +"-nat-reg-ach-prev-pca.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

elif test_run is False:
    writer = pd.ExcelWriter(outputs+"\qof-"+ short_year +"-nat-reg-ach-prev-pca.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

<h4>Populate worksheets with data</h4><h7>Writes data to saved workbook template by sheet name - startrow includes the header, consequently the data starts at row 13 startrow = 12

In [ ]:
t6_af_df.to_excel(writer, sheet_name='AF',index=False, header = False,startrow=12)
t6_bp_df.to_excel(writer, sheet_name='BP',index=False, header = False,startrow=12)
t6_chd_df.to_excel(writer, sheet_name= 'CHD',index=False, header = False,startrow=12)
t6_hf_df.to_excel(writer, sheet_name='HF',index=False, header = False,startrow=12)
t6_hyp_df.to_excel(writer, sheet_name='HYP',index=False, header = False,startrow=12)
t6_pad_df.to_excel(writer, sheet_name='PAD',index=False, header = False,startrow=12)
t6_stia_df.to_excel(writer, sheet_name='STIA',index=False, header = False,startrow=12)
t6_ast_df.to_excel(writer, sheet_name='AST',index=False, header = False,startrow=12)
t6_copd_df.to_excel(writer, sheet_name='COPD',index=False, header = False,startrow=12)
t6_ob_df.to_excel(writer, sheet_name='OB',index=False, header = False,startrow=12)
t6_smok_df.to_excel(writer, sheet_name='SMOK',index=False, header = False,startrow=12)
t6_can_df.to_excel(writer, sheet_name='CAN',index=False, header = False,startrow=12)
t6_ckd_df.to_excel(writer, sheet_name='CKD',index=False, header = False,startrow=12)
t6_dm_df.to_excel(writer, sheet_name='DM',index=False, header = False,startrow=12)
t6_pc_df.to_excel(writer, sheet_name='PC',index=False, header = False,startrow=12)
t6_dem_df.to_excel(writer, sheet_name='DEM',index=False, header = False,startrow=12)
t6_dep_df.to_excel(writer, sheet_name='DEP',index=False, header = False,startrow=12)
t6_ep_df.to_excel(writer, sheet_name='EP',index=False, header = False,startrow=12)
t6_ld_df.to_excel(writer, sheet_name='LD',index=False, header = False,startrow=12)
t6_mh_df.to_excel(writer, sheet_name='MH',index=False, header = False,startrow=12)
t6_ost_df.to_excel(writer, sheet_name='OST',index=False, header = False,startrow=12)
t6_ra_df.to_excel(writer, sheet_name='RA',index=False, header = False,startrow=12)
t6_ndh_df.to_excel(writer, sheet_name='NDH',index=False, header = False,startrow=12)
t6_cs_df.to_excel(writer, sheet_name='CS',index=False, header = False,startrow=12)
t6_qi1_df.to_excel(writer, sheet_name='QIPDD',index=False, header = False,startrow=12)
t6_qi2_df.to_excel(writer, sheet_name='QIOA',index=False, header = False,startrow=12)
t6_qi3_df.to_excel(writer, sheet_name='QIPDDMI',index=False, header = False,startrow=12)
t6_vi_df.to_excel(writer, sheet_name='VI',index=False, header = False,startrow=12)

In [ ]:
## Saves data written to excel workbook
writer.save()

## Template 8 ICB (STP) Excel table
[Return to contents](#Table-of-Contents)

In [ ]:
## Open excel workbook template to be populated
## TEMPLATE-8-qof-YYYY-icb-ach-prev-pca.xlsx
wb = openpyxl.load_workbook(filename = templates +"\TEMPLATE-8-qof-YYYY-icb-ach-prev-pca.xlsx")

#### Name the worksheets in the TEMPLATE workbook to be used

In [ ]:
## Add the pseudonyms for worksheets in the TEMPLATE workbook to be used
T8_TITLE = wb["Title sheet"]
T8_AF = wb["AF"]
T8_BP = wb["BP"]
T8_CHD = wb["CHD"]
T8_HF = wb["HF"]
T8_HYP = wb["HYP"]
T8_PAD = wb["PAD"]
T8_STIA = wb["STIA"]
T8_AST = wb["AST"]
T8_COPD = wb["COPD"]
T8_OB = wb["OB"]
T8_SMOK = wb["SMOK"]
T8_CAN = wb["CAN"]
T8_CKD = wb["CKD"]
T8_DM = wb["DM"]
T8_PC = wb["PC"]
T8_DEM = wb["DEM"]
T8_DEP = wb["DEP"]
T8_EP = wb["EP"]
T8_LD = wb["LD"]
T8_MH = wb["MH"]
T8_OST = wb["OST"]
T8_RA = wb["RA"]
T8_NDH = wb["NDH"]
T8_CS = wb["CS"]
T8_QI_1 = wb["QIPDD"]
T8_QI_2 = wb["QIOA"]
T8_QI_3 = wb["QIPDDMI"]
T8_VI = wb["VI"]

#### Populate Publication title, Date and Link on contents page and format

In [ ]:
T8_TITLE["A7"] = QOF_title
T8_TITLE["A7"].font = NAME_FONT
T8_TITLE["B10"] = Publication_date
T8_TITLE["B10"].font = DATE_FONT
T8_TITLE["B11"].hyperlink = URL
T8_TITLE["B11"].style = "Hyperlink"
T8_TITLE["B11"].font = LINK_FONT
T8_TITLE["A68"] = RS_name
T8_TITLE["A68"].font = DATE_FONT

#### Populate copyright for worksheets and format

In [ ]:
T8_TITLE["A72"] = Copyright
T8_TITLE["A72"].font = DATE_FONT

T8_AF[stp_count_pub] = Copyright
T8_AF[stp_count_pub].font = C_FONT

T8_BP[stp_count_pub] = Copyright
T8_BP[stp_count_pub].font = C_FONT

T8_CHD[stp_count_pub] = Copyright
T8_CHD[stp_count_pub].font = C_FONT

T8_HF[stp_count_pub] = Copyright
T8_HF[stp_count_pub].font = C_FONT

T8_HYP[stp_count_pub] = Copyright
T8_HYP[stp_count_pub].font = C_FONT

T8_PAD[stp_count_pub] = Copyright
T8_PAD[stp_count_pub].font = C_FONT

T8_STIA[stp_count_pub] = Copyright
T8_STIA[stp_count_pub].font = C_FONT

T8_AST[stp_count_pub] = Copyright
T8_AST[stp_count_pub].font = C_FONT

T8_COPD[stp_count_pub] = Copyright
T8_COPD[stp_count_pub].font = C_FONT

T8_OB[stp_count_pub] = Copyright
T8_OB[stp_count_pub].font = C_FONT

T8_SMOK[stp_count_pub] = Copyright
T8_SMOK[stp_count_pub].font = C_FONT 

T8_CAN[stp_count_pub] = Copyright
T8_CAN[stp_count_pub].font = C_FONT

T8_CKD[stp_count_pub] = Copyright
T8_CKD[stp_count_pub].font = C_FONT

T8_DM[stp_count_pub] = Copyright
T8_DM[stp_count_pub].font = C_FONT

T8_PC[stp_count_pub] = Copyright
T8_PC[stp_count_pub].font = C_FONT

T8_DEM[stp_count_pub] = Copyright
T8_DEM[stp_count_pub].font = C_FONT

T8_DEP[stp_count_pub] = Copyright
T8_DEP[stp_count_pub].font = C_FONT

T8_EP[stp_count_pub] = Copyright
T8_EP[stp_count_pub].font = C_FONT

T8_LD[stp_count_pub] = Copyright
T8_LD[stp_count_pub].font = C_FONT

T8_MH[stp_count_pub] = Copyright
T8_MH[stp_count_pub].font = C_FONT

T8_OST[stp_count_pub] = Copyright
T8_OST[stp_count_pub].font = C_FONT 

T8_RA[stp_count_pub] = Copyright
T8_RA[stp_count_pub].font = C_FONT

T8_NDH[stp_count_pub] = Copyright 
T8_NDH[stp_count_pub].font = C_FONT

T8_CS[stp_count_pub] = Copyright
T8_CS[stp_count_pub].font = C_FONT

T8_QI_1[stp_count_pub] = Copyright
T8_QI_1[stp_count_pub].font = C_FONT

T8_QI_2[stp_count_pub] = Copyright
T8_QI_2[stp_count_pub].font = C_FONT

T8_QI_3[stp_count_pub] = Copyright
T8_QI_3[stp_count_pub].font = C_FONT

T8_VI[stp_count_pub] = Copyright
T8_VI[stp_count_pub].font = C_FONT

In [ ]:
## Add Source text (Source: Data source, NHS England) to worksheets and format
T8_AF[stp_source_ref] = Source
T8_AF[stp_source_ref].font = SOURCE_FONT
T8_BP[stp_source_ref] = Source
T8_BP[stp_source_ref].font = SOURCE_FONT
T8_CHD[stp_source_ref] = Source
T8_CHD[stp_source_ref].font = SOURCE_FONT
T8_HF[stp_source_ref] = Source
T8_HF[stp_source_ref].font = SOURCE_FONT
T8_HYP[stp_source_ref] = Source
T8_HYP[stp_source_ref].font = SOURCE_FONT
T8_PAD[stp_source_ref] = Source
T8_PAD[stp_source_ref].font = SOURCE_FONT 
T8_STIA[stp_source_ref] = Source
T8_STIA[stp_source_ref].font = SOURCE_FONT 

T8_AST[stp_source_ref] = Source
T8_AST[stp_source_ref].font = SOURCE_FONT
T8_COPD[stp_source_ref] = Source
T8_COPD[stp_source_ref].font = SOURCE_FONT

T8_OB[stp_source_ref] = Source
T8_OB[stp_source_ref].font = SOURCE_FONT
T8_SMOK[stp_source_ref] = Source
T8_SMOK[stp_source_ref].font = SOURCE_FONT

T8_CAN[stp_source_ref] = Source
T8_CAN[stp_source_ref].font = SOURCE_FONT
T8_CKD[stp_source_ref] = Source
T8_CKD[stp_source_ref].font = SOURCE_FONT 
T8_DM[stp_source_ref] = Source
T8_DM[stp_source_ref].font = SOURCE_FONT 
T8_NDH[stp_source_ref] = Source 
T8_NDH[stp_source_ref].font = SOURCE_FONT 
T8_PC[stp_source_ref] = Source
T8_PC[stp_source_ref].font = SOURCE_FONT 

T8_DEM[stp_source_ref] = Source
T8_DEM[stp_source_ref].font = SOURCE_FONT 
T8_DEP[stp_source_ref] = Source
T8_DEP[stp_source_ref].font = SOURCE_FONT
T8_EP[stp_source_ref] = Source
T8_EP[stp_source_ref].font = SOURCE_FONT
T8_LD[stp_source_ref] = Source
T8_LD[stp_source_ref].font = SOURCE_FONT
T8_MH[stp_source_ref] = Source
T8_MH[stp_source_ref].font = SOURCE_FONT

T8_OST[stp_source_ref] = Source
T8_OST[stp_source_ref].font = SOURCE_FONT 
T8_RA[stp_source_ref] = Source
T8_RA[stp_source_ref].font = SOURCE_FONT

T8_CS[stp_source_ref] = Source
T8_CS[stp_source_ref].font = SOURCE_FONT

T8_QI_1[stp_source_ref] = Source
T8_QI_1[stp_source_ref].font = SOURCE_FONT
T8_QI_2[stp_source_ref] = Source
T8_QI_2[stp_source_ref].font = SOURCE_FONT
T8_QI_3[stp_source_ref] = Source
T8_QI_3[stp_source_ref].font = SOURCE_FONT

T8_VI[stp_source_ref] = Source
T8_VI[stp_source_ref].font = SOURCE_FONT

In [ ]:
## Add line at the bottom of the table
## max row and col (range)
for row in T8_AF.iter_cols(min_row=int(stp_source_count)
                           ,max_col=42
                           ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER

for row in T8_BP.iter_cols(min_row=int(stp_source_count)
                           ,max_col=27
                           ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER

for row in T8_CHD.iter_cols(min_row=int(stp_source_count)
                            ,max_col=52
                            ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER

for row in T8_HF.iter_cols(min_row=int(stp_source_count)
                           ,max_col=58
                           ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER

for row in T8_HYP.iter_cols(min_row=int(stp_source_count)
                            ,max_col=44
                            ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER

for row in T8_PAD.iter_cols(min_row=int(stp_source_count)
                            ,max_col=19
                            ,max_row=int(stp_source_count)):
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_STIA.iter_cols(min_row=int(stp_source_count)
                             ,max_col=52
                             ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_AST.iter_cols(min_row=int(stp_source_count),
                            max_col=51,
                            max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_COPD.iter_cols(min_row=int(stp_source_count),
                             max_col=42
                             ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_OB.iter_cols(min_row=int(stp_source_count)
                           ,max_col=19
                           ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_SMOK.iter_cols(min_row=int(stp_source_count)
                             ,max_col=43
                             ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_CAN.iter_cols(min_row=int(stp_source_count)
                            ,max_col=42
                            ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_CKD.iter_cols(min_row=int(stp_source_count)
                            ,max_col=19
                            ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER

for row in T8_DM.iter_cols(min_row=int(stp_source_count)
                           ,max_col=91
                           ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_NDH.iter_cols(min_row=int(stp_source_count)
                            ,max_col=31
                            ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_PC.iter_cols(min_row=int(stp_source_count)
                           ,max_col=19
                           ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_DEM.iter_cols(min_row=int(stp_source_count)
                            ,max_col=34
                            ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_DEP.iter_cols(min_row=int(stp_source_count)
                            ,max_col=32
                            ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_EP.iter_cols(min_row=int(stp_source_count)
                           ,max_col=19
                           ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_LD.iter_cols(min_row=int(stp_source_count)
                           ,max_col=19
                           ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_MH.iter_cols(min_row=int(stp_source_count)
                           ,max_col=74
                           ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_OST.iter_cols(min_row=int(stp_source_count)
                            ,max_col=19
                            ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_RA.iter_cols(min_row=int(stp_source_count)
                           ,max_col=34
                           ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_CS.iter_cols(min_row=int(stp_source_count)
                           ,max_col=36
                           ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_VI.iter_cols(min_row=int(stp_source_count)
                           ,max_col=53
                           ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_QI_1.iter_cols(min_row=int(stp_source_count)
                             ,max_col=11
                             ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_QI_2.iter_cols(min_row=int(stp_source_count)
                             ,max_col=11
                             ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T8_QI_3.iter_cols(min_row=int(stp_source_count)
                             ,max_col=42
                             ,max_row=int(stp_source_count)): 
   for cell in row:
       cell.border = T_BORDER

#### Populate ICB (STP) table titles and message to worksheets and format

In [ ]:
T8_AF_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="AF"].reset_index(drop = True)
T8_AF["A8"] = T8_AF_TITLE.at[0, "TABLE_TITLE"]
T8_AF["A9"] = message_af
T8_AF["A9"].font = MESSAGE_FONT

T8_BP_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="BP"].reset_index(drop = True)
T8_BP["A8"] = T8_BP_TITLE.at[0, "TABLE_TITLE"]
T8_BP["A9"] = message_bp
T8_BP["A9"].font = MESSAGE_FONT

T8_CHD_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="CHD"].reset_index(drop = True)
T8_CHD["A8"] = T8_CHD_TITLE.at[0, "TABLE_TITLE"]
T8_CHD["A9"] = message_chd
T8_CHD["A9"].font = MESSAGE_FONT

T8_HF_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="HF"].reset_index(drop = True)
T8_HF["A8"] = T8_HF_TITLE.at[0, "TABLE_TITLE"]
T8_HF["A9"] = message_hf
T8_HF["A9"].font = MESSAGE_FONT

T8_HYP_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="HYP"].reset_index(drop = True)
T8_HYP["A8"] = T8_HYP_TITLE.at[0, "TABLE_TITLE"]
T8_HYP["A9"] = message_hyp
T8_HYP["A9"].font = MESSAGE_FONT

T8_PAD_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="PAD"].reset_index(drop = True)
T8_PAD["A8"] = T8_PAD_TITLE.at[0, "TABLE_TITLE"]
T8_PAD["A9"] = message_pad
T8_PAD["A9"].font = MESSAGE_FONT

T8_STIA_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="STIA"].reset_index(drop = True)
T8_STIA["A8"] = T8_STIA_TITLE.at[0, "TABLE_TITLE"]
T8_STIA["A9"] = message_stia
T8_STIA["A9"].font = MESSAGE_FONT

T8_AST_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="AST"].reset_index(drop = True)
T8_AST["A8"] = T8_AST_TITLE.at[0, "TABLE_TITLE"]
T8_AST["A9"] = message_ast
T8_AST["A9"].font = MESSAGE_FONT

T8_COPD_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="COPD"].reset_index(drop = True)
T8_COPD["A8"] = T8_COPD_TITLE.at[0, "TABLE_TITLE"]
T8_COPD["A9"] = message_copd
T8_COPD["A9"].font = MESSAGE_FONT

T8_OB_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="OB"].reset_index(drop = True)
T8_OB["A8"] = T8_OB_TITLE.at[0, "TABLE_TITLE"]
T8_OB["A9"] = message_ob
T8_OB["A9"].font = MESSAGE_FONT

T8_SMOK_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="SMOK"].reset_index(drop = True)
T8_SMOK["A8"] = T8_SMOK_TITLE.at[0, "TABLE_TITLE"]
T8_SMOK["A9"] = message_smok
T8_SMOK["A9"].font = MESSAGE_FONT

T8_CAN_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="CAN"].reset_index(drop = True)
T8_CAN["A8"] = T8_CAN_TITLE.at[0, "TABLE_TITLE"]
T8_CAN["A9"] = message_can
T8_CAN["A9"].font = MESSAGE_FONT

T8_CKD_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="CKD"].reset_index(drop = True)
T8_CKD["A8"] = T8_CKD_TITLE.at[0, "TABLE_TITLE"]
T8_CKD["A9"] = message_ckd
T8_CKD["A9"].font = MESSAGE_FONT

T8_DM_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="DM"].reset_index(drop = True)
T8_DM["A8"] = T8_DM_TITLE.at[0, "TABLE_TITLE"]
T8_DM["A9"] = message_dm
T8_DM["A9"].font = MESSAGE_FONT

T8_NDH_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="NDH"].reset_index(drop = True)
T8_NDH["A8"] = T8_NDH_TITLE.at[0, "TABLE_TITLE"]
T8_NDH["A9"] = message_ndh
T8_NDH["A9"].font = MESSAGE_FONT

T8_PC_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="PC"].reset_index(drop = True)
T8_PC["A8"] = T8_PC_TITLE.at[0, "TABLE_TITLE"]
T8_PC["A9"] = message_pc
T8_PC["A9"].font = MESSAGE_FONT

T8_DEM_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="DEM"].reset_index(drop = True)
T8_DEM["A8"] = T8_DEM_TITLE.at[0, "TABLE_TITLE"]
T8_DEM["A9"] = message_dem
T8_DEM["A9"].font = MESSAGE_FONT

T8_DEP_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="DEP"].reset_index(drop = True)
T8_DEP["A8"] = T8_DEP_TITLE.at[0, "TABLE_TITLE"]
T8_DEP["A9"] = message_dep
T8_DEP["A9"].font = MESSAGE_FONT

T8_EP_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="EP"].reset_index(drop = True)
T8_EP["A8"] = T8_EP_TITLE.at[0, "TABLE_TITLE"]
T8_EP["A9"] = message_ep
T8_EP["A9"].font = MESSAGE_FONT

T8_LD_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="LD"].reset_index(drop = True)
T8_LD["A8"] = T8_LD_TITLE.at[0, "TABLE_TITLE"]
T8_LD["A9"] = message_ld
T8_LD["A9"].font = MESSAGE_FONT

T8_MH_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="MH"].reset_index(drop = True)
T8_MH["A8"] = T8_MH_TITLE.at[0, "TABLE_TITLE"]
T8_MH["A9"] = message_mh
T8_MH["A9"].font = MESSAGE_FONT

T8_OST_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="OST"].reset_index(drop = True)
T8_OST["A8"] = T8_OST_TITLE.at[0, "TABLE_TITLE"]
T8_OST["A9"] = message_ost
T8_OST["A9"].font = MESSAGE_FONT

T8_RA_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="RA"].reset_index(drop = True)
T8_RA["A8"] = T8_RA_TITLE.at[0, "TABLE_TITLE"]
T8_RA["A9"] = message_ra
T8_RA["A9"].font = MESSAGE_FONT

T8_CS_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="CS"].reset_index(drop = True)
T8_CS["A8"] = T8_CS_TITLE.at[0, "TABLE_TITLE"]
T8_CS["A9"] = message_cs
T8_CS["A9"].font = MESSAGE_FONT

T8_VI_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="VI"].reset_index(drop = True)
T8_VI["A8"] = T8_VI_TITLE.at[0, "TABLE_TITLE"]
T8_VI["A9"] = message_vi
T8_VI["A9"].font = MESSAGE_FONT

T8_QI_1_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="QI_1"].reset_index(drop = True)
T8_QI_1["A8"] = T8_QI_1_TITLE.at[0, "TABLE_TITLE"]
T8_QI_1["A9"] = message_qi1
T8_QI_1["A9"].font = MESSAGE_FONT

T8_QI_2_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="QI_2"].reset_index(drop = True)
T8_QI_2["A8"] = T8_QI_2_TITLE.at[0, "TABLE_TITLE"]
T8_QI_2["A9"] = message_qi2
T8_QI_2["A9"].font = MESSAGE_FONT

T8_QI_3_TITLE = stp_table_titles.loc[stp_table_titles["GROUP_CODE"]=="QI_3"].reset_index(drop = True)
T8_QI_3["A8"] = T8_QI_3_TITLE.at[0, "TABLE_TITLE"]
T8_QI_3["A9"] = message_qi3
T8_QI_3["A9"].font = MESSAGE_FONT

#### Populate Fyears in indicator group tables

In [ ]:
## Populate current and previous financial years to all indicator groups at icb (stp) level
T8_AF["D11"] = Previous_year
T8_AF["H11"] = Fyear
T8_AF["M11"] = Previous_year
T8_AF["O11"] = Fyear
T8_AF["R11"] = Previous_year
T8_AF["U11"] = Fyear

T8_BP["D11"] = Previous_year
T8_BP["F11"] = Fyear
T8_BP["H11"] = Previous_year
T8_BP["J11"] = Fyear
T8_BP["M11"] = Previous_year
T8_BP["P11"] = Fyear

T8_CHD["D11"] = Previous_year
T8_CHD["H11"] = Fyear
T8_CHD["O11"] = Previous_year
T8_CHD["Q11"] = Fyear
T8_CHD["T11"] = Previous_year
T8_CHD["W11"] = Fyear

T8_HF["D11"] = Previous_year
T8_HF["H11"] = Fyear
T8_HF["M11"] = Previous_year
T8_HF["O11"] = Fyear
T8_HF["R11"] = Previous_year
T8_HF["U11"] = Fyear

T8_HYP["D11"] = Previous_year
T8_HYP["H11"] = Fyear
T8_HYP["O11"] = Previous_year
T8_HYP["Q11"] = Fyear
T8_HYP["T11"] = Previous_year
T8_HYP["W11"] = Fyear

T8_PAD["D11"] = Previous_year
T8_PAD["H11"] = Fyear
T8_PAD["M11"] = Previous_year
T8_PAD["O11"] = Fyear

T8_STIA["D11"] = Previous_year
T8_STIA["H11"] = Fyear
T8_STIA["O11"] = Previous_year
T8_STIA["Q11"] = Fyear
T8_STIA["T11"] = Previous_year
T8_STIA["W11"] = Fyear

T8_AST["D11"] = Previous_year
T8_AST["H11"] = Fyear
T8_AST["N11"] = Previous_year
T8_AST["P11"] = Fyear
T8_AST["S11"] = Previous_year
T8_AST["V11"] = Fyear

T8_COPD["D11"] = Previous_year
T8_COPD["H11"] = Fyear
T8_COPD["M11"] = Previous_year
T8_COPD["O11"] = Fyear
T8_COPD["R11"] = Previous_year
T8_COPD["U11"] = Fyear

T8_OB["D11"] = Previous_year
T8_OB["H11"] = Fyear
T8_OB["M11"] = Previous_year
T8_OB["O11"] = Fyear

T8_SMOK["D11"] = Previous_year
T8_SMOK["F11"] = Fyear
T8_SMOK["H11"] = Previous_year
T8_SMOK["J11"] = Fyear
T8_SMOK["M11"] = Previous_year
T8_SMOK["P11"] = Fyear

T8_CAN["D11"] = Previous_year
T8_CAN["H11"] = Fyear
T8_CAN["M11"] = Previous_year
T8_CAN["O11"] = Fyear
T8_CAN["R11"] = Previous_year
T8_CAN["U11"] = Fyear

T8_CKD["D11"] = Previous_year
T8_CKD["H11"] = Fyear
T8_CKD["M11"] = Previous_year
T8_CKD["O11"] = Fyear

T8_DM["D11"] = Previous_year
T8_DM["H11"] = Fyear
T8_DM["N11"] = Previous_year
T8_DM["P11"] = Fyear
T8_DM["S11"] = Previous_year
T8_DM["V11"] = Fyear

T8_NDH["D11"] = Previous_year
T8_NDH["H11"] = Fyear
T8_NDH["L11"] = Previous_year
T8_NDH["N11"] = Fyear
T8_NDH["Q11"] = Previous_year
T8_NDH["T11"] = Fyear

T8_PC["D11"] = Previous_year
T8_PC["H11"] = Fyear
T8_PC["M11"] = Previous_year
T8_PC["O11"] = Fyear

T8_DEM["D11"] = Previous_year
T8_DEM["H11"] = Fyear
T8_DEM["M11"] = Previous_year
T8_DEM["O11"] = Fyear
T8_DEM["R11"] = Previous_year
T8_DEM["U11"] = Fyear

T8_DEP["D11"] = Previous_year
T8_DEP["H11"] = Fyear
T8_DEP["M11"] = Previous_year
T8_DEP["O11"] = Fyear
T8_DEP["R11"] = Previous_year
T8_DEP["U11"] = Fyear

T8_EP["D11"] = Previous_year
T8_EP["H11"] = Fyear
T8_EP["M11"] = Previous_year
T8_EP["O11"] = Fyear

T8_LD["D11"] = Previous_year
T8_LD["H11"] = Fyear
T8_LD["M11"] = Previous_year
T8_LD["O11"] = Fyear

T8_MH["D11"] = Previous_year
T8_MH["H11"] = Fyear
T8_MH["M11"] = Previous_year
T8_MH["O11"] = Fyear
T8_MH["R11"] = Previous_year
T8_MH["U11"] = Fyear

T8_OST["D11"] = Previous_year
T8_OST["H11"] = Fyear
T8_OST["M11"] = Previous_year
T8_OST["O11"] = Fyear

T8_RA["D11"] = Previous_year
T8_RA["H11"] = Fyear
T8_RA["M11"] = Previous_year
T8_RA["O11"] = Fyear
T8_RA["R11"] = Previous_year
T8_RA["U11"] = Fyear

T8_CS["D11"] = Previous_year
T8_CS["F11"] = Fyear
T8_CS["I11"] = Previous_year
T8_CS["K11"] = Fyear
T8_CS["N11"] = Previous_year
T8_CS["Q11"] = Fyear

T8_VI["D11"] = Previous_year
T8_VI["E11"] = Fyear
T8_VI["J11"] = Previous_year
T8_VI["L11"] = Fyear
T8_VI["O11"] = Previous_year
T8_VI["R11"] = Fyear

T8_QI_1["D11"] = Fyear
T8_QI_1["F11"] = Fyear

T8_QI_2["D11"] = Fyear
T8_QI_2["F11"] = Fyear

T8_QI_3["D11"] = Fyear
T8_QI_3["F11"] = Fyear
T8_QI_3["H11"] = Fyear

#### Populate group points in table headers

In [ ]:
## Populate indicator group points for the current year in table headers
T8_AF["O12"] = "Total Achievement Score "+group_points_af
T8_BP["J12"] = "Total Achievement Score "+group_points_bp
T8_CHD["Q12"] ="Total Achievement Score "+ group_points_chd
T8_HF["O12"] = "Total Achievement Score "+group_points_hf
T8_HYP["Q12"] = "Total Achievement Score "+group_points_hyp
T8_PAD["O12"] = "Total Achievement Score "+group_points_pad
T8_STIA["Q12"] = "Total Achievement Score "+group_points_stia
T8_AST["P12"] = "Total Achievement Score "+group_points_ast
T8_COPD["O12"] = "Total Achievement Score "+group_points_copd
T8_OB["O12"] = "Total Achievement Score "+group_points_ob
T8_SMOK["J12"] = "Total Achievement Score "+group_points_smok
T8_CAN["O12"] = "Total Achievement Score "+group_points_can
T8_CKD["O12"] = "Total Achievement Score "+group_points_ckd
T8_DM["P12"] = "Total Achievement Score "+group_points_dm
T8_NDH["N12"] = "Total Achievement Score "+group_points_ndh
T8_PC["O12"] = "Total Achievement Score "+group_points_pc
T8_DEM["O12"] = "Total Achievement Score "+group_points_dem
T8_DEP["O12"] = "Total Achievement Score "+group_points_dep
T8_EP["O12"] = "Total Achievement Score "+group_points_ep
T8_LD["O12"] = "Total Achievement Score "+group_points_ld
T8_MH["O12"] = "Total Achievement Score "+group_points_mh
T8_OST["O12"] = "Total Achievement Score "+group_points_ost
T8_RA["O12"] = "Total Achievement Score "+group_points_ra
T8_CS["K12"] = "Total Achievement Score "+group_points_cs
T8_VI["L12"] = "Total Achievement Score "+group_points_vi
T8_QI_1["F12"] = "Total Achievement Score "+group_points_qi1
T8_QI_2["F12"] = "Total Achievement Score "+group_points_qi2
T8_QI_3["F12"] = "Total Achievement Score "+group_points_qi3

## Populate indicator group points for the previous year in table headers
T8_AF["M12"] = "Total Achievement Score "+previous_gr_points_af
T8_BP["H12"] = "Total Achievement Score "+previous_gr_points_bp
T8_CHD["O12"] = "Total Achievement Score "+previous_gr_points_chd
T8_HF["M12"] = "Total Achievement Score "+previous_gr_points_hf
T8_HYP["O12"] = "Total Achievement Score "+previous_gr_points_hyp
T8_PAD["M12"] = "Total Achievement Score "+previous_gr_points_pad
T8_STIA["O12"] = "Total Achievement Score "+previous_gr_points_stia
T8_AST["N12"] = "Total Achievement Score "+previous_gr_points_ast
T8_COPD["M12"] = "Total Achievement Score "+previous_gr_points_copd
T8_OB["M12"] = "Total Achievement Score "+previous_gr_points_ob
T8_SMOK["H12"] = "Total Achievement Score "+previous_gr_points_smok
T8_CAN["M12"] = "Total Achievement Score "+previous_gr_points_can
T8_CKD["M12"] = "Total Achievement Score "+previous_gr_points_ckd
T8_DM["N12"] = "Total Achievement Score "+previous_gr_points_dm
T8_NDH["L12"] = "Total Achievement Score "+previous_gr_points_ndh
T8_PC["M12"] = "Total Achievement Score "+previous_gr_points_pc
T8_DEM["M12"] = "Total Achievement Score "+previous_gr_points_dem
T8_DEP["M12"] = "Total Achievement Score "+previous_gr_points_dep
T8_EP["M12"] = "Total Achievement Score "+previous_gr_points_ep
T8_LD["M12"] = "Total Achievement Score "+previous_gr_points_ld
T8_MH["M12"] = "Total Achievement Score "+previous_gr_points_mh
T8_OST["M12"] = "Total Achievement Score "+previous_gr_points_ost
T8_RA["M12"] = "Total Achievement Score "+previous_gr_points_ra
T8_CS["I12"] = "Total Achievement Score "+previous_gr_points_cs
T8_VI["J12"] = "Total Achievement Score "+previous_gr_points_vi

#### Populate indicator points in table headers

In [ ]:
## Populate indicator points for the current year in table headers
T8_AF["Y12"] = "Achievement Score "+ind_points_af001
T8_AF["AA12"] = "Achievement Score "+ind_points_af006
T8_AF["AI12"] = "Achievement Score "+ind_points_af007

T8_BP["T12"] = "Achievement Score "+ind_points_bp002

T8_CHD["AA12"] = "Achievement Score "+ind_points_chd001
T8_CHD["AC12"] = "Achievement Score "+ind_points_chd005
T8_CHD["AK12"] = "Achievement Score "+ind_points_chd008
T8_CHD["AS12"] = "Achievement Score "+ind_points_chd009

T8_HF["Y12"] = "Achievement Score "+ind_points_hf001
T8_HF["AA12"] = "Achievement Score "+ind_points_hf003
T8_HF["AI12"] = "Achievement Score "+ind_points_hf005
T8_HF["AQ12"] = "Achievement Score "+ind_points_hf006
T8_HF["AY12"] = "Achievement Score "+ind_points_hf007

T8_HYP["AA12"] = "Achievement Score "+ind_points_hyp001
T8_HYP["AC12"] = "Achievement Score "+ind_points_hyp003
T8_HYP["AK12"] = "Achievement Score "+ind_points_hyp007

T8_PAD["R12"] = "Achievement Score "+ind_points_pad001

T8_STIA["AA12"] = "Achievement Score "+ind_points_stia001
T8_STIA["AC12"] = "Achievement Score "+ind_points_stia007
T8_STIA["AK12"] = "Achievement Score "+ind_points_stia010
T8_STIA["AS12"] = "Achievement Score "+ind_points_stia011

T8_AST["Z12"] = "Achievement Score "+ind_points_ast005
T8_AST["AB12"] = "Achievement Score "+ind_points_ast006
T8_AST["AJ12"] = "Achievement Score "+ind_points_ast007
T8_AST["AR12"] = "Achievement Score "+ind_points_ast008

T8_COPD["Y12"] = "Achievement Score "+ind_points_copd009
T8_COPD["AA12"] = "Achievement Score "+ind_points_copd008
T8_COPD["AI12"] = "Achievement Score "+ind_points_copd010

T8_OB["R12"] = "Achievement Score "+ind_points_ob002

T8_SMOK["T12"] = "Achievement Score "+ind_points_smok002
T8_SMOK["AB12"] = "Achievement Score "+ind_points_smok004
T8_SMOK["AJ12"] = "Achievement Score "+ind_points_smok005

T8_CAN["Y12"] = "Achievement Score "+ind_points_can001
T8_CAN["AA12"] = "Achievement Score "+ind_points_can004
T8_CAN["AI12"] = "Achievement Score "+ind_points_can005

T8_CKD["R12"] = "Achievement Score "+ind_points_ckd005

T8_DM["Z12"] = "Achievement Score "+ind_points_dm017
T8_DM["AB12"] = "Achievement Score "+ind_points_dm006
T8_DM["AJ12"] = "Achievement Score "+ind_points_dm012
T8_DM["AR12"] = "Achievement Score "+ind_points_dm014
T8_DM["AZ12"] = "Achievement Score "+ind_points_dm019
T8_DM["BH12"] = "Achievement Score "+ind_points_dm020
T8_DM["BP12"] = "Achievement Score "+ind_points_dm021
T8_DM["BX12"] = "Achievement Score "+ind_points_dm022
T8_DM["CF12"] = "Achievement Score "+ind_points_dm023

T8_NDH["X12"] = "Achievement Score "+ind_points_ndh001

T8_PC["R12"] = "Achievement Score "+ind_points_pc001

T8_DEM["Y12"] = "Achievement Score "+ind_points_dem001
T8_DEM["AA12"] = "Achievement Score "+ind_points_dem004

T8_DEP["Y12"] = "Achievement Score "+ind_points_dep003

T8_EP["R12"] = "Achievement Score "+ind_points_ep001

T8_LD["R12"] = "Achievement Score "+ind_points_ld004

T8_MH["Y12"] = "Achievement Score "+ind_points_mh001
T8_MH["AA12"] = "Achievement Score "+ind_points_mh002
T8_MH["AI12"] = "Achievement Score "+ind_points_mh003
T8_MH["AQ12"] = "Achievement Score "+ind_points_mh006
T8_MH["AY12"] = "Achievement Score "+ind_points_mh007
T8_MH["BG12"] = "Achievement Score "+ind_points_mh011
T8_MH["BO12"] = "Achievement Score "+ind_points_mh012

T8_OST["R12"] = "Achievement Score "+ind_points_ost004

T8_RA["Y12"] = "Achievement Score "+ind_points_ra001
T8_RA["AA12"] = "Achievement Score "+ind_points_ra002

T8_CS["U12"] = "Achievement Score "+ind_points_cs005
T8_CS["AC12"] = "Achievement Score "+ind_points_cs006

T8_VI["V12"] = "Achievement Score "+ind_points_vi001
T8_VI["AD12"] = "Achievement Score "+ind_points_vi002
T8_VI["AL12"] = "Achievement Score "+ind_points_vi003
T8_VI["AT12"] = "Achievement Score "+ind_points_vi004

T8_QI_1["H12"] = "Achievement Score "+ind_points_qipdd009
T8_QI_1["J12"] = "Achievement Score "+ind_points_qipdd010

T8_QI_2["H12"] = "Achievement Score "+ind_points_qioa011
T8_QI_2["J12"] = "Achievement Score "+ind_points_qioa012

T8_QI_3["K12"] = "Achievement Score "+ind_points_qipddmi001
T8_QI_3["S12"] = "Achievement Score "+ind_points_qipddmi002
T8_QI_3["AA12"] = "Achievement Score "+ind_points_qipddmi003
T8_QI_3["AI12"] = "Achievement Score "+ind_points_qipddmi004

<h2>Save Workbook

In [ ]:
## Save workbook
if test_run is True:
    wb.save(f"test_folder\\qof-"+ short_year +"-icb-ach-prev-pca.xlsx")
elif test_run is False:
    wb.save(outputs+"\qof-"+ short_year +"-icb-ach-prev-pca.xlsx")

<h2>Write data to saved excel workbook

In [ ]:
## Write data to saved workbook
if test_run is True:
    writer = pd.ExcelWriter(f"test_folder\\qof-"+ short_year +"-icb-ach-prev-pca.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

elif test_run is False:
    writer = pd.ExcelWriter(outputs+"\qof-"+ short_year +"-icb-ach-prev-pca.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)


<h4>Populate worksheets with data</h4><h7>Writes data to saved workbook template by sheet name - startrow includes the header, consequently the data starts at row 13 startrow = 12

In [ ]:
t8_af_df.to_excel(writer, sheet_name='AF',index=False, header = False,startrow=12)
t8_bp_df.to_excel(writer, sheet_name='BP',index=False, header = False,startrow=12)
t8_chd_df.to_excel(writer, sheet_name= 'CHD',index=False, header = False,startrow=12)
t8_hf_df.to_excel(writer, sheet_name='HF',index=False, header = False,startrow=12)
t8_hyp_df.to_excel(writer, sheet_name='HYP',index=False, header = False,startrow=12)
t8_pad_df.to_excel(writer, sheet_name='PAD',index=False, header = False,startrow=12)
t8_stia_df.to_excel(writer, sheet_name='STIA',index=False, header = False,startrow=12)
t8_ast_df.to_excel(writer, sheet_name='AST',index=False, header = False,startrow=12)
t8_copd_df.to_excel(writer, sheet_name='COPD',index=False, header = False,startrow=12)
t8_ob_df.to_excel(writer, sheet_name='OB',index=False, header = False,startrow=12)
t8_smok_df.to_excel(writer, sheet_name='SMOK',index=False, header = False,startrow=12)
t8_can_df.to_excel(writer, sheet_name='CAN',index=False, header = False,startrow=12)
t8_ckd_df.to_excel(writer, sheet_name='CKD',index=False, header = False,startrow=12)
t8_dm_df.to_excel(writer, sheet_name='DM',index=False, header = False,startrow=12)
t8_pc_df.to_excel(writer, sheet_name='PC',index=False, header = False,startrow=12)
t8_dem_df.to_excel(writer, sheet_name='DEM',index=False, header = False,startrow=12)
t8_dep_df.to_excel(writer, sheet_name='DEP',index=False, header = False,startrow=12)
t8_ep_df.to_excel(writer, sheet_name='EP',index=False, header = False,startrow=12)
t8_ld_df.to_excel(writer, sheet_name='LD',index=False, header = False,startrow=12)
t8_mh_df.to_excel(writer, sheet_name='MH',index=False, header = False,startrow=12)
t8_ost_df.to_excel(writer, sheet_name='OST',index=False, header = False,startrow=12)
t8_ra_df.to_excel(writer, sheet_name='RA',index=False, header = False,startrow=12)
t8_ndh_df.to_excel(writer, sheet_name='NDH',index=False, header = False,startrow=12)
t8_cs_df.to_excel(writer, sheet_name='CS',index=False, header = False,startrow=12)
t8_qi1_df.to_excel(writer, sheet_name='QIPDD',index=False, header = False,startrow=12)
t8_qi2_df.to_excel(writer, sheet_name='QIOA',index=False, header = False,startrow=12)
t8_qi3_df.to_excel(writer, sheet_name='QIPDDMI',index=False, header = False,startrow=12)
t8_vi_df.to_excel(writer, sheet_name='VI',index=False, header = False,startrow=12)

In [ ]:
## Saves data written to excel workbook
writer.save()

## Template 9 Sub ICB (CCG) Excel table
[Return to contents](#Table-of-Contents)

In [ ]:
## Open excel workbook template to be populated
## TEMPLATE-9-qof-YYYY-sicbl-ach-prev-pca.xlsx
wb = openpyxl.load_workbook(filename = templates +"\TEMPLATE-9-qof-YYYY-sicbl-ach-prev-pca.xlsx")

#### Name the worksheets in the TEMPLATE workbook to be used

In [ ]:
## Add the pseudonyms for worksheets in the TEMPLATE workbook to be used
T9_TITLE = wb["Title sheet"]
T9_AF = wb["AF"]
T9_BP = wb["BP"]
T9_CHD = wb["CHD"]
T9_HF = wb["HF"]
T9_HYP = wb["HYP"]
T9_PAD = wb["PAD"]
T9_STIA = wb["STIA"]
T9_AST = wb["AST"]
T9_COPD = wb["COPD"]
T9_OB = wb["OB"]
T9_SMOK = wb["SMOK"]
T9_CAN = wb["CAN"]
T9_CKD = wb["CKD"]
T9_DM = wb["DM"]
T9_PC = wb["PC"]
T9_DEM = wb["DEM"]
T9_DEP = wb["DEP"]
T9_EP = wb["EP"]
T9_LD = wb["LD"]
T9_MH = wb["MH"]
T9_OST = wb["OST"]
T9_RA = wb["RA"]
T9_NDH = wb["NDH"]
T9_CS = wb["CS"]
T9_QI_1 = wb["QIPDD"]
T9_QI_2 = wb["QIOA"]
T9_QI_3 = wb["QIPDDMI"]
T9_VI = wb["VI"]

#### Populate Publication title, Date and Link on contents page and format

In [ ]:
T9_TITLE["A7"] = QOF_title
T9_TITLE["A7"].font = NAME_FONT
T9_TITLE["B10"] = Publication_date
T9_TITLE["B10"].font = DATE_FONT
T9_TITLE["B11"].hyperlink = URL
T9_TITLE["B11"].style = "Hyperlink"
T9_TITLE["B11"].font = LINK_FONT
T9_TITLE["A68"] = RS_name
T9_TITLE["A68"].font = DATE_FONT

#### Populate copyright for worksheets and format

In [ ]:
T9_TITLE["A72"] = Copyright
T9_TITLE["A72"].font = DATE_FONT

T9_AF[ccg_count_pub] = Copyright
T9_AF[ccg_count_pub].font = C_FONT

T9_BP[ccg_count_pub] = Copyright
T9_BP[ccg_count_pub].font = C_FONT

T9_CHD[ccg_count_pub] = Copyright
T9_CHD[ccg_count_pub].font = C_FONT

T9_HF[ccg_count_pub] = Copyright
T9_HF[ccg_count_pub].font = C_FONT

T9_HYP[ccg_count_pub] = Copyright
T9_HYP[ccg_count_pub].font = C_FONT

T9_PAD[ccg_count_pub] = Copyright
T9_PAD[ccg_count_pub].font = C_FONT 

T9_STIA[ccg_count_pub] = Copyright
T9_STIA[ccg_count_pub].font = C_FONT 

T9_AST[ccg_count_pub] = Copyright
T9_AST[ccg_count_pub].font = C_FONT

T9_COPD[ccg_count_pub] = Copyright
T9_COPD[ccg_count_pub].font = C_FONT

T9_OB[ccg_count_pub] = Copyright
T9_OB[ccg_count_pub].font = C_FONT

T9_SMOK[ccg_count_pub] = Copyright
T9_SMOK[ccg_count_pub].font = C_FONT 

T9_CAN[ccg_count_pub] = Copyright
T9_CAN[ccg_count_pub].font = C_FONT

T9_CKD[ccg_count_pub] = Copyright
T9_CKD[ccg_count_pub].font = C_FONT 

T9_DM[ccg_count_pub] = Copyright
T9_DM[ccg_count_pub].font = C_FONT 

T9_PC[ccg_count_pub] = Copyright
T9_PC[ccg_count_pub].font = C_FONT 

T9_DEM[ccg_count_pub] = Copyright
T9_DEM[ccg_count_pub].font = C_FONT 

T9_DEP[ccg_count_pub] = Copyright
T9_DEP[ccg_count_pub].font = C_FONT

T9_EP[ccg_count_pub] = Copyright
T9_EP[ccg_count_pub].font = C_FONT

T9_LD[ccg_count_pub] = Copyright
T9_LD[ccg_count_pub].font = C_FONT

T9_MH[ccg_count_pub] = Copyright
T9_MH[ccg_count_pub].font = C_FONT

T9_OST[ccg_count_pub] = Copyright
T9_OST[ccg_count_pub].font = C_FONT 

T9_RA[ccg_count_pub] = Copyright
T9_RA[ccg_count_pub].font = C_FONT

T9_NDH[ccg_count_pub] = Copyright 
T9_NDH[ccg_count_pub].font = C_FONT 

T9_CS[ccg_count_pub] = Copyright
T9_CS[ccg_count_pub].font = C_FONT

T9_QI_1[ccg_count_pub] = Copyright
T9_QI_1[ccg_count_pub].font = C_FONT

T9_QI_2[ccg_count_pub] = Copyright
T9_QI_2[ccg_count_pub].font = C_FONT

T9_QI_3[ccg_count_pub] = Copyright
T9_QI_3[ccg_count_pub].font = C_FONT

T9_VI[ccg_count_pub] = Copyright
T9_VI[ccg_count_pub].font = C_FONT

In [ ]:
## Add Source text (Source: Data source, NHS England) to worksheets and format
T9_AF[ccg_source_ref] = Source
T9_AF[ccg_source_ref].font = SOURCE_FONT
T9_BP[ccg_source_ref] = Source
T9_BP[ccg_source_ref].font = SOURCE_FONT
T9_CHD[ccg_source_ref] = Source
T9_CHD[ccg_source_ref].font = SOURCE_FONT
T9_HF[ccg_source_ref] = Source
T9_HF[ccg_source_ref].font = SOURCE_FONT
T9_HYP[ccg_source_ref] = Source
T9_HYP[ccg_source_ref].font = SOURCE_FONT
T9_PAD[ccg_source_ref] = Source
T9_PAD[ccg_source_ref].font = SOURCE_FONT 
T9_STIA[ccg_source_ref] = Source
T9_STIA[ccg_source_ref].font = SOURCE_FONT 

T9_AST[ccg_source_ref] = Source
T9_AST[ccg_source_ref].font = SOURCE_FONT
T9_COPD[ccg_source_ref] = Source
T9_COPD[ccg_source_ref].font = SOURCE_FONT

T9_OB[ccg_source_ref] = Source
T9_OB[ccg_source_ref].font = SOURCE_FONT
T9_SMOK[ccg_source_ref] = Source
T9_SMOK[ccg_source_ref].font = SOURCE_FONT 

T9_CAN[ccg_source_ref] = Source
T9_CAN[ccg_source_ref].font = SOURCE_FONT
T9_CKD[ccg_source_ref] = Source
T9_CKD[ccg_source_ref].font = SOURCE_FONT 
T9_DM[ccg_source_ref] = Source
T9_DM[ccg_source_ref].font = SOURCE_FONT 
T9_NDH[ccg_source_ref] = Source 
T9_NDH[ccg_source_ref].font = SOURCE_FONT
T9_PC[ccg_source_ref] = Source
T9_PC[ccg_source_ref].font = SOURCE_FONT

T9_DEM[ccg_source_ref] = Source
T9_DEM[ccg_source_ref].font = SOURCE_FONT 
T9_DEP[ccg_source_ref] = Source
T9_DEP[ccg_source_ref].font = SOURCE_FONT
T9_EP[ccg_source_ref] = Source
T9_EP[ccg_source_ref].font = SOURCE_FONT
T9_LD[ccg_source_ref] = Source
T9_LD[ccg_source_ref].font = SOURCE_FONT
T9_MH[ccg_source_ref] = Source
T9_MH[ccg_source_ref].font = SOURCE_FONT

T9_OST[ccg_source_ref] = Source
T9_OST[ccg_source_ref].font = SOURCE_FONT 
T9_RA[ccg_source_ref] = Source
T9_RA[ccg_source_ref].font = SOURCE_FONT
 
T9_CS[ccg_source_ref] = Source
T9_CS[ccg_source_ref].font = SOURCE_FONT

T9_QI_1[ccg_source_ref] = Source
T9_QI_1[ccg_source_ref].font = SOURCE_FONT
T9_QI_2[ccg_source_ref] = Source
T9_QI_2[ccg_source_ref].font = SOURCE_FONT
T9_QI_3[ccg_source_ref] = Source
T9_QI_3[ccg_source_ref].font = SOURCE_FONT

T9_VI[ccg_source_ref] = Source
T9_VI[ccg_source_ref].font = SOURCE_FONT

In [ ]:
## Add line at the bottom of the table
## max row and col (range)
for row in T9_AF.iter_cols(min_row=int(ccg_source_count)
                           ,max_col=42
                           ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER

for row in T9_BP.iter_cols(min_row=int(ccg_source_count)
                           ,max_col=27
                           ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER

for row in T9_CHD.iter_cols(min_row=int(ccg_source_count)
                            ,max_col=52
                            ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER

for row in T9_HF.iter_cols(min_row=int(ccg_source_count)
                           ,max_col=58
                           ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER

for row in T9_HYP.iter_cols(min_row=int(ccg_source_count)
                            ,max_col=44
                            ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER

for row in T9_PAD.iter_cols(min_row=int(ccg_source_count)
                            ,max_col=19
                            ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_STIA.iter_cols(min_row=int(ccg_source_count)
                             ,max_col=52
                             ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_AST.iter_cols(min_row=int(ccg_source_count)
                            ,max_col=51
                            ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_COPD.iter_cols(min_row=int(ccg_source_count)
                             ,max_col=42
                             ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_OB.iter_cols(min_row=int(ccg_source_count)
                           ,max_col=19
                           ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_SMOK.iter_cols(min_row=int(ccg_source_count)
                             ,max_col=43
                             ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_CAN.iter_cols(min_row=int(ccg_source_count)
                            ,max_col=42
                            ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_CKD.iter_cols(min_row=int(ccg_source_count)
                            ,max_col=19
                            ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER

for row in T9_DM.iter_cols(min_row=int(ccg_source_count)
                           ,max_col=91
                           ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_NDH.iter_cols(min_row=int(ccg_source_count)
                            ,max_col=31
                            ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_PC.iter_cols(min_row=int(ccg_source_count)
                           ,max_col=19
                           ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_DEM.iter_cols(min_row=int(ccg_source_count)
                            ,max_col=34
                            ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_DEP.iter_cols(min_row=int(ccg_source_count)
                            ,max_col=32
                            ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_EP.iter_cols(min_row=int(ccg_source_count)
                           ,max_col=19
                           ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_LD.iter_cols(min_row=int(ccg_source_count)
                           ,max_col=19
                           ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_MH.iter_cols(min_row=int(ccg_source_count)
                           ,max_col=74
                           ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_OST.iter_cols(min_row=int(ccg_source_count)
                            ,max_col=19
                            ,max_row=int(ccg_source_count)):
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_RA.iter_cols(min_row=int(ccg_source_count)
                           ,max_col=34
                           ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_CS.iter_cols(min_row=int(ccg_source_count)
                           ,max_col=36
                           ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_VI.iter_cols(min_row=int(ccg_source_count)
                           ,max_col=53
                           ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_QI_1.iter_cols(min_row=int(ccg_source_count)
                             ,max_col=11
                             ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_QI_2.iter_cols(min_row=int(ccg_source_count)
                             ,max_col=11
                             ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T9_QI_3.iter_cols(min_row=int(ccg_source_count)
                             ,max_col=42
                             ,max_row=int(ccg_source_count)): 
   for cell in row:
       cell.border = T_BORDER

#### Populate SUB ICB (CCG) table titles and message to worksheets and format

In [ ]:
T9_AF_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="AF"].reset_index(drop = True)
T9_AF["A8"] = T9_AF_TITLE.at[0, "TABLE_TITLE"]
T9_AF["A9"] = message_af
T9_AF["A9"].font = MESSAGE_FONT

T9_BP_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="BP"].reset_index(drop = True)
T9_BP["A8"] = T9_BP_TITLE.at[0, "TABLE_TITLE"]
T9_BP["A9"] = message_bp
T9_BP["A9"].font = MESSAGE_FONT

T9_CHD_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="CHD"].reset_index(drop = True)
T9_CHD["A8"] = T9_CHD_TITLE.at[0, "TABLE_TITLE"]
T9_CHD["A9"] = message_chd
T9_CHD["A9"].font = MESSAGE_FONT

T9_HF_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="HF"].reset_index(drop = True)
T9_HF["A8"] = T9_HF_TITLE.at[0, "TABLE_TITLE"]
T9_HF["A9"] = message_hf
T9_HF["A9"].font = MESSAGE_FONT

T9_HYP_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="HYP"].reset_index(drop = True)
T9_HYP["A8"] = T9_HYP_TITLE.at[0, "TABLE_TITLE"]
T9_HYP["A9"] = message_hyp
T9_HYP["A9"].font = MESSAGE_FONT

T9_PAD_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="PAD"].reset_index(drop = True)
T9_PAD["A8"] = T9_PAD_TITLE.at[0, "TABLE_TITLE"]
T9_PAD["A9"] = message_pad
T9_PAD["A9"].font = MESSAGE_FONT

T9_STIA_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="STIA"].reset_index(drop = True)
T9_STIA["A8"] = T9_STIA_TITLE.at[0, "TABLE_TITLE"]
T9_STIA["A9"] = message_stia
T9_STIA["A9"].font = MESSAGE_FONT

T9_AST_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="AST"].reset_index(drop = True)
T9_AST["A8"] = T9_AST_TITLE.at[0, "TABLE_TITLE"]
T9_AST["A9"] = message_ast
T9_AST["A9"].font = MESSAGE_FONT

T9_COPD_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="COPD"].reset_index(drop = True)
T9_COPD["A8"] = T9_COPD_TITLE.at[0, "TABLE_TITLE"]
T9_COPD["A9"] = message_copd
T9_COPD["A9"].font = MESSAGE_FONT

T9_OB_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="OB"].reset_index(drop = True)
T9_OB["A8"] = T9_OB_TITLE.at[0, "TABLE_TITLE"]
T9_OB["A9"] = message_ob
T9_OB["A9"].font = MESSAGE_FONT

T9_SMOK_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="SMOK"].reset_index(drop = True)
T9_SMOK["A8"] = T9_SMOK_TITLE.at[0, "TABLE_TITLE"]
T9_SMOK["A9"] = message_smok
T9_SMOK["A9"].font = MESSAGE_FONT

T9_CAN_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="CAN"].reset_index(drop = True)
T9_CAN["A8"] = T9_CAN_TITLE.at[0, "TABLE_TITLE"]
T9_CAN["A9"] = message_can
T9_CAN["A9"].font = MESSAGE_FONT

T9_CKD_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="CKD"].reset_index(drop = True)
T9_CKD["A8"] = T9_CKD_TITLE.at[0, "TABLE_TITLE"]
T9_CKD["A9"] = message_ckd
T9_CKD["A9"].font = MESSAGE_FONT

T9_DM_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="DM"].reset_index(drop = True)
T9_DM["A8"] = T9_DM_TITLE.at[0, "TABLE_TITLE"]
T9_DM["A9"] = message_dm
T9_DM["A9"].font = MESSAGE_FONT

T9_PC_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="PC"].reset_index(drop = True)
T9_PC["A8"] = T9_PC_TITLE.at[0, "TABLE_TITLE"]
T9_PC["A9"] = message_pc
T9_PC["A9"].font = MESSAGE_FONT

T9_DEM_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="DEM"].reset_index(drop = True)
T9_DEM["A8"] = T9_DEM_TITLE.at[0, "TABLE_TITLE"]
T9_DEM["A9"] = message_dem
T9_DEM["A9"].font = MESSAGE_FONT

T9_DEP_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="DEP"].reset_index(drop = True)
T9_DEP["A8"] = T9_DEP_TITLE.at[0, "TABLE_TITLE"]
T9_DEP["A9"] = message_dep
T9_DEP["A9"].font = MESSAGE_FONT

T9_EP_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="EP"].reset_index(drop = True)
T9_EP["A8"] = T9_EP_TITLE.at[0, "TABLE_TITLE"]
T9_EP["A9"] = message_ep
T9_EP["A9"].font = MESSAGE_FONT

T9_LD_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="LD"].reset_index(drop = True)
T9_LD["A8"] = T9_LD_TITLE.at[0, "TABLE_TITLE"]
T9_LD["A9"] = message_ld
T9_LD["A9"].font = MESSAGE_FONT

T9_MH_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="MH"].reset_index(drop = True)
T9_MH["A8"] = T9_MH_TITLE.at[0, "TABLE_TITLE"]
T9_MH["A9"] = message_mh
T9_MH["A9"].font = MESSAGE_FONT

T9_OST_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="OST"].reset_index(drop = True)
T9_OST["A8"] = T9_OST_TITLE.at[0, "TABLE_TITLE"]
T9_OST["A9"] = message_ost
T9_OST["A9"].font = MESSAGE_FONT

T9_RA_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="RA"].reset_index(drop = True)
T9_RA["A8"] = T9_RA_TITLE.at[0, "TABLE_TITLE"]
T9_RA["A9"] = message_ra
T9_RA["A9"].font = MESSAGE_FONT

T9_NDH_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="NDH"].reset_index(drop = True)
T9_NDH["A8"] = T9_NDH_TITLE.at[0, "TABLE_TITLE"]
T9_NDH["A9"] = message_ndh
T9_NDH["A9"].font = MESSAGE_FONT

T9_CS_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="CS"].reset_index(drop = True)
T9_CS["A8"] = T9_CS_TITLE.at[0, "TABLE_TITLE"]
T9_CS["A9"] = message_cs
T9_CS["A9"].font = MESSAGE_FONT

T9_QI_1_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="QI_1"].reset_index(drop = True)
T9_QI_1["A8"] = T9_QI_1_TITLE.at[0, "TABLE_TITLE"]
T9_QI_1["A9"] = message_qi1
T9_QI_1["A9"].font = MESSAGE_FONT

T9_QI_2_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="QI_2"].reset_index(drop = True)
T9_QI_2["A8"] = T9_QI_2_TITLE.at[0, "TABLE_TITLE"]
T9_QI_2["A9"] = message_qi2
T9_QI_2["A9"].font = MESSAGE_FONT

T9_QI_3_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="QI_3"].reset_index(drop = True)
T9_QI_3["A8"] = T9_QI_3_TITLE.at[0, "TABLE_TITLE"]
T9_QI_3["A9"] = message_qi3
T9_QI_3["A9"].font = MESSAGE_FONT

T9_VI_TITLE = ccg_table_titles.loc[ccg_table_titles["GROUP_CODE"]=="VI"].reset_index(drop = True)
T9_VI["A8"] = T9_VI_TITLE.at[0, "TABLE_TITLE"]
T9_VI["A9"] = message_vi
T9_VI["A9"].font = MESSAGE_FONT

#### Populate Fyears in indicator group tables

In [ ]:
## Populate current and previous financial years to all indicator groups at sub icb (ccg) level
T9_AF["D11"] = Previous_year
T9_AF["H11"] = Fyear
T9_AF["M11"] = Previous_year
T9_AF["O11"] = Fyear
T9_AF["R11"] = Previous_year
T9_AF["U11"] = Fyear

T9_BP["D11"] = Previous_year
T9_BP["F11"] = Fyear
T9_BP["H11"] = Previous_year
T9_BP["J11"] = Fyear
T9_BP["M11"] = Previous_year
T9_BP["P11"] = Fyear

T9_CHD["D11"] = Previous_year
T9_CHD["H11"] = Fyear
T9_CHD["O11"] = Previous_year
T9_CHD["Q11"] = Fyear
T9_CHD["T11"] = Previous_year
T9_CHD["W11"] = Fyear

T9_HF["D11"] = Previous_year
T9_HF["H11"] = Fyear
T9_HF["M11"] = Previous_year
T9_HF["O11"] = Fyear
T9_HF["R11"] = Previous_year
T9_HF["U11"] = Fyear

T9_HYP["D11"] = Previous_year
T9_HYP["H11"] = Fyear
T9_HYP["O11"] = Previous_year
T9_HYP["Q11"] = Fyear
T9_HYP["T11"] = Previous_year
T9_HYP["W11"] = Fyear

T9_PAD["D11"] = Previous_year
T9_PAD["H11"] = Fyear
T9_PAD["M11"] = Previous_year
T9_PAD["O11"] = Fyear

T9_STIA["D11"] = Previous_year
T9_STIA["H11"] = Fyear
T9_STIA["O11"] = Previous_year
T9_STIA["Q11"] = Fyear
T9_STIA["T11"] = Previous_year
T9_STIA["W11"] = Fyear

T9_AST["D11"] = Previous_year
T9_AST["H11"] = Fyear
T9_AST["N11"] = Previous_year
T9_AST["P11"] = Fyear
T9_AST["S11"] = Previous_year
T9_AST["V11"] = Fyear

T9_COPD["D11"] = Previous_year
T9_COPD["H11"] = Fyear
T9_COPD["M11"] = Previous_year
T9_COPD["O11"] = Fyear
T9_COPD["R11"] = Previous_year
T9_COPD["U11"] = Fyear

T9_OB["D11"] = Previous_year
T9_OB["H11"] = Fyear
T9_OB["M11"] = Previous_year
T9_OB["O11"] = Fyear

T9_SMOK["D11"] = Previous_year
T9_SMOK["F11"] = Fyear
T9_SMOK["H11"] = Previous_year
T9_SMOK["J11"] = Fyear
T9_SMOK["M11"] = Previous_year
T9_SMOK["P11"] = Fyear

T9_CAN["D11"] = Previous_year
T9_CAN["H11"] = Fyear
T9_CAN["M11"] = Previous_year
T9_CAN["O11"] = Fyear
T9_CAN["R11"] = Previous_year
T9_CAN["U11"] = Fyear

T9_CKD["D11"] = Previous_year
T9_CKD["H11"] = Fyear
T9_CKD["M11"] = Previous_year
T9_CKD["O11"] = Fyear

T9_DM["D11"] = Previous_year
T9_DM["H11"] = Fyear
T9_DM["N11"] = Previous_year
T9_DM["P11"] = Fyear
T9_DM["S11"] = Previous_year
T9_DM["V11"] = Fyear

T9_PC["D11"] = Previous_year
T9_PC["H11"] = Fyear
T9_PC["M11"] = Previous_year
T9_PC["O11"] = Fyear

T9_DEM["D11"] = Previous_year
T9_DEM["H11"] = Fyear
T9_DEM["M11"] = Previous_year
T9_DEM["O11"] = Fyear
T9_DEM["R11"] = Previous_year
T9_DEM["U11"] = Fyear

T9_DEP["D11"] = Previous_year
T9_DEP["H11"] = Fyear
T9_DEP["M11"] = Previous_year
T9_DEP["O11"] = Fyear
T9_DEP["R11"] = Previous_year
T9_DEP["U11"] = Fyear

T9_EP["D11"] = Previous_year
T9_EP["H11"] = Fyear
T9_EP["M11"] = Previous_year
T9_EP["O11"] = Fyear

T9_LD["D11"] = Previous_year
T9_LD["H11"] = Fyear
T9_LD["M11"] = Previous_year
T9_LD["O11"] = Fyear

T9_MH["D11"] = Previous_year
T9_MH["H11"] = Fyear
T9_MH["M11"] = Previous_year
T9_MH["O11"] = Fyear
T9_MH["R11"] = Previous_year
T9_MH["U11"] = Fyear

T9_OST["D11"] = Previous_year
T9_OST["H11"] = Fyear
T9_OST["M11"] = Previous_year
T9_OST["O11"] = Fyear

T9_RA["D11"] = Previous_year
T9_RA["H11"] = Fyear
T9_RA["M11"] = Previous_year
T9_RA["O11"] = Fyear
T9_RA["R11"] = Previous_year
T9_RA["U11"] = Fyear

T9_NDH["D11"] = Previous_year
T9_NDH["H11"] = Fyear
T9_NDH["L11"] = Previous_year
T9_NDH["N11"] = Fyear
T9_NDH["Q11"] = Previous_year
T9_NDH["T11"] = Fyear

T9_CS["D11"] = Previous_year
T9_CS["F11"] = Fyear
T9_CS["I11"] = Previous_year
T9_CS["K11"] = Fyear
T9_CS["N11"] = Previous_year
T9_CS["Q11"] = Fyear

T9_VI["D11"] = Previous_year
T9_VI["E11"] = Fyear
T9_VI["J11"] = Previous_year
T9_VI["L11"] = Fyear
T9_VI["O11"] = Previous_year
T9_VI["R11"] = Fyear

T9_QI_1["D11"] = Fyear
T9_QI_1["F11"] = Fyear

T9_QI_2["D11"] = Fyear
T9_QI_2["F11"] = Fyear

T9_QI_3["D11"] = Fyear
T9_QI_3["F11"] = Fyear
T9_QI_3["H11"] = Fyear

#### Populate group points in table headers

In [ ]:
## Populate group points for the current year in table headers
T9_AF["O12"] = "Total Achievement Score "+group_points_af
T9_BP["J12"] = "Total Achievement Score "+group_points_bp
T9_CHD["Q12"] ="Total Achievement Score "+ group_points_chd
T9_HF["O12"] = "Total Achievement Score "+group_points_hf
T9_HYP["Q12"] = "Total Achievement Score "+group_points_hyp
T9_PAD["O12"] = "Total Achievement Score "+group_points_pad
T9_STIA["Q12"] = "Total Achievement Score "+group_points_stia
T9_AST["P12"] = "Total Achievement Score "+group_points_ast
T9_COPD["O12"] = "Total Achievement Score "+group_points_copd
T9_OB["O12"] = "Total Achievement Score "+group_points_ob
T9_SMOK["J12"] = "Total Achievement Score "+group_points_smok
T9_CAN["O12"] = "Total Achievement Score "+group_points_can
T9_CKD["O12"] = "Total Achievement Score "+group_points_ckd
T9_DM["P12"] = "Total Achievement Score "+group_points_dm
T9_PC["O12"] = "Total Achievement Score "+group_points_pc
T9_DEM["O12"] = "Total Achievement Score "+group_points_dem
T9_DEP["O12"] = "Total Achievement Score "+group_points_dep
T9_EP["O12"] = "Total Achievement Score "+group_points_ep
T9_LD["O12"] = "Total Achievement Score "+group_points_ld
T9_MH["O12"] = "Total Achievement Score "+group_points_mh
T9_OST["O12"] = "Total Achievement Score "+group_points_ost
T9_RA["O12"] = "Total Achievement Score "+group_points_ra
T9_NDH["N12"] = "Total Achievement Score "+group_points_ndh
T9_CS["K12"] = "Total Achievement Score "+group_points_cs
T9_QI_1["F12"] = "Total Achievement Score "+group_points_qi1
T9_QI_2["F12"] = "Total Achievement Score "+group_points_qi2
T9_QI_3["F12"] = "Total Achievement Score "+group_points_qi3
T9_VI["L12"] = "Total Achievement Score "+group_points_vi

## Populate group points for the previous year in table headers
T9_AF["M12"] = "Total Achievement Score "+previous_gr_points_af
T9_BP["H12"] = "Total Achievement Score "+previous_gr_points_bp
T9_CHD["O12"] = "Total Achievement Score "+previous_gr_points_chd
T9_HF["M12"] = "Total Achievement Score "+previous_gr_points_hf
T9_HYP["O12"] = "Total Achievement Score "+previous_gr_points_hyp
T9_PAD["M12"] = "Total Achievement Score "+previous_gr_points_pad
T9_STIA["O12"] = "Total Achievement Score "+previous_gr_points_stia
T9_AST["N12"] = "Total Achievement Score "+previous_gr_points_ast
T9_COPD["M12"] = "Total Achievement Score "+previous_gr_points_copd
T9_OB["M12"] = "Total Achievement Score "+previous_gr_points_ob
T9_SMOK["H12"] = "Total Achievement Score "+previous_gr_points_smok
T9_CAN["M12"] = "Total Achievement Score "+previous_gr_points_can
T9_CKD["M12"] = "Total Achievement Score "+previous_gr_points_ckd
T9_DM["N12"] = "Total Achievement Score "+previous_gr_points_dm
T9_PC["M12"] = "Total Achievement Score "+previous_gr_points_pc
T9_DEM["M12"] = "Total Achievement Score "+previous_gr_points_dem
T9_DEP["M12"] = "Total Achievement Score "+previous_gr_points_dep
T9_EP["M12"] = "Total Achievement Score "+previous_gr_points_ep
T9_LD["M12"] = "Total Achievement Score "+previous_gr_points_ld
T9_MH["M12"] = "Total Achievement Score "+previous_gr_points_mh
T9_OST["M12"] = "Total Achievement Score "+previous_gr_points_ost
T9_RA["M12"] = "Total Achievement Score "+previous_gr_points_ra
T9_NDH["L12"] = "Total Achievement Score "+previous_gr_points_ndh
T9_CS["I12"] = "Total Achievement Score "+previous_gr_points_cs
T9_VI["J12"] = "Total Achievement Score "+previous_gr_points_vi

#### Populate indicator points in table headers

In [ ]:
## Populate indicator points for current year in table headers
T9_AF["Y12"] = "Achievement Score "+ind_points_af001
T9_AF["AA12"] = "Achievement Score "+ind_points_af006
T9_AF["AI12"] = "Achievement Score "+ind_points_af007

T9_BP["T12"] = "Achievement Score "+ind_points_bp002

T9_CHD["AA12"] = "Achievement Score "+ind_points_chd001
T9_CHD["AC12"] = "Achievement Score "+ind_points_chd005
T9_CHD["AK12"] = "Achievement Score "+ind_points_chd008
T9_CHD["AS12"] = "Achievement Score "+ind_points_chd009

T9_HF["Y12"] = "Achievement Score "+ind_points_hf001
T9_HF["AA12"] = "Achievement Score "+ind_points_hf003
T9_HF["AI12"] = "Achievement Score "+ind_points_hf005
T9_HF["AQ12"] = "Achievement Score "+ind_points_hf006
T9_HF["AY12"] = "Achievement Score "+ind_points_hf007

T9_HYP["AA12"] = "Achievement Score "+ind_points_hyp001
T9_HYP["AC12"] = "Achievement Score "+ind_points_hyp003
T9_HYP["AK12"] = "Achievement Score "+ind_points_hyp007

T9_PAD["R12"] = "Achievement Score "+ind_points_pad001

T9_STIA["AA12"] = "Achievement Score "+ind_points_stia001
T9_STIA["AC12"] = "Achievement Score "+ind_points_stia007
T9_STIA["AK12"] = "Achievement Score "+ind_points_stia010
T9_STIA["AS12"] = "Achievement Score "+ind_points_stia011

T9_AST["Z12"] = "Achievement Score "+ind_points_ast005
T9_AST["AB12"] = "Achievement Score "+ind_points_ast006
T9_AST["AJ12"] = "Achievement Score "+ind_points_ast007
T9_AST["AR12"] = "Achievement Score "+ind_points_ast008

T9_COPD["Y12"] = "Achievement Score "+ind_points_copd009
T9_COPD["AA12"] = "Achievement Score "+ind_points_copd008
T9_COPD["AI12"] = "Achievement Score "+ind_points_copd010

T9_OB["R12"] = "Achievement Score "+ind_points_ob002

T9_SMOK["T12"] = "Achievement Score "+ind_points_smok002
T9_SMOK["AB12"] = "Achievement Score "+ind_points_smok004
T9_SMOK["AJ12"] = "Achievement Score "+ind_points_smok005

T9_CAN["Y12"] = "Achievement Score "+ind_points_can001
T9_CAN["AA12"] = "Achievement Score "+ind_points_can004
T9_CAN["AI12"] = "Achievement Score "+ind_points_can005

T9_CKD["R12"] = "Achievement Score "+ind_points_ckd005

T9_DM["Z12"] = "Achievement Score "+ind_points_dm017
T9_DM["AB12"] = "Achievement Score "+ind_points_dm006
T9_DM["AJ12"] = "Achievement Score "+ind_points_dm012
T9_DM["AR12"] = "Achievement Score "+ind_points_dm014
T9_DM["AZ12"] = "Achievement Score "+ind_points_dm019
T9_DM["BH12"] = "Achievement Score "+ind_points_dm020
T9_DM["BP12"] = "Achievement Score "+ind_points_dm021
T9_DM["BX12"] = "Achievement Score "+ind_points_dm022
T9_DM["CF12"] = "Achievement Score "+ind_points_dm023

T9_PC["R12"] = "Achievement Score "+ind_points_pc001

T9_DEM["Y12"] = "Achievement Score "+ind_points_dem001
T9_DEM["AA12"] = "Achievement Score "+ind_points_dem004

T9_DEP["Y12"] = "Achievement Score "+ind_points_dep003

T9_EP["R12"] = "Achievement Score "+ind_points_ep001

T9_LD["R12"] = "Achievement Score "+ind_points_ld004

T9_MH["Y12"] = "Achievement Score "+ind_points_mh001
T9_MH["AA12"] = "Achievement Score "+ind_points_mh002
T9_MH["AI12"] = "Achievement Score "+ind_points_mh003
T9_MH["AQ12"] = "Achievement Score "+ind_points_mh006
T9_MH["AY12"] = "Achievement Score "+ind_points_mh007
T9_MH["BG12"] = "Achievement Score "+ind_points_mh011
T9_MH["BO12"] = "Achievement Score "+ind_points_mh012

T9_OST["R12"] = "Achievement Score "+ind_points_ost004

T9_RA["Y12"] = "Achievement Score "+ind_points_ra001
T9_RA["AA12"] = "Achievement Score "+ind_points_ra002

T9_NDH["X12"] = "Achievement Score "+ind_points_ndh001

T9_CS["U12"] = "Achievement Score "+ind_points_cs005
T9_CS["AC12"] = "Achievement Score "+ind_points_cs006

T9_QI_1["H12"] = "Achievement Score "+ind_points_qipdd009
T9_QI_1["J12"] = "Achievement Score "+ind_points_qipdd010

T9_QI_2["H12"] = "Achievement Score "+ind_points_qioa011
T9_QI_2["J12"] = "Achievement Score "+ind_points_qioa012

T9_QI_3["K12"] = "Achievement Score "+ind_points_qipddmi001
T9_QI_3["S12"] = "Achievement Score "+ind_points_qipddmi002
T9_QI_3["AA12"] = "Achievement Score "+ind_points_qipddmi003
T9_QI_3["AI12"] = "Achievement Score "+ind_points_qipddmi004

T9_VI["V12"] = "Achievement Score "+ind_points_vi001
T9_VI["AD12"] = "Achievement Score "+ind_points_vi002
T9_VI["AL12"] = "Achievement Score "+ind_points_vi003
T9_VI["AT12"] = "Achievement Score "+ind_points_vi004

<h2>Save Workbook

In [ ]:
## Save workbook
if test_run is True:
    wb.save(f"test_folder\\qof-"+ short_year +"-sicbl-ach-prev-pca.xlsx")
elif test_run is False:
    wb.save(outputs+"\qof-"+ short_year +"-sicbl-ach-prev-pca.xlsx")

<h2>Write data to saved excel workbook

In [ ]:
## Write data to saved workbook
if test_run is True:
    writer = pd.ExcelWriter(f"test_folder\\qof-"+ short_year +"-sicbl-ach-prev-pca.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

elif test_run is False:
    writer = pd.ExcelWriter(outputs+ "\qof-"+ short_year +"-sicbl-ach-prev-pca.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

<h4>Populate worksheets with data</h4><h7>Writes data to saved workbook template by sheet name - startrow includes the header, consequently the data starts at row 13 startrow = 12

In [ ]:
t9_af_df.to_excel(writer, sheet_name='AF',index=False, header = False,startrow=12)
t9_bp_df.to_excel(writer, sheet_name='BP',index=False, header = False,startrow=12)
t9_chd_df.to_excel(writer, sheet_name= 'CHD',index=False, header = False,startrow=12)
t9_hf_df.to_excel(writer, sheet_name='HF',index=False, header = False,startrow=12)
t9_hyp_df.to_excel(writer, sheet_name='HYP',index=False, header = False,startrow=12)
t9_pad_df.to_excel(writer, sheet_name='PAD',index=False, header = False,startrow=12)
t9_stia_df.to_excel(writer, sheet_name='STIA',index=False, header = False,startrow=12)
t9_ast_df.to_excel(writer, sheet_name='AST',index=False, header = False,startrow=12)
t9_copd_df.to_excel(writer, sheet_name='COPD',index=False, header = False,startrow=12)
t9_ob_df.to_excel(writer, sheet_name='OB',index=False, header = False,startrow=12)
t9_smok_df.to_excel(writer, sheet_name='SMOK',index=False, header = False,startrow=12)
t9_can_df.to_excel(writer, sheet_name='CAN',index=False, header = False,startrow=12)
t9_ckd_df.to_excel(writer, sheet_name='CKD',index=False, header = False,startrow=12)
t9_dm_df.to_excel(writer, sheet_name='DM',index=False, header = False,startrow=12)
t9_pc_df.to_excel(writer, sheet_name='PC',index=False, header = False,startrow=12)
t9_dem_df.to_excel(writer, sheet_name='DEM',index=False, header = False,startrow=12)
t9_dep_df.to_excel(writer, sheet_name='DEP',index=False, header = False,startrow=12)
t9_ep_df.to_excel(writer, sheet_name='EP',index=False, header = False,startrow=12)
t9_ld_df.to_excel(writer, sheet_name='LD',index=False, header = False,startrow=12)
t9_mh_df.to_excel(writer, sheet_name='MH',index=False, header = False,startrow=12)
t9_ost_df.to_excel(writer, sheet_name='OST',index=False, header = False,startrow=12)
t9_ra_df.to_excel(writer, sheet_name='RA',index=False, header = False,startrow=12)
t9_ndh_df.to_excel(writer, sheet_name='NDH',index=False, header = False,startrow=12)
t9_cs_df.to_excel(writer, sheet_name='CS',index=False, header = False,startrow=12)
t9_qi1_df.to_excel(writer, sheet_name='QIPDD',index=False, header = False,startrow=12)
t9_qi2_df.to_excel(writer, sheet_name='QIOA',index=False, header = False,startrow=12)
t9_qi3_df.to_excel(writer, sheet_name='QIPDDMI',index=False, header = False,startrow=12)
t9_vi_df.to_excel(writer, sheet_name='VI',index=False, header = False,startrow=12)

In [ ]:
## Saves data written to excel workbook
writer.save()

<h1><b>Practice Excel tables</b></h2>

## Template 10 CV practice Excel table
[Return to contents](#Table-of-Contents)

In [ ]:
## Open excel workbook template to be populated
## TEMPLATE-10-qof-YYYY-prac-ach-prev-pca-cv.xlsx
wb = openpyxl.load_workbook(filename = templates +"\Practices\TEMPLATE-10-qof-YYYY-prac-ach-prev-pca-cv.xlsx")

#### Name the worksheets in the TEMPLATE workbook to be used

In [ ]:
## Add pseudonyms for worksheets in CV higher group here
T10_TITLE = wb["Title sheet"]
T10_AF = wb["AF"]
T10_BP = wb["BP"]
T10_CHD = wb["CHD"]
T10_HF = wb["HF"]
T10_HYP = wb["HYP"]
T10_LVSD = wb["LVSD"]
T10_PAD = wb["PAD"]
T10_STIA = wb["STIA"]

#### Populate Publication title, Date and Link on contents page and format

In [ ]:
T10_TITLE["A7"] = QOF_title
T10_TITLE["A7"].font = NAME_FONT
T10_TITLE["B11"] = Publication_date
T10_TITLE["B11"].font = DATE_FONT
T10_TITLE["B12"].hyperlink = URL
T10_TITLE["B12"].style = "Hyperlink"
T10_TITLE["B12"].font = LINK_FONT
T10_TITLE["A15"] = Notes
T10_TITLE["A15"].font = DATE_FONT
T10_TITLE["A35"] = RS_name
T10_TITLE["A35"].font = DATE_FONT

#### Populate copyright for worksheets and format

In [ ]:
T10_TITLE["A39"] = Copyright
T10_TITLE["A39"].font = DATE_FONT

T10_AF[prac_count_pub] = Copyright
T10_AF[prac_count_pub].font = C_FONT

T10_BP[prac_count_pub] = Copyright
T10_BP[prac_count_pub].font = C_FONT

T10_CHD[prac_count_pub] = Copyright
T10_CHD[prac_count_pub].font = C_FONT

T10_HF[prac_count_pub] = Copyright
T10_HF[prac_count_pub].font = C_FONT

T10_HYP[prac_count_pub] = Copyright
T10_HYP[prac_count_pub].font = C_FONT

T10_LVSD[prac_count_pub] = Copyright
T10_LVSD[prac_count_pub].font = C_FONT

T10_PAD[prac_count_pub] = Copyright
T10_PAD[prac_count_pub].font = C_FONT 

T10_STIA[prac_count_pub] = Copyright
T10_STIA[prac_count_pub].font = C_FONT 

In [ ]:
## Add Source text (Source: Data source, NHS England) to worksheets and format
T10_AF[source_count_ref] = Source
T10_AF[source_count_ref].font = SOURCE_FONT

T10_BP[source_count_ref] = Source
T10_BP[source_count_ref].font = SOURCE_FONT

T10_CHD[source_count_ref] = Source
T10_CHD[source_count_ref].font = SOURCE_FONT

T10_HF[source_count_ref] = Source
T10_HF[source_count_ref].font = SOURCE_FONT

T10_HYP[source_count_ref] = Source
T10_HYP[source_count_ref].font = SOURCE_FONT

T10_LVSD[source_count_ref] = Source
T10_LVSD[source_count_ref].font = SOURCE_FONT

T10_PAD[source_count_ref] = Source
T10_PAD[source_count_ref].font = SOURCE_FONT 

T10_STIA[source_count_ref] = Source
T10_STIA[source_count_ref].font = SOURCE_FONT 

In [ ]:
## Add line at the bottom of the table
## max row and col (range)
for row in T10_AF.iter_cols(min_row=int(source_count_pub)
                            ,max_col=44
                            ,max_row=int(source_count_pub)):
   for cell in row:
       cell.border = T_BORDER

for row in T10_BP.iter_cols(min_row=int(source_count_pub)
                            ,max_col=29
                            ,max_row=int(source_count_pub)):
   for cell in row:
       cell.border = T_BORDER

for row in T10_CHD.iter_cols(min_row=int(source_count_pub)
                             ,max_col=54
                             ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

for row in T10_HF.iter_cols(min_row=int(source_count_pub)
                            ,max_col=60
                            ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

for row in T10_HYP.iter_cols(min_row=int(source_count_pub)
                             ,max_col=46
                             ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

for row in T10_LVSD.iter_cols(min_row=int(source_count_pub)
                              ,max_col=14
                              ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

for row in T10_PAD.iter_cols(min_row=int(source_count_pub)
                             ,max_col=21
                             ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T10_STIA.iter_cols(min_row=int(source_count_pub)
                              ,max_col=54
                              ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

#### Populate table titles and message to worksheets and format

In [ ]:
## Populate table titles and message for indicators in CV higher group
T10_AF_TITLE = prac_table_titles_cv.loc[prac_table_titles_cv["GROUP_CODE"]=="AF"].reset_index(drop = True)
T10_AF["A8"] = T10_AF_TITLE.at[0, "TABLE_TITLE"]
T10_AF["A9"] = message_af
T10_AF["A9"].font = MESSAGE_FONT

T10_BP_TITLE = prac_table_titles_cv.loc[prac_table_titles_cv["GROUP_CODE"]=="BP"].reset_index(drop = True)
T10_BP["A8"] = T10_BP_TITLE.at[0, "TABLE_TITLE"]
T10_BP["A9"] = message_bp
T10_BP["A9"].font = MESSAGE_FONT

T10_CHD_TITLE = prac_table_titles_cv.loc[prac_table_titles_cv["GROUP_CODE"]=="CHD"].reset_index(drop = True)
T10_CHD["A8"] = T10_CHD_TITLE.at[0, "TABLE_TITLE"]
T10_CHD["A9"] = message_chd
T10_CHD["A9"].font = MESSAGE_FONT

T10_HF_TITLE = prac_table_titles_cv.loc[prac_table_titles_cv["GROUP_CODE"]=="HF"].reset_index(drop = True)
T10_HF["A8"] = T10_HF_TITLE.at[0, "TABLE_TITLE"]
T10_HF["A9"] = message_hf
T10_HF["A9"].font = MESSAGE_FONT

T10_HYP_TITLE = prac_table_titles_cv.loc[prac_table_titles_cv["GROUP_CODE"]=="HYP"].reset_index(drop = True)
T10_HYP["A8"] = T10_HYP_TITLE.at[0, "TABLE_TITLE"]
T10_HYP["A9"] = message_hyp
T10_HYP["A9"].font = MESSAGE_FONT

T10_LVSD_TITLE = prac_table_titles_cv.loc[prac_table_titles_cv["GROUP_CODE"]=="LVSD"].reset_index(drop = True)
T10_LVSD["A8"] = T10_LVSD_TITLE.at[0, "TABLE_TITLE"]


T10_PAD_TITLE = prac_table_titles_cv.loc[prac_table_titles_cv["GROUP_CODE"]=="PAD"].reset_index(drop = True)
T10_PAD["A8"] = T10_PAD_TITLE.at[0, "TABLE_TITLE"]
T10_PAD["A9"] = message_pad
T10_PAD["A9"].font = MESSAGE_FONT

T10_STIA_TITLE = prac_table_titles_cv.loc[prac_table_titles_cv["GROUP_CODE"]=="STIA"].reset_index(drop = True)
T10_STIA["A8"] = T10_STIA_TITLE.at[0, "TABLE_TITLE"]
T10_STIA["A9"] = message_stia
T10_STIA["A9"].font = MESSAGE_FONT

#### Populate Fyears in indicator group tables

In [ ]:
## Populate current and previous financial years to indicator groups in CV higher group
T10_AF["H11"] = Previous_year
T10_AF["K11"] = Fyear
T10_AF["O11"] = Previous_year
T10_AF["Q11"] = Fyear
T10_AF["T11"] = Previous_year
T10_AF["W11"] = Fyear

T10_BP["H11"] = Previous_year
T10_BP["I11"] = Fyear
T10_BP["J11"] = Previous_year
T10_BP["L11"] = Fyear
T10_BP["O11"] = Previous_year
T10_BP["R11"] = Fyear

T10_CHD["H11"] = Previous_year
T10_CHD["K11"] = Fyear
T10_CHD["Q11"] = Previous_year
T10_CHD["S11"] = Fyear
T10_CHD["V11"] = Previous_year
T10_CHD["Y11"] = Fyear

T10_HF["H11"] = Previous_year
T10_HF["K11"] = Fyear
T10_HF["O11"] = Previous_year
T10_HF["Q11"] = Fyear
T10_HF["T11"] = Previous_year
T10_HF["W11"] = Fyear

T10_HYP["H11"] = Previous_year
T10_HYP["K11"] = Fyear
T10_HYP["Q11"] = Previous_year
T10_HYP["S11"] = Fyear
T10_HYP["V11"] = Previous_year
T10_HYP["Y11"] = Fyear

T10_LVSD["H11"] = Previous_year
T10_LVSD["K11"] = Fyear

T10_PAD["H11"] = Previous_year
T10_PAD["K11"] = Fyear
T10_PAD["O11"] = Previous_year
T10_PAD["Q11"] = Fyear

T10_STIA["H11"] = Previous_year
T10_STIA["K11"] = Fyear
T10_STIA["Q11"] = Previous_year
T10_STIA["S11"] = Fyear
T10_STIA["V11"] = Previous_year
T10_STIA["Y11"] = Fyear

#### Populate group points in table headers

In [ ]:
## Populate indicator group points for current year
T10_AF["Q12"] = "Total Achievement Score "+group_points_af
T10_BP["L12"] = "Total Achievement Score "+group_points_bp
T10_CHD["S12"] ="Total Achievement Score "+ group_points_chd
T10_HF["Q12"] = "Total Achievement Score "+group_points_hf
T10_HYP["S12"] = "Total Achievement Score "+group_points_hyp
T10_PAD["Q12"] = "Total Achievement Score "+group_points_pad
T10_STIA["S12"] = "Total Achievement Score "+group_points_stia

## Populate indicator group points for previous year
T10_AF["O12"] = "Total Achievement Score "+previous_gr_points_af
T10_BP["J12"] = "Total Achievement Score "+previous_gr_points_bp
T10_CHD["Q12"] = "Total Achievement Score "+previous_gr_points_chd
T10_HF["O12"] = "Total Achievement Score "+previous_gr_points_hf
T10_HYP["Q12"] = "Total Achievement Score "+previous_gr_points_hyp
T10_PAD["O12"] = "Total Achievement Score "+previous_gr_points_pad
T10_STIA["Q12"] = "Total Achievement Score "+previous_gr_points_stia

#### Populate indicator points in table headers

In [ ]:
## Populate indicator points for current year in table headers
T10_AF["AA12"] = "Achievement Score "+ind_points_af001
T10_AF["AC12"] = "Achievement Score "+ind_points_af006
T10_AF["AK12"] = "Achievement Score "+ind_points_af007

T10_BP["V12"] = "Achievement Score "+ind_points_bp002

T10_CHD["AC12"] = "Achievement Score "+ind_points_chd001
T10_CHD["AE12"] = "Achievement Score "+ind_points_chd005

T10_CHD["AM12"] = "Achievement Score "+ind_points_chd008
T10_CHD["AU12"] = "Achievement Score "+ind_points_chd009

T10_HF["AA12"] = "Achievement Score "+ind_points_hf001
T10_HF["AC12"] = "Achievement Score "+ind_points_hf003
T10_HF["AK12"] = "Achievement Score "+ind_points_hf005
T10_HF["AS12"] = "Achievement Score "+ind_points_hf006
T10_HF["BA12"] = "Achievement Score "+ind_points_hf007

T10_HYP["AC12"] = "Achievement Score "+ind_points_hyp001
T10_HYP["AE12"] = "Achievement Score "+ind_points_hyp003
T10_HYP["AM12"] = "Achievement Score "+ind_points_hyp007

T10_PAD["T12"] = "Achievement Score "+ind_points_pad001

T10_STIA["AC12"] = "Achievement Score "+ind_points_stia001
T10_STIA["AE12"] = "Achievement Score "+ind_points_stia007
T10_STIA["AM12"] = "Achievement Score "+ind_points_stia010
T10_STIA["AU12"] = "Achievement Score "+ind_points_stia011

<h2>Save Workbook

In [ ]:
## Save workbook
if test_run is True:
    wb.save(f"test_folder\\qof-"+ short_year +"-prev-ach-pca-cv-prac.xlsx")
elif test_run is False:
    wb.save(outputs+"\qof-" + short_year +"-prev-ach-pca-cv-prac.xlsx")

<h2>Write data to saved excel workbook

In [ ]:
## Write data to saved workbook
if test_run is True:
    writer = pd.ExcelWriter(f"test_folder\\qof-"+ short_year +"-prev-ach-pca-cv-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

elif test_run is False:
    writer = pd.ExcelWriter(outputs+"\qof-"+ short_year +"-prev-ach-pca-cv-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

<h4>Populate worksheets with data</h4><h7>Writes data to saved workbook template by sheet name - startrow includes the header, consequently the data starts at row 13 startrow = 12

In [ ]:
t10_17_af_df.to_excel(writer, sheet_name='AF',index=False, header = False,startrow=12)
t10_17_bp_df.to_excel(writer, sheet_name='BP',index=False, header = False,startrow=12)
t10_17_chd_df.to_excel(writer, sheet_name= 'CHD',index=False, header = False,startrow=12)
t10_17_hf_df.to_excel(writer, sheet_name='HF',index=False, header = False,startrow=12)
t10_17_hyp_df.to_excel(writer, sheet_name='HYP',index=False, header = False,startrow=12)
t10_17_lvsd_df.to_excel(writer, sheet_name='LVSD',index=False, header = False,startrow=12)
t10_17_pad_df.to_excel(writer, sheet_name='PAD',index=False, header = False,startrow=12)
t10_17_stia_df.to_excel(writer, sheet_name='STIA',index=False, header = False,startrow=12)

In [ ]:
## Saves data written to excel workbook
writer.save()

## Template 11 RESP practice Excel table
[Return to contents](#Table-of-Contents)

In [ ]:
## Open excel workbook template to be populated
## TEMPLATE-11-qof-YYYY-prac-ach-prev-pca-resp.xlsx
wb = openpyxl.load_workbook(filename = templates +"\Practices\TEMPLATE-11-qof-YYYY-prac-ach-prev-pca-resp.xlsx")

#### Name the worksheets in the TEMPLATE workbook to be used

In [ ]:
## Add pseudonyms for worksheets in RESP higher group
T11_TITLE = wb["Title sheet"]
T11_AST = wb["AST"]
T11_COPD = wb["COPD"]

#### Populate Publication title, Date and Link on contents page and format

In [ ]:
T11_TITLE["A7"] = QOF_title
T11_TITLE["A7"].font = NAME_FONT
T11_TITLE["B11"] = Publication_date
T11_TITLE["B11"].font = DATE_FONT
T11_TITLE ["B12"].hyperlink = URL
T11_TITLE ["B12"].style = "Hyperlink"
T11_TITLE ["B12"].font = LINK_FONT
T11_TITLE["A15"] = Notes
T11_TITLE["A15"].font = DATE_FONT
T11_TITLE["A29"] = RS_name
T11_TITLE["A29"].font = DATE_FONT

#### Populate copyright for worksheets and format

In [ ]:
T11_TITLE["A33"] = Copyright
T11_TITLE["A33"].font = DATE_FONT

T11_AST[prac_count_pub] = Copyright
T11_AST[prac_count_pub].font = C_FONT

T11_COPD[prac_count_pub] = Copyright
T11_COPD[prac_count_pub].font = C_FONT

In [ ]:
## Add Source text (Source: Data source, NHS England) to worksheets and format
T11_AST[source_count_ref] = Source
T11_AST[source_count_ref].font = SOURCE_FONT

T11_COPD[source_count_ref] = Source
T11_COPD[source_count_ref].font = SOURCE_FONT

In [ ]:
## Add line at the bottom of the table
for row in T11_AST.iter_cols(min_row=int(source_count_pub)
                             ,max_col=53
                             ,max_row=int(source_count_pub)): #max row and col (range)
   for cell in row:
       cell.border = T_BORDER
       
for row in T11_COPD.iter_cols(min_row=int(source_count_pub)
                              ,max_col=44
                              ,max_row=int(source_count_pub)): #max row and col (range)
   for cell in row:
       cell.border = T_BORDER

#### Populate table titles for worksheets and format

In [ ]:
## Populate table titles and message for indicators in RESP higher group
filter = (prac_table_titles_resp["GROUP_CODE"]=="AST")
T11_AST_TITLE = prac_table_titles_resp[filter].reset_index(drop = True)
T11_AST["A8"] = T11_AST_TITLE.at[0, "TABLE_TITLE"]
T11_AST["A9"] = message_ast
T11_AST["A9"].font = MESSAGE_FONT

filter = (prac_table_titles_resp["GROUP_CODE"]=="COPD")
T11_COPD_TITLE = prac_table_titles_resp[filter].reset_index(drop = True)
T11_COPD["A8"] = T11_COPD_TITLE.at[0, "TABLE_TITLE"]
T11_COPD["A9"] = message_copd
T11_COPD["A9"].font = MESSAGE_FONT

#### Populate Fyears in indicator group tables

In [ ]:
## Populate current and previous financial years to indicator groups in RESP higher group
T11_AST["H11"] = Previous_year
T11_AST["K11"] = Fyear
T11_AST["P11"] = Previous_year
T11_AST["R11"] = Fyear
T11_AST["U11"] = Previous_year
T11_AST["X11"] = Fyear

T11_COPD["H11"] = Previous_year
T11_COPD["K11"] = Fyear
T11_COPD["O11"] = Previous_year
T11_COPD["Q11"] = Fyear
T11_COPD["T11"] = Previous_year
T11_COPD["W11"] = Fyear

#### Populate group points in table headers

In [ ]:
## Populate indicator group points for current year
T11_AST["R12"] = "Total Achievement Score "+group_points_ast
T11_COPD["Q12"] = "Total Achievement Score "+group_points_copd

## Populate indicator group points for previous year
T11_AST["P12"] = "Total Achievement Score "+previous_gr_points_ast
T11_COPD["O12"] = "Total Achievement Score "+previous_gr_points_copd

#### Populate indicator points in table headers

In [ ]:
## Populate indicator points for current year in table headers
T11_AST["AB12"] = "Achievement Score "+ind_points_ast005
T11_AST["AD12"] = "Achievement Score "+ind_points_ast006
T11_AST["AL12"] = "Achievement Score "+ind_points_ast007
T11_AST["AT12"] = "Achievement Score "+ind_points_ast008

T11_COPD["AA12"] = "Achievement Score "+ind_points_copd009
T11_COPD["AC12"] = "Achievement Score "+ind_points_copd008
T11_COPD["AK12"] = "Achievement Score "+ind_points_copd010

<h2>Save Workbook

In [ ]:
## Save workbook
if test_run is True:
    wb.save(f"test_folder\\qof-"+ short_year +"-prev-ach-pca-resp-prac.xlsx")
elif test_run is False:
    wb.save(outputs+"\qof-"+ short_year +"-prev-ach-pca-resp-prac.xlsx")

<h2>Write data to saved excel workbook

In [ ]:
## Write data to saved workbook
if test_run is True:
    writer = pd.ExcelWriter(f"test_folder\\qof-"+ short_year +"-prev-ach-pca-resp-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

elif test_run is False:
    writer = pd.ExcelWriter(outputs+"\qof-"+ short_year +"-prev-ach-pca-resp-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

<h4>Populate worksheets with data</h4><h7>Writes data to saved workbook template by sheet name - startrow includes the header, consequently the data starts at row 13 startrow = 12

In [ ]:
t10_17_ast_df.to_excel(writer, sheet_name='AST',index=False, header = False,startrow=12)
t10_17_copd_df.to_excel(writer, sheet_name='COPD',index=False, header = False,startrow=12)

In [ ]:
## Saves data written to excel workbook
writer.save()

## Template 12 LS practice Excel table
[Return to contents](#Table-of-Contents)

In [ ]:
## Open excel workbook template to be populated
## TEMPLATE-12-qof-YYYY-prac-ach-prev-pca-ls.xlsx
wb = openpyxl.load_workbook(filename = templates +"\Practices\TEMPLATE-12-qof-YYYY-prac-ach-prev-pca-ls.xlsx")

#### Name the worksheets in the TEMPLATE workbook to be used

In [ ]:
## Add the pseudonyms for worksheets in the TEMPLATE workbook to be used
T12_TITLE = wb["Title sheet"]
T12_OB = wb["OB"]
T12_SMOK = wb["SMOK"]

#### Populate Publication title, Date and Link on contents page and format

In [ ]:
T12_TITLE["A7"] = QOF_title
T12_TITLE["A7"].font = NAME_FONT
T12_TITLE["B11"] = Publication_date
T12_TITLE["B11"].font = DATE_FONT
T12_TITLE ["B12"].hyperlink = URL
T12_TITLE ["B12"].style = "Hyperlink"
T12_TITLE ["B12"].font = LINK_FONT
T12_TITLE["A15"] = Notes
T12_TITLE["A15"].font = DATE_FONT
T12_TITLE["A29"] = RS_name
T12_TITLE["A29"].font = DATE_FONT

#### Populate copyright for worksheets and format

In [ ]:
T12_TITLE["A33"] = Copyright
T12_TITLE["A33"].font = DATE_FONT

T12_OB[prac_count_pub] = Copyright
T12_OB[prac_count_pub].font = C_FONT

T12_SMOK[prac_count_pub] = Copyright
T12_SMOK[prac_count_pub].font = C_FONT

In [ ]:
## Add Source text (Source: Data source, NHS England) to worksheets and format
T12_OB[source_count_ref] = Source
T12_OB[source_count_ref].font = SOURCE_FONT

T12_SMOK[source_count_ref] = Source
T12_SMOK[source_count_ref].font = SOURCE_FONT

In [ ]:
## Add line at the bottom of the table 
## max row and col (range)      
for row in T12_OB.iter_cols(min_row=int(source_count_pub)
                            ,max_col=21
                            ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T12_SMOK.iter_cols(min_row=int(source_count_pub)
                              ,max_col=45
                              ,max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

#### Populate table titles for worksheets and format

In [ ]:
## Populate table titles and message for indicators in LS higher group
T12_OB_TITLE = prac_table_titles_ls.loc[prac_table_titles_ls["GROUP_CODE"]=="OB"].reset_index(drop = True)
T12_OB["A8"] = T12_OB_TITLE.at[0, "TABLE_TITLE"]
T12_OB["A9"] = message_ob
T12_OB["A9"].font = MESSAGE_FONT

T12_SMOK_TITLE = prac_table_titles_ls.loc[prac_table_titles_ls["GROUP_CODE"]=="SMOK"].reset_index(drop = True)
T12_SMOK["A8"] = T12_SMOK_TITLE.at[0, "TABLE_TITLE"]
T12_SMOK["A9"] = message_smok
T12_SMOK["A9"].font = MESSAGE_FONT

#### Populate Fyears in indicator group tables

In [ ]:
## Populate current and previous years for indicator groups in LS higher group
T12_OB["H11"] = Previous_year
T12_OB["K11"] = Fyear
T12_OB["O11"] = Previous_year
T12_OB["Q11"] = Fyear

T12_SMOK["H11"] = Previous_year
T12_SMOK["I11"] = Fyear
T12_SMOK["J11"] = Previous_year
T12_SMOK["L11"] = Fyear
T12_SMOK["O11"] = Previous_year
T12_SMOK["R11"] = Fyear

#### Populate group points in table headers

In [ ]:
## Populate indicator group points for current year
T12_OB["O12"] = "Total Achievement Score "+group_points_ob
T12_SMOK["J12"] = "Total Achievement Score "+group_points_smok

## Populate indicator group points for previous year
T12_OB["Q12"] = "Total Achievement Score "+previous_gr_points_ob
T12_SMOK["L12"] = "Total Achievement Score "+previous_gr_points_smok

#### Populate indicator points in table headers

In [ ]:
## Populate indicator points for current year in table headers
T12_OB["T12"] = "Achievement Score "+ind_points_ob002

T12_SMOK["V12"] = "Achievement Score "+ind_points_smok002
T12_SMOK["AD12"] = "Achievement Score "+ind_points_smok004
T12_SMOK["AL12"] = "Achievement Score "+ind_points_smok005

<h2>Save Workbook

In [ ]:
## Save workbook
if test_run is True:
    wb.save(f"test_folder\\qof-"+ short_year +"-prev-ach-pca-ls-prac.xlsx")
elif test_run is False:
    wb.save(outputs+"\qof-"+ short_year +"-prev-ach-pca-ls-prac.xlsx")

<h2>Write data to saved excel workbook

In [ ]:
## Write data to saved workbook
if test_run is True:
    writer = pd.ExcelWriter(f"test_folder\\qof-"+ short_year +"-prev-ach-pca-ls-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

elif test_run is False:
    writer = pd.ExcelWriter(outputs+"\qof-"+ short_year +"-prev-ach-pca-ls-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

<h4>Populate worksheets with data</h4><h7>Writes data to saved workbook template by sheet name - startrow includes the header, consequently the data starts at row 13 startrow = 12

In [ ]:
t10_17_ob_df.to_excel(writer, sheet_name='OB',index=False, header = False,startrow=12)
t10_17_smok_df.to_excel(writer, sheet_name='SMOK',index=False, header = False,startrow=12)

In [ ]:
## Saves data written to excel workbook
writer.save()

## Template 13 HD practice Excel table
[Return to contents](#Table-of-Contents)

In [ ]:
## Open excel workbook template to be populated
## TEMPLATE-13-qof-YYYY-prac-ach-prev-pca-hd.xlsx
wb = openpyxl.load_workbook(filename = templates +"\Practices\TEMPLATE-13-qof-YYYY-prac-ach-prev-pca-hd.xlsx")

#### Name the worksheets in the TEMPLATE workbook to be used

In [ ]:
## Add the pseudonyms for worksheets in the TEMPLATE workbook to be used
T13_TITLE = wb["Title sheet"]
T13_CAN = wb["CAN"]
T13_CKD = wb["CKD"]
T13_DM = wb["DM"]
T13_NDH = wb["NDH"]
T13_PC = wb["PC"]

#### Populate Publication title, Date and Link on contents page and format

In [ ]:
T13_TITLE["A7"] = QOF_title
T13_TITLE["A7"].font = NAME_FONT
T13_TITLE["B11"] = Publication_date
T13_TITLE["B11"].font = DATE_FONT
T13_TITLE ["B12"].hyperlink = URL
T13_TITLE ["B12"].style = "Hyperlink"
T13_TITLE ["B12"].font = LINK_FONT
T13_TITLE["A15"] = Notes
T13_TITLE["A15"].font = DATE_FONT
T13_TITLE["A32"] = RS_name
T13_TITLE["A32"].font = DATE_FONT

#### Populate copyright for worksheets and format

In [ ]:
T13_TITLE["A38"] = Copyright
T13_TITLE["A38"].font = DATE_FONT

T13_CAN[prac_count_pub] = Copyright
T13_CAN[prac_count_pub].font = C_FONT

T13_CKD[prac_count_pub] = Copyright
T13_CKD[prac_count_pub].font = C_FONT

T13_DM[prac_count_pub] = Copyright
T13_DM[prac_count_pub].font = C_FONT

T13_NDH[prac_count_pub] = Copyright
T13_NDH[prac_count_pub].font = C_FONT

T13_PC[prac_count_pub] = Copyright
T13_PC[prac_count_pub].font = C_FONT

In [ ]:
## Add Source text (Source: Data source, NHS England) to worksheets and format
T13_CAN[source_count_ref] = Source
T13_CAN[source_count_ref].font = SOURCE_FONT
T13_CKD[source_count_ref] = Source
T13_CKD[source_count_ref].font = SOURCE_FONT 
T13_DM[source_count_ref] = Source
T13_DM[source_count_ref].font = SOURCE_FONT 
T13_NDH[source_count_ref] = Source 
T13_NDH[source_count_ref].font = C_FONT 
T13_PC[source_count_ref] = Source
T13_PC[source_count_ref].font = SOURCE_FONT 

In [ ]:
## Add line at the bottom of the table   
## max row and col (range)    
for row in T13_CAN.iter_cols(min_row=int(source_count_pub)
                             ,max_col=44,
                             max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T13_CKD.iter_cols(min_row=int(source_count_pub)
                             ,max_col=21,
                             max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

for row in T13_DM.iter_cols(min_row=int(source_count_pub)
                            ,max_col=93,
                            max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T13_NDH.iter_cols(min_row=int(source_count_pub)
                             ,max_col=33,
                             max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T13_PC.iter_cols(min_row=int(source_count_pub)
                            ,max_col=21,
                            max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

#### Populate table titles for worksheets and format

In [ ]:
## Populate table titles and message for indicators in HD higher group
T13_CAN_TITLE = prac_table_titles_hd.loc[prac_table_titles_hd["GROUP_CODE"]=="CAN"].reset_index(drop = True)
T13_CAN["A8"] = T13_CAN_TITLE.at[0, "TABLE_TITLE"]
T13_CAN["A9"] = message_can
T13_CAN["A9"].font = MESSAGE_FONT

T13_CKD_TITLE = prac_table_titles_hd.loc[prac_table_titles_hd["GROUP_CODE"]=="CKD"].reset_index(drop = True)
T13_CKD["A8"] = T13_CKD_TITLE.at[0, "TABLE_TITLE"]
T13_CKD["A9"] = message_ckd
T13_CKD["A9"].font = MESSAGE_FONT

T13_DM_TITLE = prac_table_titles_hd.loc[prac_table_titles_hd["GROUP_CODE"]=="DM"].reset_index(drop = True)
T13_DM["A8"] = T13_DM_TITLE.at[0, "TABLE_TITLE"]
T13_DM["A9"] = message_dm
T13_DM["A9"].font = MESSAGE_FONT

T13_NDH_TITLE = prac_table_titles_hd.loc[prac_table_titles_hd["GROUP_CODE"]=="NDH"].reset_index(drop = True)
T13_NDH["A8"] = T13_NDH_TITLE.at[0, "TABLE_TITLE"]
T13_NDH["A9"] = message_ndh
T13_NDH["A9"].font = MESSAGE_FONT

T13_PC_TITLE = prac_table_titles_hd.loc[prac_table_titles_hd["GROUP_CODE"]=="PC"].reset_index(drop = True)
T13_PC["A8"] = T13_PC_TITLE.at[0, "TABLE_TITLE"]
T13_PC["A9"] = message_pc
T13_PC["A9"].font = MESSAGE_FONT

#### Populate Fyears in indicator group tables

In [ ]:
## Populate current and previous financial years for indicator groups in HD higher group
T13_CAN["H11"] = Previous_year
T13_CAN["K11"] = Fyear
T13_CAN["O11"] = Previous_year
T13_CAN["Q11"] = Fyear
T13_CAN["T11"] = Previous_year
T13_CAN["W11"] = Fyear

T13_CKD["H11"] = Previous_year
T13_CKD["K11"] = Fyear
T13_CKD["O11"] = Previous_year
T13_CKD["Q11"] = Fyear

T13_DM["H11"] = Previous_year
T13_DM["K11"] = Fyear
T13_DM["P11"] = Previous_year
T13_DM["R11"] = Fyear
T13_DM["U11"] = Previous_year
T13_DM["X11"] = Fyear

T13_NDH["H11"] = Previous_year
T13_NDH["K11"] = Fyear
T13_NDH["O11"] = Previous_year
T13_NDH["Q11"] = Fyear
T13_NDH["T11"] = Previous_year
T13_NDH["W11"] = Fyear

T13_PC["H11"] = Previous_year
T13_PC["K11"] = Fyear
T13_PC["O11"] = Previous_year
T13_PC["Q11"] = Fyear

#### Populate group points in table headers

In [ ]:
## Populate current years indicator group points
T13_CAN["Q12"] = "Total Achievement Score "+group_points_can
T13_CKD["Q12"] = "Total Achievement Score "+group_points_ckd
T13_DM["R12"] = "Total Achievement Score "+group_points_dm
T13_NDH["Q12"] = "Total Achievement Score "+group_points_ndh
T13_PC["Q12"] = "Total Achievement Score "+group_points_pc

## Populate previous years indicator group points
T13_CAN["O12"] = "Total Achievement Score "+previous_gr_points_can
T13_CKD["O12"] = "Total Achievement Score "+previous_gr_points_ckd
T13_DM["P12"] = "Total Achievement Score "+previous_gr_points_dm
T13_NDH["O12"] = "Total Achievement Score "+previous_gr_points_ndh
T13_PC["O12"] = "Total Achievement Score "+previous_gr_points_pc

#### Populate indicator points in table headers

In [ ]:
## Populate current years indicator points
T13_CAN["AA12"] = "Achievement Score "+ind_points_can001
T13_CAN["AC12"] = "Achievement Score "+ind_points_can004
T13_CAN["AK12"] = "Achievement Score "+ind_points_can005

T13_CKD["T12"] = "Achievement Score "+ind_points_ckd005

T13_DM["AB12"] = "Achievement Score "+ind_points_dm017
T13_DM["AD12"] = "Achievement Score "+ind_points_dm006
T13_DM["AL12"] = "Achievement Score "+ind_points_dm012
T13_DM["AT12"] = "Achievement Score "+ind_points_dm014
T13_DM["BB12"] = "Achievement Score "+ind_points_dm019
T13_DM["BJ12"] = "Achievement Score "+ind_points_dm020
T13_DM["BR12"] = "Achievement Score "+ind_points_dm021
T13_DM["BZ12"] = "Achievement Score "+ind_points_dm022
T13_DM["CH12"] = "Achievement Score "+ind_points_dm023

T13_NDH["AA12"] = "Achievement Score "+ind_points_ndh001

T13_PC["T12"] = "Achievement Score "+ind_points_pc001

<h2>Save Workbook

In [ ]:
## Save workbook
if test_run is True:
    wb.save(f"test_folder\\qof-"+ short_year +"-prev-ach-pca-hd-prac.xlsx")
elif test_run is False: 
    wb.save(outputs+"\qof-"+ short_year +"-prev-ach-pca-hd-prac.xlsx")

<h2>Write data to saved excel workbook

In [ ]:
## Write data to saved workbook
if test_run is True:
    writer = pd.ExcelWriter(f"test_folder\\qof-"+ short_year +"-prev-ach-pca-hd-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

elif test_run is False:
    writer = pd.ExcelWriter(outputs+"\qof-"+ short_year +"-prev-ach-pca-hd-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

<h4>Populate worksheets with data</h4><h7>Writes data to saved workbook template by sheet name - startrow includes the header, consequently the data starts at row 13 startrow = 12

In [ ]:
t10_17_can_df.to_excel(writer, sheet_name='CAN',index=False, header = False,startrow=12)
t10_17_ckd_df.to_excel(writer, sheet_name='CKD',index=False, header = False,startrow=12)
t10_17_dm_df.to_excel(writer, sheet_name='DM',index=False, header = False,startrow=12)
t10_17_ndh_df.to_excel(writer, sheet_name='NDH',index=False, header = False,startrow=12)
t10_17_pc_df.to_excel(writer, sheet_name='PC',index=False, header = False,startrow=12)

In [ ]:
## Saves data written to excel workbook
writer.save()

## Template 14 NEU practice Excel table
[Return to contents](#Table-of-Contents)

In [ ]:
## Open excel workbook template to be populated
## TEMPLATE-14-qof-YYYY-prac-ach-prev-pca-neu.xlsx
wb = openpyxl.load_workbook(filename = templates +"\Practices\TEMPLATE-14-qof-YYYY-prac-ach-prev-pca-neu.xlsx")

#### Name the worksheets in the TEMPLATE workbook to be used

In [ ]:
## Add the pseudonyms for worksheets in the TEMPLATE workbook to be used
T14_TITLE = wb["Title sheet"]
T14_DEM = wb["DEM"]
T14_DEP = wb["DEP"]
T14_EP = wb["EP"]
T14_LD = wb["LD"]
T14_MH = wb["MH"]

#### Populate Publication title, Date and Link on contents page and format

In [ ]:
T14_TITLE["A7"] = QOF_title
T14_TITLE["A7"].font = NAME_FONT
T14_TITLE["B11"] = Publication_date
T14_TITLE["B11"].font = DATE_FONT
T14_TITLE ["B12"].hyperlink = URL
T14_TITLE ["B12"].style = "Hyperlink"
T14_TITLE ["B12"].font = LINK_FONT
T14_TITLE["A15"] = Notes
T14_TITLE["A15"].font = DATE_FONT
T14_TITLE["A32"] = RS_name
T14_TITLE["A32"].font = DATE_FONT

#### Populate copyright for worksheets and format

In [ ]:
T14_TITLE["A38"] = Copyright
T14_TITLE["A38"].font = DATE_FONT

T14_DEM[prac_count_pub] = Copyright
T14_DEM[prac_count_pub].font = C_FONT

T14_DEP[prac_count_pub] = Copyright
T14_DEP[prac_count_pub].font = C_FONT

T14_EP[prac_count_pub] = Copyright
T14_EP[prac_count_pub].font = C_FONT

T14_LD[prac_count_pub] = Copyright
T14_LD[prac_count_pub].font = C_FONT

T14_MH[prac_count_pub] = Copyright
T14_MH[prac_count_pub].font = C_FONT

In [ ]:
## Add Source text (Source: Data source, NHS England) to worksheets and format
T14_DEM[source_count_ref] = Source
T14_DEM[source_count_ref].font = SOURCE_FONT

T14_DEP[source_count_ref] = Source
T14_DEP[source_count_ref].font = SOURCE_FONT

T14_EP[source_count_ref] = Source
T14_EP[source_count_ref].font = SOURCE_FONT

T14_LD[source_count_ref] = Source
T14_LD[source_count_ref].font = SOURCE_FONT

T14_MH[source_count_ref] = Source
T14_MH[source_count_ref].font = SOURCE_FONT

In [ ]:
## Add line at the bottom of the table  
## max row and col (range)     
for row in T14_DEM.iter_cols(min_row=int(source_count_pub),
                             max_col=36,
                             max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T14_DEP.iter_cols(min_row=int(source_count_pub),
                             max_col=34,
                             max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T14_EP.iter_cols(min_row=int(source_count_pub),
                            max_col=21,
                            max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T14_LD.iter_cols(min_row=int(source_count_pub),
                            max_col=21,
                            max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T14_MH.iter_cols(min_row=int(source_count_pub),
                            max_col=76,
                            max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

#### Populate table titles in worksheets and format

In [ ]:
## Populate table titles and message for indicators in NEU higher group
T14_DEM_TITLE = prac_table_titles_neu.loc[prac_table_titles_neu["GROUP_CODE"]=="DEM"].reset_index(drop = True)
T14_DEM["A8"] = T14_DEM_TITLE.at[0, "TABLE_TITLE"]
T14_DEM["A9"] = message_dem
T14_DEM["A9"].font = MESSAGE_FONT

T14_DEP_TITLE = prac_table_titles_neu.loc[prac_table_titles_neu["GROUP_CODE"]=="DEP"].reset_index(drop = True)
T14_DEP["A8"] = T14_DEP_TITLE.at[0, "TABLE_TITLE"]
T14_DEP["A9"] = message_dep
T14_DEP["A9"].font = MESSAGE_FONT

T14_EP_TITLE = prac_table_titles_neu.loc[prac_table_titles_neu["GROUP_CODE"]=="EP"].reset_index(drop = True)
T14_EP["A8"] = T14_EP_TITLE.at[0, "TABLE_TITLE"]
T14_EP["A9"] = message_ep
T14_EP["A9"].font = MESSAGE_FONT

T14_LD_TITLE = prac_table_titles_neu.loc[prac_table_titles_neu["GROUP_CODE"]=="LD"].reset_index(drop = True)
T14_LD["A8"] = T14_LD_TITLE.at[0, "TABLE_TITLE"]
T14_LD["A9"] = message_ld
T14_LD["A9"].font = MESSAGE_FONT

T14_MH_TITLE = prac_table_titles_neu.loc[prac_table_titles_neu["GROUP_CODE"]=="MH"].reset_index(drop = True)
T14_MH["A8"] = T14_MH_TITLE.at[0, "TABLE_TITLE"]
T14_MH["A9"] = message_mh
T14_MH["A9"].font = MESSAGE_FONT

#### Populate Fyears in indicator group tables

In [ ]:
## Populate current and previous financial years for indicator groups in NEU higher group
T14_DEM["H11"] = Previous_year
T14_DEM["K11"] = Fyear
T14_DEM["O11"] = Previous_year
T14_DEM["Q11"] = Fyear
T14_DEM["T11"] = Previous_year
T14_DEM["W11"] = Fyear

T14_DEP["H11"] = Previous_year
T14_DEP["K11"] = Fyear
T14_DEP["O11"] = Previous_year
T14_DEP["Q11"] = Fyear
T14_DEP["T11"] = Previous_year
T14_DEP["W11"] = Fyear

T14_EP["H11"] = Previous_year
T14_EP["K11"] = Fyear
T14_EP["O11"] = Previous_year
T14_EP["Q11"] = Fyear

T14_LD["H11"] = Previous_year
T14_LD["K11"] = Fyear
T14_LD["O11"] = Previous_year
T14_LD["Q11"] = Fyear

T14_MH["H11"] = Previous_year
T14_MH["K11"] = Fyear
T14_MH["O11"] = Previous_year
T14_MH["Q11"] = Fyear
T14_MH["T11"] = Previous_year
T14_MH["W11"] = Fyear

#### Populate group points in table headers

In [ ]:
## Populate current years indicator group points
T14_DEM["Q12"] = "Total Achievement Score "+group_points_dem
T14_DEP["Q12"] = "Total Achievement Score "+group_points_dep
T14_EP["Q12"] = "Total Achievement Score "+group_points_ep
T14_LD["Q12"] = "Total Achievement Score "+group_points_ld
T14_MH["Q12"] = "Total Achievement Score "+group_points_mh

## Populate previous years indicator group points
T14_DEM["O12"] = "Total Achievement Score "+previous_gr_points_dem
T14_DEP["O12"] = "Total Achievement Score "+previous_gr_points_dep
T14_EP["O12"] = "Total Achievement Score "+previous_gr_points_ep
T14_LD["O12"] = "Total Achievement Score "+previous_gr_points_ld
T14_MH["O12"] = "Total Achievement Score "+previous_gr_points_mh


<h4>Populate indicator points in table headers

In [ ]:
## Populate current years indicator points
T14_DEM["AA12"] = "Achievement Score "+ind_points_dem001
T14_DEM["AC12"] = "Achievement Score "+ind_points_dem004

T14_DEP["AA12"] = "Achievement Score "+ind_points_dep003

T14_EP["T12"] = "Achievement Score "+ind_points_ep001

T14_LD["T12"] = "Achievement Score "+ind_points_ld004

T14_MH["AA12"] = "Achievement Score "+ind_points_mh001
T14_MH["AC12"] = "Achievement Score "+ind_points_mh002
T14_MH["AK12"] = "Achievement Score "+ind_points_mh003
T14_MH["AS12"] = "Achievement Score "+ind_points_mh006
T14_MH["BA12"] = "Achievement Score "+ind_points_mh007
T14_MH["BI12"] = "Achievement Score "+ind_points_mh011
T14_MH["BQ12"] = "Achievement Score "+ind_points_mh012

<h2>Save Workbook

In [ ]:
## Save workbook
if test_run is True:
    wb.save(f"test_folder\\qof-"+ short_year +"-prev-ach-pca-neu-prac.xlsx")
elif test_run is False:
    wb.save(outputs+"\qof-"+ short_year +"-prev-ach-pca-neu-prac.xlsx")

<h2>Write data to saved excel workbook

In [ ]:
## Write data to saved workbook
if test_run is True:
    writer = pd.ExcelWriter(f"test_folder\\qof-"+ short_year +"-prev-ach-pca-neu-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

elif test_run is False:
    writer = pd.ExcelWriter(outputs+"\qof-"+ short_year +"-prev-ach-pca-neu-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

<h4>Populate worksheets with data</h4><h7>Writes data to saved workbook template by sheet name - startrow includes the header, consequently the data starts at row 13 startrow = 12

In [ ]:
t10_17_dem_df.to_excel(writer, sheet_name='DEM',index=False, header = False,startrow=12)
t10_17_dep_df.to_excel(writer, sheet_name='DEP',index=False, header = False,startrow=12)
t10_17_ep_df.to_excel(writer, sheet_name='EP',index=False, header = False,startrow=12)
t10_17_ld_df.to_excel(writer, sheet_name='LD',index=False, header = False,startrow=12)
t10_17_mh_df.to_excel(writer, sheet_name='MH',index=False, header = False,startrow=12)

In [ ]:
## Saves data written to excel workbook
writer.save()

## Template 15 MS practice Excel table
[Return to contents](#Table-of-Contents)

In [ ]:
## Open excel workbook template to be populated
## TEMPLATE-15-qof-YYYY-prac-ach-prev-pca-ms.xlsx
wb = openpyxl.load_workbook(filename = templates +"\Practices\TEMPLATE-15-qof-YYYY-prac-ach-prev-pca-ms.xlsx")

#### Name the worksheets in the TEMPLATE workbook to be used

In [ ]:
## Add the pseudonyms for worksheets in the TEMPLATE workbook to be used
T15_TITLE = wb["Title sheet"]
T15_OST = wb["OST"]
T15_RA = wb["RA"]

#### Populate Publication title, Date and Link on contents page and format

In [ ]:
T15_TITLE["A7"] = QOF_title
T15_TITLE["A7"].font = NAME_FONT
T15_TITLE["B11"] = Publication_date
T15_TITLE["B11"].font = DATE_FONT
T15_TITLE ["B12"].hyperlink = URL
T15_TITLE ["B12"].style = "Hyperlink"
T15_TITLE ["B12"].font = LINK_FONT
T15_TITLE["A15"] = Notes
T15_TITLE["A15"].font = DATE_FONT
T15_TITLE["A29"] = RS_name
T15_TITLE["A29"].font = DATE_FONT

#### Populate copyright for worksheets and format

In [ ]:
T15_TITLE["A35"] = Copyright
T15_TITLE["A35"].font = DATE_FONT

T15_OST[prac_count_pub] = Copyright
T15_OST[prac_count_pub].font = C_FONT

T15_RA[prac_count_pub] = Copyright
T15_RA[prac_count_pub].font = C_FONT

In [ ]:
## Add Source text (Source: Data source, NHS England) to worksheets and format
T15_OST[source_count_ref] = Source
T15_OST[source_count_ref].font = SOURCE_FONT 

T15_RA[source_count_ref] = Source
T15_RA[source_count_ref].font = SOURCE_FONT

In [ ]:
## Add line at the bottom of the table       
for row in T15_OST.iter_cols(min_row=int(source_count_pub),
                             max_col=21,
                             max_row=int(source_count_pub)): #max row and col (range)
   for cell in row:
       cell.border = T_BORDER
       
for row in T15_RA.iter_cols(min_row=int(source_count_pub),
                            max_col=36,
                            max_row=int(source_count_pub)): #max row and col (range)
   for cell in row:
       cell.border = T_BORDER

#### Populate table titles in worksheets and format

In [ ]:
## Populate table titles and message for indicators in MS higher group
T15_OST_TITLE = prac_table_titles_ms.loc[prac_table_titles_ms["GROUP_CODE"]=="OST"].reset_index(drop = True)
T15_OST["A8"] = T15_OST_TITLE.at[0, "TABLE_TITLE"]
T15_OST["A9"] = message_ost
T15_OST["A9"].font = MESSAGE_FONT

T15_RA_TITLE = prac_table_titles_ms.loc[prac_table_titles_ms["GROUP_CODE"]=="RA"].reset_index(drop = True)
T15_RA["A8"] = T15_RA_TITLE.at[0, "TABLE_TITLE"]
T15_RA["A9"] = message_ra
T15_RA["A9"].font = MESSAGE_FONT

#### Populate Fyears in indicator group tables

In [ ]:
## Populate current and previous financial years for indicator groups in MS higher group
T15_OST["H11"] = Previous_year
T15_OST["K11"] = Fyear
T15_OST["O11"] = Previous_year
T15_OST["Q11"] = Fyear

T15_RA["H11"] = Previous_year
T15_RA["K11"] = Fyear
T15_RA["O11"] = Previous_year
T15_RA["Q11"] = Fyear
T15_RA["T11"] = Previous_year
T15_RA["W11"] = Fyear

#### Populate group points in table headers

In [ ]:
## Populate current years indicator group points 
T15_OST["O12"] = "Total Achievement Score "+group_points_ost
T15_RA["O12"] = "Total Achievement Score "+group_points_ra

## Populate previous years indicator group points 
T15_OST["Q12"] = "Total Achievement Score "+previous_gr_points_ost
T15_RA["Q12"] = "Total Achievement Score "+previous_gr_points_ra

#### Populate indicator points in table headers

In [ ]:
## Populate current years indicator points 
T15_OST["T12"] = "Achievement Score "+ind_points_ost004

T15_RA["AA12"] = "Achievement Score "+ind_points_ra001
T15_RA["AC12"] = "Achievement Score "+ind_points_ra002

<h2>Save Workbook

In [ ]:
## Save workbook
if test_run is True:
    wb.save(f"test_folder\\qof-"+ short_year +"-prev-ach-pca-ms-prac.xlsx")
elif test_run is False:
    wb.save(outputs+"\qof-"+ short_year +"-prev-ach-pca-ms-prac.xlsx")

<h2>Write data to saved excel workbook

In [ ]:

## Write data to saved workbook
if test_run is True:
    writer = pd.ExcelWriter(f"test_folder\\qof-"+ short_year +"-prev-ach-pca-ms-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

elif test_run is False:
    writer = pd.ExcelWriter(outputs+"\qof-"+ short_year +"-prev-ach-pca-ms-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

<h4>Populate worksheets with data</h4><h7>Writes data to saved workbook template by sheet name - startrow includes the header, consequently the data starts at row 13 startrow = 12

In [ ]:
t10_17_ost_df.to_excel(writer, sheet_name='OST',index=False, header = False,startrow=12)
t10_17_ra_df.to_excel(writer, sheet_name='RA',index=False, header = False,startrow=12)

In [ ]:
## Saves data written to excel workbook
writer.save()

## Template 16 FER OBS GYN practice Excel table
[Return to contents](#Table-of-Contents)

In [ ]:
## Open excel workbook template to be populated
## TEMPLATE-16-qof-YYYY-prac-ach-prev-pca-fer-obs-gyn.xlsx
wb = openpyxl.load_workbook(filename = templates +"\Practices\TEMPLATE-16-qof-YYYY-prac-ach-prev-pca-fer-obs-gyn.xlsx")

#### Name the worksheets in the TEMPLATE workbook to be used

In [ ]:
## Add the pseudonyms for worksheets in the TEMPLATE workbook to be used
T16_TITLE = wb["Title sheet"]
T16_CS = wb["CS"]

#### Populate Publication title, Date and Link on contents page and format

In [ ]:
T16_TITLE["A7"] = QOF_title
T16_TITLE["A7"].font = NAME_FONT
T16_TITLE["B11"] = Publication_date
T16_TITLE["B11"].font = DATE_FONT
T16_TITLE ["B12"].hyperlink = URL
T16_TITLE ["B12"].style = "Hyperlink"
T16_TITLE ["B12"].font = LINK_FONT
T16_TITLE["A15"] = Notes
T16_TITLE["A15"].font = DATE_FONT
T16_TITLE["A28"] = RS_name
T16_TITLE["A28"].font = DATE_FONT

#### Populate copyright for worksheets and format

In [ ]:
T16_TITLE["A34"] = Copyright
T16_TITLE["A34"].font = DATE_FONT

T16_CS[prac_count_pub] = Copyright
T16_CS[prac_count_pub].font = C_FONT

In [ ]:
## Add Source text (Source: Data source, NHS England) to worksheets and format
T16_CS[source_count_ref] = Source
T16_CS[source_count_ref].font = C_FONT

In [ ]:
## Add line at the bottom of the table  
## max row and col (range)     
for row in T16_CS.iter_cols(min_row=int(source_count_pub),
                            max_col=38,
                            max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

#### Populate table titles to worksheets and format

In [ ]:
## Populate table titles and message for indicators in FER OBS GYN higher group
T16_CS_TITLE = prac_table_titles_gyn.loc[prac_table_titles_gyn["GROUP_CODE"]=="CS"].reset_index(drop = True)
T16_CS["A8"] = T16_CS_TITLE.at[0, "TABLE_TITLE"]
T16_CS["A9"] = message_cs
T16_CS["A9"].font = MESSAGE_FONT

#### Populate Fyears in indicator group tables

In [ ]:
## Populate current and previous financial years for indicator groups in FER OBS GYN higher group
T16_CS["H11"] = Previous_year
T16_CS["I11"] = Fyear
T16_CS["K11"] = Previous_year
T16_CS["M11"] = Fyear
T16_CS["P11"] = Previous_year
T16_CS["S11"] = Fyear

#### Populate group points in table headers

In [ ]:
## Add current years indicator group points
T16_CS["M12"] = "Total Achievement Score "+group_points_cs

## Add previous years indicator group points
T16_CS["K12"] = "Total Achievement Score "+previous_gr_points_cs

#### Populate indicator points in table headers

In [ ]:
## Add current years indicator points
T16_CS["W12"] = "Achievement Score "+ind_points_cs005
T16_CS["AE12"] = "Achievement Score "+ind_points_cs006

<h2>Save Workbook

In [ ]:
## Save workbook
if test_run is True:
    wb.save(f"test_folder\\qof-"+ short_year +"-prev-ach-pca-fer-obs-gyn-prac.xlsx")
elif test_run is False:
    wb.save(outputs+"\qof-"+ short_year +"-prev-ach-pca-fer-obs-gyn-prac.xlsx")

<h2>Write data to saved excel workbook

In [ ]:
## Write data to saved workbook
if test_run is True:
    writer = pd.ExcelWriter(f"test_folder\\qof-"+ short_year +"-prev-ach-pca-fer-obs-gyn-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

elif test_run is False:
    writer = pd.ExcelWriter(outputs+"\qof-"+ short_year +"-prev-ach-pca-fer-obs-gyn-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

<h4>Populate worksheets with data</h4><h7>Writes data to saved workbook template by sheet name - startrow includes the header, consequently the data starts at row 13 startrow = 12

In [ ]:
t10_17_cs_df.to_excel(writer, sheet_name='CS',index=False, header = False,startrow=12)

In [ ]:
## Saves data written to excel workbook
writer.save()

## Template 17 QI practice Excel table
[Return to contents](#Table-of-Contents)

In [ ]:
## Open excel workbook template to be populated
## TEMPLATE-17-qof-YYYY-prac-ach-pca-qual-imp.xlsx
wb = openpyxl.load_workbook(filename = templates +"\Practices\TEMPLATE-17-qof-YYYY-prac-ach-pca-qual-imp.xlsx")

#### Name the worksheets in the TEMPLATE workbook to be used

In [ ]:
## Add the pseudonyms for worksheets in the TEMPLATE workbook to be used
T17_TITLE = wb["Title sheet"]
T17_QI_1 = wb["QIPDD"]
T17_QI_2 = wb["QIOA"]
T17_QI_3 = wb["QIPDDMI"]

#### Populate Publication title, Date and Link on contents page and format

In [ ]:
T17_TITLE["A7"] = QOF_title
T17_TITLE["A7"].font = NAME_FONT
T17_TITLE["B11"] = Publication_date
T17_TITLE["B11"].font = DATE_FONT
T17_TITLE ["B12"].hyperlink = URL
T17_TITLE ["B12"].style = "Hyperlink"
T17_TITLE ["B12"].font = LINK_FONT
T17_TITLE["A15"] = Notes
T17_TITLE["A15"].font = DATE_FONT
T17_TITLE["A30"] = RS_name
T17_TITLE["A30"].font = DATE_FONT

#### Populate copyright for worksheets and format

In [ ]:
T17_TITLE["A36"] = Copyright
T17_TITLE["A36"].font = DATE_FONT

T17_QI_1[prac_count_pub] = Copyright
T17_QI_1[prac_count_pub].font = C_FONT

T17_QI_2[prac_count_pub] = Copyright
T17_QI_2[prac_count_pub].font = C_FONT

T17_QI_3[prac_count_pub] = Copyright
T17_QI_3[prac_count_pub].font = C_FONT

In [ ]:
## Add Source text (Source: Data source, NHS England) to worksheets and format
T17_QI_1[source_count_ref] = Source
T17_QI_1[source_count_ref].font = SOURCE_FONT

T17_QI_2[source_count_ref] = Source
T17_QI_2[source_count_ref].font = SOURCE_FONT

T17_QI_3[source_count_ref] = Source
T17_QI_3[source_count_ref].font = SOURCE_FONT

In [ ]:
## Add line at the bottom of the table   
## max row and col (range)    
for row in T17_QI_3.iter_cols(min_row=int(source_count_pub),
                              max_col=44,
                              max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T17_QI_1.iter_cols(min_row=int(source_count_pub),
                              max_col=13,
                              max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER
       
for row in T17_QI_2.iter_cols(min_row=int(source_count_pub),
                              max_col=13,
                              max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

#### Populate table titles to worksheets and format

In [ ]:
## Populate table titles and message for indicators in QI higher group
T17_QI_1_TITLE = prac_table_titles_qi.loc[prac_table_titles_qi["GROUP_CODE"] == "QI_1"].reset_index(drop = True)
T17_QI_1["A8"] = T17_QI_1_TITLE.at[0, "TABLE_TITLE"]
T17_QI_1["A9"] = message_qi1
T17_QI_1["A9"].font = MESSAGE_FONT

T17_QI_2_TITLE = prac_table_titles_qi.loc[prac_table_titles_qi["GROUP_CODE"] == "QI_2"].reset_index(drop = True)
T17_QI_2["A8"] = T17_QI_2_TITLE.at[0, "TABLE_TITLE"]
T17_QI_2["A9"] = message_qi2
T17_QI_2["A9"].font = MESSAGE_FONT

T17_QI_3_TITLE = prac_table_titles_qi.loc[prac_table_titles_qi["GROUP_CODE"] == "QI_3"].reset_index(drop = True)
T17_QI_3["A8"] = T17_QI_3_TITLE.at[0, "TABLE_TITLE"]
T17_QI_3["A9"] = message_qi3
T17_QI_3["A9"].font = MESSAGE_FONT

<h4>Populate Fyears in indicator group tables

In [ ]:
## Populate current financial year for indicator groups in QI higher group
## QI indicators only contain data for the current year
T17_QI_1["H11"] = Fyear
T17_QI_1["I11"] = Fyear

T17_QI_2["H11"] = Fyear
T17_QI_2["I11"] = Fyear

T17_QI_3["H11"] = Fyear
T17_QI_3["I11"] = Fyear
T17_QI_3["K11"] = Fyear

<h4>Populate group points in table headers

In [ ]:
## Populate indicator group points for current year
T17_QI_1["I12"] = "Total Achievement Score "+group_points_qi1
T17_QI_2["I12"] = "Total Achievement Score "+group_points_qi2
T17_QI_3["I12"] = "Total Achievement Score "+group_points_qi3

<h4>Populate indicator points in table headers

In [ ]:
## Populate indicator points for current year in table headers
T17_QI_1["K12"] = "Achievement Score "+ind_points_qipdd009
T17_QI_1["M12"] = "Achievement Score "+ind_points_qipdd010

T17_QI_2["K12"] = "Achievement Score "+ind_points_qioa011
T17_QI_2["M12"] = "Achievement Score "+ind_points_qioa012

T17_QI_3["N12"] = "Achievement Score "+ind_points_qipddmi001
T17_QI_3["V12"] = "Achievement Score "+ind_points_qipddmi002
T17_QI_3["AD12"] = "Achievement Score "+ind_points_qipddmi003
T17_QI_3["AL12"] = "Achievement Score "+ind_points_qipddmi004

<h2>Save Workbook

In [ ]:
## Save workbook
if test_run is True:
    wb.save(f"test_folder\\qof-"+ short_year +"-prev-ach-qual-imp-prac.xlsx")
elif test_run is False:
    wb.save(outputs+"\qof-"+ short_year +"-prev-ach-qual-imp-prac.xlsx")

<h2>Write data to saved excel workbook

In [ ]:
## Write data to saved workbook
if test_run is True:
    writer = pd.ExcelWriter(f"test_folder\\qof-"+ short_year +"-prev-ach-qual-imp-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

elif test_run is False:
    writer = pd.ExcelWriter(outputs+"\qof-"+ short_year +"-prev-ach-qual-imp-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

<h4>Populate worksheets with data</h4><h7>Writes data to saved workbook template by sheet name - startrow includes the header, consequently the data starts at row 13 startrow = 12

In [ ]:
t10_17_qi1_df.to_excel(writer, sheet_name='QIPDD',index=False, header = False,startrow=12)
t10_17_qi2_df.to_excel(writer, sheet_name='QIOA',index=False, header = False,startrow=12)
t10_17_qi3_df.to_excel(writer, sheet_name='QIPDDMI',index=False, header = False,startrow=12)

In [ ]:
## Saves data written to excel workbook
writer.save()

## Template 19 VI practice Excel table
[Return to contents](#Table-of-Contents)

In [ ]:
## Open excel workbook template to be populated
## TEMPLATE-19-qof-YYYY-prac-ach-prev-pca-vi.xlsx
wb = openpyxl.load_workbook(filename = templates +"\Practices\TEMPLATE-19-qof-YYYY-prac-ach-prev-pca-vi.xlsx")

#### Name the worksheets in the TEMPLATE workbook to be used

In [ ]:
## Add the pseudonyms for worksheets in the TEMPLATE workbook to be used
T19_TITLE = wb["Title sheet"]
T19_VI = wb["VI"]

#### Populate Publication title, Date and Link on contents page and format

In [ ]:
T19_TITLE["A7"] = QOF_title
T19_TITLE["A7"].font = NAME_FONT
T19_TITLE["B11"] = Publication_date
T19_TITLE["B11"].font = DATE_FONT
T19_TITLE ["B12"].hyperlink = URL
T19_TITLE ["B12"].style = "Hyperlink"
T19_TITLE ["B12"].font = LINK_FONT
T19_TITLE["A15"] = Notes
T19_TITLE["A15"].font = DATE_FONT
T19_TITLE["A28"] = RS_name
T19_TITLE["A28"].font = DATE_FONT

#### Populate copyright for worksheets and format

In [ ]:
T19_TITLE["A34"] = Copyright
T19_TITLE["A34"].font = DATE_FONT

T19_VI[prac_count_pub] = Copyright
T19_VI[prac_count_pub].font = C_FONT

In [ ]:
## Add Source text (Source: Data source, NHS England) to worksheets and format
T19_VI[source_count_ref] = Source
T19_VI[source_count_ref].font = SOURCE_FONT

In [ ]:
## Add line at the bottom of the table 
## max row and col (range)      
for row in T19_VI.iter_cols(min_row=int(source_count_pub),
                            max_col=38,
                            max_row=int(source_count_pub)): 
   for cell in row:
       cell.border = T_BORDER

#### Populate table titles to worksheets and format

In [ ]:
## Populate table titles and message for indicators in VI higher group
T19_VI_TITLE = prac_table_titles_vi.loc[prac_table_titles_vi["GROUP_CODE"]=="VI"].reset_index(drop = True)
T19_VI["A8"] = T19_VI_TITLE.at[0, "TABLE_TITLE"]
T19_VI["A9"] = message_vi
T19_VI["A9"].font = MESSAGE_FONT

#### Populate Fyears in indicator group tables

In [ ]:
## Populate current and previous financial years for indicator group in VI higher group
T19_VI["H11"] = Fyear
T19_VI["L11"] = Previous_year
T19_VI["N11"] = Fyear
T19_VI["Q11"] = Previous_year
T19_VI["T11"] = Fyear

#### Populate group points in table headers

In [ ]:
## Populate indicator group points for previous year
T19_VI["L12"] = "Total Achievement Score "+previous_gr_points_vi

## Populate indicator group points for current year
T19_VI["N12"] = "Total Achievement Score "+group_points_vi

#### Populate indicator points in table headers

In [ ]:
## Populate indicator points for current year in table headers
T19_VI["X12"] = "Achievement Score "+ind_points_vi001
T19_VI["AF12"] = "Achievement Score "+ind_points_vi002
T19_VI["AN12"] = "Achievement Score "+ind_points_vi003
T19_VI["AV12"] = "Achievement Score "+ind_points_vi004

<h2>Save Workbook

In [ ]:
## Save workbook
if test_run is True:
    wb.save(f"test_folder\\qof-"+ short_year +"-prev-ach-pca-vi-prac.xlsx")
elif test_run is False:
    wb.save(outputs+"\qof-"+ short_year +"-prev-ach-pca-vi-prac.xlsx")

<h2>Write data to saved excel workbook

In [ ]:
## Write data to saved workbook in specified folder
if test_run is True:
    writer = pd.ExcelWriter(f"test_folder\\qof-"+ short_year +"-prev-ach-pca-vi-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

elif test_run is False:
    writer = pd.ExcelWriter(outputs+"\qof-"+ short_year +"-prev-ach-pca-vi-prac.xlsx", engine="openpyxl") 
    writer.book = wb
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)

<h4>Populate worksheets with data</h4><h7>Writes data to saved workbook template by sheet name - startrow includes the header, consequently the data starts at row 13 startrow = 12

In [ ]:
t10_17_vi_df.to_excel(writer, sheet_name='VI',index=False, header = False,startrow=12)

In [ ]:
## Saves data written to excel workbook
writer.save()